In [ ]:
# Step 1: Install required libraries
!pip install -q sentence-transformers

# Step 2: Import required libraries
from sentence_transformers import SentenceTransformer, util
import torch
import pickle

# Step 3: Load a multilingual SentenceTransformer model - here we only consider the retrieval prompt.
model = SentenceTransformer('sentence-transformers/LaBSE')


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 27.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 20.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 27.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 70.4 MB/s eta 0:00:00


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/461 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/2.22k [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.36M [00:00<?, ?B/s]

2_Dense/config.json:   0%|          | 0.00/114 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.36M [00:00<?, ?B/s]

In [ ]:
!pip install -qU sentence-transformers faiss-cpu
from sklearn.preprocessing import normalize
import faiss

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 275.9/275.9 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 50.8 MB/s eta 0:00:00


In [ ]:
# Step 4: Precompute document embeddings (fine-tuning part)
# Replace this with your actual dataset of Sinhala documents.
documents = [
      "දිනය -  2016.06.10\nස්ථාන භාර නිළධාරී,\nපළාත් පාලන පුහුණු මධ්‍යස්ථානය,\nනවයාලතැන්න,\nකටුගස්තොට.\nපළාත් පාලන ආයතන කාර්යාල කළමනාකරණය සම්බන්ධයෙන් නවක කළමනාකරණ සහකාරවරුන් දැනුවත් කිරීමේ වැඩමුළුව - මධ්‍යම පළාත \nඋක්ත කරුණ සම්බන්ධයෙන් මධ්‍යම පළාත් පාලන කොමසාරිස්ගේ අංක සීපීසී/සීඑල්ජී/01/AC/පුහුණු/1/2/2016-07 හා 2016.05.06 දිනැති ලිපිය හා බැඳේ.\n02. ඒ අනුව උක්ත වැඩසටහන සඳහා මෙම ආයතනය නියෝජනය කරමින් පහත නම් සඳහන් නිළධාරීන් සහභාගී වන බව කාරුණිකව දන්වමි.\n01. S.රම්‍යා ප්‍රභා මිය - කලමනාකරණ සහකාර. \n02. H.M.N.සෙව්වන්දි මිය - කලමනාකරණ සහකාර. \nනාගරික ලේකම්, \nමාතලේ."
    ,
      "නම :\nලිපිනය : \nදුරකථන අංකය :\nදිනය :\nදිස්ත්‍රික් සම්බන්ධීකරණ නිලධාරී මගින් සාමාන්‍යාධිකාරී තුමා,\nශ්‍රී ලංකා සමාජ ආරක්ෂණ මණ්ඩලය.\nවිශ්‍රාම වැටුප් යෝජනා ක්‍රමයේ නැවත දායකත්වය ලැබීම.\nඉහත දායක අංකය යටතේ සමාජ ආරක්ෂණ විශ්‍රාම වැටුප් යෝජනා ක්‍රමයට දායක අංක ........................ යටතේ දායකත්වය ලබාසිටි මගේ දායකත්වය අක්‍රීය ව ඇත. \nමාහට දායකත්ව යෝජනා ක්‍රමයට වෙනස් කර ගැනීමට අවශ්‍යව ඇත.\nඒ අනුව මාහට සුරැකුම / ඉසුරු යෝජනා ක්‍රමයේ නව සාමාජිකත්වය ලබාදෙන ලෙසත් පැරණි ගිණුමේ මුදල් නව ගිණුමට බැරකර දෙන ලෙසත් කාරුණිකව ඉල්ලමි.\nනව යෝජනා ක්‍රමයේ දායකත්වය ලැබීම සඳහා පහත ලේඛන මේ සමග බාරදෙමි.\n1. පැරණි ගිණුමේ ඔප්පුව\n2. පැරණි ගිණුමේ වාරික මුදල් සටහන් පොත / ජයා පිටපත\n3. ජාතික හැඳුනුම්පතේ ජයා පිටපත\n4. බැංකුවට මුදල් ගෙවූ SSB10 බැරපත් වල ජයා පිටපත්\nසමාජ ආරක්ෂණ විශ්‍රාම වැටුප් යෝජනා ක්‍රමයේ  නැවත දායකත්වය ලැබීම සඳහා ඉහත ලේඛන බාරගත් අතර, ඔබගේ නව දායක  අංකය ........................ බව දන්වමි. \nවාරික මුදල් ගෙවීම සඳහා ................................ අංක දරණ පාස්පොත් නිකුත් කර ඇත.\n නව ඔප්පුව ඉදිරියේදී ලැබෙනු ඇත."
    ,
      "ලේකම්\nවිදේශරැකියා අමාත්‍යාංශය,\nඅංක 51/2/1,දෙවන මහල ,\nඇසට්අ ආකේඩ් ගොඩනැගිල්ල,\nකොළඹ 01.\nඅභ්‍යන්තර ස්ථාන මාරුවීම.\nජේ.ඒ.ටී.ආර්. ජයසූරිය මිය (සංවර්ධන නිලධාරි iii) පත්වීම් ලිපි අංක CS/DOS/30926.\nඋක්ත කරුණ සම්බන්ධයෙන් මා මගින් ඔබ අමතා ඇති නාඋල ප්‍රාදේශිය ලේකම්ගේ අංක 6/1/7/1/38 හා 2016.06.08 දිනැති ලිපිය හා  බැඳේ.\n02.ඒ අනුව ඔබ අමාත්‍යාංශ යටතේ නාඋල ප්‍රාදේශිය ලේකම් කාර්යාලයේ  සේවයට වාර්තා කල සංවර්ධන නිලධාරි ජේ.ඒ.ටී.ආර්. ජයසූරිය මියගේ සේවයට වාර්තා කිරිමට අදාළ ලිපි ලේඛන මේ සමඟ අමුණා කාරුණිකව ඉදිරිපත් කරමි.\nතේජානි තිලකරත්න,\nඅතිරේක දිස්ත්‍රික් ලේකම්,\nමාතලේ  දිස්ත්‍රික් ලේකම් වෙනුවට."
    ,
      "දිනය 2015.11.03\nඅධ්‍යක්‍ෂ ජනරාල්,\nජනලේඛන හා සංඛ්‍යාලේඛන දෙපාර්තමේන්තුව.\nජන හා නිවාස සංගණනය හා ආර්ථික සංගණනයේ දැනට නිකුත් වී ඇති දත්ත පිළිබඳව දිස්ත්‍රික් සංඛ්‍යාලේඛන අංශ ප්‍රධානීන් දැනුවත් කිරීමේ වැඩසටහන - 2015.11.13 \nඉහත කරුණට අදාල ඔබේ අංක 11/ECR/2015 හා 2015.11.02 දිනැති ලිපිය හා බැඳේ.\n2. ඉහත වැඩසටහන සඳහා සහභාගී වීමට දිස්ත්‍රික් ප්‍රවාහන සැලැස්මට අනුව ප්‍රවාහන පහසුකම් සපයා ගන්නා ලෙස දැනුම් දී ඇති අතර මෙම කාර්යාලයට අයත් වාහනයෙන් මහනුවර හා කෑගල්ල සංඛ්‍යාලේඛනඥ වරුන්ටද ප්‍රවාහන පහසුකම් සැලසිය යුතුවේ. \nදැනට මෙම කාර්යාලයට අනුයුක්ත රියදුරු ශල්‍ය කර්මයකට ලක්ව දින කිහිපයක සිට නිවාඩු ලබා සිටින බැවින්, එදිනට මෙම කාර්යාලයට අයත් වාහනය පාවිච්චි කළ නොහැකි තත්වයක් ඇති වී ඇත. \nඒ අනුව ඉහත දිස්ත්‍රික්ක 3හි ප්‍රධානින්ගේ ප්‍රවාහන පහසුකම් සපයා ගත යුතු ආකාරය සම්බන්ධයෙන් උපදෙස් පතමි.\nදිස්ත්‍රික් සංඛ්‍යාලේඛනඥ,\nමාතලේ."
    ,
      "දිනය 2016.05.23\nඅධ්‍යක්‍ෂ ජනරාල්, \nජනලේඛන හා සංඛ්‍යා ලේඛන දෙපාර්තමේන්තුව,\n306/71,\nබත්තරමුල්ල.\nප්‍රජා විද්‍යාත්මක හා සෞඛ්‍ය සමීක්ෂණය .\nඉහත සඳහන් සමීක්ෂණය සඳහා සහභාගී වන නිලධාරීන් සඳහා නවාතැන් සැපයීම සඳහා නිවසක් කුලියට ගැනීමට ඔබ විසින් මා වෙත එවන ලද අංක 1/DHS/Accomadation 2015 දරණ 2016.04.27 දිනැති ලිපිය අනුව මිල ගණන් කැඳවන ලදී. \nප්‍රාදේශීය වශයෙන් මිල ගණන් කැඳවන ලද අතර මාතලේ නගරයේ කාර්යාල කීපයක හා ග්‍රාම නිලධාරී ආසන්න වසම්වල දැන්වීම් ප්‍රසිද්ධ කරන ලදී.\nලංසු මිල ගණන් 2016.04.27 දින විවෘත කරන ලදී.\n එක් මිල ගණනක් පමණක් ලැබී තිබුණි. \nවිස්තර පහත සඳහන් වේ,\n01. ජී.කේ.එන්.ඒ. ද සිල්වා මහතා   මාසික කුලිය 40,000/-\nමෙම නිවස මාතලේ නගරයේ ශ්‍රී සංඝමිත්තා බාලිකා පාසල ඉදිරිපිට ඔබ විසින් අවශ්‍ය බව දන්වා ඇති පහසුකම් සහිත නිවාසයකි. \nමිල ගණන් වල සහතික කල පිටපතක් මේ සමඟ එවන අතර නිවාස හිමිකරු සමඟ සාකච්ඡා කර සුදුසුනම් කුලියට ගැනීමට කටයුතු කරන මෙන් දන්වා සිටිමි.\nනීල් ද අල්විස් ,\nදිස්ත්‍රික් ලේකම් ,\nදිස්ත්‍රික් ලේකම් කාර්යාලය මාතලේ."
    ,
      "මගේ අංකය : DSKU/SWD/SS/SSB4/3/\nසිහිකැඳවීමයි\nසමාජ ආරක්ෂණ විශ්‍රාම වැටුප් යෝජනා ක්‍රමය\nජාතික හැඳුනුම්පත් අංකය ලබාගැනීම\nදායක අංක - \nසමාජ ආරක්ෂණ විශ්‍රාම වැටුප් යෝජනා ක්‍රමයේ ඉසුරු/ සුරැකුම යෝජනා ක්‍රමයට ඉහත දායක අංක යටතේ දායකත්වය ලැබීමේදී ඔබ විසින් ලබාදී ඇති අයදුම්පත්‍රය හා බැඳේ.\nමෙම යෝජනා ක්‍රමයේ දායකත්වය ලබාගැනීමේදී ජාතික හැඳුනුම්පත් අංකය අනිවාර්ය බව දන්වන අතර, ඔබ අයදුම්පත්‍රය පිරවීමේදී ජාතික හැඳුනුම්පත් අංකය සඳහන් නොකිරීම මත ඔබගේ අයදුම්පත්‍රය පරිගණකගත කිරීමේ ගැටළුවක් පැන නැගී ඇති බවද දන්වමි.\nකරුණාකර ඔබගේ ජාතික හැඳුනුම්පත් අංකය 037-2221895 දුරකථන අංකය වෙත ඉතා ඉක්මනින් දැනුම් දීමට කටයුතු කර පසුව එහි ජයා පිටපතක් මෙම ලිපියේ පිටපතක් සමග අප වෙත\nඑවීමටද කටයුතු කරන ලෙස දන්වමි.\nලේඛන යොමුකළ යුතු ලිපිනය\nදිස්ත්‍රික් සම්බන්ධීකරණ නිලධාරී, සමාජ ආරක්ෂණ විශ්‍රාම වැටුප් අංශය,\nදිස්ත්‍රික් ලේකම් කාර්යාලය,\nකුරුණෑගල.\nදිස්ත්‍රික් සම්බන්ධීකරණ නිලධාරී."
    ,
      "ලේකම්,\nස්වදේශ කටයුතු අමාත්‍යාංශය,\nනිදහස් චතුරශ්‍රය ,\nකොළඹ 07.\nසංවර්ධන නිලධාරි තනතුරෙහි රාජකාරි සඳහා වාර්තා කිරීම.\nඋක්ත කරුණ සම්බන්ධයෙන් මා මගින් ඔබ අමතා ඇති ලග්ගල පල්ලෙගම ප්‍රාදේශිය ලේකම්ගේ 2016.05.30 හා 2016.06.10 දිනැති ලිපි හා බැඳේ.\nඒ අනුව ස්වදේශ කටයුතු අමාත්‍යාංශය යටතේ ලග්ගල පල්ලෙගම ප්‍රාදේශිය ලේකම් කාර්යාලයේ සේවයට වාර්තා කල පහත නම් සඳහන් නිලධාරින්ගේ සේවයට වාර්තා කිරීමේ ආකෘති පත්‍ර මේ සමඟ අමුණා කාරුණිකව ඉදිරිපත් කරමි.\n1.එච්,එම්.යූ.ජී.ජයන්ත හේරත්\n11.එල්.ඩබ්.බස්නායක \nතේජානි තිලකරත්න,\nඅතිරේක දිස්ත්‍රික් ලේකම්,\nමාතලේ  දිස්ත්‍රික් ලේකම් වෙනුවට."
    ,
      "දිනය 2016.03.28\nපළාත් පාලන සහකාර කොමසාරිස් - මාතලේ මඟින්,\nපළාත් පාලන කොමසාරිස් - මධ්‍යම පළාත මඟින්,\nව්‍යාපෘති අධ්‍යක්‍ෂ,\nපිළිසරු ව්‍යාපෘතිය,\nමධ්‍යම පරිසර අධිකාරිය, \n\,\n104, ඩෙන්සිල් කොබ්බෑකඩුව මාවත,\nබත්තරමුල්ල.\nමාතලේ මහා නගර සභාවේ කොම්පෝස්ට් අංගනය වැඩිදියුණු කිරීම.\nමධ්‍යම පරිසර අධිකාරියේ පිළිසරු ව්‍යාපෘතිය මඟින් ප්‍රතිපාදන ලබාදීම මත මාතලේ මහා නගර සභාවේ කොම්පෝස්ට් අංගනය වැඩිදියුණු කිරීමේ ව්‍යාපෘතියේ ඉදිකිරීම් වලට අදාලවූ අවසන් බිල්පත කොන්ත්‍රාත් ආයතනය විසින් මාතලේ මහා නගර සභාව වෙත ඉදිරිපත් කර ඇත.\n02. එම බිල්පතට අදාල මුදල ප්‍රතිපූර්ණය කිරීම සඳහා අදාල නිර්දේශ සහිතව මේ සමඟ ඔබ වෙත ඉදරිපත් කරමි.\nනාගරික කොමසාරිස්, \nමාතලේ."
    ,
      "මගේ අංකය : සේ.අ.අ.ගෙවීම්/දී.කම්./පොදු උපදෙස්/2016,\nසේ.අ.අ.ගෙවීම් අංශය ,\n2016.05.18,\nසහකාර කම්කරු කොමසාරිස් ,\nදිස්ත්‍රික් කම්කරු කාර්යාලය ,\nමඩකලපුව .\nඅංක 136 වන ප්‍රගති සමාලෝචන රැස්වීමේදී ගත් වැදගත් තීරණ. \nශ්‍රී ලංකා මහ බැංකුව සහ කම්කරු දෙපාර්තුමේන්තුව මඟින් ඉහත අංක දරණ ප්‍රගති සමාලෝචන රැස්වීමේදී පහත කරුණු ක්‍රියාත්මක කිරීමට එකඟ වී ඇත. \nඒවා ක්‍රියාත්මක කිරීමට මෙසේ යොමු කරමි.\nකම්කරු දෙපාර්තුමේන්තුවේ පරිගණක පද්ධතියේ හා RN වාර්තාවට අනුව සාමාජික අංක ඒකාබද්ධ වී ඇති විට හා එම ඒකාබද්ධ වූ අංක සඳහා ශ්‍රී ලංකා මහ බැංකුවේ ශේෂයක් නොපෙන්වන විටදී අදාළ අංක වල ඒකාබද්ධයක් ඇති බව තීරණ ලිපියේ  Other Remarks වල ඇතුළත් කිරීමටත්,\n  උදා : 256/A/001 Amalgamated with 257/A/002 in department of labour තීරණ ලිපියේ හි අර්ථලාභියාගේ සම්පුර්ණ නම නොයොදා මුලකුරු සමඟ නම පමණක්  ඇතුළත් කිරීම සහ හවුල් ගිණුමක් දී ඇති අවස්ථාවකදී අර්ථලාභියාගේ නම ගිණුම් හිමියාගේ නම යටතේ යෙදීමටත් , \nතවද 60% , 40% වශයෙන් ප්‍රතිලාභ කොටස් වශයෙන් නිදහස් කිරීමේ දී අදාළ කොටස් වෙන වෙනම බැර විය යුතු  අය හා කොටස් බෙදිය යුතු කාලය පැහැදිලිව ඇතුළත් ලිපියක් තීරණ ලිපිය සමඟ යැවීමටත් ,\n අක්‍රීය ආයතනවල සාමාජික නමෙහි අනන්‍යතාවය සනාථ කර ගැනීමේ දී ඊට අදාළව ශ්‍රී ලංකා මහ බැංකුවෙන් දායක මුදල් විස්තර ඉල්ලුම් කිරීමේ දී අවශ්‍යතාවය පැහැදිලිව දන්වා යැවීමටත්,\n ප්‍රතිලාභ නිදහස් කිරීමේ දී දේශීය ගිණුම් අංක යොමු කිරීමේ දී අදාළ ගිණුම් පිටපත් අධිකාරී වෙත නොයැවීමටත්, \n විදේශ ගිණුම් සදහා නිර්දේශ කිරීමේ දී අදාළ ගිණුම් පිටපත් තීරණ ලිපි හා අනු තීරණ ලිපි සමග යැවීමටත් , \n සක්‍රීය‍ ආයතන වල අදාළ ප්‍රතිලාභ මුදල් අවිනිශ්චිත සාමාජික අංක වලට (90,000 ට වැඩි) ගිණුම් ගත වී ඇති විට ඒවා අදාළ ආයතන මගින් ස්ථිර අංකයකට මාරු කර ගැනීමටත් ,\n වැසූ ආයතනවල අදාළ අවිනිශ්චිත අංකයෙහි අනන්‍යතාවය  සනාථ කරගෙන අධිකාරී වෙත යැවීමටත්,\n ප්‍රතිලාභ ගෙවීම් සදහා යොමු කරන තීරණ ලිපිවල අර්ථලාභීන්ගේ බැංකු ශාඛා කේත අංකය (Branch Code)අදාළ බැංකු ගිණුම් අංකයේ ඇතුළත් අංකයම ද යන්න පරික්ෂා කිරීමෙන් පසු තීරණ ලිපි යැවීමටත්,\n අදාළ බැංකු ගිණුම් අංකය හා අදාළ ශාඛා අංකය නොගැළපෙන විට දී එහි අනන්‍යතාවය අදාළ ශාඛා කළමනාකාරතුමා  මඟින් සනාථ කර ඉදිරි පියවර ගැනීමට කටයුතු කිරීමටත් , \n අදාළ බැංකු ගිණුම් අංකයට අදාළ ගිණුම් පිටපත් ලබා ගැනීමේදී අදාළ ශාඛා අංකය ද ඇතුළත් පරිදි අදාළ පිටපත ලබා  ගැනීමටත්,\n අදාළ ගිණුම මව් ශාඛාවේ , උප ශාඛාවක් වූ විට දී (විශේෂයෙන් මහජන බැංකු ගිණුම් වලදී) එහි කේත අංකය ඇතුළත් නොවේ නම් එහි ප්‍රධාන ශාඛාවේ කේත අංකයට ප්‍රධාන ශාඛාව තෝරා එහි උප ශාඛාවක් යන්න සනාථ කර යැවීමටත් කටයුතු කරන්න.\nනියෝජ්‍ය කම්කරු කොමසාරිස් ,\nසේ.අ.අ. අංශය ."
    ,
      "සහකාර අධ්‍යක්ෂ (නැගෙනහිර).\nකෝවිල් කුලම් පුරාවස්තු ස්ථානය සම්බන්ධව.\nඉහත ස්ථානය සම්බන්ධයෙන් 2015.03.12 දිනැතිව මා වෙත ලැබුණු ඇමතුම හා සබැඳේ.\n02. ඒ අනුව සෙල්වරාජා නැමැති මන්ත්‍රීවරයෙකු විසින් දන්වා සිටියේ මෙම ස්ථානය මේ වන විට යම්කිසි පිරිසක් විනාසකාරි ක්‍රියාකාරකම් සිදු කරන බවයි. \nඑබැවින් මේ සම්බන්ධයෙන් පරික්ෂාකොට බලා සුදුසු ක්‍රියාමාර්ග ගන්න මෙන් කාරුණිකව දන්වා සිටිමි.\nඑන්. බී. කරව්ගහන්ග,\nවැ.ආ. සහකාර අධ්‍යක්ෂ (නිශ්චළ දේපල ).\nඅත්සන් කලේ  ආචාර්ය සෙනරත් දිසානායක ,\nපුරාවිද්‍යා අධ්‍යක්ෂ ජනරාල්.\nපිටපත\nදිස්ත්‍රික් ලේකම් - දිස්ත්‍රික් ලේකම් කාර්යාලය, මඩකලපුව - කරු. දැන ගැනීම සඳහා"
    ,
      "මධ්‍යම පළාත් ආණ්ඩුකාරවර,\nගරු නිලුකා ඒකනායක මැතිතුමිය,\nආණ්ඩුකාර කාර්යාලය,\nමහනුවර.\nගරු ආණ්ඩුකාරතුමියෙනි,\nපොදු මගී ප්‍රවාහන සේවය සඳහා ව්‍යුහාත්මක ප්‍රතිසංස්කරණ ක්‍රියාත්මක කිරීම.\nඔබ තුමිය සමග මා මෑතකදී සාකච්ඡා කරන ලද මධ්‍යම පළාත් මගී ප්‍රවාහන සේවා අධිකාරිය විසින් පිළියෙළ කරන ලද  ඉහත ව්‍යපෘති වාර්තාව හා බැඳේ.\nඒ අනුව එම ව්‍යාපෘති වාර්තාව ඔබතුමියගේ කාරුණික අවධානය සඳහා ඉදිරිපත් කරන අතර ඒ මගින් ඉදිරිපත් කර ඇති කරුණු සාකච්ඡා කිරීමට සුදුසු දිනයක් ලබා දෙන මෙන් කාරුණිකව ඉල්ලමි.\nගාමිණී රාජරත්න,\nලේකම්.\nපිටපත් : \nගරු  එදිරිවීර වීරවර්ධන \nමධ්‍යම පළාත් මාර්ග සංවර්ධන, ප්‍රවාහන, විදුලිබල හා බලශක්ති සහ නිවාස හා ඉදිකිරීම් අමාත්‍ය - කාරුණික දැනගැනීම සඳහා\nගරු  එරික් වීරවර්ධන\nසභාපති\nමධ්‍යම පළාත් මගී ප්‍රවාහන සේවා අධිකාරිය - කාරුණික දැනගැනීම සඳහා"
    ,
      "දිනය 2016.06.22 \nමානව සම්පත් විධායක,\nකීල්ස් සුපර් ආයතනය,\nඅංක 489, විලියම් ගොපල්ලව මාවත,\nමහනුවර.\nමාතලේ දිස්ත්‍රික් රැකියා 100 වැඩසටහන \nමිනිස් බල හා රැකි රක්ෂා දෙපාර්තමේන්තුවේ, මාතලේ දිස්ත්‍රික් ලේකම් කාර්යාලීය වෘත්තීය මාර්ගෝපදේශ අංශය හා මහජන රැකියා සේවා මධ්‍යස්ථානය මඟින් සංවිධානය කරන ඉහත සඳහන් වැඩසටහන 2016 ජුලි මස 08 වැනි දින පෙ.ව. 10.00 සිට ප.ව. 2.00 දක්වා මාතලේ ප්‍රාදේශීය ලේකම් කාර්යාල පරිශ්‍රයේදී පැවැත්වේ.\nමෙම වැඩසටහන සඳහා මාතලේ දිස්ත්‍රික්කය පුරා විසිරී සිටින රැකියා අපේක්‍ෂිත තරුණ තරුණියන් 200ක් පමණ සහභාගී වේ. \nඔවුන්ගේ අධ්‍යාපන හා වෘත්තීය සුදුසුකම් අනුව එදින ම පෞද්ගලික අංශයේ විවිධ තනතුරු සඳහා යොමුකිරීමට බලාපොරොත්තු වන බැවින් ඔබ ආයතනයට ද මෙම මහඟු අවස්ථාවට සහභාගී වන ලෙස කාරුණිකව දන්වා සිටිමි.\nමෙම කාර්යය සඳහා ඔබගේ පැමිණීම ඉතාමත් අගය කොට සලකමි.\nතේජානි තිලකරත්න,\nඅතිරේක දිස්ත්‍රික් ලේකම්, \nමාතලේ දිස්ත්‍රික් ලේකම් වෙනුවට.\n(කරුණාකර, ආයතනයේ පැමිණීම තහවුරු කරගැනීමට හා ස්ථානයේ ඉඩ පහසුකම් සලසා ගැනීම සඳහා පහත දුරකථන අංකවලට අමතන්න.)\nනෙවිල් කුමාර - 071-9177695, 066-2222824."
    ,
      "ඒකාබද්ධ සේවා අධ්‍යක්ෂ ජනරාල්,\nරාජ්‍ය පරිපාලන හා කළමනාකරණ අමාත්‍යාංශය.\nනිදහස් චතුරශ්‍රය,\nකොළඹ 07.\nසංවර්ධන නිලධාරි තනතුරෙහි රාජකාරි සඳහා වර්තා කිරීම.\nඋක්ත කරුණ හ සම්බන්ධයෙන් මා  ඇමතූ විදේශ රැකියා අමාත්‍යාංශ ලේකම්ගේ  අංක MFE/RAD/3/4 vol(iii) හා  2016.04.26 දිනැති ලිපිය හා බැඳේ. \n02. ඒ අනුව විදේශ රැකියා අමාත්‍යාංශය යටතේ මාතලේ දිස්ත්‍රික් ලේකම් කාර්යාලයේ සේවයට වාර්තා කල සංවර්ධන නිලධාරි ඒ.එම්.පී.කේ. රත්නායක මහතාගේ සේවයට වාර්තා කිරීමේ ආකෘති පත්‍රය මේ සමඟ අමුණා කාරුණිකව ඉදිරිපත් කරමි.\nසුභාෂිණී විජේසිංහ,\nපරිපාලන නිලධාරී,\nමාතලේ දිස්ත්‍රික් ලේකම් වෙනුවට.\nපිටපත් :- \n01. ලේකම්,  ස්වදේශ කටයුතු අමාත්‍යාංශ\t කා.දැ.ගැ.ස.\n02. ලේකම් , විදේශ රැකියා අමාත්‍යාංශය\t කා.දැ.ගැ.ස.\n03.ප්‍රාදේශීය ලේකම් , මාතලේ \t\t කා.දැ.ගැ.ස.\n04. ප්‍රධාන ගණකාධිකාරි\t\t              කා.දැ.ගැ.ස.\n05.නිවාඩු විෂය\t\t\t             කා.දැ.ගැ.ස.\n06.පෞද්ගලික ලිපි ගොනුව \t\t              කා.දැ.ගැ.ස."
    ,
      "ජාතික ආපදා සහන සේවා මධ්‍යස්ථානය.\nස්වභාවික ආපදාවක් නිසා හානියට හෝ බලපෑමකට ලක්වූ නිවසක් සඳහා හානි ප්‍රතිපුර්ණය කිරීමේ ඉල්ලුම් පත්‍රය. \n(මෙම ඉල්ලුම් පත්‍රය ජාතික රක්ෂණ භාර අරමුදලින් රක්ෂණ වන්දි ලබාගැනීම සඳහා පමණි)\nදිස්ත්‍රික්කය:  \nප්‍රාදේශීය ලේකම් කොට්ඨාශය:\nග්‍රාම නිලධාරී කොට්ඨාශය:\nගම/පෙදෙස/වීදීය:\nසිදුවී ඇති ආපදාව:\nආපදාව සිදු වූ කාලසීමාව (දින ..................... සිට .......................දක්වා)\n(මෙම කොටස ඉල්ලුම්කරු විසින් සම්පුර්ණ කල යුතුය.)\n1 ඉල්ලුම්කරුගේ සම්පුර්ණ නම:\n1.1 මුලකුරු සමඟ නම:\n2 ජාතික හැඳුනුම්පත් අංකය:\n3 දුරකථන අංකය (ස්ථාවර/ජංගම):\n4 බලපෑමට/ හානියට පත් නිවසෙහි ලිපිනය:\n4.1 වරිපනම් අංකය:\n5 තාවකාලික රැඳවුම් ස්ථානයක හෝ ඥාතින්ගේ නිවාසයක රැදී සිටින්නේ නම් එහි ලිපිනය:\n6 ඉල්ලුම්කරුගේ පවුලේ සාමාජිකයින් සංඛ්‍යාව:\n7 ඉල්ලුම්කරුගේ රැකියාව:\n8 බලපෑමට/ හානියට පත් නිවසේ අයිතිය පිළිබඳ විස්තරය (අනවශ්‍ය වචන කපා හරින්න).\n8.1 බලපෑමට/ හානියට පත් නිවස ඉල්ලුම්කරු සතුය/ කුලී හිමිකාරිත්වයක් පමණි/ බලපත්‍රලාභී/ වෙනත්\n8.2 කුලී නිවසියෙකු නම් නිවසෙහි පදිංචි වී සිටි කාලය: අවුරුදු 5 ට අඩු/ අවුරුදු 5 ත් 10 ත් අතර/ අවුරුදු 10 ත් 20 ත් අතර/අවුරුදු 20 ට වැඩි\n8.3 හානියට පත් නිවසට රක්ෂණාවරණයක් ලබා ගෙන තිබේද? ඔව්/ නැත ඔව් නම් රක්ෂණය කර ඇති ආයතනය\n9 බලපෑමට/ හානියට පත් නිවස පිළිබඳ විස්තරය\n9.1 නිවසේ මුළු බිම් ප්‍රමාණය:\n9.1.1 නිවාස ගංවතුරින් යටවුවානම් යටවූ ප්‍රමාණය ආසන්න වශයෙන් අඩි:\n9.1.2 බලපෑමට/ හානියට පත් නිවසේ මුලික කොටස් ඉදිකරන ලද්දේ කුමන ද්‍රව්‍ය වලින්ද?\nවහල:\nබිත්ති:\nගෙබිම:\nඅත්තිවාරම:\nවැසිකිලිය:\n9.2 බලපෑමට/ හානියට පත් නිවසේ මුලිකාංගයන්හි මුළු තක්සේරු වටිනාකම කොපමණද? රු.\n9.2.1\nවහල රු:\nබිත්ති රු :\nගෙබිම රු:\nඅත්තිවාරම රු:\nවැසිකිලිය රු:\nගෘහ උපකරණ රු:\nගෘහස්ත විදුලිබල පද්ධතිය රු:\nගෘහස්ත ජලනල පද්ධතිය රු:\n10 පුර්ණ හානියට පත් නිවසක් නම්, ඉදිකර ඇති ඉඩමෙහි අයිතිය සනාථ කිරීම සඳහා වන ලේඛණ (ඔප්පුව, හිමිකම් සහතිකය, බලපත්‍රය) මෙම ලේඛණයට අමුණා තිබිය යුතුය.\n11 බැංකු ගිණුමේ තොරතුරු (පහත සඳහන් පරිදි ඇතුලත් කරන්න. අනවශ්‍ය වචන කපා හරින්න).\n11.1 ගිණුම් අංකය(ඉල්ලුම්කරුගේ):\n11.2 ගිණුම් වර්ගය: ජංගම ගිණුම/ ඉතිරි කිරීමේ ගිණුම/ ස්ථාවර ගිණුම/ වෙනත් (සඳහන් කරන්න).\n11.3 බැංකුවේ නම සහ ලිපිනය:\nමා විසින් ඉහත සඳහන් කර ඇති සියළු තොරතුරු සත්‍ය හා නිවැරදි බවට සහතික වෙමි. \nඉහත ආපදාවෙන් බලපෑමට/ හානියට පත් වී ඇති නිවාසය නැවත අළුත්වැඩියා කරගැනීම/ ඉදිකරගැනීම/ පිරිසුදු කරගැනීම සඳහා ජාතික රක්ෂණ භාර අරමුදලින් රක්ෂණ වන්දි මුදල් ලබාදෙන ලෙස ඉල්ලා සිටිමි. \nතවද, ජාතික රක්ෂණ භාර අරමුදලින් රක්ෂණ වන්දි මුදල් ප්‍රදානය කිරීමේදී අදාළ කොන්දේසි වලටද එකඟ වෙමි.\nදිනය \nඅයදුම්කරුගේ අත්සන"
    ,
      "අධ්‍යක්ෂ,\nජාතික වැඩිහිටි මහලේකම් කාර්යාලය,\nදෙවන මහල,\nසෙත්සිරිපාය,\nබත්තරමුල්ල.\nඅක්ෂි කාච ලබා ගැනීම.\nඋක්ත කරුණට අදාලව පහත නම් සඳහන් අයගේ අක්ෂි කච ලබා ගැනීම සඳහා ප්‍රාදේශීය ලේකම්ගේ නිර්දේශය සහිත වෛද්‍ය වාර්තාව සමග අයදුම්පත අවශ්‍ය කටයුතු සඳහා මේ සමග අමුණා එවන බව කාරුණිකව දන්වා සිටිමි.\nප්‍රාදේශීය ලේකම්,\nමාතලේ."
    ,
      "සභාපති,\nජාතික ළමා ආරක්ෂක අධිකාරිය,\nනො.330, තලවතුගොඩ පාර,\nමාදිවෙල, \nශ්‍රී ජයවර්ධනපුර.\nග්‍රාමීය මට්ටමින් ළමුන්ගේ සුරක්ෂිතතාව වර්ධනය කිරීම උදෙසා ප්‍රජා වැඩ සටහන් ක්‍රියාත්මක කරන නිලධාරීන් දැනුවත් කිරීමේ දීප ව්‍යාප්ත වැඩ සටහන.\nඉහත වැඩ සටහන සඳහා ඔබගේ අංක PU/NPA/Community Intervention/2016/01/01 හා 2016.05.02 දිනැති ලිපිය හා බැඳේ.\n02. එමගින් එවන ලද අංක 450777 දරණ රු. 15,000/= ක් වටිනා චෙක්පත සදහා වූ අංක Z/22 776281 දරන 2016/05/13 දිනැති මුදල් භාරගැනීමේ කුවිතාන්සිය අවශ්‍ය කටයුතු සඳහා මේ සමග අමුණා එවමි.\n ප්‍රාදේශීය ලේකම්,\nමාතලේ."
    ,
      "මාතලේ දිස්ත්‍රික්කයේ සියලුම ප්‍රාදේශීය ලේකම්වරුන් වෙත,\nආර්ථික සංවර්ධන අමාත්‍යාංශයට අනුයුක්තව සිටි සංවර්ධන සහකාර/සංවර්ධන ක්ෂේත්‍ර සහකාර නිලධාරින් නිශ්චිත අමාත්‍යාංශයකට අනුයුක්ත නොකිරීම.\nඋක්ත කරුණ සම්බන්ධයෙන් මා වෙත යොමුකර ඇති ස්වදේශ කටයුතු අමාත්‍යාංශ ලේකම්ගේ අංක HAF/1/05/සංනි/ඒකාබද්ධ නොවන පොදු හා 2016/05/09 දිනැති ලිපිය හා බැඳේ.\n02. ඒ අනුව පෙර ආර්ථිකසංවර්ධන අමාත්‍යාංශයේ සිට ස්වදේශ කටයුතු අමාත්‍යාංශයට  අනුයුක්ත කරන ලද, සේවා ව්‍යවස්ථාවක් යටතට නොගැනෙන  නිලධාරින් අතුරින් සංවර්ධන නිළධාරි සේවා ව්‍යවස්ථාව ක්‍රියාත්මක වන දිනට සේවයේ යෙදී සිටි නිළධාරීන් හට සංවර්ධන නිළධාරි සේවයට අන්තර්ග්‍රහණය විය හැකි ලෙස අදාල නිලධාරින් දැනුවත් කරන මෙන් මා වෙත දන්වා ඇත.\n03. එම නිසා , ඉහත කරුණ සම්බන්ධයෙන් ඔබ කාර්යාලයේ සිටින සංවර්ධන සහකාර/සංවර්ධන ක්ෂේත්‍ර සහකාර නිලධාරින් මේ පිළිබඳව දැනුවත්කර   මා වෙත වාර්තා කරන මෙන් කාරුණිකව දන්වා සිටිමි.\nතේජානි තිලකරත්න,\nඅතිරේක දිස්ත්‍රික් ලේකම්.\nමාතලේ දිස්ත්‍රික් ලේකම් වෙනුවට."
    ,
      "මහත්මියණි/ මහත්මයාණෙනි\nසමාජ ආරක්ෂණ විශ්‍රාම වැටුප් යෝජනා ක්‍රමය\nවාරික මුදල් සටහන් පොත ලබාදීම. \nදායක අංක -\nරාජ්‍ය විශ්‍රාම වැටුපකට හිමිකම් නොකියන ජනතාව වෙනුවෙන් ක්‍රියාත්මක සමාජ ආරක්ෂණ විශ්‍රාම වැටුප් යෝජනා ක්‍රමයේ  දායකත්වය ලබා ඔබගේ වැඩිහිටි දිවියට ලබාගත් සුරක්ෂිතභාවය වෙනුවෙන් මා සතුටට පත්වෙමි.\nඔබගේ ඉදිරි වාරික මුදල් ගෙවීම සඳහා අංක ............................................ දරණ මුදල් වාරික සටහන් පොත මේ සමග එවමි.\nඔබගේ වාරික මුදල රු. ................................. ක් වන අතර, එම වාරික මුදල මාසිකව/ත්‍රෛමාසිකව/ වාර්ෂිකව ගෙවීමට කටයුතු කළ යුතුය.\nඔබගේ නියමිත දිනයට වාරික ගෙවීමට කටයුතු කළ යුතුය. \nවාරිකයක් ගෙවීමේදී දින 30 ක සහන කාලයක් හිමිවේ. \nප්‍රමාදව වාරික ගෙවීමේදී හා අක්‍රමවත් වාරික එකතු කර ගෙවීමේදී ඒ සඳහා ප්‍රමාද ගාස්තුවක් අයකරනු ලැබේ.\nඅඛණ්ඩව හෝ කඩින් කඩ හෝ වන පරිදි වාරික 10 ක් නොගෙවා සිටීමෙන් දායකත්වය අහෝසි වන බව අවධාරණය කරමි.\nවිශ්‍රාම වැටුප් යෝජනා ක්‍රමයට අදාළ ඔබට හිමි ඔප්පුව ඉදිරියේදී ලැබෙනු ඇත.\nමේ පිළිබඳව සියළුම විමසීම් සඳහා 037 2221895 දුරකතනයෙන් ලබාගත හැකි අතර, එසේ නොමැතිනම් ලිඛිතව දන්වා එවීමට කටයුතු කරන ලෙසද වැඩිදුරටත් දන්වමි.\nඅබලතාවයක් හෝ මරණයක් සිදුවීමේදී වන්දි මුදල් ලබාගැනීම සඳහා අදාළ සිදුවීම සිදුවී දින 90 ක් ඇතුළත දැනුම්දිය යුතු වේ.\nදිස්ත්‍රික් සම්බන්ධීකරණ නිලධාරී"
    ,
      "සාමාන්‍යාධිකාරී,\nමධ්‍යම පළාත් මාර්ග සංවර්ධන අධිකාරිය,\nබෝගම්බර,\nමහනුවර.\nඅම්පිටිය ගාල්තැන්න මාර්ගය - පැති බැම්ම කඩා වැටීම, තලාතුඔය කන්ද කැපු තැන්න මාර්ගය වැඩි දියුණු කිරීම,කොන්ත්‍රාත් පැකේජය - JICA/PIU/CP/CON/CP3-1\nඋක්ත කරුණ සම්බන්ධව ප්‍රධාන ලේකම් විසින් ඔබටද පිටපතක් සහිතව මවෙත යොමු කර ඇති අංක CPC/CS/03/04/09/-06 හා 2016.05.31 දිනැති ලිපිය හා බැඳේ.\nඒ අනුව, නියමිත ප්‍රසම්පාදන කටයුතු සිදුකර අදාල කර්මාන්තය කිරීමට අනුමැතිය ලැබී ඇති හෙයින් ඒ සඳහා අනුමත කල ඇස්තමේන්තුවේ පිටපතක් ඉදිරිපත් කරන ලෙස දන්වමි.\nපී. එස්. පතිරණ,\nසහකාර ලේකම්."
    ,
      "දිනය 2012.04.04 \nප්‍රාදේශීය ලේකම් මඟින්,\nසහකාර සම්පත් සංවර්ධන සහකාර,\nප්‍රාදේශීය ලේකම් කාර්යාලය,\nදිස්ත්‍රික් මාසික රැස්වීම. \nඵලදායිතා ප්‍රවර්ධන අමාත්‍යංශයේ මිනිස්බල හා රැකිරක්ෂා දෙපාර්තමේන්තුවේ මානව සම්පත් සංවර්ධන සහකාර නිලධාරීන්ගේ සහ වෘත්තීය මාර්ගෝපදේශ නිලධාරීන්ගේ 2012 අප්‍රේල් මස 05 වන දින පැවැත්වීමට නියමිතව තිබූ දිස්ත්‍රික් මාසික රැස්වීමට සහභාගීවීම(දිවි නැගුම තුන් වන අදියරේ සමාරම්භක උත්සවයට සියළුම නිලධාරීන් සහභාගී විය යුතු නිසා) අපහසු බව අදාළ නිලධාරීන් මා වෙත දැනුම් දී ඇත.\n02. එම නිසා එදිනට පැවැත්වීමට සංවිධානය කර තිබූ මාසික රැස්වීම 2012/04/20 දින මාතලේ දිස්ත්‍රික් ලේකම් කාර්යාලයේ වෘත්තීය මාර්ගෝපදේශ අංශයේදී පැවැත්වීමට කටයුතු සංවිධානය කර ඇති බැවින් එදිනට අනිවාර්යයෙන්ම සහභාගීවන ලෙස කාරුණිකව දන්වමි.\nවසන්තා වර්ණකුළසූරිය,\nසහකාර දිස්ත්‍රික් ලේකම්,\nමාතලේ දිස්ත්‍රික් ලේකම් වෙනුවට.\nපිටපත - අධ්‍යක්‍ෂ ජනරාල් - මිනිස්බල හා රැකිරක්ෂා දෙපාර්තමේන්තුව."
    ,
      "සියලුම විදුහල්පතිවරුන් වෙත,\n2016 අ.පො.ස.(උ. පෙළ) උපකාරක සම්මන්ත්‍රණ මාලාව සඳහා පොදු උපදෙස් මාලාව.\nමෙවර පැවැත්වීමට නියමිත උක්ත සම්මන්ත්‍රණ මාලාව සඳහා පහත දැක්වෙන පොදු උපදෙස් අනුගමනය කරන  ලෙස කාරුණිකව දන්වමි.\n01. මෙම සම්මන්ත්‍රණ මාලාව අදාළ කාලසටහනට අනුකූලවම පැවැත්වීමට කටයුතු කළ යුතුය.\n02.සම්මන්ත්‍රණ පැවැත්වීම සඳහා සම්පත්දායකයන් ලෙස සුදුසුකම් සහිත ප්‍රවීන ගුරුවරු, ගුරු උපදේශක වරු, කථිකාචාර්ය වරු, සහභාගී කරවා ගත යුතුය.\n03. අවශ්‍ය අවස්තාවලදී ශිෂ්‍ය සංඛ්‍යාව අඩු පාසල්වල සිසුන් ආසන්නයේ පිහිටි උසස් පෙළ සහිත වෙනත් පාසල්වලට යොමුකර සම්මන්ත්‍රණ සඳහා සහභාගී වීමට කටයුතු කල හැකි අතර මේ සඳහා කලාප අධ්‍යාපන අධ්‍යක්ෂකගේ සහයෝගය ලබාගත යුතුය.\n04.ප්‍රශ්නපත්‍ර මුද්‍රණය සඳහා පාසල් සංවර්ධන සමිතියේ මුදල් වැය කිරීම සඳහා අනුමැතිය ලබාදෙන අතර විශාල පිරිවැයක් වැයකර සිදුකරන මෙම සම්මන්ත්‍රණ වලින් උපරිම ප්‍රයෝජනය ලබාගැනීම සඳහා සිසුන් සහභාගී කර වීමට සියළුම විදුහල්පතිවරුන් අවශ්‍ය පියවර ගත යුතුය.\n05. විවිධ විෂයන් වෙනුවෙන් පැවැත්වූ සම්මන්ත්‍රණ සඳහා සහභාගී වූ ශිෂ්‍ය සංඛ්‍යාව හා සම්පත් දායකයින් පිලිබඳ විස්තර ඇතුලත් අධීක්ෂණ වාර්තා ජාතික පාසල් විදුහල්පතිවරුන් විසින් ජාතික පාසල් අධ්‍යක්ෂ වෙතටත්, පළාත් පාසල්වල විදුහල්පතිවරුන් විසින් පළාත් අධ්‍යාපන අධ්‍යක්ෂ වෙතටත් යොමු කළ යුතුය.\nඑච්. යූ. ප්‍රේමතිලක,\nඅතිරේක ලේකම් (අධ්‍යාපන  සංවර්ධන),\nඅධ්‍යාපන ලේකම් වෙනුවට.\nපිටපත් \n1. පළාත් අධ්‍යාපන අධ්‍යක්ෂ.\n2. කලාප  අධ්‍යාපන අධ්‍යක්ෂ."
    ,
      "2012 ජන හා නිවාස  සංගණනයේ  ප්‍රධාන තොරතුරු එළි දැක්වීමේ සම්මන්ත්‍රණය.\n2012 වර්ෂයේ  ජන සංගණන හා ප්‍රජා විද්‍යා අංශය විසින් පවත්වන ලද  ජන හා නිවාස  සංගණනයේ  ප්‍රධාන තොරතුරු එළි දැක්වීමේ සම්මන්ත්‍රණයක් මේ වන විටත් පවත්වා ඇත .\nඑම  සම්මන්ත්‍රණ මාලාවේ  දෙවන සම්මන්ත්‍රණය 2015 මැයි මස 12 වන දින පෙ.ව 8.30 සිට 1.30 දක්වා අධ්‍යක්ෂ ජනරාල් ගේ ප්‍රධානත්වයෙන් ජයික් හිල්ටන් රෙසිඩෙන්සීස් හෝටලයේ (JAIC Hiltion Residences)බෝල් රූම් 4 ශාලවේ (Ball Room 4)පැවැත්වීමට කටයුතු යොදා ඇත.\nපළමු සම්මන්ත්‍රණය සඳහා සහභාගි නොවූ සංඛ්‍යාලේඛනඥ සහ සංඛ්‍යාලේඛන නිලධාරියෙකු මෙම සම්මන්ත්‍රණයට සහභාගිවන ලෙස කාරුණිකව දන්වා සිටිමි.\nතවද සංඛ්‍යාලේඛනඥ වරයාට අමතරව සහභාගි වන නිලධාරියා,දිස්ත්‍රික්ක වශයෙන් පවත්වන සම්මන්ත්‍රණ වලදී දේශන පැවැත්වීමට හැකි නිලධාරියෙකු විය යුතු වේ.\nතවද රාජකාරි නිවාඩු දින 02 ක් සහ ගමන් වියදම් අධ්‍යක්ෂ   ජනරාල් විසින් අනුමත කර ඇත.\nඒ සඳහා නියමිත වේලාවට සහභාගි වන ලෙස දන්වා සිටිමි.\nඅධ්‍යක්ෂ, \nජන සංගණන හා ප්‍රජා විද්‍යා අංශය.\n අචාර්‍ය අමර සතරසිංහ,\nඅධ්‍යක්ෂ   ජනරාල්."
    ,
      "සාමාන්‍යාධිකාරී,\nපළාත් මගී ප්‍රවාහන සේවා අධිකාරිය.\nඅධ්‍යක්ෂ මණ්ඩල වාර්තා ලබාගැනීම.\nඔබ ආයතනයේ අධ්‍යක්ෂ මණ්ඩලය වෙත ඉදිරිපත් කරනු ලබන අධ්‍යක්ෂ මණ්ඩල පත්‍රිකා අධ්‍යක්ෂ මණ්ඩලයට ඉදිරිපත් කිරීමට පෙර එහි කෙටුම්පතක් අධ්‍යක්ෂ මණ්ඩල රැස්වීම පැවැත්වීමට නියමිත දිනට පෙර එනම් අවම වශයෙන් දින 3 ට පෙර මා වෙත ඉදිරිපත් කිරීමට කටයුතු කරන ලෙස කාරුණිකව දන්වමි.\n එම රැස්වීමෙන් පසුව සතියක් තුලදී අදාල වාර්තාව ලබාදෙන ලෙසද වැඩිදුරටත් සඳහන් කරමි.\n02. 2015 ජනවාරි මස සිට මේ දක්වා ගන්නා ලද අධ්‍යක්ෂ මණ්ඩල තීරණවල පිටපත් මේ වන තෙක් මා වෙත ලැබී නොමැත.\n නොපමාව එම වාර්තාද ම වෙත එවන ලෙස කාරුණිකව දන්වමි.\nගාමිණී රාජරත්න,\nලේකම්,\nමධ්‍යම පළාත් මාර්ග සංවර්ධන ප්‍රවාහන විදුලිබල හා බලශක්ති සහ නිවාස හා ඉදිකිරීම් අමාත්‍යාංශය."
    ,
      "දිස්ත්‍රික් සම්බන්ධීකරණ නිලධාරී මගින්,\nසාමාන්‍යාධිකාරී,\nසමාජ ආරක්ෂණ මණ්ඩලය.\nනම :\nලිපිනය : \nදිනය : \nදුරකථන අංකය :\nදායක අංකය : \nජාතික හැඳුනුම්පත් අංකය :\nදායකත්වය සක්‍රිය කර ගැනීම.\nඉහත දායක අංකය යටතේ දායකත්වය ලබා ඇති මා වාරික නිසි පරිදි නොගෙවීම මත දායකත්වය අක්‍රිය තත්වයට පත්ව ඇත.\n කරුණාකර නැවත හිඟ වාරික මුදල් හා ප්‍රමාද ගාස්තු අයකර දායකත්වය සක්‍රිය කර නැවත යෝජනා ක්‍රමයට ඇතුලත් කර ගන්නා ලෙස කාරුණිකව ඉල්ලා සිටිමි. \nඉදිරියේ දී නිසි පරිදි  වාරික මුදල් ගෙවීමට කටයුතු කරන බවද කාරුණිකව දන්වා සිටිමි.\nදිනය                      අත්සන \nසාමාන්‍යාධිකාරී,\nසමාජ ආරක්ෂණ මණ්ඩලය.\nදායකත්වය සක්‍රිය කර ගැනීම\nඉහත දායක අංක සඳහන් දායකයා 2016 ........................................ දින ..................................... අංක දරණ රිසිට්පත් යටතේ රු. ..................................... ගෙවා සාමාජිකත්වය සක්‍රිය කර ගන්නා ලදී. \nඅදාළ මුදල ගිණුම්ගත කිරීම සඳහා මේ සමග රිසිට්පත හා බැංකු බැරපත යොමු කරමි.\nඑච්.කේ.එම්. රංජිත් දිසානායක,\nදිස්ත්‍රික් සම්බන්ධීකරණ නිලධාරී."
    ,
      "දිස්ත්‍රික් මාසික රැස්වීම් වාර්තාව\nදිනය-2015/05/08\nවේලාව-පෙ.ව 9.30 \nස්ථානය- දිස්ත්‍රික් ලේකම් කාර්යාලය වෘත්තීය මාර්ගෝපදේශ අංශය\nආගම සිහි කිරීමෙන් අනතුරුව දිස්ත්‍රික්  වෘත්තීය මාර්ගෝපදේශ නිලධාරි එස්.එම්.ටී .ද අල්විස් මහතා විසින් රැස්වීම  මෙහෙය වන් ලදී.\nඑම අවස්ථාවට සහභාගි වූ සියළුම ප්‍රා.ලේ.කාර්යාල වල මා.ස.ස.සහකාර නිලධාරින් සහ දිස්ත්‍රික් ලේකම් කාර්‍යාලයේ නිලධාරින් විසින් පෙර මාසයේ එනම් අප්‍රේල් මස ප්‍රගතිය සහ මැයි මස ඉදිරි වැඩ සැලැස්ම පිලිබඳව සාකච්ඡා කරන ලදී.\nඅනු අංකය,ප්‍රාදේශිය ලේකම් කොට්ටාශය,සංවිධානය කර ඇති වැඩසටහන, පවත්වනු ලබන දිනය හා ස්ථානය"
    ,
      "මැනුම් අධිකාරි,\nසඳහා ඉඩම් අත්කර ගැනීම\nඅංක .......... හා ................ දිනැති  / අවසන් ගම් සිතියමේ අංක ............. දරන කැබලි වශයෙන් පෙන්නුම් කර ඇති ඉඩම්වල මායිම්, පිහිටීම හා බිම් ප්‍රමාණය දැක්වෙන විස්තරයක් මාවෙත එවන ලෙස කාරුණිකව දන්වමි. \nරජයට අත්කර ගත් ඉඩම්, ඉඩම් ලියාපදිචිකිරීමේ කාර්යාලයේ ලිපාපදිංචි කිරීම සඳහා මෙම විස්තර අවශ්‍යවන බවට තවදුරටත් දන්වමි.\nප්‍රාදේශීය ලේකම්/ අත්කර ගැනීම් නිලධාරි,\n ප්‍රාදේශීය ලේකම් කොට්ඨාශය,\nප්‍රාදේශීය ලේකම් කාර්යාලය."
    ,
      "මගේ අංකය : DSKU/SWD/SS/PENSN/4/16\nසමාජ ආරක්ෂණ විශ්‍රාම වැටුප් යෝජනා ක්‍රමයේ විශ්‍රාම වැටුප් පොත ලබා දීම.\nඔබට හිමි විශ්‍රාම වැටුප් පොත .................................... ප්‍රාදේශීය ලේකම් කාර්යාලය වෙත එවා ඇති අතර බදාදා දිනක පෙ.ව. .............................. ට ප්‍රාදේශීය ලේකම් කාර්යාලය වෙත ගොස් ඔබගේ විශ්‍රාම වැටුප් පොත ලබා ගන්නා ලෙස දන්වමි.\nමේ සමග එවන ආකෘතිය පුරවා පොත ලද බවට දිස්ත්‍රික් සම්බන්ධීකරණ නිලධාරී, සමාජ ආරක්ෂණ විශ්‍රාම වැටුප්  අංශය, දිස්ත්‍රික් ලේකම් කාර්යාලය, කුරුණෑගල යන ලිපිනයට එවන ලෙස දන්වන අතර සෑම මසකම 15 වන දින සිට 20 වන දින දක්වා නම්කරන ලද ග්‍රාමීය බැංකුවෙන් විශ්‍රාම වැටුප් මුදල් ලබාගත හැකි බව ද වැඩිදුරටත් දන්වමි.\nඑච්.කේ.එම්. රංජිත් දිසානායක,\nදිස්ත්‍රික් සම්බන්ධීකරණ නිලධාරී."
    ,
      "අධ්‍යක්ෂ ජනරාල්,\nමිනිස්බල හා රැකී රක්ෂා දෙපාර්තමේන්තුව,\n9 වන මහල,\nසෙත්සිරිපාය(දෙවන අදියර ),\nබත්තරමුල්ල.\nමාසික රැස්වීම් වාර්තාව - සැප්තැම්බර් මාසය\nමාතලේ දිස්ත්‍රික්කයේ වෘත්තිය මාර්ගෝපදේශ නිළධාරින්ගේ සහ මානව සම්පත් සංවර්ධන නිළධාරින්ගේ මාසික රැස්වීම 2015.09.04 වන දින වෘත්තිය මාර්ගෝපදේශ මධ්‍යස්ථානයේ දී පැවැත්වු අතර ඒ සඳහා සහභාගි වූ නිළධාරින්ගේ අත්සන් ලේඛනය පසුගිය මාසයේ ප්‍රගතිය, මෙම මස කිරීමට අපේක්ෂිත වැඩසටහන් පිලිබඳ විස්තරය සහ රැස්වීම් වාර්තාව මේ සමඟ කාරුණිකව ඉදිරිපත් කරමි.\nචමිල් සකලසූරිය,\nසහකාර දිස්ත්‍රික් ලේකම්,\nමාතලේ  දිස්ත්‍රික් ලේකම් වෙනුවට."
    ,
      "දිස්ත්‍රික් ලේකම්-මාතලේ මගින්,\nඅධ්‍යක්ෂ ජනරාල්,\nමිනිස් බල හා රැකි රක්ෂා දෙපාර්තමේන්තුව,\n9 වන මහල,\nසෙත්සිරිපාය 2 වන අදියර,\nබත්තරමුල්ල.\nවැඩිහිටි ජනගහනය ශ්‍රම බලකායට දායක කර ගැනීම පිලිබඳ සමීක්ෂණයේ තොරතුරු රැස් කිරීම.\nඋක්ත මැයෙන් යුතු ඔබගේ අංක DME/03/01/13/24 (01) දරණ හා 3013-04-30 දිනැති ලිපිය හා බැඳේ .\n02. එම ලිපියෙ සඳහන් පරිදි මේ වන විට මා විසින් රැස්කර ගත් සමීක්ෂණ තොරතුරු මේ සමඟ ඉදිරිපත් කරන අතර ඉදිරියේදි තවදුරටත් එම තොරතුරු රැස් කර කඩිනමින් ඔබ වෙත ඉදිරිපත් කරන බව මෙයින් කාරුණිකව දන්වා සිටිමී.\nමෙයට විශ්වාසී,\nඒ.ජී.එස්. ජයතිස්ස.\nමානව සම්පත් සංවර්ධන සහකාර,\nඅධ්‍යක්ෂ ජනරාල්,\nමිනිස් බල හා රැකි රක්ෂා දෙපාර්තමේන්තුව,\n9 වන මහල,\nසෙත්සිරිපාය 2 වන අදියර,\nබත්තරමුල්ල.\nඅවශ්‍ය කටයුතු සඳහා කාරුණිකව ඉදිරිපත් කරමි.\nඑච්.ඩබ්.එම්.එස්.විජේසිංහ,\nපරිපාලන නිලධාරි,\nදිස්ත්‍රික් ලේකම් වෙනුවට."
    ,
      "මධ්‍යම පළාත් ප්‍රධාන ලේකම්,\nප්‍රධාන  ලේකම් කාර්යාලය, \nපල්ලෙකැලේ,\nකුණ්ඩසාලේ.\nදොරුවාදෙණිය හා පත්තුණුපිටිය කම්බි පාලම ඉදිකිරිම .\nකො අංක CPC/PRDA/GO-2013- ප්‍රතිපාදන ලබාගැනීම.\nඋක්ත කරුණ සම්බන්ධයෙන් මධ්‍යම පළාත් මාර්ග සංවර්ධන අධිකාරියේ සාමාන්‍යාධිකාරී විසින් ඔබ මඟින් පළාත් පාලන සහ පළාත්  සභා අමාත්‍යාංශය වෙත යොමු කිරීම සඳහා ම වෙත ඒවා ඇති අංක CPC/PRDA/1/3/1/8/52 හා 2016/05/10 දින දරන ලිපිය හා අදාළ ඇමුණුම් මේ සමඟ යොමු කරමි.\nවැඩ අවසන් කර ජනතා අයිතියට පත් කර යථෝක්ත කර්මාන්තය වෙනුවෙන් පළාත් මාර්ග සංවර්ධන අධිකාරිය විසින් අදාළ කොන්ත්‍රාත් සමාගමට රු  6,804,264.36 ගෙවීම නිර්දේශ කරන අතර එය කඩිනමින් පළාත් මර්ග සංවර්ධන අධිකාරියට ලබාදීමට අවශ්‍ය කටයුතු සලසා දෙන මෙන් කාරුණිකව ඉල්ලා සිටිමි.\nතවද, මේ සම්බන්ධව මාගේ සමාංක හා 2016.03.02, 2015.6.30 දිනැති ලිපි මඟින් ද ඉල්ලීම් කර ඇති බව වැඩිදුරටත් දන්වා සිටිමි.\nගාමිණී රාජරත්න,\nලේකම්."
    ,
      "විදුහල්පති,\nම/ශ්‍රි සංඝමිත්තා බාලිකා මහා විද්‍යාලය,\nමාතලේ.\nවෘත්තිය මංපෙත් වැඩසටහන 2010\nමිනිස් බල හා රැකියා දෙපාර්තමේන්තුවේ වෘත්තීය මාර්ගෝපදේශ අංශය මගින් විද්වත් වෘත්තීකයින්ගේ සංගමයේ පුහුණු ආයතන හා රැකියා ආයතන සම්බන්ධ කර ගනිමින්  උක්ත වැඩසටහන පැවැත්වීමට කටයුතු සංවිධානය කර ඇත.\n02.මෙම වැඩසටහන මගින් \n01.පාසල් සිසුන් තුල ඇති හැකියා ලැදියා හා වටිනාකම් පිලිබඳ ස්වයං තක්සේරුවක් ලබා දීම හා ඒ අනුව ගැලපෙන විවිධ වෘත්තීය මංපෙත් හඳුන්වා දීම.\n02.තෝරාගත්  වෘත්තීය මංපෙතට පිවිස වෘත්තිය දියුණුව සලසා ගැනීම සඳහා සුදුසු වෘත්තිය සැලැස්මක් පාසලින් පිටවීමට පෙර සකසා ගැනීමට මග පෙන්වීම.\n03.වැඩිදුරට තොරතුරු ලබා ගැනිම සඳහා තොරතුරු පත්‍රිකා සමඟ කවුළු විවෘත කර තිබීම.\n04.ඉහත ක්‍රියාමාර්ග නිසා පාසල්  කාලය තුලදීම වැඩ ලෝකය සම්බන්ධව අවබෝධය ලබාදීම හේතුවෙන් පාසල් කාලයෙන් පසු රැකියා සෙවීමේ කාලය අඩු වීම මගින් ශ්‍රම බලකායට අවම කාලයක් තුල ඵලදායි පුද්ගලයන් එක් වීම හරහා රටේ සංවර්ධනයට දායක කරවීම.\nඋක්ත වැඩසටහන පැවැත්වීම සඳහා ඔබ විද්‍යාලීය ශ්‍රවණාගාරය 2010/10/26 වෙනි දින පෙ.ව.8.00 සිට  ප.ව. 1.30 දක්වා වෙන් කර දෙන මෙන් කාරුණිකව ඉල්ලා සිටිමි.\nනිරෝෂණ කරුණාතිලක,\nසහකාර දිස්ත්‍රික්  ලේකම් මාතලේ,\nවෙනුවට. \nඅ/ක එච්.එම්. ගාමිණී සෙනෙවිරත්න \nදිස්ත්‍රික් ලේකම් - මාතලේ"
    ,
      "මගේ අංකය : DSKU/SWD/SS/P1FORM/4/63\nදිනය : 2016.\nසමාජ ආරක්ෂණ විශ්‍රාම වැටුප් යෝජනා ක්‍රමයේ විශ්‍රාම වැටුප ලබා දීම.\nරාජ්‍ය විශ්‍රාම වැටුපකට හිමිකම් නැති ජනතාව වෙනුවෙන් ක්‍රියාත්මක කරනු ලබන සමාජ ආරක්ෂණ විශ්‍රාම වැටුප් යෝජනා ක්‍රමයේ විශ්‍රාම වැටුප ලබා ගැනීම සඳහා ඔබ හිමිකම් ලබා ඇති බව සතුටින් දන්වමි.\nවිශ්‍රාම වැටුප ලබා ගැනීම සඳහා මේ සමග එවනු ලබන SSB PI ආකෘතිය සම්පුර්ණ කර ග්‍රාම නිලධාරී ගේ නිර්දේශය සහිතව එහි අංක 6 යටතේ ඉල්ලා ඇති ලේඛන ද සමග පෙ.ව. .................................... ට කුරුණෑගල දිස්ත්‍රික් ලේකම් කාර්යාලයේ (කච්චේරියේ) සමාජ ආරක්ෂණ අංශය වෙත භාර දෙන ලෙස කාරුණිකව දන්වා සිටිමි.\nලේඛන සම්පුර්ණ කිරීමේදී පහත සඳහන් උපදෙස් පිළිබඳව අවධානය යොමු කරන ලෙස දන්වා සිටිමි.\n1. ආකෘති පත්‍රය නිවැරදිව සම්පුර්ණ කර ග්‍රාම නිලධාරී මගින් නිර්දේශ කර තිබිය යුතුය.\n2. උප්පැන්න සහතිකයේ ජායා පිටපත / හැඳුනුම්පතේ ජායා පිටපත / ග්‍රාමීය බැංකු පොතේ ජායා පිටපත ලබා දිය යුතුවේ.\nඑම පිටපත් ග්‍රාම නිලධාරී මගින් සහතික කර තිබීම.\n3. ඔප්පුවේ / උප්පැන්න සහතිකයේ / හැඳුනුම්පතේ වාසගම සහිත නම හෝ උපන් දිනයේ යම් වෙනසක් තිබේ නම් ඒ සඳහා දිවුරුම් ප්‍රකාශයක් ඉදිරිපත් කිරීම.\n4. ග්‍රාමීය බැංකුවේ ගිණුම තනි පුද්ගල ගිණුමක් විය යුතුය. \n(ප්‍රාදේශීය සමුපකාර ග්‍රාමීය බැංකු ගිණුම් වලංගු නොවේ)\n5. මුදල් වාරික සටහන් පොත හා ඔප්පුවේ මුදල් පිටපත ඉදිරිපත් කළ යුතු අතර සමාජ ආරක්ෂණ ඔප්පුව නොමැත්තේ නම් ඔප්පුව නැති බවට ලිපියක් ඉදිරිපත් කළ යුතුය.\nඑච්.කේ.එම්. රංජිත් දිසානායක,\nදිස්ත්‍රික් සම්බන්ධීකරණ නිලධාරී. \nකාර්යාලීය ප්‍රයෝජනය සඳහා පමණි.\nපහත ලේඛන භාර ගතිමි.\n1.උප්පැන්න සහතිකයේ  පිටපත\n2. උප්පැන්න සහතිකය වෙනුවට ඉදිරිපත් කරන දිවුරුම් ප්‍රකාශයක්\n3. ජාතික හැඳුනුම්පතේ  දෙපැත්තේම ජායා පිටපතක්\n4. නමේ වෙනස සම්බන්ධව දිවුරුම් ප්‍රකාශයක්\n5. වාරික සටහන් පොත / කාඩ්පත \n6. සමාජ ආරක්ෂණ ඔප්පුව\n7. ග්‍රාමීය බැංකු මගින් නිකුත් කරන ලද පස්පොතේ ජායා පිටපතක්\n8. ළගදී ලබාගත් ජායා රූප දෙකක්  (පාස්පෝට් ප්‍රමාණයේ)\nඑච්.කේ.එම්. රංජිත් දිසානායක,\nදිස්ත්‍රික් සම්බන්ධීකරණ නිලධාරී."
    ,
      "දිනය 2016/06/22\nරැකියා 100 වැඩසටහන සම්බන්ධවයි.\nකම්කරු හා වෘත්තීය සබඳතා අමාත්‍යංශයේ මිනිස්බල හා රැකිරක්ෂා දෙපාර්තමේන්තුව මඟින් මාතලේ දිස්ත්‍රික් ලේකම් කාර්යාලය සහ මාතලේ ප්‍රාදේශීය ලේකම් කාර්යාලය එක්ව සංවිධානය කරනු ලබන උක්ත වැඩසටහන 2016/07/08 වන දින පෙ.ව. 9.00 සිට ප.ව. 4.00 දක්වා මාතලේ ප්‍රාදේශීය ලේකම් කාර්යාලයේ දී පැවැත්වීමට කටයුතු සංවිධානය කර ඇත.\n02. මාතලේ දිස්ත්‍රික් ලේකම් කාර්යාලයේ  වෘත්තීය මාර්ගෝපදේශ අංශය සහ රැකියා සේවා මධ්‍යස්ථානය මඟින්  වෘත්තීය මාර්ගෝපදේශ වැඩසටහන් පවත්වා, ලියාපදිංචි කරගත් රැකියා අපේක්‍ෂිත තරුණ තරුණියන්ට රැකියා 100 ක් ලබාදීමේ අරමුණින් මෙම වැඩසටහන සංවිධානය කරනු ලැබේ.\n03. මේ සඳහා ප්‍රදේශයේ රැකියා ඇබෑර්තු පවතින පෞද්ගලික ආයතන, කර්මාන්තශාලා, අර්ධ රාජ්‍ය ආයතන, බැංකු සහ දිස්ත්‍රික්කයෙන් පිට ප්‍රමුඛ පෙලේ රැකියා ආයතන මෙම වැඩසටහනට සම්බන්ධකර ගනු ලැබේ.\n04. රැකියාවක් ලබාගැනීම සඳහා ඔබගේ ජීවදත්ත තොරතුරු ඇතුළත් අයදුම්පත් කීපයක් සහ අදාළ සහතික පත් රැගෙන සම්මුඛ පරීක්ෂණයට සහභාගී වී ඔබට ගැලපෙන රැකියාව තෝරාගෙන ඔබගේ ජීවිතය සාර්ථක කර ගැනීමට අවස්ථාව ලබාගන්නා ලෙස කාරුණිකව දන්වා සිටිමි.\n05.මෙම වැඩසටහන සඳහා සහභාගී වන/නොවන බව වෘත්තීය මාර්ගෝපදේශ නිලධාරී, දිස්ත්‍රික් ලේකම් කාර්යාලය, මාතලේ යන ලිපිනයට පහත කොටස පුරවා 2016/06/30 දිනට පෙර අප වෙත එවන ලෙස ද කාරුණිකව ඉල්ලා සිටිමි.\nතේජානි තිලකරත්න,\nඅතිරේක දිස්ත්‍රික් ලේකම්, \nමාතලේ දිස්ත්‍රික් ලේකම් වෙනුවට.\n2016/07/08 වන දින.......... වන මා මාතලේ ප්‍රාදේශීය ලේකම් කාර්යාලයේ පැවැත්වෙන රැකියා 100 වැඩසටහන සඳහා සහභාගී වෙමි/නොවෙමි.\nඅත්සන"
    ,
      "දිස්ත්‍රික් ලේකම්,\nදිස්ත්‍රික් ලේකම් කාර්යාලය,\nමාතලේ.\nමසකට පැය 20 සීමාව ඉක්මවා අතිකාල සේවයේ යෙදීමට අනුමැතිය ඉල්ලීම.\nමෙම කාර්යාලයේ සේවය කරන පහත සඳහන් නිලධාරීන් විසින් 2016 ජුනි මස පැය 20 සීමාව ඉක්මවා අතිකාල සේවයේ යෙදීමට අනුමැතිය ඉල්ලා ඉදිරිපත්කර ඇති ඉල්ලුම් පත් පහක් (05 ක්) මාගේ නිර්දේශය සහිතව අවශ්‍ය කටයුතු සඳහා මේ සමඟ කාරුණිකව ඉදිරිපත් කරමි.\nඅනු අංකය. නිලධාරියාගේ නාම හා තනතුර. නිර්දේශ කරන පැය ගණන.\nඑල්.ජී.ලයනල් රූපසිංහ මයා - රියදුරු \nජී.ජී.නිහාල් ගුණවර්ධන මයා - රියදුරු \nජී.එච්.ජී.ඉලංගරත්න මයා - කා.කා.ස.I\nඑස්.සී.කොඩිතුවක්කු මයා - කා.කා.ස.I\nඊ.ජී.හේමන්ත උදේනිශාන්ත මයා - කා.සේ.සේ.III\nප්‍රාදේශීය ලේකම්, \nපල්ලේපොල."
    ,
      "ප්‍රසම්පාදන දැන්වීම\nදිස්ත්‍රික් ලේකම් කාර්යාලය - මාතලේ.\nමාතලේ දිස්ත්‍රික් ලේකම් කාර්යාලයේ ප්‍රසම්පාදන කමිටු සභාපති වන මවිසින් පහත සඳහන් ව්‍යාපෘති සඳහා 2016.07.04 වැනි දින පස්වරු 02.30 දක්වා ICTAD/CIDA ආයතනයේ ලියාපදිංචි සුදුසුකම්ලත් ලංසුකරුවන්ගෙන් මුද්‍රා තබන ලද ලංසු පත් භාරගනු ලැබේ.\nව්‍යාපෘතිය ICகொள்முதல்TAD/CIDA ලියාපදිංචිය \nසෘජු ඉදිකිරීම් පිරිවැය ආසන්න (රු. මි) වැට් රහිත \nආපසු නොගෙවන ප්‍රසම්පාදන ලියවිලි ගාස්තු \nආපසු ගෙවන ලංසු ඇපකරය\n01. එක් එක් ව්‍යාපෘතිය සඳහා වෙන් වෙන් වශයෙන් ලංසු ඉදිරිපත් කළ යුතු ය.\n02. ලංසුපත් 2016.06.17 දින සිට 2016.07.01 වැනි දින පස්වරු 03.00 දක්වා සතියේ වැඩ කරන දිනයන්හි ආපසු නොගෙවන ලංසු ලියවිලි ගාස්තු මෙම කාර්යාලයේ සරප් වෙත ගෙවා ලිඛිත ඉල්ලීමක් ඉදිරිපත්කර ලංසු ලියකියවිලි මාතලේ දිස්ත්‍රික් ලේකම් කාර්යාලයේ ඉංජිනේරු අංශයෙන් ලබාගත හැකිය.\n03.ICTAD/CIDA ලියාපදිංචිය තහවුරු කර ගැනීමෙන් පසුව ලංසුපත් නිකුත් කරනු ලැබේ.\n04. ප්‍රසම්පාදන ලියවිලි ඉදිරිපත් කිරීමේදී ප්‍රසම්පාදනකරුගේ වැට් බඩු ලියාපදිංචියක් ඇත්නම් ඊට අදාල අංකය සඳහන් කිරීම අනිවාර්ය වන අතර අදාල සහතිකයේ පිටපතක්ද ටෙන්ඩර් පත්‍රයට අමුණා එවිය යුතු ය.\n05. ඉහත ආපසු ගෙවන ලංසු සුරක්ෂණ ගාස්තු (Bid Security) දිස්ත්‍රික් ලේකම් කාර්යාලයේ සරප් වෙත මුදලින් ගෙවීම හෝ ශ්‍රී ලංකා මහබැංකුව පිළිගත් වාණිජ බැංකුවක් මඟින් ලබාගත් 2016.08.04 වන දින දක්වා වලංගු ඇපකරයක් මඟින් ඉදිරිපත් කළ හැකි ය.\n06. සම්පූර්ණ කරන ලද ලංසු මුල් පිටපත හා අතිරේක පිටපතක් වශයෙන් 2016.07.04 වන දින පස්වරු 02.30 ට ප්‍රථම ප්‍රසම්පාදන කමිටුවේ සභාපති, දිස්ත්‍රික් ලේකම් කාර්යාලය, මාතලේ. නමින් දිස්ත්‍රික් ලේකම් කාර්යාලයේ දිස්ත්‍රික් ඉංජිනේරු නිල කාමරයේ තබා ඇති ප්‍රසම්පාදන පෙට්ටියේ බහාලමින් හෝ ලියාපදිංචි තැපෑලෙන් ඉදිරිපත් කළ හැකි අතර, ප්‍රසම්පාදනපත් බහාලන කවරයේ උඩ වම් කෙළවරේ ඉහළ  කර්මාන්තයේ නම අනිවාර්යයෙන්ම සඳහන් කළ යුතු ය.\n௦7.2016.07.04 වන දින පස්වරු 02.45 ට ප්‍රසම්පාදන කමිටුව රැස්වී ලංසුපත් විවෘත කරනු ලබන අතර එම අවස්ථාවට ලංසු ඉදිරිපත් කරන ලංසුකරුවන්ට හෝ ඔහු වෙනුවෙන් ලිඛිතව බලය පවරනු ලබන නියෝජිතයෙකුට සහභාගී විය හැකි ය.\n08. ඉහත ව්‍යාපෘතිය සඳහා වැඩි විස්තර මාතලේ දිස්ත්‍රික් ලේකම් කාර්යාලයේ ඉංජිනේරු අංශයෙන් ලබාගත හැකි ය. (දුරකථන අංකය 066-2222319)\n09. ලංසු සඳහා වන අවසාන තීරණය ප්‍රසම්පාදන කමිටුවේ තීරණය වේ.\nසභාපති,\nදිස්ත්‍රික් ලේකම් කාර්යාලයීය ප්‍රසම්පාදන කමිටුව,\nදිස්ත්‍රික් ලේකම් කාර්යාලය - මාතලේ."
    ,
      "එරික් වීරවර්ධන මහතා,\nසභාපති,\nමධ්‍යම පළාත් මගී ප්‍රවාහන සේවා අධිකාරිය.\nශ්‍රී ලංකීය පොදු මගී ප්‍රවාහන සේවය තුල ව්‍යුහාත්මක ප්‍රතිසංස්කරණ සඳහා නව යෝජනා.\nමධ්‍යම පළාත් මගී ප්‍රවාහන අධිකාරියේ සේවක වැටුප් සහ දීමනා ගෙවීම.\nඉහත කරුණට අදාලව 2016.5.12 දින ජනාධිපති ලේකම්ගේ ප්‍රධානත්වයෙන් පවත්වන ලද සාකච්ඡාවේදී ගනු ලැබූ තීරණ සම්බන්ධවයි.\nඒ අනුව, තීරණය කරන ලද මගී ප්‍රවාහනය තුල ව්‍යුහාත්මක යෝජනා සිදුකිරීම සම්බන්ධයෙන් වන ව්‍යාපෘති යෝජනාව වියදම් ඇස්තමේන්තු සමඟ සකස් කොට අදාල පාර්ශව වෙත යොමු කිරීම සඳහා නොපමාව මා වෙත ලබා දෙන ලෙස කාරුණිකව දැනුම් දෙමි.\nගාමිණී රාජරත්න,\nලේකම්.\nපිටපත් : ගරු  එදිරිවීර වීරවර්ධන මහතා ,\nමධ්‍යම පළාත් මාර්ග සංවර්ධන, ප්‍රවාහන, විදුලිබල හා බලශක්ති සහ නිවාස හා ඉදිකිරීම් අමාත්‍ය"
    ,
      "දිනය 2010.01.15 \nදිස්ත්‍රික් ලේකම් මඟින්,\nඅධ්‍යක්‍ෂ,\nවෘත්තීය මාර්ගෝපදේශ අංශය,\nකම්කරු සබඳතා සහ මිනිස්බල අමාත්‍යංශය,\n64, කෙලින් වීදිය,\nබත්තරමුල්ල.\nමාතලේ දිස්ත්‍රික් මාසික ඉදිරි වැඩසටහන් වාර්තාව  2010 ජනවාරි \nඋක්ත කරුණ සම්බන්ධයෙන් මානව සම්පත් සංවර්ධන සහකාර නිලධාරීන්ගේ 2010 වසර සඳහා වෘත්තීය මාර්ගෝපදේශ අංශයට අදාලව නියමිත වර්ෂික ඉලක්ක ලබා දුන් පසු ඉතිරි වැඩසටහන් සංවිධානය කර ගන්නා බව මානව සම්පත් සංවර්ධන සහකාර නිලධාරීන් සහ වෘත්තීය මාර්ගෝපදේශ නිලධාරීන් විසින් සාකච්ඡා කර ගත් බව කාරුණිකව දන්වමි.\nඑම්.අයි.කුමාරි,\nමානව සම්පත් සංවර්ධන සහකාරඅධ්‍යක්‍ෂ,\nවෘත්තීය මාර්ගෝපදේශ අංශය,\nකම්කරු සබඳතා සහ මිනිස්බල අමාත්‍යංශය,\n64, කෙලින් වීදිය,\nබත්තරමුල්ල.\nඉදිරිපත් කරමි. \nනිරෝෂණ කරුණාතිලක \nසහකාර දිස්ත්‍රික් ලේකම්\nමාතලේ දිස්ත්‍රික් ලේකම් වෙනුවට"
    ,
      "අධ්‍යක්ෂ,\nවෘත්තිය මාර්ගෝපදේශ අංශය,\nකම්කරු සබඳතා සහ මිනිස්බල අමාත්‍යාංශය,\n64,කෙලින් වීදිය, \nබත්තරමුල්ල.\nරැකියා සමාජ නායකයින් පුහුණු කිරීමේ වැඩමුළුව.\nමාතලේ දිස්ත්‍රික්කයේ උක්ත වැඩමුළුව 2008.09.01- 02 දෙදින මහනුවර පොල්ගොල්ල සමූපකාර සංවර්ධන ආයතනයේ හි පැවැත්වීය .\nඒ සඳහා ප්‍රාදේශිය ලේකම් කොට්ටාශ මට්ටමින් සහභාගි කර වූ රැකියා සමාජ නායකයින් ගණන හා අදාල අත්සන් ලේඛනයේ පිටපතක් ද මේ සමග ඉදිරිපත් කරමි. \nප්‍රාදේශිය ලේකම් කොට්ටාශය      යොමු කල රැකියා සමාජ නායකයන් ගණන\nමාතලේ\t\t      \t\t03\nඋකුවෙල\t\t\t\t\t03\nරත්තොට\t\t\t\t\t07\nදඹුල්ල\t\t\t\t\t\t04\nනාඋල\t\t\t\t\t\t03\nගලේවෙල\t\t\t\t\t07\nලග්ගල-පල්ලෙගම\t\t\t\t03\nවිල්ගමුව \t\t\t\t\t03\nඅඹන්ගඟ කෝරලය \t\t\t\t04\nයටවත්ත\t\t\t\t\t-\nපල්ලෙපොල\t\t\t        -\nඊ .එම්. කේ .ගුණරත්න \nමානව සම්පත් සංවර්‍ධන සහකාර අධ්‍යක්ෂ \nවෘත්තිය මාර්ගෝපදේශ අංශය\nකම්කරු සබඳතා සහ මිනිස්බල අමාත්‍යාංශය\nඉදිරි කටයුතු සඳහා ඉදිරිපත් කරමි.\t\t \nතේජානි තිලකරත්න\nඅතිරේක දිස්ත්‍රික් ලේකම්\nමාතලේ  දිස්ත්‍රික් ලේකම් වෙනුවට \nපිටපත - අධ්‍යක්ෂ, රැකියා සැලසුම් අංශය,බත්තරමුල්ල-දැ.ගැ.ස"
    ,
      "දිස්ත්‍රික් සම්බන්ධීකරණ නිලධාරී මගින්,\nසාමාන්‍යාධිකාරී,\nසමාජ ආරක්ෂණ මණ්ඩලය\nනම :\nලිපිනය : .\nදිනය : \nදුරකථන අංකය :\nදායක අංකය :\nජාතික හැඳුනුම්පත් අංකය :\nඔප්පු පිටපතක් ලබා ගැනීම\nඉහත සාමාජික අංකය යටතේ  සමාජ ආරක්ෂණ විශ්‍රාම වැටුප් යෝජනා ක්‍රමයට දායකත්වය ලබා ඇති මා වෙත සමාජ ආරක්ෂණ ඔප්පුව ...........හේතුවෙන් නැවත ඔප්පු පිටපතක් මා වෙත ලබා දීමට කටයුතු කරන මෙන්  කාරුණිකව ඉල්ලමි.\nමින් පසු වාරික නිසි පරිදි ගෙවීම සිදුකරන බවද සඳහන් කරමි.\nදිනය:                       අත්සන :\nසාමාන්‍යාධිකාරී (සමාජ ආරක්ෂණ)\nඔප්පු පිටපතක් ලබා ගැනීම\nසමාජ ආරක්ෂණ විශ්‍රාම වැටුප් යෝජනා ක්‍රමයට දායකත්වය ලබා ඇති ඉහත දායක අංක සඳහන් දායකයා සඳහා ඔප්පු පිටපතක් නිකුත් කර මා වෙත දැනුම් දීමට කටයුතු කරන මෙන් කාරුණිකව ඉල්ලමි.\nඑච්.කේ.එම්. රංජිත් දිසානායක,\nදිස්ත්‍රික් සම්බන්ධීකරණ නිලධාරී."
    ,
      "වෙත :- ගණකාධිකාරී(මුදල්),\nවෙතින් :- ගොඩනැගිලි කළමනාකරු.\nදිනය :- 2012.12.18\nමගේ අංකය :- ED/7/74/3/2/v\nවිෂය :- ප්‍රතිපාදන ලබා ගැනීම.\nඅමාත්‍යාංශය තුළ ස්ථාපනය කර ඇති අභ්‍යන්තර දුරකථන පද්ධතිය අකුණුසැරවලින් ආරක්ෂා කර ගැනීම සඳහා විදුලි තතැඳුම් කිරීමට අවශ්‍ය විදුලිමය ද්‍රව්‍ය මිළදී ගැනීම සඳහා රු.47,710/- ක මුදලක් ගණකාධිකාරී(ගෙවීම්) වෙත වෙන් කර යැවීමට කටයුතු කරන මෙන් කාරුණිකව ඉල්ලා සිටිමි.\nඅදාළ වියදම් ඇස්තමේන්තුවල ඡායා පිටපත් හා අදාළ මිළදී ගැනීම් සඳහා අතිරේක ලේකම්(පාසල් සැපයීම්,සේවා හා ඉදිකිරීම්)ගෙන් ලද අනුමැතියේ ඡායා පිටපත් මේ සමඟ අමුණා ඇති බව කරුණාවෙන් සලකන්න.\nමේ සම්බන්ධව ඔබෙන් ලැබෙන සහයෝගය අතිශයින් අගයමි.\nඒ.එච්.එච්.කේ.වීරකෝන්,\nගොඩනැගිලි කළමනාකරු."
    ,
      "පළාත් අධ්‍යාපන අධ්‍යක්ෂ,\nපළාත් අධ්‍යාපන දෙපාර්තමේන්තුව,\nත්‍රිකුණාමලය.\nඋපකෘත විශේෂ පාසලක් ලෙස ලියාපදිංචි කිරීම - ආයතනය\nඅංක 26/4 ඒ, බවුන්ඩ්රි පාර, මඩකලපුව දකුණ යන ස්ථානයෙහි තරුණ ක්‍රිස්තියානි සංගමය විසින් පවත්වාගෙන යනු ලබන විශේෂ අවශ්‍යතා සහිත සිසුන් සඳහා YMCA ආයතනය අධ්‍යාපන අමාත්‍යංශයේ ලියාපදිංචි උපකෘත විශේෂ පාසලක් ලෙස අනුමත කරගැනීම සඳහා එම ආයතනයේ ලේකම් විසින් ඉල්ලීමක් ඉදිරිපත්කර තිබේ.\nයථොක්ත පාසල ලියාපදිංචි කිරීම සලකා බැලීම සඳහා පහත තොරතුරු අවශ්‍ය වේ.\n1. දුලබතා සහිත සිසුන් සඳහා පාසලක් පවත්වාගෙන යාමට හැකිවන පරිදි සමිති සමාගම් ආඥා පනත යටතේ සමිතියක් වශයෙන් ලිපදිංචි කරන ලද සහතිකයේ පිටපතක් හෝ පාර්ලිමේන්තු පනතක් මගින් ලැබී ඇති අනුමැතියේ පිටපතක්.\n2. සමාජසේවා ආයතනයක් ලෙස පවත්වාගෙන යාමට සමාජ සේවා අමාත්‍යංශයෙන් ලබා ඇති අනුමැතිය.\n3. පාසල් භූමිය හා ගොඩනැගිලි පිලිබඳ පාසල් පාලක මණ්ඩලය සතු අයිතිය තහවුරු කෙරෙන ලිපි (ඉඩම් ඔප්පු ආදිය).\n4. පාසල ආරම්භයේ සිට මේ දක්වා අධ්‍යාපන ලබන සිසුන්ගේ සංඛ්‍යාව හා ස්වාභාවය ගැහැණු පිරිමි වශයෙන් .\n5. ගුරුවරුන් තෝරාගන්නා ලද ක්‍රමවේදය ගුරු සංඛ්‍යාව ගැහැණු පිරිමි වශයෙන්.\n6. පාසැල්හි පාසල් අධ්‍යාපන කටයුතු හා අනෙකුත් වැඩසටහන් ක්‍රියාත්මක කරන කාලසටහන. \n7. මෙම ආයතනය අධ්‍යාපන ආයතනයක් පවත්වාගෙන යාම සඳහා ආයතනය සතු මූල්‍ය වත්කම්.\n8. පාසලේ ආරම්භයේ සිට මේ දක්වා පවතින භෞතික සම්පත් පිලිබඳ තොරතුරු හා ඒවායේ  වටිනාකම. \n9. සාමාන්‍ය පාසල්වලට ඇතුළත් කර අධ්‍යපනය ලබාගත නොහැකි දුබලතා සහිත සිසුන් පමණක් අධ්‍යාපනය ලබන බව."
    ,
      "මගේ අංකය - 1/3/13/1\nදිනය - 2010.10.20\nවිදුහල්පති,\nම/ශ්‍රී සංඝමිත්තා බාලිකා මහා විද්‍යාලය,\nමාතලේ.\nවෘත්තීය මංපෙත් වැඩසටහන 2010.\nමිනිස් බල හා රැකි රක්ෂා දෙපාර්තමේන්තුවේ වෘත්තීය මාර්ගෝපදේශ අංශය මගින් විද්වත් වෘත්තිකයින්ගේ සංගමයේ පුහුණු ආයතන හා රැකියා ආයතන සම්බන්ධ කර ගනිමින් උක්ත වැඩසටහන පැවැත්වීමට කටයුතු සංවිධානය කර ඇත.\n02. මෙම වැඩසටහන මගින් පාසල් සිසුන් තුල ඇති හැකියා ලැදියා හා වටිනාකම් පිළිබඳ ස්වයං තක්සේරුවක් ලබා දීම හා ඒ අනුව ගැලපෙන විවිධ වෘත්තීය මංපෙත් හඳුන්වා දීම.\n\tතෝරාගත් වෘත්තීය මංපෙතට පිවිස වෘත්තීය දියුණුව සලසා ගැනීම සඳහා සුදුසු වෘත්තීය සැලැස්මක් පාසලින් පිටවීමට පෙර සකසා ගැනීමට මග පෙන්වීම.\n\tවැඩිදුර තොරතුරු ලබා ගැනීම සඳහා තොරතුරු පත්‍රිකා සමග කවුළු විවෘත කර තිබීම.\n\tඉහත ක්‍රියා මාර්ග නිසා පාසල් කාලය තුලදීම වැඩ ලෝකය සම්බන්ධව අවබෝධය ලබාදීම හේතුවෙන් පාසල් කාලයෙන් පසු රැකියා සෙවීමේ කාලය අඩුවීම මගින් ශ්‍රම බලකායට අවම කාලයක් තුල ඵලදායි පුද්ගලයින් එක් වීම හරහා රටේ සංවර්ධනයට දායක කරවීම.\n\tඋක්ත වැඩසටහන පැවැත්වීම සඳහා ඔබ විද්‍යාලයීය ශ්‍රවණාගාරය 2010/10/26 වෙනි දින පෙ:ව: 8.00 සිට ප:ව: 1.30 දක්වා වෙන් කර දෙන මෙන් කාරුණිකව ඉල්ලා සිටිමි.\nනිරෝෂණ කරුණාතිලක,\nසහකාර දිස්ත්‍රික් ලේකම් - මාතලේ,\nඅ/ක : එච්.එම්.ගාමිණී සෙනෙවිරත්න,\nදිස්ත්‍රික් ලේකම් - මාතලේ වෙනුවට."
    ,
      "මගේ අංකය : සේ.අ.අ.ගෙවීම්/දී.කම්./පොදු උපදෙස්/2016\nසේ.අ.අ.ගෙවීම් අංශය \n2016.05.18\nසහකාර කම්කරු කොමසාරිස් ,\nදිස්ත්‍රික් කම්කරු කාර්යාලය ,\nමඩකලපුව .\nඅංක 136 වන ප්‍රගති සමාලෝචන රැස්වීමේදී ගත් වැදගත් තීරණ. \nශ්‍රී ලංකා මහ බැංකුව සහ කම්කරු දෙපාර්තුමේන්තුව මඟින් ඉහත අංක දරණ ප්‍රගති සමාලෝචන රැස්වීමේ දී පහත කරුණු ක්‍රියාත්මක කිරීමට එකඟ වී ඇත.\n ඒවා ක්‍රියාත්මක කිරීමට මෙසේ යොමු කරමි.\nකම්කරු දෙපාර්තුමේන්තුවේ පරිගණක පද්ධතියේ හා RN වාර්තාවට අනුව සාමාජික අංක ඒකාබද්ධ වී ඇති විට හා එම ඒකාබද්ධ වූ අංක සඳහා ශ්‍රී ලංකා මහ බැංකුවේ ශේෂයක් නොපෙන්වන විටදී අදාළ අංක වල ඒකාබද්ධයක් ඇති බව තීරණ ලිපියේ  Other Remarks වල ඇතුළත් කිරීමටත්,\nඋදා : 256/A/001 Amalgamated with 257/A/002 in department of labour\nතීරණ ලිපියේ Account name හි අර්ථලාභියාගේ සම්පුර්ණ නම නොයොදා මුලකුරු සමඟ නම පමණක් ඇතුළත් කිරීම සහ හවුල් ගිණුමක් දී ඇති අවස්ථාවකදී අර්ථලාභියාගේ නම ගිණුම් හිමියාගේ නම යටතේ යෙදීමටත් ,\nතවද 60%,40% වශයෙන් ප්‍රතිලාභ කොටස් වශයෙන් නිදහස් කිරීමේ දී අදාළ කොටස් වෙන වෙනම බැර විය යුතු අය හා කොටස් බෙදිය යුතු කාලය පැහැදිලිව ඇතුළත් ලිපියක් තීරණ ලිපිය සමඟ යැවීමටත්,\n අක්‍රීය ආයතනවල සාමාජික නමෙහි අනන්‍යතාවය සනාථ කර ගැනීමේ දී ඊට අදාළව ශ්‍රී ලංකා මහ බැංකුවෙන් දායක මුදල් විස්තර ඉල්ලුම් කිරීමේ දී අවශ්‍යතාවය පැහැදිලිව දන්වා යැවීමටත්,"
    ,
      "ගුරු  විදුහල්පති මගින්,\nඑම්.සිවකුමාර් මයා,\nමුරකරු ,\nගුරු විද්‍යාලය ,\nමඩකලපුව.\nපෙර සේවා ස්ථානය සංශෝධනය කිරීම.\nමගේ අංක ඊඩි /05/67/12/60/iv වෙළුම හා 2016.02.09 දිනැති ලිපිය මඟින් මඩකලපුව ජාතික අධ්‍යාපන විද්‍යා පීඨය වෙත ලබා දෙන ලද අධ්‍යාපන  අමාත්‍යාංශයේ පුස්තකාල සේවක තනතුරට පත් කිරීමේ ලිපිය හා බැඳේ.\nඑම ලිපියෙහි ලිපිනයේ සඳහන් ඔබගේ වර්තමාන සේවා ස්ථානය මඩකලපුව ගුරු විද්‍යාලය ලෙස ද නව සේවා ස්ථානය මඩ/වින්සන්ට් බාලිකා විද්‍යාලය ලෙස ද නිවැරදි කරන බව කාරුණිකව දන්වා සිටිමි.\nඉහත ලිපියේ අනෙකුත් කොන්දේසි එලෙසම පවතින බව වැඩි දුරටත් දන්වා සිටිමි.\nඑච්.ඒ.එස්.නිශාන්ත පෙරේරා\nපරිපාලන නිලධාරී (ආයතන)\n පිටපත්\nවිගණකාධිපති \nඅධ්‍යක්‍ෂ ජනරාල් - විශ්‍රාම වැටුප් දෙපාර්තුමේන්තුව. \nප්‍රධාන ගණකාධිකාරී. \nපළාත් අධ්‍යාපන අධ්‍යක්‍ෂ \t- නැගෙනහිර පළාත.\nකලාප අධ්‍යාපන අධ්‍යක්‍ෂ \t- මඩකලපුව. \nගුරු විදුහල්පති - ගුරු විද්‍යාලය. \nපීඨාධිපති - මඩ/වින්සන්ට් බාලිකා විද්‍යාලය. \nපෞද්ගලික ලිපිගොනුව."
    ,
      "මගේ අංකය - සීපීසී/සීඑස්/3/3/4\nදිනය - 2014.01.06\nලේකම්,\nමධ්‍යම පළාත් අමාත්‍යාංශය,\nසියළු අමාත්‍යාංශ.\nප්‍රාග්ධන ආයෝජන වැඩසටහන - 2013 - මධ්‍යම පළාත\n2013.12.31 දිනට අවසන් ප්‍රගති වාර්තා ලබා ගැනීම.\n(අවිච්ඡේද හා අතරැඳි බිල්පත් ඇතුලත්ව)\nජාතික, පළාත් හා දිස්ත්‍රික්ක මට්ටමේ අවසාන ප්‍රගති සමාලෝචන කටයුතු, ඇගයීම් හා සමාජ බලපෑම් ඇගයීම, ඉදිරි සැලසුම් සම්පාදන කටයුතු වැනි අවශ්‍යතාවයන් උදෙසා 2013 මධ්‍යම පළාත් සභා සංවර්ධන සැලැස්මට අදාල පහත සඳහන් ප්‍රගති වාර්තා හා විස්තර 2014.01.14 දිනට හෝ පෙර මෙම කාර්යාලයට එවීමට කටයුතු කරන මෙන් කාරුණිකව ඉල්ලා සිටිමි.\n(01) මධ්‍යම පළාත් ආයෝජන වැඩසටහනට අදාල සවිස්තරාත්මක ප්‍රගති වාර්තාව.\n2013 වර්ෂය තුළ ඔබ අමාත්‍යාංශය මඟින් පළාත තුළ විවිධ මුදල් මූලයන් යටතේ (පළාත් සභා, රේඛීය අමාත්‍යාංශ, දෙපාර්තමේන්තු විශේෂ හා විදේශාධාර ව්‍යාපෘති) ක්‍රියාත්මක කරන ලද සියලු සංරචක වලට අයත් සංවර්ධන වැඩසටහන්වල (ගරු පළාත් සභා මන්ත්‍රී ආධාර ව්‍යාපෘති ද ඇතුළත්ව) ප්‍රගතිය යෝජනා මට්ටමින් මේ යටතේ ඉදිරිපත් කිරීමට කාරුණිකව කටයුතු කරන්න. ඒ සමඟ පහත සඳහන් වාර්තාද වෙන් වෙන් වශයෙන් ඉදිරිපත් කරන්න.\n(1.1) 2013 වර්ෂයේදී අවිච්ඡේද වූ ව්‍යාපෘති පිළිබඳ විස්තර හා එක් එක් ව්‍යාපෘතියේ අවිච්ඡේද මුදල.\n(1.2)  2013 වර්ෂයේ අතරැදි බිල්පත් විස්තර හා වටිනාකම.\nඇම්.ආර්.ජී.එච්.දසනායක,\nනියෝජ්‍ය ප්‍රධාන ලේකම්(සැලසුම්),\nප්‍රධාන ලේකම් වෙනුවට."
    ,
      "එස්.නිරෝෂිනි මයා/මිය/මෙනවිය ,\nමන්ඩපතාඩි ,\nකුණිකුඩායි ,\nමඩකලපුව .\nජාතික හැඳුනුම්පත් අංකය - 936961169V.\nඅධ්‍යාපන අමාත්‍යාංශයේ ප්‍රාථමික ශිල්පීය නොවන සේවක ගණයේ III ශ්‍රේණියේ පාසල් කම්කරු තනතුරට පත් කිරීම.\nඔබ අධ්‍යාපන අමාත්‍යංශය යටතේ 2016-04-18 දින සිට ක්‍රියාත්මක වන පරිදි පාසල් කම්කරු තනතුරට පත් කර ඇති බව සතුටින් දන්වනු කැමැත්තෙමි.\n02.ඒ අනුව සිසිලියා ජාතික පාසල වෙත පැමිණ විදුහල්පති වෙතින් රාජකාරි භාර ගෙන මා වෙත වාර්තා කරන්න.\n03. ඔබ විසින් මෙම තනතුර ජනතාව වෙනුවෙන් වන භාරයක් ලෙස දරනු ලැබිය යුතුය. \nඑම තනතුරට අදාල සියලු රාජකාරි හා වගකීම් මෙම දේශයේ හා එහි ජනතාවගේ යහපත වෙනුවෙන් කාර්යක්‍ෂමව හා අනලස්ව ඉටු කළ යුතු වේ. \n04. ඔබ ශ්‍රී ලංකා ප්‍රජාතාන්ත්‍රික සමාජවාදී ජනරජයට උපරිම වශයෙන් පක්‍ෂපාතී විය යුතු අතර , අදාල පරිදි රාජකාරියේදී රහස්‍ය භාවය පුර්ණ වශයෙන් ආරක්ෂා කළ යුතුය.\n05. ඔබ රජයේ සේවයේ නියැලී සිටින කාලය තුළදී වෙනත් ආයතනයක රැකියාවක නියැලීමට හෝ වෙනත් වෘත්තියක නියැලීමට හෝ ඔබට හිමිකමක් නොමැත.\n06. මෙම පත්වීම ස්ථිරය. \nඑතකුදු වුවත් ඔබ වර්ෂ 03 ක පරිවාස කාලයකට යටත් විය යුතුවේ. \nමෙම පත්වීම් ලිපියෙන් බැඳුණු යුතුකම් හා වගකීම් නොපිරිහෙලා ඉටුකරමින්ද, යහපත් හැසිරීම තුළින්ද, කාර්යක්ෂම ලෙස සේවය කිරීම තුළින්ද ඔබ රජයේ  සේවය සඳහා  සුදුස්සෙකු බව එම කාලය තුළදී සනාථ කළ යුතුවේ. \nඑසේම පරිවාස කාලය තුළදී ඔබ විසින් සම්පුර්ණ කළ යුතු මෙම පත්වීම් ලිපියෙහි දක්වා ඇති අනෙකුත් අවශ්‍යතා සපුරාලීමට ද ඔබ වගබලා ගත යුතුය. \nමෙම සියලු අවශ්‍යතා යථාපරිදි සපුරාලීමට ඔබ සමත් වුවහොත් පමණක් පරිවාස කාලය අවසානයේ මෙම පත්වීම ස්ථිර කරනු ලැබේ. \nඔබගේ පත්වීම ස්ථිර තත්ත්වයට පත්වනුයේ එම පත්වීම ස්ථිර කළ බවට පත්කිරීම් බලධාරියා විසින් විධිමත් ලිපියක් නිකුත් කිරීමෙන් පසුව පමණක් වේ. \nඑතෙක් ඔබගේ පත්වීම ස්ථිර කර නොමැති සේ සැළකේ."
    ,
      "මගේ අංකය - සීපීසී/සීඑස්/03/04/04/01\nදිනය - 2012.05.28\n අධ්‍යක්ෂ,\nකලාපීය ආර්ථික සංවර්ධන ආයතනය,\nයටිනුවර වීදිය,\nමහනුවර.\nශ්‍රවණාගාරය වෙන්කර ගැනීම.\nමධ්‍යම පළාත් ප්‍රධාන ලේකම් කාර්යාලය මගින් සංවිධානය කරනු ලබන ප්‍රගති සමාලෝචන රැස්වීමක් 2012.06.12 දින පෙ.ව. 9.00ට කලාපීය ආර්ථික සංවර්ධන ආයතනයේ (REDA) ශ්‍රවණාගාරයේදී පැවැත්වීමට කටයුතු යොදා ඇත.\nඑම රැස්වීම පැවැත්වීම සඳහා අදාල දිනයේදී ශ්‍රවණාගාරය වෙන්කර දෙනමෙන් කාරුණිකව ඉල්ලා සිටිමි.\nඑම්.ආර්.ජී.එච්.දසනායක,\nනියෝජ්‍ය ප්‍රධාන ලේකම්(සැලසුම්)."
    ,
      "මගේ අංකය - 1/2/9\nදිනය - 2016.06.27\nමෝටර් රථ පරීක්ෂක,\nමෝටර් රථ ප්‍රවාහන දෙපාර්තමේන්තුව,\nමාතලේ.\nරියදුරු බලපත්‍ර ගාස්තු අයකිරීම.\nඋක්ත කරුණට අදාලව මෝටර් රථ ප්‍රවාහන කොමසාරිස් ජනරාල්ගේ අංක DMT/HL/TAX/01-01 හා 2014.09.02 දිනැති ලිපිය හා බැඳේ.\n02. ඒ අනුව 2016.06.23 දින රැස් කරන ලද මෝටර් රථ ප්‍රවාහන දෙපාර්තමේන්තුවට අයත් ආදායම 74,500/- නිවැරදි බවට සහතික කරන අතර ඒ සඳහා අංක 606769 දරණ මුදල් භාරගැනීමේ කුවිතාන්සිය නිකුත් කර ඇති බව කාරුණිකව දන්වා සිටිමි.\n(ආදායම් ලැයිස්තුව හා මුදල් භාරගැනීමේ කුවිතාන්සිය මේ සමඟ අමුණා ඇත.)\nපී.ඩී.ආනන්ද ද අල්විස්,\nප්‍රධාන ගණකාධිකාරි,\nමාතලේ දිස්ත්‍රික් ලේකම් වෙනුවට."
    ,
      "සියලුම කලාප අධ්‍යාපන අධ්‍යක්ෂවරුන් වෙත,\nකලාප කාර්යාල ඇගයීම සඳහා සකස් කර ඇති දත්ත පාදකය ක්‍රියාත්මක කිරීම.\nඅධ්‍යාපන ලේකම්ගේ ED/01/44/03/05/03 අංක දරණ 2016.01.04 දින එවන ලද ලිපියට අමතර වශයෙනි.\nඉහත ලිපිය මගින් කලාප අධ්‍යපන කාර්යාලවල ඇගයීම් ක්‍රියාවලිය සඳහා සකස් කර ඇති දත්ත විද්‍යුත් පාදකය සම්බන්ධව කරුණු දක්වා ඇත. \nඊට අමතරව පහත සඳහන් කරුණු කෙරෙහිද ඔබේ අවධානය යොමු කරමි. \n1. අධ්‍යපන අමාත්‍යාංශයේ කළමනාකරණ හා ප්‍රමිති ශාඛාව මගින් ඉහත දත්ත පාදකය සම්බන්ධව කලාප හා පළාත් ප්‍රමිති සම්බන්ධීකරණ නිලධාරීන් සඳහා පුහුණු වැඩසටහනක් 2016.01.25 හා 26 දිනවල පවත්වන ලදී.\n2. එම පුහුණු වැඩමුළුවට සහභාගී වූ කළමනාකරණ හා ප්‍රමිති නිලධාරීන් මගින් කලාප කාර්යාලයේ එක් එක් අංශවලට ( සැලසුම්, සංවර්ධන, මූල්‍ය, සාමාන්‍ය හා ආයතනික, අධ්‍යාපන පරිපාලන ) අදාළ දත්ත, මෙම දත්ත පාදකයට ඇතුලත් කිරීම සඳහා වගකීම දැරිය හැකි නිලධාරීන් පුහුණු කරවා ගැනීම.\n3. හඳුන්වාදෙන දත්ත පාදකයට 2016 ජනවාරි මාසයේ සිට අදාළ දත්ත ඇතුලත් කිරීම ( මාසිකව ප්‍රගතිය විමසා ඇති නිර්නායකවලට අදාළ තොරතුරු මාසිකව සම්පූර්ණ කිරීම)\nලබා දී ඇති උපදෙස් පරිදි කටයුතු කොට කලාප කාර්යාලවලින් සිදු කෙරෙන කාර්යයන් තවදුරටත් කාර්යක්ෂමව හා ඵලදායිව ඉටුකර ගැනීමට ඔබේ අවධානය යොමු කරන ලෙස කාරුණිකව දන්වමි.\nඑස්. යූ. විජේරත්න,\nඅතිරේක ලේකම් (ප්‍රතිපත්ති, සැලසුම් හා කාර්යසාධන සමාලෝචන ),\nඅධ්‍යාපන ලේකම් වෙනුවට."
    ,
      "වෙත : අතිරේක ලේකම්(පාසල් සැපයීම් සේවා හා ඉදිකිරීම්) මඟින්,\n අතිරේක ලේකම් (සැලසුම් හා කාර්යසාධන සමාලෝචන)\nවෙතින් : අධ්‍යක්ෂ (පාසල් ගොඩනැගිලි)\nදිනය : 2013.11.26\nමගේ අංකය : ED/7/99/4/2/2\nවිෂය : අනවසරයෙන් හානිකර පරිදි පිහිනුම් තටාකයක් ඉදිකිරීම - කෑගලු විද්‍යාලය\nඋක්ත කරුණ සම්බන්ධයෙන් මා වෙත යොමු කරන ලද ඔබගේ ඊඩී/03/52/1/12/1(iv) හා 2013.10.31 ලිපිය හා බැඳේ.\nඒ අනුව මෙම කාර්යය ආදි ශිෂ්‍ය සංගමය මඟින් මෙහෙයවන අතර මේ වන විට ඔවුන් බිම් සකස් කිරීමේ කටයුතු සඳහා පමණක් අවසර ගෙන එම කටයුතු ඉටු කරමින් පවතී. \nමෙම ලිපියේ සඳහන් වන ආකාරයට කඳු සහිත භූමියක මෙම ඉදිකිරීම් කටයුතු කරගෙන යාම නිසා ජලය පහළට ගලා බසින අතර මෙහිදී රැඳෙන ජලය උපරිම වශයෙන් පැය 24 ක කාලයක් තුළ එක්රැස් වීමකින් තොරව බැස යන බවටත්, එමඟින් අසල්වාසීන්ගේ සෞඛ්‍යයට තර්ජනයක් නොවන බවටත් විදුහල්පති සමඟ කරන ලද සාකච්ඡාවේදී අනාවරණය විය.\n තවද ඒ බවට කරුණු දක්වමින් අධ්‍යාපන අමාත්‍යංශ ලේකම්වරයා වෙත යොමු කරන ලද විදුහල්පතිගේ ලිපියද ඔබගේ කාරුණික අවධානයට යොමුකරමි.(ඇමුණුම 1)\nතවදුරටත්, උක්ත පැමිණිලිකරු විසින් සඳහන් කර ඇති පරිදි ස්වභාවිකව පිහිටි ගල මත බෝර දමා හෑරීමට කටයුතු කරමින් සිටින බවට සඳහන් කර ඇතත්, එම කටයුතු ආරක්ෂාකාරී ලෙස නවීන තාක්ෂණය මඟින් ඉටුකළ හැකි වන අතර, එවැනි ක්‍රම උපයෝගී කොටගෙන සිදු කිරීමට නියමිත බවටත් විදුහල්පති විසින් කරුණු දක්වා යොමු කර ඇති ලිපියෙහි(ඇමුණුම 1) සඳහන් වේ.\nආදි ශිෂ්‍ය සංගමය මඟින් පිහිනුම් තටාකය ඉදිකිරීම - කෑගලු විද්‍යාලය\nඉහත විදුහලේ ආදි ශිෂ්‍ය සංගමය විසින් පිහිනුම් තටාකය ඉදිකර දීම සඳහා ඉදිරිපත් වී ඇති බැවින් ඒ සඳහා අනුමැතිය ලබා දෙන ලෙස විදුහල්පති 2013-05-08 දිනැති ලිපියෙන් (ඇමුණුම 3) ඉල්ලීමක් කර ඇත.\n2.මෙම ඉදිකිරීමට අදාළ පහත සඳහන් ලියවිලි සහ සැලසුම් ද මා වෙත එවා ඇත.\n2.1.විදුහල්පතිගේ වාර්තාව (ඇමුණුම 4)\n2.2.ඇස්තමේන්තුව (අදියර I සඳහා) (ඇමුණුම 5)\n2.3.තාක්ෂණ කමිටුවේ හා මෙහෙයුම් කමිටුවේ සැලසුම් (ඇමුණුම 6)\n2.4.ආදි ශිෂ්‍ය සංගමය විසින් පිහිනුම් තටාකය ඉදිකර දීමට කැමැත්ත පළ කල ලිපිය(ඇමුණුම 7)\n2.5.මුදල් තත්ත්වය පිළිබඳ වාර්තාව (ඇමුණුම 8)\n2.6.භූමි සැලැස්ම(ඇමුණුම 9)\n2.7.වාස්තු විද්‍යාත්මක සැලසුම් (ඇමුණුම 10)\nඉහත ඉදිකිරීම්වලට අදාළ සැලසුම් අනුමත කරන අතර තවද මෙම ව්‍යාපෘතිය ආදිශිෂ්‍ය සංගමය මඟින් මුදල් උපයා කරනු ලබන කටයුත්තක් බැවින් ප්‍රසම්පාදන ක්‍රියාවලිය සඳහා අදාළ වන චක්‍රලේඛ මීට අදාළ නොවන බවද කාරුණිකව දන්වා සිටිමි.\n මෙම ඉදිකිරීම් කටයුතු සඳහා මුදල් තත්ත්වය යහපත් මට්ටමක නොපැවතිය හොත් භූමිය සකස් කොට ගොඩනැගිල්ලක් ඉදිකිරීම සඳහා වුවද භාවිතයට ගැනීම සඳහා වන ඉල්ලීම ඉටුකිරීම නිර්දේශ කරමි.\nකරු අනුමැතියට ඉදිරිපත් කරමි.\n අයි.එම්.ප්‍රනාන්දු,\nපාසල් ගොඩනැගිලි අධ්‍යක්ෂ."
    ,
      "වෙත \t\t :-  ජ්‍යෙෂ්ඨ සහකාර ලේකම් (පාලක)\nවෙතින් \t :-  ගොඩනැගිලි කළමනාකරු \nදිනය \t\t:-2011.11.11\nමගේ අංකය\t:-ED/5/74/1(පොදු II)\nවිෂය \t\t :-  පරිගණක අභ්‍යාසලාභිනියක් ලබා ගැනීම \nගොඩනැගිලි කළමනාකරණ ශාඛාවේ පරිගණක පුහුණු වන්නකුගේ සේවය 2011.07.31 දින වන තෙක් ලබා ගත් අතර ඇයගේ සේවය අවසන් වී ඇති බැවින් නැවත පරිගණක අභ්‍යාසලාභිනියක් ලබා දෙන මෙන් කාරුණිකව ඉල්ලමි.\nඑච්.එච්.කේ.වීරකෝන්,\nගොඩනැගිලි කළමනාකරු ."
    ,
      "ප්‍රධාන විධායක නිලධාරී,\nජාතික රක්ෂණ භාර අරමුදල,\nඅංක 97 මරදාන පාර,\nකොළඹ - 10\nමහත්මයාණෙනි,\nඅග්‍රහාර  වෛද්‍ය රක්ෂණ ප්‍රතිලාභ ඉල්ලීම -2016 \nමෙම කාර්යාලයට අනුයුක්තව සේවය කරන රාජ්‍ය කළමනාකරණ සහකාර ඒ.එම්.රත්නමැණිකේ මහත්මිය විසින් ඇස් කණ්ණාඩි ලබා ගැනීම සඳහා මා වෙත ඉදිරිපත් කරන ලද අග්‍රහාර වෛද්‍ය රක්ෂණ ප්‍රතිලාභ අයදුම්පත , වෛද්‍ය බිල්පත් හා පහත සඳහන් ලියවිලි මාගේ නිර්දේශය සහිතව  අවශ්‍ය කටයුතු සඳහා මේ සමග ඉදිරිපත් කරන බව කාරුණිකව දන්වමි.\n01. ඇස්  පරික්ෂා කිරීමට වෛද්‍යවරයා ඉදිරිපත් කරන සහතිකය .\n02. කණ්ණාඩි මිලදී ගත් බිල්පත.\n03. ජාතික හැදුනුම්පතේ සහතික කරන ලද පිටපත.\nමෙයට,\nඔබේ සේවයට කැපවූ ,\nබී.එස්.එම්.පි.සෙනෙවිරත්න.\nපරිපාලන නිලධාරී,\nපල්ලේපොල ප්‍රාදේශීය ලේකම් වෙනුවට ,"
    ,
      "මගේ අංකය - CPC/CS/03/03/04\nදිනය - 07.07.2014\nලේකම්,\nමධ්‍යම පළාත් සියළු අමාත්‍යාංශ.\n2014 ප්‍රාග්ධන ආයෝජන වැඩසටහන - මධ්‍යම පළාත\nමාසික ප්‍රගති වාර්තා ලබාගැනීම සම්බන්ධවයි.\nඔබ අමාත්‍යාංශය මඟින් මධ්‍යම පළාත තුළ විවිධ ප්‍රතිපාදන මූලයන් යටතේ (පළාත් සභා, රේඛීය අමාත්‍යාංශ, දෙපාර්තමේන්තු හා විදේශාධාර ව්‍යාපෘති ) ක්‍රියාත්මක කරන ලද සියළු සංරචක වලට අයත් සංවර්ධන වැඩසටහන්වල (ගරු පළාත් සභා මන්ත්‍රී ආධාර ව්‍යාපෘතිද ඇතුළත්ව ) ප්‍රගතිය පහත සඳහන් රැස්වීම් වලට ඉදිරිපත් කිරීම සඳහා අදාළ ආකෘතිය යටතේ ලබාදෙන මෙන් කාරුණිකව ඉල්ලමි.  \n02. මෙම වාර්තා ඒකාබද්ධ කර ප්‍රගති වාර්තා සකස් කර නියමිත වේලාවට ඉදිරිපත් කළ යුතු බැවින්,ඔබ අමාත්‍යාංශයට අයත් දෙපාර්තමේන්තු හා අංශ වෙතින් ප්‍රගතිය ලබාගැනීම සඳහා සුදුසු වැඩපිළිවෙලක් සකස් කරන මෙන් කාරුණිකව ඉල්ලා සිටිමි.\n\t1.පළාත් ප්‍රගති වාර්තාව.\n\t\tඅදාළ මාසයේ ප්‍රගතිය එලඹෙන ඊළඟ මස 10 දිනට හෝ ඊට පෙර සාරාංශ වාර්තාව සමඟ (ආකෘති අංක 01).\n\t\t2.දිස්ත්‍රික් සම්බන්ධීකරණ කමිටු රැස්වීම් වාර්තා.\n\t\tමහනුවර, මාතලේ හා නුවරඑළිය දිස්ත්‍රික් කමිටු රැස්වීම් සඳහා ඉදිරිපත් කිරීමට අදාළ ප්‍රගති වාර්තා සෑම මසකම 5 දින හෝ ඊට පෙර සාරාංශ වාර්තාවක් ලෙස (ආකෘති අංක 2).\n\t3.අමාත්‍යාංශ ලේකම්වරුන්ගේ රැස්වීම් වාර්තාව.\n\t\tමෙම රැස්වීම සෑම මසකම අවසන් සතියේ පැවැත්වෙන බැවින් එම මස 20 වෙනි දිනට හෝ ඊට පෙර සාරාංශ වාර්තාවක් ලෙස (ආකෘති අංක 3).\nඅදාළ වාර්තා නියමිත දිනට ප්‍රථම මා වෙත ලැබීමට සැලැස්වීම මඟින්, උක්ත රැස්වීම් සඳහා ප්‍රගති වාර්තා හා පරිගණක ඉදිරිපත්කිරීම් සකස් කිරීමේ දුෂ්කරතා අවම වනු ඇත.\nඇම්.ආර්.පී.එච්.දසනායක,\nනියෝජ්‍ය ප්‍රධාන ලේකම්(සැලසුම්),\nප්‍රධාන ලේකම් වෙනුවට."
    ,
      "මගේ අංකය : 16/1/4094                   \n අධ්‍යාපන අධ්‍යක්‍ෂ ,\nකලාප අධ්‍යාපන කාර්යාලය ,\nමඩකලපුව.\nවිශ්‍රාම වැටුප් ගෙවීම -  පී.රට්නසබාපති මහත්මිය\nඋක්ත කරුණ සම්බන්ධව ඔබ විසින් යොමු කරන ලද ඔබේ සමාංක  හා  දිනැති ලිපි ලේඛන හා බැදේ.\n එම අයදුම්පතෙහි අඩුපාඩු පවතින බැවින් මෙම ලිපිය සමඟ අයදුම්පත ඔබ වෙත යොමු කරමි. \nවිශ්‍රාම වැටුප් දෙපාර්තමේන්තු වෙබ් අඩවිය වෙත පිවිස එහි සඳහන් අඩුපාඩු නිවැරදි කර දින 10ක් තුල නැවත මා වෙත යොමු කරන ලෙස කාරුණිකව දන්වමි.\n www.pensions.gov.lk -> Registration Information -> Enter NIC -> Submit  \nසහකාර අධ්‍යක්‍ෂ (විශ්‍රාම වැටුප් හා ලියාපදිංචි ),\nවිශ්‍රාම වැටුප් අධ්‍යක්‍ෂ ජනරාල් වෙනුවට."
    ,
      "මගේ අංකය, \nපාලන අංශය ,\nකම්කරු දෙපාර්තමේන්තුව, \nකොළඹ  05.\n ලේකම් - කම්කරු හා වෘත්තීය සමිති සබදතා අමාත්‍යාංශය\nකම්කරු කොමසාරිස්වරුන් ,\nප්‍රධාන ගණකාධිකාරී ,\nකලාප භාර නියෝජ්‍ය කම්කරු කොමසාරිස්වරුන් ,\nනියෝජ්‍ය කම්කරු කොමසාරිස්,\nනියෝජ්‍ය අධ්‍යක්‍ෂක ,\nදිස්ත්‍රික් කර්මාන්තශාලා පරීක්ෂක ඉංජිනේරුවරුන් ,\nසහකාර කම්කරු කොමසාරිස්වරුන් ,\nජ්‍යෙෂ්ඨ කම්කරු නිලධාරීන් .\nකාර්යාලීය ලිපිනය වෙනස් වීම පිළිබඳව දැනුම්දීම.\nඅප දෙපාර්තමේන්තුවට අයත් මුලතිව්, කිලිනොච්චි, මොරවක යන උප කම්කරු කාර්යාලය යන කාර්යාල නව ගොඩනැගිලිවල ස්ථාපිත කිරීම හේතුවෙන් එම කාර්යාලවල නව තැපැල් ලිපිනයන් පහත සඳහන් පරිදි වෙනස් වූ බව කාරුණිකව දන්වා සිටීම.\n තවද , එම කාර්යාලයන්හි දුරකථන හා ෆැක්ස් අංකයන්හි කිසිදු වෙනසක් සිදු නොවූ බවද වැඩිදුරටත් දන්වා සිටිමි.\nකාර්යාලය , නව තැපැල් ලිපිනය. \nමුලතිව් උප කම්කරු කාර්යාලය , මුලතිව් පාර , හිජ්රාපුරම් , මුල්ලියාවලෛ.\nකිලිනොච්චි උප කම්කරු කාර්යාලය , 155 වන සැතපුම් කණුව , A9 පාර , කිලිනොච්චිය.\nමොරවක උප කම්කරු කාර්යාලය, සිසිලානවත්ත , වැවිල , මොරවක. \nකේ.එන්.එස්.ජි. කාරියප්පෙරුම ,\nඅතිරේක කම්කරු කොමසාරිස් ජනරාල් ,\nකම්කරු කොමසාරිස් ජනරාල් වෙනුවට.\nපිටපත් :- 1. කම්කරු කොමසාරිස් ජනරාල් \n2. ජ්‍යෙෂ්ඨ කම්කරු නිලධාරී - මුලතිව් \n3. ජ්‍යෙෂ්ඨ කම්කරු නිලධාරී - කිලිනොච්චිය \n4. ජ්‍යෙෂ්ඨ කම්කරු නිලධාරී - මොරවක්"
    ,
      "பணிப்பாளர்,\nதொழில் வழிகாடடல் பிரிவு, \nதொழிலாளர் தொடர்பு மற்றும் மனித வள அமைச்சு, \n64,பிரதான வீதி ,\nபத்தர முல்ல.\nதொழில் சார் சமூக தலைவர்களை பகிற்று விக்கும் செயலமர்வு \nமாத்தளை மாவடடத்தில் மேட்படி செயலமர்வு  2008.02.01/02 இரு தினங்களிலும் கண்டி பொல்கொல்ல  அபிவிருத்தி வலயத்தில் நடை பெற்றது.\nஅதற்காக பிரதேச செயலாளர் பிரிவுகளில் இருந்து பங்கு பெற செய்த தொழில் சார் சுமுக தலைவர்களின் தொகையும் உரிய கையொப்ப படிவத்தின் பிரதி ஒன்றையும் இத்துடன் முன்வைக்கின்றேன்.\nபிரதேச செயலாளர் பிரிவு               ஈடுபடுத்தப்படட தொழில் சார் சமூக தலைவர்களின் எண்ணிக்கை \nமாத்தளை                                                                          03\nஉகுவேலை                                                                       03\nரத்தோட                                                                              07\nதம்புள்ள                                                                               04\nநாவுல                                                                                   03\nகலேவெல                                                                           07\nலக்கல பல்லேகம                                                            03\nவிலகமுக                                                                             03\nஅம்பன்கங்கை கமேலதிக நடவடிக்கைக்காக முன்வைக்கிறேன். \nோரல                                                   04\nயடவத்தை    - பல்லே  போல                                                                       -\nபணிப்பாளர், \nதொழில் வழி காடடல் பிரிவு,  \nதொழிலாளர் தொடர்பு மற்றும் மனிதவள அமைச்சு .\n                                                                                                               தேஜாளி திலகரத்னா      \n                                                                                                                                       மேலதிக மாவடட செயலாளர் \n                                                                                                                                        மாத்தளை மாவடட செயலாளர்களுக்காக \n\nபிரதி;பணிப்பாளர் \n            தொழில் திடடமிடல் பிரிவு பத்தரமுல்ல"
    ,
      "ලේකම්, ප්‍රධාන අමාත්‍යාංශය, මධ්‍යම පළාත,\nලේකම්, කෘෂිකර්ම අමාත්‍යාංශය, මධ්‍යම පළාත,\nලේකම්, සෞඛ්‍ය අමාත්‍යාංශය, මධ්‍යම පළාත,\nලේකම්, කර්මාන්ත අමාත්‍යාංශය, මධ්‍යම පළාත,\nලේකම්, මාර්ග අමාත්‍යාංශය, මධ්‍යම පළාත.\nවාර්ෂික සංවර්ධන සැලැස්ම-2016\nපළාත් නිශ්චිත සංවර්ධන ප්‍රදාන\nඋක්ත කරුණ සම්බන්ධව මාගේ අංක සීපීසී/සීඑස්/03/02/09/1-2016 හා 2016.04.01 දිනැති ලිපිය හා බැඳේ.\n02. එහි සඳහන් පරිදි 2016 සංවර්ධන සැලැස්ම සම්බන්ධ ව්‍යාපෘති සංශෝධනය සඳහා දී තිබූ කාල සීමාව, 2016.05.30 දිනෙන් අවසන් වන බැවින්, එම දිනට පෙර සුවිශේෂී කරුණු මත කෙරෙන සියළු සංශෝධන මා වෙත එවිය යුතුය. \nඉන් පසු කිසිඳු යෝජනාවක් සංශෝධනය නොකරන ලෙස කාරුණිකව දන්වමි.\nඑම්.එස්.ප්‍රේමවංශ,\nප්‍රධාන ලේකම්,\nමධ්‍යම පළාත ."
    ,
      "ප්‍රාදේශීය ලේකම්,\nප්‍රාදේශීය ලේකම් කාර්යාලය,\nනුවරඑළිය.\n2016 වර්ෂය - පළාත් සභා ගරු මන්ත්‍රී සංවර්ධන ව්‍යාපෘති අනුමත කිරීම.\nමධ්‍යම පළාත් සභා ගරු නියෝජ්‍ය සභාපති ආර්.එම්.එස්.බී.රත්නායක මැතිතුමා,\nඉහත සඳහන් පළාත් සභා ගරු මන්ත්‍රීතුමාට ලබාදුන් රුපියල් මිලියන 2.5ක මුදල යටතේ, මෙම ලිපියේ පසු පිටෙහි සඳහන් සංවර්ධන ව්‍යාපෘති ඔබගේ ප්‍රාදේශීය ලේකම් කාර්යාලය මගින් ක්‍රියාත්මක කිරීම මෙයින් අනුමත කරමි.\n02. අනුමත සංවර්ධන ව්‍යාපෘති ක්‍රියාත්මක කරවීම සඳහා අවශ්‍ය ප්‍රතිපාදන, නියෝජ්‍ය ප්‍රධාන ලේකම් (මුදල්) විසින් ඔබ වෙත ලබා දෙනු ඇත.\n03. සංවර්ධන ව්‍යාපෘති ආරම්භ කිරීමේදී මුදල් ලබාදී ඇති ගරු මන්ත්‍රීතුමාගේ අදහස් ද විමසා බලන්නේ නම් මැනවි.\n04. සෑම මාසයක් සඳහාම ඊළඟ මාසයේ 10වන දිනට පෙර, අනුමත කර එවා ඇති සංවර්ධන ව්‍යාපෘතීන්ගේ ප්‍රගතිය මා වෙත එවීමට කටයුතු කරන ලෙස ද කාරුණිකව දන්වමි.\n05. ව්‍යාපෘති ක්‍රියාත්මක කිරීමට අපහසු යම් විශේෂ ප්‍රායෝගික ගැටලු ඇතිනම් ඒ බව මා වෙත වහාම දන්වන්න.\n06. මෙම ව්‍යාපෘති 2016.11.15 වන දිනට පෙර භෞතික වශයෙන් නිමකර, සියලුම ගෙවීම් කටයුතු ද අවසන් කළ යුතු බව කාරුණිකව දන්වා සිටිමි."
    ,
      "මගේ අංකය - 8/4/1/5/1\nදිනය - 2016.06.\nලේකම්,\nජනාධිපති අරමුදල,\nඅංක 41, රේණුකා ගොඩනැගිල්ල,\nජනාධිපති මාවත, කොළඹ 01.\nවෛද්‍ය ප්‍රතිකාර ලබා ගැනීම සඳහා ජනාධිපති අරමුදලින් මූල්‍යාධාරයක් ඉල්ලීම -  ................................. මයා\nඋක්ත කරුණ සම්බන්ධ ඔබේ අංක පීඑෆ්/එම්/................................. හා 2016.06.14 දිනැති ලිපිය හා බැඳේ.\n මාතලේ ප්‍රාදේශීය ලේකම් කොට්ඨාශයේ අංක .............................. ලිපිනයෙහි පදිංචි ........................... මහතාගේ වෛද්‍ය ආධාර අයදුම්පතේ අංක 08,09,10 හා 11 ඡේද යටතේ වූ මාගේ නිරීක්ෂණ පහත පරිදි වේ.\nඡේද අංක 08 - රෝගියාගේ පවුලේ විස්තර හා ආදායම් පිළිබඳව අයදුම් පත්‍රයේ සඳහන් තොරතුරු නිවැරදිය.\nඡේද අංක 09 - නිශ්චල හා චංචල දේපල පිළිබඳව අයදුම් පත්‍රයේ සඳහන් තොරතුරු නිවැරදිය.\nඡේද අංක 10 - යෝජිත ප්‍රතිකාර සඳහා අවශ්‍ය මුදල සොයාගෙන ඇති ආකාරය පිළිබඳව අයදුම් පත්‍රයේ සඳහන් තොරතුරු නිවැරදිය.\nඡේද අංක 12 - රෝගියා මීට පෙර ජනාධිපති අරමුදලින් මූල්‍යාධාර ලබාගෙන නැත.\n ඉහත නිරීක්ෂණ අනුව රෝගියාට ජනාධිපති අරමුදලින් මූල්‍යාධාරයක් ලබාදීම සුදුසු බව නිර්දේශ කර කාරුණිකව ඉදිරිපත් කරමි.\nකේ.පී.කේ.එල්.පී.මධුවන්ති,\nප්‍රාදේශීය ලේකම්/ මාතලේ."
    ,
      "මගේ අංකය - සීපීසී/සීඑස්/03/04/06-2(2016)\nදිනය - 2016.12.15\nප්‍රාදේශීය ලේකම්,\nප්‍රාදේශීය ලේකම් කාර්යාලය,\nඅඹගමුව.\n2016 වර්ෂය - පළාත් සභා ගරු මන්ත්‍රී සංවර්ධන ව්‍යාපෘති අනුමත කිරීම.\nඋක්ත කරුණ සම්බන්ධයෙන් මා විසින් ඔබ වෙත එවන ලද මගේ සමාංක හා 2016.11.27 දිනැති ලිපිය හා බැඳේ.\n02. එම ලිපියෙන් ඔබ ආයතනය මගින් ක්‍රියාත්මක කිරීම අනුමත කර තිබූ මධ්‍යම පළාත් සහා ගරු නියෝජ්‍ය සභාපති ආර්.එම්.එස්.බී.රත්නායක මැතිතුමාගේ පහත සඳහන් සංවර්ධන යෝජනාව එතුමාගේ ඉල්ලීම පරිදි පහත අයුරින් සංශෝධනය කර ක්‍රියාත්මක කිරීම මෙයින් අනුමත කරමි.\nඅවලංගු කරන ලද සංවර්ධන යෝජනාව මුදල    (රු.)\nඅනුමත කරන නව සංවර්ධන යෝජනාව මුදල   (රු.)"
    ,
      "මගේ අංකය : පි/එෆ්/එම්/219750 \nඔබේ අංකය : ADM/PREFU/D/175/2015\n2016 මැයි මස 23 වන දින \nප්‍රාදේශීය ලේකම් ,\nප්‍රාදේශීය මහලේකම් කාර්යාලය ,\nමන්මුනේ උතුර, මඩකලපුව.\nවෛද්‍ය ප්‍රතිකාර ලබා ගැනීම සඳහා ජනාධිපති අරමුදලෙන් මුල්‍යාධාරයක් ඉල්ලා සිටීම - වී.වීරසිංහම් මහතා\nඋක්ත කරුණට අදාල ඔබගේ සමාංක හා 2016.01.05 දිනැති ලිපිය සම්බන්ධයෙනි.\nඑම ලිපියේ රෝගියාගේ  රැකියාව, මාසික ආදායම් තත්ත්වය සම්බන්ධයෙන් සඳහන් කර නොමැත. \nනමුත් රෝගියා විසින් ඉදිරිපත් කරන ලද අයදුම්පතේ ඔහු රන්කරුවකු ලෙස රැකියාව කරමින් රු. 20,000/- ක මාසික අදායමක් ලබන බව සඳහන් කර ඇත. \nඒ අනුව රෝගියාගේ ඉහත තොරතුරු ඇතුළත් වාර්තාවක් නොපමාව මා වෙත එවන්නේ නම් මැනවි. \nනිරුක ලත්තුවාහන්දි ,\nජනධිපති සහකාර ලේකම් ,\nජනාධිපති අරමුදලේ සහකාර ලේකම් ,\nජනාධිපති ලේකම් වෙනුවට."
    ,
      "ප්‍රධාන විධායක නිලධාරී,\nජාතික රක්ෂණ භාර අරමුදල,\nඅංක. 97 මරදාන පාර,\nකොළඹ - 10.\nමහත්මයාණෙනි,\nඅග්‍රහාර වෛද්‍ය රක්ෂණ ප්‍රතිලාභ ඉල්ලීම -2016 \nමෙම කාර්යාලයට අනුයුක්තව සේවය කරන සංවර්ධන නිලධාරී අයි.ජි.ස්වර්ණ මල්ලිකා  මහත්මිය විසින් වෙනත් රෝග සඳහා  මා වෙත ඉදිරිපත් කරන ලද අග්‍රහාර වෛද්‍ය රක්ෂණ ප්‍රතිලාභ අයදුම්පත , වෛද්‍ය බිල්පත් හා පහත සඳහන් ලියවිලි මාගේ නිර්දේශය සහිතව අවශ්‍ය කටයුතු සඳහා මේ සමග ඉදිරිපත් කරන බව කාරුණිකව දන්වමි.\n01. රෝග නිශ්චය කාඩ් පතේ සහතික කරන ලද පිටපත. \n02. ජාතික හැදුනුම්පතේ සහතික කරන ලද පිටපත.\n03. උප්පැන්න සහතිකයේ සහතික කරන ලද පිටපත.\nමෙයට,\n ඔබේ සේවයට කැපවූ ,\nබී.එස්.එම්.පි.සෙනෙවිරත්න, \nපරිපාලන නිලධාරී,\nපල්ලේපොල ප්‍රාදේශීය ලේකම් වෙනුවට ,"
    ,
      "වෙත :- එස් .පල්ලියගුරු , ප්‍රධාන ගබඩා භාරකරු (පත්තලගෙදර ගබඩා සංකීර්ණය)\nවෙතින් :- අධ්‍යක්ෂ (සැපයීම්)\nමගේ අංකය :- ED/07/97/03/04/02\nදිනය :- 2011/06/\nවිෂය :- නිකුත් කිරීමේ නියෝගය 2014/ Sci/15\nඋප විෂය :- මහින්දෝදය තාක්ෂණික විද්‍යාගාර සඳහා භාණ්ඩ හා උපකරණ සැපයීම.\n01. නිකුත් කල යුතු භාණ්ඩ - මහින්දෝදය තාක්ෂණිකවිද්‍යාගාර සඳහා මිලදී ගෙන දැනට පත්තලගෙදර ගබඩාවේ ගබඩා කර ඇති ඇමුණුම 02 හි සඳහන් රසායනික ද්‍රව්‍ය\n02. නිකුත් කළ  යුත්තේ කාටද යන වග - ඇමුණුම 01 වශයෙන් ඇති ලැයිස්තුවේ සඳහන් පාසල්වල විදුහල්පතිවරුන් වෙත.\n03. භාණ්ඩ ප්‍රවාහනය සඳහා භාරදීම - තොරතුරු සටහන් කර ගනිමින් සැපයීම් ශාඛාවේ රියදුරු සංචිතයට අයත් රියදුරු මහතුන් වෙත.\n04. අතරමැදි පාර්ශවයක් හරහා එම භාණ්ඩ භාර දේ නම් එම පාර්ශවය - අදාළ නැත.\n05.ඉහත සඳහන් තොරතුරුවලට අනුකූලව අදාල භාණ්ඩ පහත සඳහන් කොන්දේසි වලට යටත්ව නිකුත් කරන මෙන් කාරුණිකව දන්වමි.\nපොදු 141නිකුත් කිරීමේදී හඳුනා ගත හැකි පරිදි සම්පූර්ණ නාමයන් හැඳින්වීම අනිවාර්ය වේ.\nඩබ්ලිව්. ඩී. ජයසේන\nඅධ්‍යක්ෂ (සැපයීම්)\nපිටපත් - 01. අධ්‍යාපන අධ්‍යක්ෂ (විද්‍යා) කරු. දැ. ගැ. පි."
    ,
      "වෙත :- අධ්‍යක්ෂ(තොරතුරු හා සන්නිවේදන තාක්ෂණ)\nමඟින් :- අතිරේක ලේකම් (ප්‍රසම්පාදන හා ඉදිකිරීම්)\nවෙතින් :- අධ්‍යාපන අධ්‍යක්ෂ (සැපයීම්)\nදිනය :- 2011.04.07\nමගේ අංකය :- ED/07/97/02/32\nවිශය :- ජයවර්ධන කලාපයේ පාසල් සඳහා විද්‍යුත් පියානෝ සහ පරාවර්තන රහිත විශේෂිත කළුලෑලි (Green Board) ලබාදීම.\nඉහත කරුණ සම්බන්ධයෙන් අතිරේක ලේකම් (අධ්‍යාපන ගුණාත්මක සංවර්ධන)ගේ අංක ED/24/1/2CT හා  2011/03/23 දිනැති ලිපිය හා බැඳේ.\n02. කොරියානු ආධාර යටතේ ලැබුණු විද්‍යුත් පියානෝ සහ පරාවර්තන රහිත විශේෂිත කළු ලෑලි (Green Board) ලබාදීමට සුදුසුකම් ලබා ඇති ජයවර්ධනපුර කලාපයේ පාසල් වෙත ඒ බව දන්වන ලිපි 2011-04-10 දින පැවැත්වීමට නියමිත උත්සවයේ දී එම පාසැල් වෙත ලබාදීමට අවශ්‍ය කටයුතු සූදානම් කිරීමට එමඟින් දැනුම් දී තිබිණි.\n03. මෙම කොරියානු ප්‍රදානයන් බෙදා හැරීම සම්බන්ධව සැපයීම් ශාඛාව වෙත මෙතෙක් ලැබී ඇති උපදෙස් අනුව ජයවර්ධනපුර කලාපයේ පාසල් 56ක් සුදුසුකම් ලබා තිබුණි. \n(කරු. ඇමුණුම I බලන්න) \nමෙම පාසැල් 56 සඳහාම මේ වන විටත් විද්‍යුත් පියානෝ ලබා දීම නිම කර ඇති බව සතුටින් දන්වමි.\n04. තවද ඒ අතරින් (ඇමුණුම් II හි) සදහන් පාසල් 15 සඳහා ද නියම කරන ලද ග්‍රීන් බෝර්ඩ් ප්‍රමාණයන් මේ වන විටත් ලබා දී ඇත. \nඑබැවින් ඉදිරියේදී ග්‍රීන් බෝර්ඩ් ලබා දීමට නියමිත පාසල් 41 (ඇමුණුම් IIIහි ) සඳහා ඒ බව දන්වමින් සකස් කළ ලිපි 41 ඉහත ඉල්ලීම පරිදි ඔබ වෙත මේ සමඟ එවන බව කාරුණිකව සලකන්න.\n05. කෙසේ වුවද අප සැපයීම් ශාඛාව වෙත මෙතෙක් ලැබී ඇති උපදෙස්වලට අනුව සුදුසුකම් ලැබූ පාසල් මෙහිදී සලකා ඇත. \nපියානෝ සහ ග්‍රීන් බෝර්ඩ් ඉතිරි තොග බෙදා දීමේ දී ගුණාත්මක සංවර්ධනය සඳහා වූ වෙනත් නිර්ණායක ද සැළකිල්ලට ගෙන තවත් අමතර පාසල් අනාගතයේ දී නම් කෙරේ නම්, විද්‍යුත් පියානෝ සම්බන්ධ එම අදාළ විෂය අධ්‍යක්ෂ වන, අධ්‍යක්ෂ (සෞන්දර්ය) විසින් ද, ග්‍රීන් බෝර්ඩ් සම්බන්ධයෙන් නම් ඒවා ගුණාත්මක   අධ්‍යාපන සංවර්ධනයට දායක වන පරිදි භාවිතයට ගැනීම සම්බන්ධව කටයුතු කරන විෂය අධ්‍යක්ෂවරයෙකු විසින් ද අදාල පාසල් නම් කලයුතුවනු ඇත. \nඑබැවින් ජයවර්ධනපුර කලාපය සඳහා සැපයීම් ශාඛාව වෙත දැනුම් දෙන ලද පාසල්වලට අමතරව තව පාසල් සඳහා විද්‍යුත් පියානෝ සහ ග්‍රීන් බෝර්ඩ් ලබා දීම සම්බන්ධ තොරතුරු හා පාසල් දැනුවත් කරන ලිපි එම අදාල විෂය අධ්‍යක්ෂවරුන් විමසීමෙන් ලබා ගතයුතු බවද කාරුණිකව සලකන්න. \nබී.ඩී.සී. බියන්විල,\nඅධ්‍යක්ෂ(සැපයීම්).\nපිටපත :- අතිරේක ලේකම්, (අධ්‍යාපන ගුණාත්මක සංවර්ධන) කරු. දැ. ගැ. පි."
    ,
      "වෙත :- ජ්‍යෙෂ්ඨ සහකාර ලේකම්(පාලන)\nවෙතින් :- ගොඩනැගිලි කළමනාකරු\nදිනය :- 2011.11.11\nමගේ අංකය :- ED/5/74/1 (පොදු II)\nවිෂය :- පරිගණක අභ්‍යාසලාභිනියක් ලබා ගැනීම\nගොඩනැගිලි කළමනාකරණ ශාඛාවේ පරිගණක පුහුණුවන්නෙකුගේ සේවය 2011.07.31 දින වන තෙක් ලබා ගත් අතර ඇයගේ සේවය අවසන් වී ඇති බැවින් නැවත පරිගණක අභ්‍යාසලාභිනියක් ලබා දෙන මෙන් කාරුණිකව ඉල්ලමි.\nඒ.එච්.එච්.කේ.වීරකෝන්,\nගොඩනැගිලි කළමනාකරු."
    ,
      "සියලුම විදුහල්පතිවරුන් වෙත,\n2016 අ.පො.ස.(උ.පෙළ) උපකාරක සම්මන්ත්‍රණ මාලාව සඳහා පොදු උපදෙස් මාලාව.\nමෙවර පැවැත්වීමට නියමිත උක්ත සම්මන්ත්‍රණ මාලාව සඳහා පහත දැක්වෙන පොදු උපදෙස් අනුගමනය කරන ලෙස කාරුණිකව දන්වමි.\n1. මෙම සම්මන්ත්‍රණ මාලාව අදාළ කාලසටහනට අනුකුලවම පැවැත්වීමට කටයුතු කළ යුතුය.\n2. සම්මන්ත්‍රණ පැවැත්වීම සඳහා සම්පත්දායකයින් ලෙස සුදුසුකම් සහිත ප්‍රවීණ ගුරුවරු , ගුරු උපදේශකවරු ,කතිකාචාර්යවරු  සහභාගී කරවා ගත යුතුය.\n3. අවශ්‍ය අවස්ථාවලදී ශිෂ්‍ය සංඛ්‍යාව අඩු පාසල් වල සිසුන් ආසන්නයේ පිහිටි උසස් පෙළ සහිත වෙනත් පාසල් වලට  යොමුකර සම්මන්ත්‍රණ සඳහා සහභාගී වීමට කටයුතු කළ හැකි අතර මේ සඳහා කලාප අධ්‍යාපන අධ්‍යක්ෂකගේ   සහයෝගය ලබාගත යුතුය.\n4. ප්‍රශ්න පත්‍ර මුද්‍රණය සඳහා පාසල් සංවර්ධන සමිතියේ මුදල් වැය කිරීම සඳහා අනුමැතිය ලබාදෙන අතර විශාල පිරිවැයක් වැයකර සිදුකරන මෙම සම්මන්ත්‍රණ වලින් උපරිම ප්‍රයෝජනය ලබාගැනීම සඳහා සිසුන් සහභාගී කර  වීමට සියලුම විදුහල්පතිවරුන් අවශ්‍ය පියවර ගත යුතුය.\n5. විවිධ විෂයන් වෙනුවෙන් පැවැත්වූ සම්මන්ත්‍රණ සඳහා සහභාගී වූ ශිෂ්‍ය සංඛ්‍යාව හා සම්පත් දායකයින් පිළිබඳ විස්තර ඇතුළත් අධීක්‍ෂණ වාර්තා ජාතික පාසල් විදුහල්පතිවරුන් විසින් ජාතික පාසල් අධ්‍යක්‍ෂ වෙතටත් ,පළාත් පාසල්වල විදුහල්පතිවරුන් විසින් පළාත් අධ්‍යාපන අධ්‍යක්‍ෂ වෙතටත් යොමු කළ යුතුය.\n  එච්.යු.ප්‍රේමතිලක,\nඅතිරේක ලේකම් ( අධ්‍යාපන ගුණාත්මක සංවර්ධන )\nඅධ්‍යාපන ලේකම් වෙනුවට.\nපිටපත් : පළාත් අධ්‍යාපන අධ්‍යක්‍ෂ.\n\t   කලාප අධ්‍යාපන අධ්‍යක්‍ෂ."
    ,
      "මධ්‍යම පළාත් කාර්යසාධන වාර්තාව සකස් කිරීම - 2011\n2011 වර්ෂය වෙනුවෙන් ඉදිරිපත් කරනු ලබන කාර්ය සාධන වාර්තාව සකස් කිරීම සඳහා ඔබ අමාත්‍යාංශය හා ඒ යටතේ ඇති දෙපාර්තමේන්තුවල/ ආයතනය/ අංශය සම්බන්ධයෙන් වන තොරතුරු අවශ්‍යව ඇත.\n අමාත්‍යාංශය මඟින් තොරතුරු එවීමේදී දෙපාර්තමේන්තු මට්ටමෙන් තොරතුරු ගෙන්වා ගෙන අමාත්‍යාංශය හා දෙපාර්තමේන්තු වෙන් වෙන් වශයෙන් සකස් කොට තොරතුරු සකස් කර එවන මෙන් දන්වමි.\n02. ඒ අනුව, පහත ශීර්ෂ යටතේ සකස් කරන ලද තොරතුරු 2012.03.09 දිනට ප්‍රථම මෘදුකාංග පිටපතක් මඟින් මා වෙත ලැබීමට සලස්වන ලෙස ඉල්ලමි. \nතොරතුරු ad.central.gov@gmail.com විද්‍යුත් තැපැල් ලිපිනය ඔස්සේද එවිය හැකි බව වැඩිදුරටත් සඳහන් කරමි.\nI.ආයතනයේ නම\nII.ආයතනයේ දර්ශනය හා මෙහෙවර\nIII.ආයතනයේ ප්‍රධාන අරමුණු හා කාර්යයන්\nIV.සංවිධාන සටහන\nV.2011 වර්ෂය සඳහා සැලසුම් කරන ලද හා ඉටු කරන ලද කාර්යයන්\nVI.ඔබ ආයතනයට අයත් විෂය පථයට අදාල 2011 ජනවාරි මසයේදී එක් එක් අංශ සඳහා පැවති තත්ත්වය\nVII.2011.12.31 වන විට එම තත්ත්වයන්ගෙන් සිදුවී ඇති ප්‍රගතිය (මෙම ප්‍රගතිය නිමවුම් සහ ප්‍රතිඵල ලෙස බෙදා වෙන් කොට එවීමට හැකි නම් වඩාත් ඵලදායි වේ.)\nVIII.2011 වර්ෂයේ ලබා ගත් විශේෂ ජයග්‍රහණයන්\nIX.2011 වර්ෂයේ සිදුකරන ලද අනෙකුත් විශේෂ කාර්යයන් ඉතාමත් පැහැදිලි ලෙස කෙටියෙන්\n03. ඉදිරිපත් කරනු ලබන තොරතුරු පහත පිරිවිතරයන් ඔස්සේ සකස් විය යුතුය.\nMicrosoft Office 2003 - MS Word වැඩසටහනක් ආකාරයෙන් A4 ප්‍රමාණයේ පිටු වල Portrait ආකාරයට සැකසීම.\nඅකුරු වර්ගය (Font) FM Bindumathi හා අකුරු වල ප්‍රමාණය  (Font Size)10 වීම (මාතෘකා සඳහා අකුරු වල ප්‍රමාණය 11 වීම.)\nපේළි අතර පරතරය (line spacing) 1 වීම.\nපිටු සැකසීමේදී Marging සඳහා Left 1.5'', Right 1'' බැගින්ද Top Mottom 1/2'' ලෙසද ගැනීම.\n04. ඉහත පිරිවිතරයන්ට පරිබාහිරව ලේඛණ සකස් කිරීමෙන් වලකින ලෙසට සඳහන් කරන අතර, අප්‍රේල් මාසයට පෙර මෙම කාර්යය නිමා කළ යුතු බැවින් ඒ සඳහා වැඩි අවධානයක් යොමු කොට ඉල්ලා ඇති තොරතුරු නියමිත දිනට හෝ ඊට පෙර මා වෙත ලැබීමට සලස්වන ලෙස වැඩිදුරටත් ඉල්ලමි. \nගාමිණී රාජරත්න,\nනියෝජ්‍ය ප්‍රධාන ලේකම්(පාලන),\nමධ්‍යම පළාත් ප්‍රධාන ලේකම් වෙනුවට ."
    ,
      "මගේ අංකය : සී.පී.සී/කේ.එම්.සී/නා.ලේ/06/කළ.ක \nනාගරික ලේකම් දෙපාර්තමේන්තුව,\nමහ නගර සභාව,\nමහනුවර.\n2016.06.\nකළමණාකරණ කමිටු රැස්වීම.\nමහනුවර මහ නගර සභාවේ කළමණාකරණ කමිටු රැස්වීම ගරු නගරාධිපතිතුමාගේ අණ පරිදි 2016-06-21 වන අඟහරුවාදා දින ප.ව.3.00 ට මහනුවර නගර ශාලාවේදී පැවැත්වෙන බවත්, එම රැස්වීමට සහභාගි වන ලෙසත් මෙයින් කාරුණිකව දන්වා සිටිමි.\nඒ.එච්.එම්.ඉන්දිකා කුමාරි අබේසිංහ)\nනාගරික ලේකම්.\nන්‍යාය පත්‍රය\n01. 2016 මැයි මස මහා සභාවේදී ගරු නාගරික මන්ත්‍රීවරුන් විසින් වාචිකව හා ලිඛිතව එක් එක් දෙපාර්තමේන්තුවට/ අංශයට අදාළව ඉදිරිපත් කරන ලද පැමිණිලි/පෙත්සම් සම්බන්ධයෙන් ගෙන ඇති ක්‍රියා මාර්ගයන් පිළිබඳව සාකච්ඡා කිරීම.\n02. එක් එක් දෙපාර්තමේන්තු මගින් ඉටුකරනු ලබන වැඩසටහන් පිළිබඳ ප්‍රගතිය සාකච්ඡා කිරීම.\n03.වෙනත් කරුණු."
    ,
      "මගේ අංකය  BDF/D/Hem/m/m/15\nධීවර හා ජලජ සම්පත් දෙපාර්තමේන්තුව.\nදිස්ත්‍රික් ධීවර කාර්යාලය ,\nමඩකලපුව.\n2016.06.30 \nඅධ්‍යක්‍ෂ ජනරාල් ,\nධීවර හා ජලජ සම්පත් දෙපාර්තමේන්තුව.\n1996 අංක 02 දරණ ධීවර හා ජලජ සම්පත් පනත යටතේ බලාත්මක වන අංක 859/3 සහ 1995.02.20 දරණ අති විශේෂ ගැසට් පත්‍රය මගින් සංශෝධිත අංක 457/46 සහ 1987.01.19 වෙනි දින අති විශේෂ ගැසට් පත්‍රයේ ප්‍රකාශිත 1986 හැඹිලි(පර්ස්සීන්) දැල් පිළිබඳ නියෝග මාලාවට අනුව මසුන් ඇල්ලීම සඳහා අවසර පත්‍ර ලබා දීම.\nඋක්ත කරුණට අදාලව ඔබ විසින් ම වෙත එවන ලද ඔබගේ සමාංක DFAR/FM/AD19/577 හා 2016.06.29 වෙනි දිනැති , ෆැක්ස් ලිපියේ කොන්දේසි වලට අනුකූලව , 2016 වර්ෂයට මෙම දිස්ත්‍රික් ධීවර කාර්යාලය මගින් කිසිම ධීවරයෙකුට, හැඹිලිදැල්(පර්සීන්) මගින් මසුන් ඇල්ලීම සඳහා , අවසර දී නොමැති බව, ඔබගේ ඉදිරි කටයුතු සඳහා කාරුණිකව දන්වා සිටිමි.\nආර්.සී.කෲස් ,\nසහකාර අධ්‍යක්‍ෂ ,\nමඩකලපුව."
    ,
      "ප්‍රදේශීය ලේකම්,\nප්‍රදේශීය ලේකම් කාර්යාලය ,\nගලේවෙල .\nදිස්ත්‍රික් සම්බධීකරණ කමිටු තීරණ දැනුම් දීම. \nපොලිස් මුරපොලක් ඉදි කිරීම සඳහා ඉඩමක් ලබා ගැනීම.\n2016.03.04 දින මාතලේ දිස්ත්‍රික් ලේකම් කාර්යාලයේ පැවති දිස්ත්‍රික් සම්බන්ධීකරණ කමිටු රැස්වීමේදී සාකච්ඡා කල කරුණු හා බැඳේ .\n02. ඔබ කොට්ඨාසයේ මකුළුගස් වැව ප්‍රදේශයේ පොලිස් මුරපොලක් ඇති බවත් , එය බඳු ක්‍රමය යටතේ පෞද්ගලික ස්ථානයක පිහිටුවා ඇති බැවින්, එය ස්ථිරව ඉදිකිරිම සඳහා සෞඛ්‍ය දෙපාර්තමේන්තුව ඉඩමක් හඳුනා ගෙන ඇති බවට යෝජනා කරන ලදී.\n එසේ නමුත් එම ඉඩම සෞඛ්‍ය දෙපාර්තමේන්තුව විසින් පරිහරණය කරන ලද ඉඩමක්ද යන්න පිළිබඳවද සොයා බලා පොලිස් මුරපොල ඉදිකිරිම සඳහා එම හඳුනා ගෙන ඇති ඉඩම ලබා දීමට ඇති හැකියාව පිළිබඳව පරීක්ෂා කරබලා මේ සමග එවන ලද ආකෘති පත්‍රයට අනුව තොරතුරු සම්පුර්ණ කර 2016.06.15 දිනට පෙර මා වෙත ලැබීමට සලස්වන මෙන් කාරුණිකව දන්වා සිටිමි. \nඅයි.ඩි.පි.විජේතිලක, \nඅතිරේක දිස්ත්‍රික් ලේකම් (ඉඩම්).\nඅත්/කළේ නීල් ද අල්විස්, \nදිස්ත්‍රික් ලේකම්/දිසාපති,\nමාතලේ .\nපිටපත - 01. පොලිස් ස්ථානාධිපති , ගලේවෙල  - කාරුණික දැන ගැනීම සඳහා"
    ,
      "සියළුම දිස්ත්‍රික් දිවිනැගුම අධ්‍යක්ෂවරුන් වෙත,\nවැටුප් සාරාංශ වාර්තා -2016\nඔබ දිස්ත්‍රික්කය  විසින් නිලධාරී වැටුප් ගෙවීම සඳහා සකස් කළ වැටුප් සාරංශ වාර්තාවේ පිටපතක් සෑම මාසයකම 19 වෙනි දිනට හෝ එදිනට පෙර ලබා දීමට කටයුතු කළ යුතු බව කාරුණිකව දන්වා සිටිමි.\nඑම වැටුප් සාරාංශ වාර්තාව සමඟ වැය විෂය 331-02-02-1001 යටතේ අදාළ වන හරස් සටහන් විශ්ලේෂණයක් ද භාණ්ඩාගාරය වෙත ඉදිරිපත් කිරීම අනිවාර්ය කර ඇත.\nපහත ආකෘතිය යටතේ 1001 වැය විෂය හරස් සටහනේ විශ්ලේෂණ වාර්තාවක් ද  ඉදිරිපත් කල යුතුය.\nවැන්දඹු හා අනත්දරු අරමුදල්  දායකය. \nරජයේ නිලධාරීන්ගේ අත්තිකාරම් බී ගිණුමේ අවකරණ.  \nරජයේ නිවාස කුලී  .\nවෙනත් (විස්තර කළ යුතුය) . \nඑකතුව .\n විශේෂයෙන් මෙම වාර්තා නියමිත දිනට  හෝ ඊට පෙර ලබා දීමට කටයුතු කළ යුතු අතර එසේ නියමිත දිනට අදාළ වාර්තා ලබා දීමට අපොහොසත් වන දිස්තික්ක වෙත භාණ්ඩාගාරයෙන් අක් මුදල් ලබා දීම ප්‍රමාද වීම හේතු කොට ගෙන වැටුප් සඳහා අක් මුදල් නියමිත දිනට  නිදහස් කිරීමට නොහැකි වනු ඇත.\nසුමිත් ජේ. කුමාර,\nගණකාධිකාරී(වැටුප් හා ගෙවීම්),\nප්‍රධාන ගණකාධිකාරී වෙනුවට."
    ,
      "සියළුම දිස්ත්‍රික් දිවිනැගුම අධ්‍යක්ෂවරුන් වෙත,\nඋත්සව අත්තිකාරම් ගෙවීම - 2016\n2016 වර්ෂය සඳහා උත්සව අත්තිකාරම් ගෙවීමේදී කටයුතු කළ යුතු ආකාරය පිළිබඳව 07/2015 දිනැති රාජ්‍ය පරිපාලන චක්‍රලේඛයට යොමු වේ.\n02. අදාළ විධිවිධාන ප්‍රකාර උත්සව අත්තිකාරම් ගෙවීම සඳහා දිස්ත්‍රික් දිවිනැගුම කාර්යාල කටයුතු කළ යුතු අතර, මේ සඳහා 2016 වර්ෂය තුළ ඔබ වෙත රජයේ නිලධාරීන්ගේ  අත්තිකාරම් බී ගිණුම් කටයුතු සඳහා ලබා දී ඇති ප්‍රතිපාදන යොදා ගත හැකිය. \nඅවශ්‍ය අතිරේක ප්‍රතිපාදන හා අක් මුදල් දිස්ත්‍රික්  දිවිනැගුම කාර්යාලය වෙත නිදහස් කිරීමට කටයුතු කරනු ලබන අතර, ලබා දී ඇති ප්‍රතිපාදන සීමාවන් තුළ ණය ගෙවීම් සිදු කළ යුතුය.\n03. මෙම ණය ගෙවීම් පිළිබඳ ගෙවීම් කටයුතු නිවැරදිව දිස්ත්‍රික් දිවිනැගුම කාර්යාලය විසින් සිදු කළ යුතු බැවින් මෙම කටයුතු දිස්ත්‍රික් ගණකාධීකාරීවරයගේ පූර්ණ අධීක්ෂණය යටතේ සිදු කළ යුතුය.\nපාලන ගිණුම්, කේවල ශේෂ ලැයිස්තු නඩත්තු කළ යුතුවේ.\n මාසිකව හරස් සටහන් මගින් අයකිරීම් ගිණුම් ගත කර, ගිණුම් සාරාංශ මගින් එම තොරතුරු ප්‍රධාන කාර්යාලයට වාර්තා කළ යුතුය.\nතවද, ත්‍රෛමාසික සැසඳුම් පිළියෙළ කිරීමේදී හා වර්ෂ අවසාන රජයේ නිලධාරීන්ගේ අත්තිකාරම් ගිණුම පිළියෙල කිරීමේදී අවශ්‍ය තොරතුරු ඔබ විසින් නිවැරදිව හා ක්‍රමානුකූලව ප්‍රධාන කාර්යාලය වෙත ලබා දිය යුතුය.\n04. මේ සම්බන්ධයෙන් ඔබගේ කඩිනම් අවධානය යොමු කර 2016 වර්ෂයේ උත්සව අත්තිකාරම් ගෙවීම සඳහා අවශ්‍ය කටයුතු සැලසුම් කර ගන්නා ලෙස කාරුණිකව දන්වමි.\nසුමිත් ජේ. කුමාර,\nගණකාධීකාරී (වැටුප් හා ගෙවීම්),\nඅධ්‍යක්ෂ ජනරාල් වෙනුවට."
    ,
      "රාජ්‍ය ආයතන සඳහා දේශීය ආහාර සැපයීමේ කණ්ඩායමට බඳවා ගැනීමේ සම්මුඛ පරීක්ෂණය. \nමාතලේ දිස්ත්‍රික්කය තුළ ක්‍රියාත්මක කිරීමට නියමිත දේශීය ආහාර නිෂ්පාදකයින් කණ්ඩායමක් හඳුනා ගැනීමේ වැඩසටහනට සහභාගී වීමට ඔබ විසින් එවන ලද අයදුම්පත් අප වෙත ලැබී ඇත.\nඅයදුම්පත් එවීමෙන් ඔබ දැක්වූ සහයෝගයට ස්තූතිවන්ත වෙමි.\n02. ඒ අනුව සුදුසු අයදුම්කරුවන් තෝරා ගැනීම සඳහා වන සම්මුඛ පරීක්ෂණය 2016 ................ වන දින පෙ.ව. 9.30 ට දිස්ත්‍රික් ලේකම් කාර්යාලයේ කෘෂිකර්ම අංශයේ දී පැවැත්වීමට නියමිතය.\nඑබැවින් එදින නියමිත වෙලාවට පෙර පැමිණ සම්මුඛ පරික්ෂණයට සහභාගී වන මෙන් කාරුණිකව දන්වා සිටිමි.\n03. සම්මුඛ පරීක්ෂණය සඳහා රැගෙන ආ යුතු ලියකියවිලි පහතින් දක්වා ඇත.\n1. ඔබගේ ජාතික හැදුනුම්පත. \n2.ඔබ වසමට ආදාළ ග්‍රාම නිලධාරී චරිත සහතියකයක්. \n3. දේශීය ආහාර සකස් කිරීම පිළිබඳව ඔබ මීට පෙර පුහුණුවීමක් ලබා ඇත්නම් ඊට අදාල  සහතිකපත් හා ඒ පිළිබඳ විස්තර. \n4. සම්මුඛ පරීක්ෂණය සඳහා වැදගත් වේ යැයි ඔබ සිතන අනෙකුත් ලියවිලි .\nඊ.එම්.ඩි.එස්.එන්.ඒකනායක ,\nදිස්ත්‍රික් කෘෂිකර්ම අධ්‍යක්‍ෂ ,\nමාතලේ දිස්ත්‍රික් ලේකම් වෙනුවට."
    ,
      "මගේ අංකය : සීපීසී/කේඑම්සී/නා.ලේ/06/ග.ග.උ.ක \nමහ නගර සභාව,\nමහනුවර.\n2016.06.\nගරු මන්ත්‍රීතුමනි/මහත්මයාණෙනි,\nමහනුවර මහා නගර සභාවේ ගමනා ගමන උපදේශක කමිටු රැස්වීම.\nමහනුවර මහ නගර සභාවේ ගමනාගමන උපදේශක කමිටුවේ රැස්වීමක් 2016-06-07 වන දින ප.ව. 3.00 ට මහනුවර නගර ශාලාවේදී පැවැත්වේ.\nඒ සඳහා අනිවාර්යයෙන් සහභාගී වන ලෙස ගරු නගරාධිපතිතුමාගේ නියමය පරිදි මෙයින් කාරුණිකව දන්වා සිටිමි.\nඑම රැස්වීමේ න්‍යාය පත්‍රය පහත සඳහන් වේ.\nමෙයට - නියමය පරිදි,\n(ඒ.එච්.එම්.ඉන්දිකා කුමාරි අබේසිංහ),\nනාගරික ලේකම්."
    ,
      "පීඨාධිපති/කළමණාකරු /විදුහල්පති මගින්,\nඑම්.සිවකුමාර් මයා/මිය/මෙනෙවිය,\nමුරකරු ,\nමඩකලපුව ජාතික අධ්‍යාපන විද්‍යා පීඨය ,\nමඩකලපුව.\nප්‍රාථමික ශිල්පීය නොවන 1 ශ්‍රේණියේ පුස්තකාල සේවක තනතුරට පත් කිරීම.\nප්‍රාථමික ශිල්පීය නොවන සේවක ගණය සඳහා වන බඳවා ගැනීමේ පටිපාටියේ 11 වන වගන්තිය අනුව ඔබ මඩකලපුව ජාතික අධ්‍යාපන විද්‍යා පීඨයේ 1 ශ්‍රේණියේ පුස්තකාල සේවක තනතුරට පත් කරනු ලැබේ.\n02. ඒ අනුව ඔබට රාජ්‍ය පරිපාලන චක්‍රලේඛ අංක 06/2006 (IV)හි  රු. 11,730-10X120-10X130-10X145-12X160-17,600/-(PL1-2006ඒ ) වැටුප් පරිමාණය හිමි වේ. \nමෙම ශ්‍රේණියට අදාල වැටුප්  පියවර රු.14375.00 වේ.\n 03. මෙම තනතුර ස්ථිරය. \nවිශ්‍රාම වැටුප් සහිතය. \nඔබගේ පත්වීම කලින් අවසන් කරනු නොලැබුවහොත්, ඔබගේ පත්වීමෙහි ස්ථිර කරන ලද ලිපියක් ඔබට ලැබෙන තෙක් ,ඔබගේ පත්වීමේ දින සිට අවුරුද්දක කාලයක්  හෝ එම කාලය දීර්ඝ කිරීම අවස්ථානුරූපයයි සලකනු ලැබුවහොත් එසේ දීර්ඝ කළ කාලයක් හෝ ඔබ වැඩ බැලීමේ පදනම මත සේවය කළ යුතුය.\n  04. පූර්ව පත්වීම් ලිපියෙහි ඇතුළත් අනෙකුත් කොන්දේසි වර්තමාන බඳවා ගැනීමේ පටිපාටියේ විධිවිධාන වලට  යටත්ව තවදුරටත් එසේම වලංගු වනු ඇත.\n  05. මෙම ලිපිය ලද බවත් , මෙම පත්වීම භාරගැනීමට ඔබගේ කැමැත්ත /අකමැත්ත විදුහල්පති/ පීඨාධිපති/ කළමණාකරු මගින් මා වෙත ලියා දන්වන ලෙසත් දන්වමි.\n  ජි.එල්.එස්.සේනාධීර ,\n අතිරේක ලේකම් (පාලන ) ,\n අධ්‍යාපන ලේකම් වෙනුවට .\n  පිටපත් \nවිගණකාධිපති \nප්‍රධාන ගණකාධිකාරී \nඅධ්‍යක්‍ෂ ජනරාල් -විශ්‍රාම වැටුප් දෙපාර්තමේන්තුව \nප්‍රධාන කොමසාරිස් - ගුරු අධ්‍යාපන\nපීඨාධිපති/කළමනාකරු/විදුහල්පති - මඩකලපුව ජාතික අධ්‍යාපන විද්‍යා පීඨය\nපෞද්ගලික ලිපි ගොනුව"
    ,
      "මගේ අංකය - CPC/CS/3/1/3/5/1 railway\nදිනය - 2016.05.03\nප්‍රධාන ගණකාධිකාරි,\nශ්‍රී ලංකා දුම්රිය සේවය,\nතැ.පෙ. 355,\nකොළඹ 10.\nදුම්රිය බලපත්‍ර බිල්පත් සඳහා මුදල් ගෙවීම.\nඋක්ත කරුණ සම්බන්ධයෙන් මෙම කාර්යාලයට යොමු කරන ලද ඉන්වොයිස් අංක ...........................යට අදාළව ..........................මස සඳහා වූ රු. ......................... ක මුදල වෙනුවෙන් අංක ........................ හා ......................... දිනැති චෙක්පත ආකෘති පත්‍රය ද මෙයට අමුණා කාරුණිකව ඉදිරිපත් කරමි.\nපහත සඳහන් ලිපිනයට කරුණාකර ලදුපතක් එවීමට කටයුතු කරන්න.\nනියෝජ්‍ය ප්‍රධාන ලේකම්(සැලසුම්),\nප්‍රධාන ලේකම් කාර්යාලය,\nතැ.පෙ. 102,\nමධ්‍යම පළාත් සභා සංකීර්ණය,\nපල්ලෙකැලේ.\nප්‍රධාන ලේකම් වෙනුවට."
    ,
      "වෙත :- ගබඩා කළමනාකරු , ඩබ්. ඒ. කේ. ආරියරත්න මහතා\nමඟින් :- ප්‍රධාන ගබඩා භාරකරු, පත්තලගෙදර ගබඩා සංකීර්ණය\nවෙතින් :- අධ්‍යක්ෂ (සැපයීම්)\nදිනය :- 2011.03.30\nමගේ අංකය :- ED/07/97/02/32(1)\nගරු අධ්‍යාපන අමාත්‍යතුමාගේ විශේෂ උපදෙස් පරිදි කොරියානු රජය විසින් පරිත්‍යාග කරන ලද Young Chang වර්ගයේ ආධාරක සහිත විද්‍යුත් පියානෝ පාසල් වෙත ලබාදෙන ලෙස දන්වා ඇත. \nඅමුණා ඇති ලේඛනයේ සඳහන් පාසල් 16 වෙත ආධාරක සහිත විද්‍යුත් පියානෝ විධිමත් පරිදි නිකුත් කිරීමට කටයුතු කරන මෙන් කාරුණිකව දන්වමි.\nලේඛණයේ සඳහන් සෑම පාසලක් සඳහාම පියානෝ නිකුත් කිරීමේ දී පොදු 141 ආකෘති පත්‍රයන්හි, Young Chang වර්ගයේ විද්‍යුත් පියානෝ යතුරු පුවරු (අනුක්‍රමික අංකය ...) 01 යි, Young Chang වර්ගයේ විද්‍යුත් පියානෝ ආධාරක 01යි, (අත්පොත, සී.ඩී. තැටිය සහ කුඩා මයික්‍රොපෝනය සහිතව) යනුවෙන් සඳහන් කළ යුතු බව ද කාරුණිකව දන්වමි.\nමෙම භාණ්ඩ විදුහල්පතිවරුන් වෙත භාරදීමට මේ සමඟ එවා ඇති ලිපි සහ අදාළ ලියකියවිලි සමඟ 41-6652 අංක දරන වාහනයේ රියදුරු ආනන්ද ජයකොඩි මහතා මඟින් අදාළ විදුහල්පතිවරුන් වෙත භාර දීමට කටයුතු කරන මෙන් ද වැඩි දුරටත් දන්වමි.\nබී. ඩී. සී. බියන්විල,\nඅධ්‍යක්ෂ (සැපයීම්)\nපිටපත් \n1. අතිරේක ලේකම් (ප්‍රසම්පාදන හා ඉදිකිරීම්) දැ. ගැ. පි.\n2. අතිරේක ලේකම් (අධ්‍යාපන ගුණාත්මක සංවර්ධන)  දැ. ගැ. පි.\n3. අධ්‍යාපන අධ්‍යක්ෂ(සෞන්දර්ය) පියානෝ පරිහරණය සම්බන්ධව පාසල්වෙත විෂයානුබද්ධ හා තාක්ෂණික උපදෙස් ලබාදීම පිණිස."
    ,
      "වෙත :- අතිරේක ලේකම්(පාසල් සැපයීම්,සේවා හා ඉදිකිරීම්) මගින්.\n           අතිරේක ලේකම්(සැලසුම් හා කාර්ය සාධන සමාලෝචන) මගින්.\n           සහකාර අධ්‍යාපන අධ්‍යක්ෂ(දත්ත කළමනාකරණ ශාඛාව) \n           (එම්.ටී.එම්.ආකිල් මහතාගේ අවධානයට)\nවෙතින් :- ගොඩනැගිලි කළමනාකරු\nදිනය :- 2011.10.07\nමගේ අංකය :- ED/7/74/3/1/ii\nවිෂය :- රාජ්‍ය තොරතුරු කේන්ද්‍රයේ(1919) තොරතුරු යාවත්කාලීන කිරීම.\nඋක්ත කරුණ සම්බන්ධයෙන් අතිරේක ලේකම් (පාසල් සැපයීම්,සේවා  හා ඉදිකිරීම්) ඇමතූ අතිරේක ලේකම් (සැලසුම් හා කාර්ය සාධන සමාලෝචන)ගේ අංක ED/03/38/01/07/46 හා 2011.09.28 දිනැති ලිපිය හා බැඳේ.\n ඒ අනුව, රාජ්‍ය තොරතුරු කේන්ද්‍රයේ අප ශාඛාවට අදාළ තොරතුරුවල වෙනසක් සිදු වී නොමැති බව කාරුණිකව දන්වා සිටිමි.\nඒ.එච්.එච්.කේ.වීරකෝන්,\nගොඩනැගිලි කළමනාකරු."
    ,
      "වෙත :- ප්‍රධාන ගබඩාභාරකරු (පත්තලගෙදර ගබඩා සංකීර්ණය),\nවෙතින් :- අධ්‍යාපන අධ්‍යක්ෂ (පාසල් සැපයීම්).\nදිනය :- 2011-04-18\nමගේ අංකය :- ඊඩී/07/97/02/24\nවිෂය :- පාසලේ නම වෙනස් ආකාරයෙන් සඳහන් වීම නිසා පියානෝ ආපසු භාරදීම.\nබෙදා හැරීමේ ලැයිස්තුවල පාසලේ නම වෙනස් ආකරයෙන් සඳහන් වීම නිසා පහත සඳහන් පාසල  වෙත පියානෝ බෙදා නොදෙන ලදී.\nපාසලේ නම හෝ/උඩුබුද්දාව ම.වි.\nනිකුත් කිරීම් නියෝග අංකය 298647(\nලැයිස්තු අංකය 77)\nදිනය 2011/03/22\n එබැවින් බෙදා නොදුන් ඉහත සඳහන් ආධාරක සහිත පියානෝ ගබඩාව වෙත නැවත භාර ගන්නා ලෙස මෙයින් දන්වමි.\n තවද ඊට අදාළ නිකුත් කිරීමේ නියෝග අවලංගු කර නැවත භාර ගන්නා ලෙස කාරුණිකව ඉල්ලා සිටිමි.\nබී.ඩී.සී.බියන්විල,\nඅධ්‍යක්ෂ(පාසල් සැපයීම්)."
    ,
      "මගේ අංකය :- බී.ඩී.එස්/B/H.P/BT/PW,\nධීවර හා ජලජ සම්පත් දෙපාර්තමේන්තුවේ ,\nදිස්ත්‍රික් ධීවර කාර්යාලය ,\nමඩකලපුව.\nඅධ්‍යක්‍ෂක ජනරාල්,\nධීවර හා ජලජ සම්පත් දෙපාර්තමේන්තුව.\nඕෂන්ස් ටුවෙල් ටුවරීස් හොටෙල් -පාසිකුඩා -කල්කුඩා - මෙම හෝටලය ගොඩනගන ලද ස්ථානය පිළිබඳව ක්‍ෂේත්‍රයේ ගොස් පරික්ෂාකර බලා ලබා ගන්නා ලද තොරතුරු වාර්තාව.\nඋක්ත කරුණ සම්බන්ධව ඔබ විසින් ම වෙත එවන ලද ෆැක්ස් ලිපිය සම්බන්ධවයි.\nමෙම ලිපියට අනුව  උක්ත හෝටල් ගොඩනැගිල්ල තනන ලද ස්ථානය සම්බන්ධව පරික්ෂාකර බලා වාර්තාවක් ලබාදෙන ලෙස අදාල ධීවර පරීක්ෂක මහතාට ම විසින් දන්වා සිටි අතර, ඒ සඳහා එම මහතා 2016.06.05 වෙනි දින එතැනට ගොස් ලබා ගන්නා ලද පරීක්ෂණ වාර්තාව පහත සඳහන් කරමි.\n01.   අප ධීවර හා ජලජ සම්පත් අමාත්‍යාංශයට අයත් ඉඩමට යාබදවම මෙම හෝටලය තැනීම සිදුකර ඇති අතර ,1909 වර්ෂයේ  ධීවර අමාත්‍යාංශය මගින් ගොඩනගන ලද පරණ 'ජෙටි ' යක්ද එතන ඇත.\n02. මෙම ප්‍රදේශයේ මුහුදු වෙරළ ආශ්‍රිිත ධීවරයින් ගේ ධීවර යාත්‍රා නතර කර ඇත.\n03.ධීවරන්ගේ ඉල්ලීම් වලට අනුව අලුතෙන් 'ජෙටියක්' තැනීමට ධීවර හා ජලජ සම්පත් අමාත්‍යාංශය මඟින් දැනටමත් ඉදිරි කටයුතු කරගෙන යන ලදී.\n04.මුහුදු වෙරළේ සිට ගොඩට මීටර 65 ක් ධීවරයින්ට අවහිර නොවන පරිදි, ගොඩනැගිලි තැනීම් නොකල යුතුය.(මෙම කෝස්ටල් ඒරියා නිසා)\n05.  මෙම හෝටල් ගොඩනැගිල්ල සඳහා වූ ඉඩමේ හා ගොඩනැගිල්ලේ සැලැස්මවල් හරියාකාරව ඉදිරිපත් කර නොමැත.\nධීවර පරීක්ෂක මහතා විසින් ඉහත සඳහන් කර ඇති කරණා වලට අනුව බලන විට,මීට කලින් 'ශෙස්පෙරස් ' හෝටලය අසල නතර කර තබන ලද , ධීවර යාත්‍රා , එම හෝටලය ගොඩනගන ලද අවස්ථාවේදී එතැන සිට ඉහත අංක 2 හි සඳහන් කර ඇති ස්ථානයට ගෙන යන ලදී. \nමෙතැන   සිට වෙනත් ස්ථානයකට ධීවර යාත්‍රා තබන ලෙස කිව්වහොත් ධීවරයින් ගේ  විරෝධය පල කරන බවද දන්වා සිටිමි."
    ,
      "නාගරික ලේකම් දෙපාර්තමේන්තුව,\nමහා නගර සභාව,\nමහනුවර.\n2016-02-01.\nගරු මහා නගර සභා මන්ත්‍රී,\nමහනුවර.\nමහත්මයාණෙනි,\nටෙන්ඩර් මණ්ඩල රැස්වීම.\nමහනුවර මහා නගර සභාවේ ටෙන්ඩර් මණ්ඩල රැස්වීමක් 2016-02-08 වන සඳුදා දින පෙ.ව.10.00 ට මහනුවර මහා නගර සභා ශාලාවේදී පැවැත්වෙන බව ගරු නගරාධිපතිතුමාගේ අණ පරිදි කාරුණිකව දන්වමි.\nඑම රැස්වීමේ න්‍යාය පත්‍රය පහත දැක්වේ.\nමෙයට - විශ්වාසී,\nඒ.එච්.එම්.ඉන්දිකා කුමාරි අබේසිංහ,\nනාගරික ලේකම්,\nමහනුවර මහා නගර සභාව.\nන්‍යාය පත්‍රය\n01. 2016-02-08 දිනට ටෙන්ඩර් කැඳවූ අයිතම\n01.මුදල් දෙපාර්තමේන්තුවට ඩබල් කැබ් රථයක් මිළදී ගැනීම.\n02.ජල භීතිකා එන්නත් සපයා ගැනීම.\n03.ජලනල උපාංග උපකරණ සපයා ගැනීම.\n04.සාමාන්‍ය ගබඩා ද්‍රව්‍ය සපයා ගැනීම.\n05.කාර්යාල උපකරණ සපයා ගැනීම.\n06.පරිගණක හා ඒ ආශ්‍රිත උපාංග සපයා ගැනීම.\n07.තීන්ත සපයා ගැනීම.\n08.සෞඛ්‍ය ආශ්‍රිත ද්‍රව්‍ය සපයා ගැනීම.\n09.ලිපිද්‍රව්‍ය සපයා ගැනීම.\n10.ආරක්ෂිත බඳපටි සපයා ගැනීම.\n11.විදුලි උපකරණ සපයා ගැනීම.\n12.සපත්තු හා මේස් සපයා ගැනීම."
    ,
      "මගේ අංකය - CPC/CS/03/01/91\nදිනය - 2016.05.30\nකළමණාකරු,\nS&M සේවා මධ්‍යස්ථානය,\nඅංක 217/1,\nමාතලේ පාර,\nකටුගස්තොට.\nඅංක PA 4899 දරන වාහනය සේවා කිරීම.\nඉහත අංක දරන කැබ් රථයේ සේවා කිරීමේ කාලය එළඹ ඇති බැවින් පහත සඳහන් සේවාවන් කර දෙන මෙන් කාරුණිකව දන්වමි.\n1)එන්ජින් ඔයිල් මාරු කිරීම.\n2)ඔයිල් ෆිල්ටරය මාරු කිරීම.\nබිල්පත පියවීම සඳහා අමුණා ඇති වවුචරය අත්සන් කර එවීමට කටයුතු කරන ලෙස වැඩිදුරටත් දන්වමි.\nකේ.ජී.එස්.නිශ්ශංක,\nනියෝජ්‍ය ප්‍රධාන ලේකම්(සැලසුම්)"
    ,
      "එන්.නවලෝජන් මයා/මිය/මෙය,\nශ්‍රී මුරුගන් කෝවිල් පාර,\nකුරුක්කල් මඩම/උ.\nජා.හැ.අං. 940131200V\nසේවා ස්ථාන සංශෝධනය කිරීම. \nමගේ අංක ED/05/67/11/21 -1 හා 2016.04.05 දිනැති පත්වීම ලිපිය හා බැඳේ.\n02.එම ලිපියෙන් ඔබ වෙත ලබා දුන් සේවා ස්ථානය මඩ/ශිවානන්ද ජාතික පාසල ලෙස සංශෝධනය කරනු ලබන බව මින් දන්වමි.\n03. ඉහත ලිපියේ අනෙකුත් කොන්දේසි එලෙසම පවතින බව වැඩිදුරටත් දන්වමි.\nඑච්.ඒ.එස්.නිශාන්ත පෙරේරා,\nපරිපාලන නිලධාරී (ආයතන) ,\nඅධ්‍යාපන ලේකම් වෙනුවට.\nපිටපත් -පළාත් අධ්‍යාපන අධ්‍යක්‍ෂ - නැගෙනහිර.\n\tකලාප අධ්‍යාපන අධ්‍යක්‍ෂ - සමන්තුරේ/මඩකලපුව.\n\tකළමනාකරු  - සමන්තුරේ ගුරු මධ්‍යස්ථානය.\n\tවිදුහල්පති - මඩ/ශිවානන්ද ජාතික පාසල. \n\tපෞද්ගලික ලිපි ගොනුව.     \n\tකාර්ය මණ්ඩල ගොනුව."
    ,
      "ත්‍රිකුණාමලය\n2016.04.28\nජ්‍යෙෂ්ඨ මිනින්දෝරු අධිකාරී,\nත්‍රිකුණාමලය / මඩකලපුව.\nජීප් බවුසර් පිලිබඳ තොරතුරු ලබා ගැනීම.\nඋක්ත මැයෙන් අතිරේක සර්වේයර් ජනරාල්ගේ අංක ප්‍රවා 1/ගර්හිත වාහන (බවුසර්) හා 2016.04.22 දිනට මා අමතන ලද මෙයට අමුණා ඇති ලිපිය හා සම්බන්ධවයි.\n02. එහි සඳහන් පරිදි අමුණා ඇති ලේඛණයේ සඳහන් ජිප්බවුසර් හා ටේලර් පිලිබඳ විස්තර සොයා බලා ඒ පිලිබඳ වාර්තාව 2016.04.29 දිනට ප්‍රථම මටද පිටපතක් සහිතව අතිරේක සර්වේයර් ජනරාල් වෙත යැවීමට නොපමාව කටයුතු කරන මෙන් කාරුණිකව දන්වමි.\nඅදාළ අංක සහිත ලිපියේ හා එම ඇමුණුමේ පිටපතක් මේ සමග අමුණා එවමි.\nපළාත් සර්වේයර් ජනරාල්(නැගෙනහිර),\nත්‍රිකුණාමලය"
    ,
      "මාගේ අංකය : ප්‍රවා 1/ගර්හිත වාහන (බවුසර්)\nප්‍රවාහන අංශය.\n2015.05.\nපළාත් සර්වේයර් ජනරාල් (නැගෙනහිර),\nජීප් බවුසර් පිලිබඳ තොරතුරු ලබා ගැනීම.\n2008 වර්ෂයේ පැවති වාර්තා වලට අනුව සකස් කරන ලද ජීප් බවුසර් සහ ට්‍රේලර් පිලිබඳ ලේඛනයක් මේ සමග අමුණා එවන අතර මෙම ලේඛණයේ සඳහන් කුමන හෝ වාහනයක් දැනට ඔබ කාර්යාලය සතුව පවතී නම් ඒ සම්බන්ධව සම්පූර්ණ වාර්තාවක් හෝ නොඑසේ නම් එම වාහන ගර්හිත කිරීම සම්බන්ධව කුමන හෝ තොරතුරක් පවතී නම් ඒ සම්බන්ධ සම්පූර්ණ වාර්තාවක් 2016.04.29 දිනට පෙර 011-2587925 දරණ අංකයට ෆැක්ස් පණිවිඩයක් මඟින් එවන ලෙස කාරුණිකව දන්වමි.\n පී. පී. සංගක්කාර\nසර්වේයර් ජනරාල්."
    ,
      "මගේ අංකය - CPC/CS/03/01/91\nදිනය - 2016.05.30\n265/70/R/16 NANKAN වර්ගයේ ටයර් 4ක් මිලට ගැනීම.\nඅප ආයතනයේ දැනට භාවිතා කරන ලද කැබ් රථයක් සඳහා ඉහත සඳහන් කර ඇති වර්ගයේ ටයර් 4ක් මිලට ගැනීමට අවශ්‍ය වී ඇත.\nඑබැවින් ඔබගේ මිල ගණන් 2016.06.03ට පෙර පහත සඳහන් ලිපිනයට අප වෙත දන්වා එවන මෙන් කාරුණිකව දන්වා එවමි.\n(නියෝජ්‍ය ප්‍රධාන ලේකම්(සැලසුම්), ප්‍රධාන ලේකම් කාර්යාලය, තැ.පෙ. 102, මධ්‍යම පළාත්සභා සංකීර්ණය, පල්ලෙකැලේ.)\nකේ.ජී.එස්.නිශ්ශංක,\nනියෝජ්‍ය ප්‍රධාන ලේකම්(සැලසුම්)."
    ,
      "සියලුම පළාත් අධ්‍යාපන ලේකම්වරු ,\nසියලුම පළාත් අධ්‍යාපන අධ්‍යක්‍ෂවරු ,\nසියලුම කලාප අධ්‍යාපන අධ්‍යක්‍ෂවරු ,\nසියලුම විදුහල්පතිවරු ,\nඅධ්‍යාපන ග්‍රන්ථ ප්‍රකාශන උපදේශක මණ්ඩලය විසින් පාසල් පුස්තකාල භාවිතය සඳහා සුදුසු යයි අනුමත කරන ලද පොත් නාම ලේඛනය  -2015 \n1952 පෙබරවාරි 29 වන දින ලංකාණ්ඩුවේ ගැසට් පත්‍රයෙහි පල කරන ලද ස්වභාෂා, ද්විභාෂා සහ ඉංග්‍රීසි පාඨශාලාවන් පිළිබඳ ව්‍යවස්ථා සංග්‍රහයේ 19/A ඡේදයට අනුකූලව පාසල් පුස්තකාල භාවිතය සඳහා සුදුසු යයි අධ්‍යාපන ග්‍රන්ථ ප්‍රකාශන උපදේශක මණ්ඩලය විසින් අනුමත කරන ලද පොත් නාම ලේඛනය මේ සමග එවමි. \nපාසල් පුස්තකාල සඳහා පොත් මිලදී ගැනීමේ දී 01 සිට 205 දක්වා අංක ගත කර ඇති මෙම පොත් වෙත ප්‍රමුඛත්වය දී කටයුතු කරන ලෙසට මෙයින් දන්වා සිටිමි.\nගුණාත්මක පොත් පමණක් පාසල් පුස්තකාලයේ භාවිතය සඳහා යොදා ගැනීමට උනන්දු කරවීම මෙහි මුඛ්‍ය අරමුණ බවද සැලකුව මැනවි.\nඩබ්ලිව්.එම්.බන්දුසේන,\nලේකම්,\nඅධ්‍යාපන අමාත්‍යාංශය."
    ,
      "මගේ අංකය : CEQ-9434\nආයතන අංශය,\nකම්කරු දෙපාර්තමේන්තුව ,\nකොළඹ 05.\nසහකාර කම්කරු කොමසාරිස් (මඩකළපුව) මගින්,\nඑස්.පි.රාජේන්ද්‍රන් මයා ,\nරාජ්‍ය කළමනාකරණ සහකාර III,\nරාජ්‍ය කළමනාකරණ සහකාර සේවයේ III ශ්‍රේණියට අන්තර්ග්‍රහණය කිරීම.\nඋක්ත කරුණ සම්බන්ධයෙන් වන ඒකාබද්ධ සේවා අධ්‍යක්‍ෂ ජනරාල් විසින් මට ද පිටපතක් සහිතව මා මගින් ඔබ අමතා ඇති අංක PA/MAS/47237(C) හා 2016.05.09 දිනැති ලිපිය මේ සමඟ එවමි.\n02. ඒ අනුව 2013.04.02 දින සිට ක්‍රියාත්මක වන පරිදි රාජ්‍ය කළමනාකරණ සහකාර සේවයේ III ශ්‍රේණියට අන්තර්ග්‍රහණය කර ඇති බව සතුටින් දන්වා සිටිමි. \nමෙම අන්තර්ග්‍රහණය නිසා ඔබගේ වැටුපෙහි හෝ වැටුප් වර්ධක දිනයෙහි කිසිදු වෙනසක් සිදු නොවන බව කාරුණිකව දන්වා සිටිමි.\nකම්කරු කොමසාරිස් ජනරාල් වෙනුවට"
    ,
      "නාගරික ලේකම් දෙපාර්තමේන්තුව,\nමහා නගර සභාව,\nමහනුවර.\n\n2016-06-02\n\nගරු මහා නගර සභා මන්ත්‍රී\n................................ මහතා,\nමහනුවර.\n\nමහත්මයාණෙනි,\n\nනාගරික ක්‍රීඩා, තරුණ කටයුතු හා සුභසාධක සේවා පිළිබඳ ස්ථාවර කාරක සභා රැස්වීම.\n\nමහනුවර මහා නගර සභාවේ නාගරික ක්‍රීඩා තරුණ කටයුතු හා සුභසාධක සේවා පිළිබඳ ස්ථාවර කාරක සභාවේ රැස්වීමක් 2016.06.06 වන සඳුදා දින පෙ.ව.10.00 ට මහනුවර මහා නගර සභා ශාලාවේදී පැවැත්වෙන බව ගරු නගරාධිපතිතුමාගේ අණ පරිදි කාරුණිකව දන්වමි.\n\nඑම රැස්වීමේ න්‍යාය පත්‍රය පහත දැක්වේ.\n\nමෙයට - විශ්වාසී,\nඒ.එච්.එම්.ඉන්දිකා කුමාරි අබේසිංහ,\nනාගරික ලේකම්,\nමහනුවර මහා නගර සභාව.\n\nන්‍යාය පත්‍රය\n\n01. නාගරික ක්‍රීඩාපිටි සංවර්ධනය කිරීම හා ලෙදර් බෝල ක්‍රිකට් පුහුණු විකට් ඉදිකිරීම සම්බන්ධව ප්‍රධාන ක්‍රීඩා භූමි පරිපාලකගේ 2016.05.31 දිනැති වාර්තාව සලකා බැලීම."
    ,
      "මගේ අංකය : ED/07/99/6/1/2\nදිනය : 2014.05.29\nපළාත් අධ්‍යාපන අධ්‍යක්ෂ,\nපළාත් අධ්‍යාපන දෙපාර්තමේන්තුව,\nදකුණු පළාත,\nගාල්ල.\nප්‍රාථමික අංශය සඳහා වැසිකිළි පද්ධතියක් ඉදිකිරීමට අනුමැතිය ලබා දීම  - ගා/සංඝමිත්තා බා.වි. \nඋක්ත කරුණ සම්බන්ධයෙන්, විද්‍යාලයේ ප්‍රාථමික අංශයේ දෙමාපියන් පිරිසක් විසින් ප්‍රාථමික අංශයේ දැරියන්  සඳහා වැසිකිළි පද්ධතියක් ඉදිකර පරිත්‍යාග කිරීමට කැමැත්ත පල කර ඇති බව සඳහන් කරමින් අනුමැතිය ඉල්ලා එවා ඇති ඔබේ අංක දප/සැවි/06/09/02/2014 හා 2014.04.30 දිනැති ලිපිය හා බැඳේ.\nඒ අනුව, පහත සඳහන් කොන්දේසි යටතේ වැසිකිලි පද්ධතියක් ඉදිකිරිම අතිරේක ලේකම් (පාසල්  සැපයීම්,සේවා හා ඉදිකිරීම්) විසින් අනුමත කල බව කාරුණිකව දන්වා සිටිමි.\n1. මෙම ඉදිකිරීම් කටයුතු විදුහලේ ප්‍රාථමික අංශයේ දෙමාපිය පිරිසක් විසින් ඉටු කරන බැවින්  විද්‍යාලයේ සිසුන්ගෙන් මුදල් රැස්කිරීමක් නොකළ යුතු වනවා ඇත.\n2. සියලු ඉදිකිරීම් කටයුතු ගාල්ල දිස්ත්‍රික් ඉංජිනේරුවරයාගේ අධීක්ෂණය යටතේ සිදුකළ යුතු වෙයි.\n3. මෙම ඉදිකිරීම් කටයුතු සිදුකරන අවස්ථාවේදී සිසුන්ගේ ආරක්ෂාවට කිසිදු ආකාරයක බලපෑමක් සිදුනොවිය යුතු වෙයි.\n4. මෙම ඉදිකිරීම්වලට අදාළ සියලු ගනුදෙනුවල ලිපි ලේඛන ඕනෑම අවස්ථාවක පරික්ෂ කිරීමට හැකි පරිදි සුරැකිව තබාගත යුතුවේ.\n5. මෙම ඉදිකිරීම් කටයුතු සඳහා අමාත්‍යංශයෙන් කිසිදු මුදලක් ලබා දෙනු නොලැබේ.\n6. මෙම ඉදිකිර්ම අඛණ්ඩව පවත්වාගෙන කටයුතු කිරීම ඔබගේ වගකීම වනු ඇත.\nඊ.එම්. ප්‍රනාන්දු,\nපාසල් ගොඩනැගිලි අධ්‍යක්ෂ,\nඅධ්‍යාපන ලේකම් වෙනුවට.\n පිටපත් :-\n1. නියෝජ්‍ය ප්‍රධාන ලේකම් (ඉංජි.සේවා), දකුණු පළාත  - දැ.ග.ස.\n2. දිස්ත්‍රික් ඉංජිනේරු, දිස්ත්‍රික් ඉංජිනේරු කාර්යාලය, ගාල්ල -  දැ.ග.ස.\n3. කලාප අධ්‍යාපන අධ්‍යක්ෂ, කලාප අධ්‍යාපන කාර්යාලය, ගාල්ල -  දැ.ග.ස.\n4. විදුහල්පති, ගා/ සංඝමිත්තා බා.වි. -  දැ.ග.ස."
    ,
      "භාෂා/7/17/ත්‍රි/පොදු/01\n2013.06.05\nදිස්ත්‍රීක් ලේකම් මගින්, \nජාතික ඒකාබද්ධතා ප්‍රවර්ධන සහකාර,\nදිස්ත්‍රීක් ලේකම් කාර්යාලය,\nඅනුරාධපුරය.\nභාෂා සංගම් ජාතික ව්‍යාපාරයේ ගැටලු පිළිබඳ සාකච්ඡාව.\nජාතික භාෂා හා සමාජ ඒකාබද්ධතා අමාත්‍ය ගරු වාසුදේව නානායක්කාර මැතිතුමාගේ උපදෙස් පරිදි භාෂා සංගම් ජාතික ව්‍යාපාරය තුළ අනුරාධපුරය දිස්ත්‍රීක්කයට අදාළව මතුවී ඇති ගැටලු පිළිබඳ විශේෂ සාකච්ඡාවක් 2013.06.13 දින පෙ.ව. 09.30 ට අමාත්‍යාංශයීය ශ්‍රවණාගාරයේදී පැවැත්වීමට කටයුතු සංවිධානය කර ඇත.\nභාෂා සංගම් ජාතික ව්‍යාපාරයට අදාළව මේ සමඟ ඔබ වෙත යොමු කර ඇති සියලුම ආකෘති පත්‍ර සම්පුර්ණ කර 2013.06.10 දිනට ප්‍රථම 0112-883933 දරන ෆැක්ස් අංකයට යොමු කර එහි මුල් පිටපත් සාකච්ඡාව සදහා පැමිණෙන විට රැගෙන එන මෙන් කාරුණිකව දන්වා සිටිමි.\nමෙම සාකච්ඡාව සදහා ඔබගේ පැමිණීම අනිවාර්ය බව කාරුණිකව සලකන්න.\nඕ.කේ.අයි.එම්. ඕපාත,\nසහකාර ලේකම් (ජාතික භාෂා),\nජාතික භාෂා හා සමාජ ඒකාබද්ධතා අමාත්‍යාංශයේ ලේකම් වෙනුවට."
    ,
      "වෙත :- අතිරේක ලේකම්(පාලන හා මුදල්),\nවිදුලි ඉංජිනේරු/ අතිරේක අධ්‍යක්ෂ(පාසල් ගොඩනැගිලි) මගින්,\nවෙතින් :- ගොඩනැගිලි කළමනාකරු.\nදිනය :- 2014.06.06\nමගේ අංකය :- ED/7/74/3/10\nවිෂය :- විදුලි ජනක යන්ත්‍රයෙන් විදුලි බල සම්බන්ධතාවය ලබා ගැනීම.\nඋක්ත කරුණ සම්බන්ධයෙන් අංක ED/01/26/01/09/01(ix) හා 2014.05.12 දිනැතිව අධ්‍යාපන අධ්‍යක්ෂ (තොරතුරු හා සන්නිවේදන තාක්ෂණ) විසින් ඔබ ඇමතූ ලිපිය හා බැඳේ.\n02.ජාතික විදුලි බල පද්ධතියේ වරින්වර සිදුවන විදුලිය ඇණහිටීම් නිසා තොරතුරු හා සන්නිවේදන තාක්ෂණ ශාඛාවේ කටයුතු කරගෙන යාමේ දුෂ්කරතා සම්බන්ධයෙන් ඉහත ලිපිය මගින් පෙන්වා දී ඇත.\n03.අමාත්‍යාංශ පරිශ්‍රයේ විදුලි ජනක යන්ත්‍ර මගින් ජනනය වන විදුලිය ලබා දී ඇත්තේ ජාතික විදුලි බල පද්ධතියේ පන්නිපිටිය විදුලි රැහැන් මාර්ගයට සම්බන්ධ කර ඇති තාරාපැවිය(Transformer) හරහා විදුලිය සැපයීම සිදු කරනු ලබන ශාඛා සඳහා පමණි. \nජාතික විදුලි බල පද්ධතියේ අකුරේගොඩ විදුලි රැහැන් මාර්ගයට සම්බන්ධ කර ඇති තාරාපැවිය(Transformer) මගින් තොරතුරු හා සන්නිවේදන ශාඛාව සඳහා විදුලිය සපයා ඇත. \nමේ නිසා මෙම ශාඛාව සඳහා සෘජු ලෙස විදුලි ජනක යන්ත්‍රය මගින් විදුලි බලය ලබා දීමේ හැකියාවක් නොමැති අතර එම විදුලි සැපයීම සිදු කිරීම වෙනුවෙන් විදුලි තතැදුම්වල සැලකිය යුතු වෙනස්කම් රාශියක් සිදු කළ යුතුව ඇත. \nතවද විදුලි ජනක යන්ත්‍රවල සහ තාරාපැවි(Transformer) තුළ පවතින ශක්‍යතාව අනුව විදුලි බලය ලබා දීමේ හැකියාව පිළිබඳව ද ශක්‍යතා වාර්තාවක් ලබා ගත යුතුව ඇත.\n04.මේ පිළිබඳව දැනට කටයුතු සිදු කරමින් පවතින අතර, ලංකා විදුලිබල මණ්ඩලයේ විදුලි අධිකාරි ලවා ශක්‍යතා පරීක්ෂණයක් සිදු කර අදාළ නිරීක්ෂණ හා නිර්දේශ ලබා ගැනීමට කටයුතු යොදා ඇත. \nඒ අනුව අදාළ නිර්දේශ ලැබීමෙන් පසු මෙම විදුලි සැපයීම සිදු කිරීමේ හැකියාවක් පවතී ද යන්න තහවුරුකර ගැනීමේ හැකියාව ඇත.\n05.තොරතුරු හා සන්නිවේදන තාක්ෂණ ශාඛාවට අමතරව ගෙවීම් ශාඛාව හා රජයේ විගණන අංශය මගින් ද විදුලි ජනක යන්ත්‍ර හරහා විදුලිය ලබා දෙන ලෙස ඉල්ලීම් ඉදිරිපත් කර ඇති අතර, විදුලි බල මණ්ඩලය මගින් ලබා දෙන නිර්දේශ හා විදුලි ජනක යන්ත්‍රවල හා තාරාපැවි(Transformer) තුළ පවතින ශක්‍යතාව අනුව, මෙම අංශ සඳහා ලබා දිය හැකි උපරිම විදුලිබල ප්‍රමාණය සැලකිල්ලට ගෙන, නිසි විදුලි රැහැන් හා අවශ්‍ය විදුලි උපාංග සවිකර විදුලිය ලබා දීමට කටයුතු කරන බව කාරුණිකව දන්වා සිටිමි.\nකමල් අතුකෝරල,\nගොඩනැගිලි කළමනාකරු.\nපිටපත් - 01.අතිරේක ලේකම්(අධ්‍යාපන හා ගුණාත්මක සංවර්ධන)  දැ.ගැ.පි\n02.අධ්‍යාපන අධ්‍යක්ෂ(තොරතුරු හා සන්නිවේදන තාක්ෂණ) දැ.ගැ.පි\n 03.විගණන අධිකාරි(රජයේ විගණන අංශය)  දැ.ගැ.පි\n 04.ගණකාධිකාරි(ගෙවීම්) දැ.ගැ.පි"
    ,
      "වෙත :- ජ්‍යෙෂ්ඨ සහකාර ලේකම් (පාලන 1)\nවෙතින් :- අධ්‍යාපන අධ්‍යක්ෂ (පාසල් සැපයීම්)\nමගේ අංකය :- ED/07/97/02/32(1) වෙළුම 2\nවිශය :- ලොරි රථ 04 ක් අමාත්‍යාංශය වෙත අනුයුක්ත කිරීම.\nදිනය :-2011.03.03\nගරු අධ්‍යාපන අමාත්‍යතුමාගේ විශේෂ උපදෙස් මත කොරියානු ප්‍රදානයන් පාසල් වෙත බෙදා දීම සඳහා විභාග දෙපාර්තමේන්තුව විසින් ලොරි රථ 04 ක් අමාත්‍යාංශය වෙත 2011.03.01 දින සිට 2011.03.15  දින දක්වා අනුයුක්ත කර ඇත.\nකරුණාකර ඇමුණුම  බලන්න.\nලොරි රථ අංකය       රියදුරු                                              රිය සහයක  \n1.LE-39           එම්. ඒ. ප්‍රේමරත්න මයා\t\tපී. ඒ. සරත් ප්‍රේමසිරි මයා \n2.LE-3953\tආර්. එම්. ඒ. ඩී. රාජපක්ෂ මයා \tආර්. ඩී. සිරිල් මයා \n3.43-4832\tඩබ්ලිව්. ජයවර්ධන මයා \t           බී. එම්. එම්. පී. ගුණතිලක මයා\n4.43-4833\tඑම්. පී. පෙරේරා මයා \t\tඒ. ඒ. නිමල් තිලකසිරි මයා \nබෙදා හැරීම් කටයුතු වලට යොදා ගැනීම සම්බන්ධව මෙම ලොරි රථ සඳහා අවශ්‍ය ඉන්ධන ප්‍රතිපාදන ප්‍රවාහන සංචිතයෙන් ලබාදෙන ලෙසත් ලොරි රථ රියදුරන් සහ රිය සහයකයන් වෙනුවෙන් අදාල ගමන් සඳහා අතිකාල දීමනා සංයුක්ත දීමනා සහ බටා ලබා දීමට අවශ්‍ය කටයුතු සම්පාදනය කර දෙන ලෙසත් කරුණාවෙන් ඉල්ලා සිටිමි.\nබී. ඩී. සී. බියන්විල,\nඅධ්‍යාපන අධ්‍යක්ෂ (සැපයීම්)."
    ,
      "මගේ අංකය : ED/07/99/6/1/2\nපාසැල් ගොඩනැගිලි ශාඛාව,\nඅධ්‍යාපන අමාත්‍යාංශය.\n2013.    12. දින\nඅතිරේක ලේකම් (පාසල් සැපයීම්, සේවා හා ඉදිකිරීම්)\nප්‍රාථමික අංශ බුදු මැදුරක් ඉදිකිරිම සඳහා අනුමැතිය ලබා ගැනීම - ගා/ සංඝමිත්තා විද්‍යාලය.\nඉහත විදුහලේ බුදු මැදුරක් ඉදිකිරිම සඳහා විදුහලේ 1 ශ්‍රේණියේ දෙමව්පිය සංසදය මගින් මුදල් හා ද්‍රව්‍ය පරිත්‍යාග කිරීමට ඉදිරිපත් වී ඇති බැවින් ඒ සඳහා අනුමැතිය ලබා දෙන ලෙස විදුහල්පතිගේ 2013/12/02 දිනැති ලිපියෙන් ඉල්ලීමක් කර ඇත. (ඇමුණුම 1)\n2. මෙම ඉදිකිරීමට අදාළ පහත සඳහන් ලියවිලි සන සැලසුම් ද මා වෙත ඒවා ඇත.\n2.1. මෙම ඉදිකිරිම පළාත් අධ්‍යාපන අධ්‍යක්ෂ, කලාප අධ්‍යාපන අධ්‍යක්ෂ, දිස්ත්‍රික් ඉංජිනේරු (ගාල්ල ) විසින් අනුමත කරන ලද ලිපිය (ඇමුණුම 2)\n2.2. ගෘහ නිර්මාණ සැලසුම් (ඇමුණුම 3) - (ඉංජිනේරු විසින් අත්සන් කර නිල මුද්‍රාව යොදා ඇත)\n2.2. ඇස්තමේන්තුව (ඇමුණුම 4) -රු. 327,627.00 ( ඉංජිනේරු විසින් අත්සන් කර නිල මුද්‍රාව යොදා ඇත)\n2.3. මුදල් තත්ත්වය පැහැදිලි කිරීම - 1 ශ්‍රේණියේ දෙමව්පියන් විසින් රු. 330,000.00 ක මුදලක් ලබා දීමට කැමැත්ත දන්වා ඇත. (ඇමුණුම 5)\nඉහත ඉදිකිරීම්වලට අදාළ සැලසුම් අනුමත කරන අතර, 1 ශ්‍රේණියේ දෙමව්පිය සංසදයෙන් මුදල් හා ද්‍රව්‍ය භාවිත කරමින් පහත සඳහන් කොන්දේසි වලට යටත්ව බුදු මැදුර ඉදිකිරිම මා නිර්දේශ කරමි.  \n3. කොන්දේසි :\n3.1. මෙම ඉදිකිරිම් කටයුතු සඳහා විදුහලේ 1 ශ්‍රේණියේ දෙමව්පිය සංසදයෙන් මුදල් හා ද්‍රව්‍ය පමණක් යොදා ගත යුතු අතර, අනෙකුත් සිසුන්ගෙන් හෝ දෙමව්පියන්ගෙන් හෝ මුදල් රැස්කිරීමක් හෝ ද්‍රව්‍යමය ආධාර ලබා ගැනීම නොකළ යුතු වනවා ඇත.\n3.2. සියලු ඉදිකිරීම් කටයුතු දිස්ත්‍රික් ඉංජිනේරුවරයාගේ (ගාල්ල) අධීක්ෂණය යටතේ සිදුකළ යුතු වෙයි.\n3.3. මෙම ඉදිකිරීම් කටයුතු සිදුකරන අවස්ථාවේදී සිසුන්ගේ ආරක්ෂාවට කිසිදු ආකාරයක බලපෑමක් සිදුනොවිය යුතු වෙයි.\n3.4. මෙම ඉදිකිරීම් කටයුතු අමාත්‍යංශයේ කිසිඳු මුදලක් ලබා දෙනු නොලැබේ.\n3.5. චක්‍රලේඛ විධිවිධාන අනුව කටයුතු කරන්න.\nකරුණාකර අනුමැතියට ඉදිරිපත් කරමි.\nඅයි.එම්. ප්‍රනාන්දු,\nපාසල් ගොඩනැගිලි අධ්‍යක්ෂ."
    ,
      "මගේ අංකය : ඊඩි/7/99/6/1\nඉංජිනේරු (පාසල් ගොඩනැගිලි),\nඌව පළාත් අධ්‍යාපන දෙපාර්තමේන්තුව,\nබදුල්ල.\nවාර්ෂික ඉදිකිරිම 2013 (විශේෂ අධ්‍යාපන ඒකක නවීකරණය හා සංවර්ධනය) සඳහා ඇස්තමේන්තු කැඳවීම.\nවිශේෂ අධ්‍යාපන ඒකක නවීකරණය හා සංවර්ධනය වෙනුවෙන් ප්‍රතිපාදන වෙන් කර ඇති බැවින් විෂය අදාළව පහත සඳහන් ජාතික පාසල්වල ගොඩනැගිලි ඉදිකිරීම් හා පුනරුත්ථාපනය වෙනුවෙන් ඇස්තමේන්තු සකස් කර එවන මෙන් කාරුණිකව දන්වා සිටිමි.\nපාසලේ නම                                        පළාත                 කාර්යය                                                                                                                   වෙන් කර ඇති ප්‍රතිපාදන\n1. බ/ගුරුතලාව මුස්ලිම් ම.ම.වි.        ඌව පළාත        40'x20' පන්ති කාමරයේ වහලයේ උළු වෙනුවට ශීට් යෙදීම.                   1,000,000/-          \n                                                                                        දොර ජනෙල් සවි කිරීම, \n                                                                                        සිවිලිම හා විදුලි පංකා,ප\n                                                                                       ප්‍රධාන මාර්ගයේ සිට පන්ති කාමරය දක්වා පිවිසුම් මාර්ගය,  \n                                                                                       යාබදව සේදුම් කාමරය\n                                                                                        විදුලිය හා ජාල පහසුකම්   \n                                                                                       පන්ති ටයිල් කිරීම.                      \nඅයි.එම්. ප්‍රනාන්දු,\nපාසල් ගොඩනැගිලි අධ්‍යක්ෂ, \nඅධ්‍යාපන ලේකම් වෙනුවට, \nපිටපත් :\n1. විදුහල්පති - බ/ගුරුතලාව මුස්ලිම් ම.ම.වි"
    ,
      "සහකාර අධ්‍යක්ෂ ,\nමධ්‍යම පරිසර අධිකාරිය,\nදිස්ත්‍රික් කාර්යාලය ,\n10 ඒ,මල්වත්ත පාර, \nමාතලේ.\nදිස්ත්‍රික් සම්බන්ධීකරණ කමිටු තීරණ දැනුම් දීම (2016.03.04) සොබා පරිසර කේන්ද්‍රයක් ආරම්භ කිරීම. \n2016.03.04 දින පැවති දිස්ත්‍රික් සම්බන්ධීකරණ කමිටු රැස්වීමේදී මාතලේ දිස්ත්‍රික්කය තුල ඔබ අධිකාරිය මගින් සොබා පරිසර කේන්ද්‍රයක් ස්ථාපිත කිරීම සම්බන්ධයෙන් සාකච්ඡා කරන ලදී.\n02. මාතලේ දිස්ත්‍රික්කය තුල පවතින රජයේ ඉඩම් ප්‍රමාණය ඉතා සීමිත වන අතර , රාජ්‍ය ආයතන විශාල සංඛ්‍යාවකින් මාතලේ නගරය තුල තම ආයතන ස්ථාපිත කිරීම සඳහා ඉඩම් ලබා දෙන ලෙස මා වෙත ඉල්ලීම් ඉදිරිපත් කර ඇත. \nඒ අනුව ව්‍යාපෘතිය ක්‍රියාත්මක කරන ආයතනවල පවතින කඩිනම්ව ක්‍රියාත්මක වීමේ ශක්‍යතාවය අනුව  අදාල ආයතන වෙත ඉඩම් බැහැර කිරීම සඳහා ප්‍රමුඛතාවය ලබා දීමට 2016.02.01 දින පැවති දිස්ත්‍රික් සම්බන්ධීකරණ කමිටුවේදී තීරණය විය.\n03. එබැවින් ඔබ විසින් ඉල්ලා ඇති පරිදි, මාධ්‍යම පරිසර අධිකාරියේ සොබා පරිසර කේන්ද්‍රයක් ආරම්භ කිරීම සඳහා සුදුසු ඉඩම් කොටසක් ලබා දීම පිළිබඳව කටයුතු කිරීම සඳහා , වියදම් කිරීමට අපේක්ෂිත මුදල සහ අදාල කාර්ය ඉටු කිරීමට අපේක්ෂිත කාල රාමුව සහිත පහත තොරතුරු ඇතුලත් වාර්තාවක් 2016.05.20 දිනට පෙර මා වෙත ලබා දෙන ලෙස කාරුණිකව ඉල්ලා සිටිමි.\n1. ඉඩම අවශ්‍ය ස්ථානය .\n2. අවශ්‍ය ඉඩම් ප්‍රමාණය .\n3. ඒ සඳහා ව්‍යාපෘති වාර්තාව .\nඅයි.ඩි.පි.විජේතිලක ,\nඅතිරේක දිස්ත්‍රික් ලේකම් (ඉඩම්).\nනීල් ද අල්විස්,\nදිස්ත්‍රික් ලේකම් / දිසාපති ,\nමාතලේ ."
    ,
      "මිනින්දෝරු දෙපාර්තමේන්තුව\nකොළඹ 05.\n2016.01.13 දින.\nසියලුම පළාත් සර්වේයර් ජනරාල්වරුන් හා දිස්ත්‍රික් භාර ජ්‍යෙෂ්ඨ මිනින්දෝරු අධිකාරීවරුන් වෙත.\nශීර්ෂ අංක 288 යටතේ නව පොදු තැන්පත් ගිණුම් විවෘත කිරීම.\nඋක්ත කරුණ සම්බන්ධයෙන් මගේ අංක සජ/ගි/පො.තැ/06/පොදු තැන්පත් හා 2015.12.18 දිනැති ලිපිය හා බැඳේ. \nඒ අනුව නව සිගාස් වැඩසටහන තුල තෙවන පාර්ශවයන් වෙත නැවත ගෙවීම් කිරීම වෙනුවෙන් තාවකාලිකව රදවාගන්නා තැන්පතු ගිණුම් යටතේ විවෘත කරන ලද උප ගිණුම් වලට අමතරව පහත සඳහන් පරිදි උප ගිණුමක් විවෘත කිරීමට කටයුතු කරන මෙන් කාරුණිකව දන්වා සිටිමි.\nගිණුමේ නම\nගිණුම් අංකය \nTreasury Code\n2.තෙවන පාර්ශවයන් වෙත නැවත ගෙවීම් කිරීම වෙනුවෙන් තාවකාලිකව රදවාගන්නා තැන්පතු ගිණුම\nවිස්තරය\nAccounting Code\nවිවිධ \n13/77/M\nඋදාහරණ වශයෙන් මෙම ගිණුමට කරනු ලබන හර හා බැර කිරීම් ලෙස මැතිවරණ දෙපාර්තමේන්තුවෙන් ලැබෙන මුදල්, පළාත් කාර්යාලවලින් දිස්ත්‍රික් කාර්යාලවලින් ලැබෙන උත්සව අත්තිකාරම් සහ විශේෂ අත්තිකාරම් මුදල්,රක්ෂණ වන්දි මුදල් යනාදියයි.\nඅධ්‍යක්ෂ (මුදල්),\nසර්වේයර් ජනරාල්  වෙනුවට."
    ,
      "මාගේ අංක 4/4/LB/23\nජාතික බිම් කැබලි වියමණ.\nහිමිකම් ලියාපදිංචි කිරීමේ මැනුම් අංශය.\n2016.05.25\nපළාත් සර්වේයර් ජනරාල් (බස්නාහිර /දකුණ /සබරගමුව/උතුර / උතුරු මැද/ නැගෙනහිර /මධ්‍යම /වයඹ / ඌව) මගින්,\nසියළුම ජ්‍යෙෂ්ඨ මිනින්දෝරු අධිකාරීන් වෙත.\nජාතික බිම් කැබලි වියමණ මාසික ප්‍රගතිය.\nජාතික බිම් කැබලි වියමණ සම්බන්ධව සති දෙකකට වරක් ඔබ විසින් එවනු ලබන ප්‍රගතිය එවීම සඳහා භාවිතා කරන ලද ආකෘති පත්‍රය සංශෝධනය කළ අතර ඉදිරියට එම ප්‍රගතිය එවීම සඳහා මේ සමඟ අමුණා ඇති සංශෝධිත ආකෘතිය භාවිතා කරන මෙන් කාරුණිකව ඉල්ලා සිටිමි.\n2. එමෙන්ම සෑම මසකම 15 වන දින සහ 30 වන දින එම ප්‍රගතිය එවීමට කටයුතු කරන මෙන් දන්වා සිටිමි.\nඒ.එම්.එන්. බණ්ඩාර, \nනියෝජ්‍ය සර්වේයර් ජනරාල් (හිමිකම් ලියාපදංචි ).\nපිටපත්\nජ්‍යෙෂ්ඨ මිනින්දෝරු අධිකාරී (   ගාල්ල/ මාතර / හම්බන්තොට/ ගම්පහ / කොළඹ/ කළුතර / රත්නපුර / කෑගල්ල/ පුත්තලම/කුරුණෑගල/ මොණරාගල/ බදුල්ල/ අම්පාර/ ත්‍රිකුණාමලය/නුවරඑළිය/ මාතලේ/ නුවර / මන්නාරම/ මඩකලපුව/ වව්නියාව/ යාපනය/ කිලිනොච්චි/ මුලතිව්/ අනුරාධපුරය/ පොළොන්නරුව.)\nඉහත තොරතුරු නොපමාව දන්වා එවීම සඳහා"
    ,
      "මගේ අංකය - 1/8/1/2(11)                                            \n2016-06-27 \nඅධ්‍යක්‍ෂ ,\nජාතික ආපදා  සහන සේවා මධ්‍යස්ථානය ,\nවිද්‍යා මාවත ,\nකොළඹ  07.\nස්වාභාවික ආපදා සඳහා  වූ රක්ෂණාවරන වැඩසටහන යටතේ කඩිනම් ආපදා සහනාධාර ගෙවීම් වියදම් වාර්තාව (2016-05-15 සිට 2016-05-27 දක්වා) \nඋක්ත වැඩසටහන යටතේ මාතලේ දිස්ත්‍රික්කය වෙත කඩිනම් ආපදා සහන සැලසීම සඳහා ගෙවීම් කිරීමට අදාළව රු. 184425.00 ක මුදල් ලැබී ඇති අතර එම මුදල් වලින් දැනට ඉදිරිපත් වී ඇති බිල්පත් සඳහා ගෙවීම් කර ඇති ආකාරය සම්බන්ධව අදාළ ප්‍රාදේශීය ලේකම් වරුන්ගෙන් ගෙන්වා ගත් වියදම් වාර්තාවන්ට අනුව සැකසූ වියදම් වාර්තාව මේ සමග ඉදිරිපත් කරමි.\nතේජානි තිලකරත්න, \n අතිරේක දිස්ත්‍රික් ලේකම් ,\n මාතලේ දිස්ත්‍රික් ලේකම් වෙනුවට."
    ,
      "මධ්‍යම පළාත් සභාවේ,\nප්‍රධාන ලේකම්,\nආණ්ඩුකාර ලේකම්,\nසියළුම අමාත්‍යාංශ ලේකම්වරු,\nසියළුම දෙපාර්තමේන්තු හා ආයතන ප්‍රධානීන් වෙත.\nමධ්‍යම පළාත් රාජ්‍ය සේවයට අයත් රියදුරු හා කාර්යාල සේවක සේවයේ පුරප්පාඩු සම්බන්ධ විස්තර ලබා ගැනීම.\nමධ්‍යම පළාත් රාජ්‍ය සේවා කොමිෂන් සභාව පත්වීම් බලධාරියා වෙත මධ්‍යම පළාත් රාජ්‍ය සේවයේ රියදුරු හා කාර්යාල සේවක සේවයේ (කාර්යාල කම්කරු, මුරකරු, සනීපාරක්ෂක කම්කරු, බංගලා භාරකරු තනතුරුද ඇතුළත් වන පරිදි ) 2015.12.31 දිනට සියළුම ආයතනවල අනුමත සේවක සංඛ්‍යාව හා එදිනට පවත්නා පුරප්පාඩු සම්බන්ධ විස්තර 2016.01.25 දිනට පෙර මෙම කොමිෂන් සභාව  වෙත ඉදිරිපත් කරන ලෙසට මා වෙත උපදෙස් දී ඇත.\n02. ඒ අනුව මේ සමඟ අමුණා එවනු ලබන ආකෘති පත්‍රය සම්පූර්ණ කර නොපමාව එවීමට කටයුතු කරන ලෙස කාරුණිකව දන්වා සිටිමි.\n03. මේ පිළිබඳව විශේෂ අවධානය යොමුකර කඩිනම් පිළිතුරු එවීමට සලස්වන්නේ නම් ඉතා අගය කොට සලකමි.\nආර්.එම්.එන්.රත්නායක,\nලේකම්,\nමධ්‍යම පළාත් සභාවේ\nරාජ්‍ය සේවා කොමිෂන් සභාව."
    ,
      "සියළුම දිස්ත්‍රික් ලේකම්වරුන්,\nදිවිනැගුම සංවර්ධන දෙපාර්තමේන්තුව - ප්‍රතිපාදන නිදහස් කිරීම - 2016\n2016 වර්ෂයේ දිවිනැගුම සංවර්ධන දෙපාර්තමේන්තුවේ වියදම් දැරීම සඳහා   අංක 331-2-2-0 යටතේ (ජනවාරි සිට ජූනි දක්වා) ඇමුණුම පරිදි ඔබ වෙත ප්‍රතිපාදන වෙන් කර ඇත.\n02. මෙම ප්‍රතිපාදන වෙන් කළ කාර්ය සඳහා පමණක් යොදා ගන්නා ලෙස දන්වන අතර ප්‍රතිපාදන සීමාව ඉක්මවා වියදම් නොකිරීමටත් මුදල් වියදම් කිරීමේදි මුදල් රෙගුලාසි හා අනෙකුත් නීති රීති වලට අනුකූලව වියදම් දරණ බවටත්  ඔබ විසින් සහතික විය යුතුය.\n03. මෙම වියදම් දැරීම සඳහා අවශ්‍ය අක්මුදල් දිවිනැගුම දිස්ත්‍රික් අතිරේක අධ්‍යක්ෂ ජනරාල් ගේ නිළ ගිණුම වෙත කාලීනව ඉඳුරාම ප්‍රේශණය කරන අතර ඒ පිළිබඳව ඔබව දැනුවත් කරනු ලැබේ. \nමහ භාණ්ඩාගාරයට මාසික ගිණුම් සාරාංශ වාර්තා කිරීමට නියම කර ඇති දිනයට දින 02 ක ට පෙර පසුගිය මාසයේ වියදම් සාරාංශය පිළියෙල කර ප්‍රධාන ගණකාධිකාරී, දිවිනැගුම සංවර්ධන දෙපාර්තමේන්තුව, 4 වන මහල, සෙත්සිරිපාය, බත්තරමුල්ල යන ලිපිනයට ලබා දීමට කටයුතු කළ යුතු බවද කාරුණිකව දන්වමි.\nසුමිත් ජේ. කුමාර,\nගණකාධිකාරී(වැටුප් හා ගෙවීම්),\nඅධ්‍යක්ෂ ජනරාල් වෙනුවට."
    ,
      "සියළුම දිස්ත්‍රික් දිවිනැගුම අධ්‍යක්ෂවරුන්,\nදිස්ත්‍රික් ලේකම් කාර්යාලය ,\n මඩකලපුව.\nදිවිනැගුම නිවාස ලොතරැයි දිනුම් ඇදීම 16 වන දිනුම් වාරය - ත්‍යාග මුදල් ලබාදීම.\nමෙම දෙපාර්තමේන්තුව යටතේ ක්‍රියාත්මක වන දිවිනැගුම නිවාස සංවර්ධන ලොතරැයි අරමුදල යටතේ 2014 හා 2015 වර්ෂ වලට අදාළ ජයග්‍රාහී ප්‍රතිලාභීන්ගේ නාම ලේඛනය අංක DDD/HO/6/M/01/2016/Lot හා 2016.02.15 දිනැති ලිපිය මඟින් ඔබ වෙත දැන්වීමට කටයුතු කර ඇත.\n02. ඔබ දිස්ත්‍රික්කයට අදාළ ත්‍යාගහිමි ප්‍රතිලාභීන් සඳහා ගෙවිය යුතු ත්‍යාග මුදල වන රු.18,000,000.00 ක මුදල ඉඳුරාම ඔබගේ නිල ගිණුම වෙත විද්‍යුත් ප්‍රේෂණ ක්‍රමය යටතේ යැවීමට කටයුතු කර ඇත. \nඅදාළ බැර නිවේදනය ඉදිරියේදී ලබා දෙනු ඇත.\n03. මෙම මුදල පොදු තැන්පත් ගිණුමේ තෙවන පාර්ශවයක් වෙත නැවත ගෙවීම් කිරීම වෙනුවෙන් තාවකාලිකව රඳවා ගැනෙන මුදල් කාණ්ඩය යටතේ අංක 6000-0-0-13-0-123 දරණ කේතය සඳහා ලැබීම් වශයෙන් ගෙන එම කේතයෙන්ම ගෙවීම් කළ යුතුය. \nයම් ප්‍රතිලාභියෙක් සඳහා ත්‍යාග මුදල් ගෙවීමට නොහැකි වුවහොත් එම මුදල සඳහා පහත සඳහන් පරිදි අණකරන ලද චෙක්පතක් කඩිනමින් මෙම දෙපාර්තමේන්තුව වෙත ලබා දිය යුතුය.\n04. කළමණාකරු, ලංකා බැංකුව බත්තරමුල්ල, දිවිනැගුම නිවාස සංවර්ධන ලොතරැයි අරමුදල් ගිණුම් අංක 0-002938040 දරණ ගිණුම''\n05. 2012/2013 වර්ෂ වල හිඟ ත්‍යාග මුදල් ගෙවීම සඳහා ඔබ වෙත ලබා දී ඇති මුදල්ද, ලබා දී ඇති උපදෙස් අනුවම ගෙවීම් කිරීමට සටහන් කර ගන්න.\n06. ඉහත උපදෙස් පරිදි ගෙවීම් කිරීමෙන් අනතුරුව ත්‍යාග මුදල් ගෙවීම සඳහා එක් එක් වර්ෂය වෙනුවෙන් ඔබ වෙත එවන ලද මුදල සහ දරණ ලද වියදම පහත සඳහන් ආකෘති පත්‍රයට අනුව සම්පූර්ණ කර 2016.03.25 දිනට පෙර ප්‍රධාන කාර්යාලයේ ගිණුම් අංශය වෙත එවීමට කටයුතු කල යුතුය."
    ,
      "මගේ අංකය - CPC/CS/2/2/3,\nදිනය - 2016.02.02,\nඉතා වැදගත්.\nසියළුම ආයතන ප්‍රධානීන් වෙත,\nඅනුමත කාර්ය මණ්ඩල වාර්තාවට අනුව 2016.01.31 දිනට මධ්‍යම පළාත් සභාවේ සේවක සංඛ්‍යාව යාවත්කාලීන කිරීම.\nමධ්‍යම පළාත් සභාවේ කාර්ය මණ්ඩලයේ සමහර තනතුරු 2016.01.01 දිනට ස්ථාන මාරුවීම් ක්‍රියාත්මක වීම මත වෙනස්වී ඇති බැවින් 2016 වර්ෂයේදී සේවක සංඛ්‍යාව යාවත්කාලීන කරගත යුතුව ඇත.\n ඒ අනුව ඔබ අමාත්‍යාංශයේ, ගරු අමාත්‍ය පෞද්ගලික කාර්ය මණ්ඩලයේ, දෙපාර්තමේන්තුවේ හා ආයතනයේ 2016.01.31 දිනට යාවත්කාලීන කරන ලද කාර්ය මණ්ඩල තොරතුරු පසුපිටෙහි සඳහන් ආකෘතිපත්‍රානුකූලව සකස් කර 2016.02.10 දිනට පෙර ලබාදීමට අවශ්‍ය කටයුතු කරන ලෙස කාරුණිකව දැනුම් දෙමි.\nකාර්ය මණ්ඩල තොරතුරු යාවත්කාලීන කරගැනීම ඉතා වැදගත් බැවින් නියමිත දිනට තොරතුරු ලබාදී මෙම කර්තව්‍ය සාර්ථක කර ගැනීමට ඔබ දක්වන සහයෝගය පිළිබඳ අතිශයින් කෘතඥ වෙමි.\nඩබ්ලිව්.එම්.වික්‍රමරත්න\nනියෝජ්‍ය ප්‍රධාන ලේකම් (පිරිස් හා පුහුණූ)\n ප්‍රධාන ලේකම්\nමධ්‍යම පළාත"
    ,
      "මාගේ අංකය :- P&S/10009/SPF 04/පොදු\nප්‍රසම්පාදන හා සැපයීම් අංශය, \nසර්වේයර් ජනරාල් කාර්යාලය,\nකොළඹ 05.\nවාර්ෂික ගබඩා සමීක්ෂණය - 2015 \nගබඩා අංකය 092 S 005\nඔබ අංශයේ 2015 වර්ෂය සඳහා වගකිවයුතු ගබඩා භාණ්ඩ වලට අදාළ පරිගණක මුද්‍රිත පිටපත් මේ සමග එවමි.\n02. අංක 02/2011 හා 2011.06.10 දිනැති ක්‍ෂේත්‍ර මණ්ඩල චක්‍රලේඛය හා ක්‍ෂේත්‍ර  මණ්ඩල චක්‍රලේඛ ලිපි අංක 12/2015 අනුව කටයුතු කර සම්පූර්ණ කරන ලද ගබඩා සමීක්ෂණ පෝරමයෙන් පිටපත් තුනකින් යුතුව ඊලග වසරේ ජනවාරි 15 දින හෝ ඊට පෙර අප වෙත එවන මෙන් කාරුණිකව ඉල්ලා සිටිමි.\n03. තවද චක්‍රලේඛවල සඳහන් කරුණු කෙරේ විශේෂ අවධානය යොමුකර කටයුතු කරන මෙන් ඉල්ලමි.\nමිනින්දෝරු අධිකාරී,\nජ්‍යෙෂ්ඨ මිනින්දෝරු අධිකාරී වෙනුවට, \nප්‍රසම්පාදන හා සැපයීම් අංශය."
    ,
      "අධ්‍යක්‍ෂ ,\nජාතික ආපදා  සහන සේවා මධ්‍යස්ථානය ,\nවිධ්‍යා මාවත,\nකොළඹ  07. \nනිවසාධාර වියදම් වාර්තා ඉදිරිපත් කිරීම - 2016 මැයි මස 31 \n මාතලේ දිස්ත්‍රික්කයේ 2014 හා 2015 වර්ෂ වල අපදාවෙන් හානි වූ නිවාස සඳහා නිවාසාධාර ගෙවීම්  කිරීමට වැය ශිර්ෂ 106-02-04-03-2104 (11) යටතේ එවා ඇති ප්‍රතිපාදන වල 2016 මැයි 31 දිනට   වියදම් වාර්තාව අදාළ ආකෘතියට අනුව පුරවා මේ සමග ඉදිරිපත් කරමි.\n  තේජානි තිලකරත්න, \n අතිරේක දිස්ත්‍රික් ලේකම් ,\n මාතලේ දිස්ත්‍රික් ලේකම් වෙනුවට ."
    ,
      "කළමනාකරණ අධ්‍යක්‍ෂ,\nමඩුල්කැලේ  ටී ඇන්ඩ් ඉකෝ ලොජ් හෝටලය,\nනුවර .\nපරිසර හිතකාමී හෝටලයක් හා කඩපිලක් ඉදිකිරිම සඳහා සුදුසු ඉඩම් කැබැල්ලක් ලබා ගැනීම .\nඋක්ත කරුණ සම්බන්ධයෙන් පරිසර හිතකාමී හෝටලයක් ඉදි කිරීම සඳහා සුදුසු ඉඩම් කැබැල්ලක් ඉල්ලා ඔබ විසින් මා වෙත යොමු කළ 2016.03.22 දිනැති ලිපිය හා බැදේ.\n02. එමගින් විසින් තම ව්‍යාපෘතිය පුළුල් කිරීමේ අරමුණින් පරිසර හිතකාමී හෝටලයක් ඉදිකිරිම සඳහා, දඹුල්ල, සිගිරිය හෝ ලග්ගල පල්ලේගම ප්‍රාදේශීය ලේකම් කොට්ඨාස වලට අයත් ප්‍රදේශයකින් අක්කර 25-30 අතර ප්‍රමාණයේ ඉඩම් කැබැල්ලක් ලබා දෙන ලෙසට ඉල්ලීමක් ඉදිරිපත් කර ඇත.\n03. ඒ අනුව ව්‍යාපෘති ක්‍රියාත්මක කරන ආයතනවල පවතින කඩිනම්ව ක්‍රියාත්මක වීමේ ශක්‍යතාවය අනුව ප්‍රමුඛතාවය දිය හැකි බැවින්, පරිසර හිතකාමී හෝටලයක් ආරම්භ කිරීම සඳහා සුදුසු ඉඩමක් ලබා දීම පිළිබඳව වියදම් කිරීමට අපේක්ෂිත මුදල සහ අදාල කාර්ය ඉටු කිරීමට අපේක්ෂිත කාල රාමුව ඇතුලත් ව්‍යාපෘති වාර්තාව 2016.05.31 වන දිනට පෙර මා වෙත ලබා දීමට කටයුතු කරන ලෙස කාරුණිකව ඉල්ලා සිටිමි.\nඅයි.ඩි.ජි.විජේතිලක,\nඅතිරේක දිස්ත්‍රික් ලේකම් (ඉඩම්),\nඅත්/කළේ නිල් ද අල්විස්. \nදිස්ත්‍රික් ලේකම් / දිසාපති ,\nමාතලේ.\nපිටපත්  -    01. ප්‍රාදේශීය ලේකම්,දඹුල්ල     -කාරුණිකව දැන ගැනීම සඳහා\n\t   02. ප්‍රාදේශීය ලේකම්, ලග්ගල ,පල්ලේගම."
    ,
      "ජ්‍යෙෂ්ඨ මිනින්දෝරු අධිකාරී,\nදිස්ත්‍රික් මිනින්දෝරු කාර්යාලය,\nමඩකලපුව.\nඅංක 61-6597 වාහනයේ ලිපිගොනුව\nසර්වේයර් ජනරාල්ගේ 2016.05.04 දිනැති ලිපිය අනුව 2016.05.24 දින ඔබ වෙත නිදහස් කරන ලද 61-6597 වාහනයට අදාල පහත සඳහන් ලිපි ගොනු මේ සමග එවන බව කාරුණිකව දන්වමි.\n01.2010.01.05 සිට 2013.09.03 දක්වා ලිපිගොනු 01යි.\n02.2014.05.10 සිට 2016.05.24 දක්වා ලිපිගොනු 01යි.\nලිපිගොනු ලැබුණු බව දන්වන්න.\nකේ.ආර්. සරත්,\nජ්‍යෙෂ්ඨ මිනින්දෝරු අධිකාරී,\nකොළඹ දිස්ත්‍රීක්කය."
    ,
      "මගේ අංකය :- ED/07/99/5/9/8\nදිනය:- 2014.02.13 දින\nනියෝජ්‍ය ප්‍රධාන ලේකම්,\nදකුණු පළාත් ඉංජිනේරු සේවා කාර්යාලය,\nකොටුව,\nගාල්ල.\n2014 වර්ෂයේ ගුරු විද්‍යාල සහ ගුරු මධ්‍යස්ථානවල කලයුතු අලුත්වැඩියා සහ නව ඉදිකිරීම් - උණවටුන අමරසුරිය ගුරු විද්‍යාලය.\nඋක්ත කරුණ සම්බන්ධයෙන් ප්‍රධාන කොමසාරිස් ( ගුරු අධ්‍යාපන ) මගින් මා වෙත යොමු කල  ED/03/20/04/05/01 හා 2014.02.07 දිනැති ලිපිය හා බැඳේ. (ඇමුණුම 1).\nඒ අනුව ඉහත ගුරු විද්‍යාලයේ පහත සඳහන් අවශ්‍යතා ලැයිස්තුවට අදාළව ඒ ඒ උපශීර්ෂ යටතේ විස්තරාත්මක ඇස්තමේන්තු සකස් කර හැකි ඉක්මනින් මා වෙත එවන මෙන් කාරුණිකව ඉල්ලා සිටිමි.\nඅනු අංකය       \nගොඩනැගිලි අලුත්වැඩියා      \nනව ඉදිකිරීම්   \nභෝජනහාරය අලුත්වැඩියාව\nවැසිකිලි පද්ධතියක් ඉදිකිරිම\nප්‍රධාන ගොඩනැගිල්ල (වලව්ව ) සහ ගැහැණු නේවාසිකාගාර අලුත්වැඩියාව\nජලනල පද්ධතිය සකස් කිරීම\nප්‍රධාන ශාලාව අලුත්වැඩියාව\nතෙමහල් ගොඩනැගිල්ලේ වහල අලුත්වැඩියාව\nඅයි.එම්. ප්‍රනාන්දු,\nඅධ්‍යක්ෂ පාසල් ගොඩනැගිලි ,\nඅධ්‍යාපන ලේකම් වෙනුවට.\nපිටපත් :-\n1. ප්‍රධාන කොමසාරිස් (ගුරු අධ්‍යාපන).\n2. විදුහල්පති, උණවටුන අමරසුරිය ගුරු විද්‍යාලය."
    ,
      "අනුරාධපුර දිස්ත්‍රික්කයේ ලියාපදිංචි භාෂා සංගම් ගැටලු පිළිබඳ විශේෂ විමර්ශන වාර්තාව. \nඅනුරාධපුර දිස්ත්‍රික්කයේ 2012 වර්ෂය සඳහා භාෂා සංගම් 36 ක් ලියාපදිංචි කර තිබිණි. \nඑම භාෂා සංගම් අතුරින් 19ක් ආරම්භකර ක්‍රියාත්මක කළ අතර සමහර භාෂා සංගම් අතරමැද දී නැවතී ගොස් ඇත. \nනියමිත පාඨමාලා කාලය ලෙස ගැනෙන පැය 150 අවසන් කරන ලද භාෂා සංගම් සහ ගැටලු පැන නැගී ඇති භාෂා සංගම් පිළිබඳ විශේෂ සාකච්ඡාවක් 2013.06.19 වන බදාදා පෙ.ව.9.00 අනුරාධපුර දිස්ත්‍රීක් ලේකම් කාර්යාලයේ පිහිටි මාගේ නිල කාමරයේදී පවත්වන ලද අතර, ඒ සඳහා සහභාගී වූ ගුරුවරුන්ගේ සහ නිලධාරීන්ගේ  අත්සන් ලේඛනයක් ද ලබා ගන්නා ලදී.\nවිශේෂයෙන් ම මෙහිදී සාකච්ඡාවට ලක්කළේ මෙතෙක් ලියාපදිංචි භාෂා සංගම්වල ගුරු උපදේශක දීමනා ලබා ගැනීම හා ගැටලු සහගත අවස්ථාවන් පිළිබඳවයි.\n එහිදී මවිසින් සොයා අවබෝධ කරගත් සියලු තොරතුරු ඔබගේ අවධානය පිණිස යොමු කරමි.\n පී.එන්.ටී.බී.ගුණරත්න \n ජාතික ඒකාබද්ධතා හා සංවර්ධන සහකාර"
    ,
      "වෙත :- ගණකාධිකාරි (ප්‍රසම්පාදන)\nවෙතින් :- අධ්‍යක්ෂ (පාසල් සැපයීම්)\nදිනය :- 2011/03/11\nමගේ අංකය :- ED/07/97/02/32(1)\n විෂය :- කොරියන් පාසල් උපකරණ පරිත්‍යාගය විද්‍යුත් පියනෝ රේගුවෙන් නිදහස් කර පත්තලගෙදර ගබඩාව වෙත ප්‍රවාහනය සඳහා සේවා ප්‍රසම්පාදනය.\nහයවන තොගය 470 පියානෝ.\nඉහත කරුණු සම්බන්ධයෙන් මාගේ අංක  ED/04/97/2/32(1) හා 2010.11.08 දිනැති ලිපිය හා බැඳේ.\n(පිටපත අමුණා ඇත)\n02. ඒ අනුව අවසන් පියානෝ 470 තොගය ඒවායේ ආධාරක සමඟ නැව්ගත කර ඇති බව දැනුම් දී ඇති අතර ඊට අදාළ නැව්ගත කිරීම් ලියවිලිවල ෆැක්ස් පිටපත් මේ වනවිටත් අප වෙත ලැබී ඇත.\nඑම  HL/HDMU ICWB 4287640 අංක දරණ බිල්පතෙහි සහ ඊට අදාල ඉන්වොයිසියේ හා ඇසුරුම් ලැයිස්තුවේ පිටපත් අවශ්‍ය කටයුතු සඳහා මේ සමග එවමි. \nඒ අනුව මෙම තොගය රේගුවෙන් නිදහස්කර පත්තලගෙදර ගබඩා සංකීර්ණය වෙත ප්‍රවාහනය කර භාරදීම සඳහා සේවා ප්‍රසම්පාදනය කරදෙන මෙන්  කාරුණිකව ඉල්ලා සිටිමි.\n03. තවද මීට අදාළ මුල් පිටපත් අප වෙත ලැබුණු වහාම ලබාදීමට නියමිත බවද කාරුණිකව සලකන්න.\n(මේ සම්බන්ධව වැඩිදුර තොරතුරු සඳහා කරුණාකර 0112784838 සංවර්ධන සහකාර උදේනි මංගලිකා මෙනවිය විමසන්න.)\nබී. ඩී. සී. බියන්විල,\nඅධ්‍යක්ෂ(සැපයීම්)\nපිටපත- 01. අතිරේක ලේකම් (ප්‍රසම්පාදන හා ඉදිකිරීම්) - කරු. දැ. ගැ. පි."
    ,
      "පී.එන්.ඩී.බී. ගුණරත්න,\nජාතික ඒකාබද්ධතා ප්‍රවර්ධන සහකාර,\nදිස්ත්‍රීක් ලේකම් කාර්යාලය,\nඅනුරාධපුරය.\n2013.06.25\nලේකම්,\nජාතික භාෂා හා සමාජ ඒකාබද්ධතා අමාත්‍යාංශය,\n40, බුත්ගමුව පාර, රාජගිරිය.\nඅනුරාධපුර දිස්ත්‍රීක්කයේ ලියාපදිංචි භාෂා සංගම් ගැටලු පිළිබඳ විශේෂ විමර්ශන වාර්තාව \nඅනුරාධපුර දිස්ත්‍රික්කයේ 2012 වර්ෂය සඳහා භාෂා සංගම් 36 ක් ලියාපදිංචි කර තිබිණි. \nඑම භාෂා සංගම් අතුරින් 19ක් ආරම්භකර ක්‍රියාත්මක කළ අතර සමහර භාෂා සංගම් අතරමැද දී නැවතී ගොස් ඇත. \nනියමිත පාඨමාලා කාලය ලෙස ගැනෙන පැය 150 අවසන් කරන ලද භාෂා සංගම් සහ ගැටලු පැන නැගී ඇති භාෂා සංගම් පිළිබඳ විශේෂ සාකච්ඡාවක් 2013.06.19 වන බදාදා පෙ.ව.9.00 අනුරාධපුර දිස්ත්‍රීක් ලේකම් කාර්යාලයේ පිහිටි මාගේ නිල කාමරයේදී පවත්වන ලද අතර, ඒ සඳහා සහභාගී වූ ගුරුවරුන්ගේ සහ නිලධාරීන්ගේ  අත්සන් ලේඛනයක් ද ලබා ගන්නා ලදී.\nවිශේෂයෙන් ම මෙහිදී සාකච්ඡාවට ලක් කළේ මෙතෙක් ලියාපදිංචි භාෂා සංගම්වල ගුරු උපදේශක දීමනා ලබා ගැනීම හා ගැටලු සහගත අවස්ථාවන් පිළිබඳවයි. \nඑහිදී මවිසින් සොයා අවබෝධ කරගත් සියලු තොරතුරු ඔබගේ අවධානය පිණිස යොමු කරමි.\nපී.එන්.ඩී.බී. ගුණරත්න,\nජාතික ඒකාබද්ධතා ප්‍රවර්ධන සහකාර,\nපිටපත :-\n1.ගරු වාසුදේව නානායක්කාර මැතිතුමා (ජාතික භාෂා සමාජ ඒකාබද්ධතා අමාත්‍ය) කා.දැ.ගැ.පි.\n2.ජ්‍යෙෂ්ඨ සහකාර ලේකම් (ජාතික භාෂා) කා.දැ.ගැ.පි.\n3. අතිරේක ලේකම් (ජාතික භාෂා) කා.දැ.ගැ.පි.\n4.අධ්‍යක්ෂ ජනරාල් (ජාතික භාෂා ) කා.දැ.ගැ.පි."
    ,
      "මගේ අංකය : ඊඩි/07/99//4/2/5\nදිනය : 2014.01.30\nනියෝජ්‍ය ප්‍රධාන ලේකම්,\nසබරගමු පළාත් ඉංජිනේරු සේවා කාර්යාලය,\nනව නගරය,\nරත්නපුර.\nප්‍රාථමික අංශය අසල ඇති අනතුරුදායක තාප්පය පිළිබඳව දැනුම් දීම - සුමන බාලිකා විද්‍යාලය\nඋක්ත කරුණ සම්බන්ධයෙන් විදුහල්පති විසින් මා වෙත යොමු කරන ලද SBV/1/අධ්‍යා.පරි./4 /ඉදිකිරිම හා අලුත්වැඩියා අංක දරණ 2014.01.23 දිනැති ලිපිය හා බැඳේ (පිටපතක් අමුණා ඇත).\nඒ අනුව, අනතුරුදායක තාප්පය ප්‍රතිසංස්කරණය කිරීම සඳහා වන ඇස්තමේන්තුව සකස්කර එවන මෙන් කාරුණිකව දන්වා සිටිමි.\nඊ.එම්.ප්‍රනාන්දු.\nපාසල් ගොඩනැගිලි අධ්‍යක්ෂ,\nඅධ්‍යාපන ලේකම් වෙනුවට.\nපිටපත් :-\n1) නියෝජ්‍ය අධ්‍යක්ෂ( පාසල් ගොඩනැගිලි ) - සබරගමුව පළාත් සභාව \n2) පළාත් අධ්‍යාපන අධ්‍යක්ෂ - සබරගමුව පළාත් අධ්‍යාපන දෙපාර්තමේන්තුව \n3) විදුහල්පති - සුමන බාලිකා විද්‍යාලය"
    ,
      "සභාපති,\nසුළු ප්‍රසම්පාදන කමිටුව,\nඅධ්‍යාපන අමාත්‍යාංශය. \nප්‍රසම්පාදන මාර්ගෝපදේශ 8:13:4 අනුව කොන්ත්‍රාත් මුදලින් 10% වැඩි කොන්ත්‍රාත් විචලන අනුමත කිරීම -  ගා/අමරසුරිය ගුරු විදුහල වලව්ව හා පිරිමි නේවාසිකාගාරය අලුත්වැඩියාව.\nඋක්ත කරුණ සම්බන්ධයෙන් අධ්‍යාපන ලේකම්ගේ අංක ඊඩි/07/101/08/TEC/2013 හා 2013-04-09 දිනැති ලිපියෙන් පත් කර තිබු ඇගයීම් කමිටුව විසින් දකුණු පළාත් අධ්‍යාපන අධ්‍යක්ෂ නිර්දේශ කර, එවා තිබු කොන්ත්‍රාත් විචලන ලේඛනය ඇගයීමට ලක් කර එම ඇගයීම් වාර්තාව ඔබ වෙත ඉදිරිපත් කිරීමට මා වෙත භාර දී ඇත.\nප්‍රසම්පාදන කමිටුවේ තීරණය ලබා ගැනීම සඳහා ඇගයීම් වාර්තාව මීට යා කර ඉදිරිපත් කරමි.\nඅයි.එම්. ප්‍රනාන්දු,\nපාසල් ගොඩනැගිලි අධ්‍යක්ෂ."
    ,
      "මගේ අංකය - CPC/CS/2/2/1 (II)\nදිනය - 2016.03.12\nඉතා වැදගත්.\nනියෝජ්‍ය ප්‍රධාන ලේකම් (සැලසුම්),\n2016 වර්ෂය තුළ බඳවා ගැනීමට යෝජිත තනතුරු සඳහා අවශ්‍ය ප්‍රතිපාදන වෙන් කර ගැනීම.\nඔබ ආයතනයේ අනුමත කාර්ය මණ්ඩලය තුළ දැනට පවත්නා පුරප්පාඩු හා ඉදිරියේදී ඇතිවන පුරප්පාඩු සඳහා බඳවා ගැනීමට යෝජිත තනතුරු තිබේ නම් 2016 වර්ෂය අවසන් වීමට ප්‍රථම බඳවා ගැනීම් සිදු කිරීමේ කටයුතු සිදුකළ යුතුව ඇත.\n02. ඒ අනුව අදාළ තනතුරු සඳහා බඳවා ගැනීම් සිදුකිරීමට අවශ්‍ය ප්‍රතිපාදන ඔබගේ වාර්ෂික ඇස්තමේන්තුවෙන් සලසා ගත යුතු අතර, එසේ සලසා ගෙන නොමැති නම් පමණක් අමතර ප්‍රතිපාදන ලබාගැනිම පිණිස ඉල්ලීම් කිරීම සඳහා පහතින් දක්වා ඇති ආකෘතිය ප්‍රකාරව සකස් කරන ලද තොරතුරු 2016.03.31 වෙනි දිනට ප්‍රථම එවීමට කටයුතු කරන ලෙස කාරුණිකව දන්වා සිටිමි.\nතනතුරු නාමය\nඅනුමත සංඛ්‍යාව\nසිටින සංඛ්‍යාව\nපුරප්පාඩු සංඛ්‍යාව\n2016 වර්ෂය තුළ බඳවා ගැනීමට අවශ්‍ය තනතුරු සංඛ්‍යාව\nඑකී තනතුරුවලට බඳවා ගැනීම සඳහා මසකට අවශ්‍ය ප්‍රතිපාදන රු.\nඑම්.එස්.ප්‍රේමවංශ,\nප්‍රධාන ලේකම්,\nමධ්‍යම පළාත."
    ,
      "මගේ අංකය - ඩී.එම්.යු./මාලේ/15/03/05\nදිනය - 2016.05.17\nඅධ්‍යක්ෂ ජනරාල්,\nආපදා කළමනාකරණ මධ්‍යස්ථානය,\nවිද්‍යා මාවත,\nකොළඹ 07.\nහදිසි ආපදා ප්‍රතිචාර දැක්වීම සඳහා මුදල් ප්‍රතිපාදන ලබා ගැනීම.\nමේ දිනවල පවතින අයහපත් කාලගුණික තත්වය හේතුවෙන් මාතලේ දිස්ත්‍රික්කයේ ප්‍රාදේශීය ලේකම් කොට්ඨාශ කිහිපයක ආපදා තත්වයන් මෙන්ම මාර්ග කිහිපයකටද පස් කඳු හා ගස් කඩා වැටීම හේතුවෙන් අවහිර වී ඇති බව  වාර්තා වී ඇති අතර ජාතික ගොඩනැගිලි පර්යේෂණ සංවිධානය විසින් නායයෑමේ අවදානම් දිස්ත්‍රික්කයක් ලෙසද නම් කර ඇත.\n02. ඒ අනුව පවතින අයහපත් කාලගුණික තත්වයේ වර්ධනයක් දක්නට ලැබෙන බැවින් හදිසි ප්‍රතිචාර දැක්වීමේ කටයුතු සඳහා රු. ලක්ෂ 05 ප්‍රතිපාදන මෙම දිස්ත්‍රික්කය වෙත ලබාදෙන ලෙස කාරුණිකව ඉල්ලා සිටිමි.\nතේජානි තිලකරත්න,\nඅතිරේක දිස්ත්‍රික් ලේකම්,\nමාතලේ දිස්ත්‍රික් ලේකම්/දිසාපති වෙනුවට."
    ,
      "මාගේ අංකය : CG/L/6/III වෙලුම\nමැනුම් ක්‍ෂේත්‍ර සහායක අංශය, \nමිනින්දෝරු දෙපාර්තමේන්තුව,\nකොළඹ - 05.\n2016.06.20 දින පත්වීම ස්ථීර කිරීම - ජේෂ්ඨ මිනින්දෝරු නිලධාරී (මඩකලපුව) මගින් උ.එම්. එස්.එම්. සාපි මහතා SA/M/1600)\nඔබගේ මැනුම් ක්‍ෂේත්‍ර සහායක සේවයේ III පන්තියේ තනතුරේ පත්වීම 2012.09.27 දින සිට ස්ථීර කරමි.\nඅත්/කළේ : පී.එම්.පී. උදයකාන්ත,\nසර්වේයර් ජනරාල්.\nඩී.ඒ.එස්.වී. වික්‍රමරත්න,\nසහකාර අධ්‍යක්ෂ (පාලන)\nපිටපත්\n01.පළාත් සර්වේයර් ජනරාල් (නැගෙනහිර පළාත) - කරු.දැ.ගැ.ස. හා අ.ක.ස.\n02. පෞද්ගලික ලිපි ගොනුවට"
    ,
      "2016.05.17\nප්‍රාදේශීය ලේකම් ,\nප්‍රාදේශීය ලේකම්  කාර්යාලය ,\nපල්ලේපොල.\nස්වයංක්‍රීය භාෂා පරිවර්තන මෘදුකාංගයක් සකස් කිරීම සඳහා අවශ්‍ය දත්ත එක්රැස් කිරීම.\nසිංහල දෙමළ භාෂා පරිවර්තකයන්ගේ දැඩි හිඟය රජයේ භාෂා පරිවර්තන කාර්යයන්හි දී දැඩි බාධකයක්ව පවතින බැවින් , ඒ සඳහා විසදුමක් වශයෙන් සිංහල - දෙමළ ස්වයංක්‍රීය භාෂා පරිවර්තන මෘදුකාංගයක් සකස් කොට මෙම දෙපාර්තමේන්තුවේ වෙබ් අඩවියට ඇතුළත් කිරීමට තීරණය කර ඇත. \n02. දැනට ගුගල් ආයතනය මඟින් සිංහල - දෙමළ භාෂා ද්විත්වය සඳහා පරිවර්තන පහසුකම් සලසා ඇති නමුත් දෙපාර්තමේන්තුව මඟින් සකස් කිරීමට යෝජිත මෙම මෘදුකාංගය රාජ්‍ය ලේඛන පදනම් කර ගනිමින්, භාෂා පරිවර්තකයින්ට ප්‍රයෝජනවත් වන පරිදි සකස් කර ඇත.\n03. මෙම මෘදුකාංගය මොරටුව විශ්ව විද්‍යාලය විසින් සකස් කරනු ලබන අතර ඒ සඳහා අවශ්‍ය දත්ත,එනම් සිංහල - දෙමළ ලේඛන දෙපාර්තමේන්තුව විසින් සැපයිය යුතු වේ. \nමෙහි ප්‍රථම අදියරේදී සරල ලිපි භාවිත කරන අතර සිංහල හා දෙමළ භාෂා වලින් සකස් කරන ලද ලිපි විශාල ප්‍රමාණයක් අවශ්‍ය වේ.\n04. එහෙයින් ඔබ කාර්යාලයේ සහ ඔබ දිස්ත්‍රික්කයේ , ප්‍රාදේශීය ලේකම් කාර්යාලවල නිතර නිතර භාවිතා කරන සිංහල හා දෙමළ භාෂා වලින් සකස් කෙරුණ ලිපි මෙම දෙපාර්තමේන්තුව වෙත ලබා දීමට කටයුතු කරන මෙන් කාරුණිකව ඉල්ලා සිටිමි. \nලිපිවල ලිපිනයන් අවශ්‍ය නොවන අතර අන්තර්ගතය පමණක් මේ සඳහා සැහේ. \nතවද මේවායේ මෘදු පිටපත් (Soft Copy) ලබා දීමට හැකි නම් අපගේ කාර්යය වඩාත් පහසු වනු ඇත.\n05. මෙය ජාතික අවශ්‍යතාවක් බැවින් මෙම කාර්යය කඩිනමින් ඉටු කර ගැනීම සඳහා ඔබ දක්වන සහයෝගය ඉතාමත් අගය කොට සළකමි.\nනීතිඥ ඩබ්ලිව්.ඒ.ජයවික්‍රම, \nරාජ්‍ය භාෂා කොමසාරිස්."
    ,
      "දිස්ත්‍රික් ලේකම්/දිස්ත්‍රික් දිවිනැගුම අධ්‍යක්ෂ,\nදිස්ත්‍රික් ලේකම් කාර්යාලය,\nමඩකලපුව.\n''වස විස නැති රටක්'' වැඩසටහන පිළිබඳ සැලසුම එළිදැක්වීම  හා ප්‍රදර්ශනය බණ්ඩාරනායක ජාත්‍යයන්තර සම්මන්ත්‍රණ ශාලා පරිශ්‍රය - 2016 මාර්තු 06,07 සහ 08.\nඋක්ත වැඩසටහන සම්බන්ධ දිවිනැගුම අධ්‍යක්ෂ ජනරාල්ගේ අංක DDD/HO/ADG හා 2016.02.29 දිනැති උපදෙස් මාලාව හා බැඳේ.\n02. ඒ අනුව අලෙවි අංශයෙන් මා වෙත එවා ඇති පරිදි අදාළ වියදම් දැරීම සඳහා පහත සඳහන් පරිදි ප්‍රතිපාදන වෙන් කරනු ලැබේ.\nවැය විෂය  වැය විස්තරය\nමුදල රු. 331-2-2-3-2502   ජීවනෝපාය සංවර්ධන  93,850.00\n03. මෙම ප්‍රතිපාදන අදාළ කටයුතු සඳහා යොදා ගන්නා ලෙස දන්වන අතර ලැබී ඇති මුළු ප්‍රතිපාදන සීමාව ඉක්මවා වියදම් නොකීරීමටත් අදාළ මුදල් වියදම් කිරීමේ දී මුදල් රෙගුලාසි හා අනෙකුත් නීති රීති වලට අනුකූලව වියදම් දරණ බවටත් ඔබ විසින් සහතික විය යුතුය.\n04. මෙම වියදම් දැරීම සඳහා අවශ්‍ය අක්මුදල් දිවිනැගුම දිස්ත්‍රික් අතිරේක අධ්‍යක්ෂ ජනරාල් ගේ නිළ ගිණුම වෙත කාලීනව ඉඳුරාම ප්‍රේශණය කරන අතර ඒ පිළිබඳව ඔබව දැනුවත් කරනු ලැබේ.\nඒ අනුව සෑම මසක් අවසන් වී දින 03 ක් ඇතුළත පසුගිය මාසයේ වියදම් සාරාංශය පිළියෙල කර ප්‍රධාන ගණකාධිකාරී, දිවිනැගුම සංවර්ධන දෙපාර්තමේන්තුව, 4 වන මහල, ''සෙත්සිරිපාය'', බත්තරමුල්ල ලිපිනයට ලබා දීමට කටයුතු කළ යුතු වේ.  \nකේ.කේ.එල්.චන්ද්‍රතිලක,\nගණකාධිකාරී(සහනාධාර හා වත්කම්)."
    ,
      "සියළුම ආයතන ප්‍රධානීන් වෙත,\nමධ්‍යම පළාත් සභාවේ සේවක සංඛ්‍යාව 2016.03.11 දිනට යාවත්කාලීන කිරීම.\nඔබ ආයතනයේ කාර්ය මණ්ඩලයේ, 2016.03.31 දිනට යාවත්කාලීන කරන ලද කාර්ය මණ්ඩල තොරතුරු ලබා ගැනීම අත්‍යවශ්‍යව පවතී.\n02. ඒ අනුව පසුපිටෙහි සඳහන් ආකෘතිපත්‍රානුකූලව තත්‍ය සේවක සංඛ්‍යාව ශ්‍රේණි වශයෙන් සඳහන් කර තොරතුරු ලබාදීමට ඔබගේ විශේෂ අවධානය යොමු කර, සම්පූර්ණ කරන ලද තොරතුරු 2016.04.24 වෙනි දිනට පෙර ලබා දීමට අවශ්‍ය කටයුතු කරන ලෙස කාරුණිකව දැනුම් දෙමි.\n04. මෙම කාර්තව්‍ය  ඉතා වැදගත් බැවින් නියමිත දිනට තොරතුරු ලබාදී ඔබ දක්වන සහයෝගය පිළිබඳව අතිශයින් කෘතඥ වෙමි.\nඑම්.එස්.ප්‍රේමවංශ\nප්‍රධාන ලේකම්\nමධ්‍යම පළාත."
    ,
      "ප්‍රධාන ලේකම්,\nමධ්‍යම පළාත.\n2016 වර්ෂය තුළ බඳවා ගැනීමට යෝග්‍ය තනතුරු සඳහා අවශ්‍ය ප්‍රතිපාදන වෙන් කරවා ගැනීම.\nඉහත කරුණට අදාළව ඔබගේ අංක CPC/CS/2/2/1 (ii) හා 2016.03.17 දිනැති ලිපිය සම්බන්ධයෙනි.\n2. මෙම අංශයේ දැනට පවතින පුරප්පාඩු පිළිබඳ ආකෘතියට අනුව සකස් කර මේ සමඟ එවන බව කාරුණිකව දන්වා සිටිමි.\nතනතුරු නාමය.\nඅනුමත සංඛ්‍යාව.\nසිටින සංඛ්‍යාව.\nපුරප්පාඩු සංඛ්‍යාව.\n2016 වර්ෂය තුළ බඳවා ගැනීමට අවශ්‍ය තනතුරු සංඛ්‍යාව.\n ඒ.ඒ.වීරකෝන්,\nඅධ්‍යක්ෂ (ක්‍රමසම්පාදන)"
    ,
      "පළාත් අධ්‍යාපන අධ්‍යක්ෂ,\nශ්‍රී ලංකා රජයේ අධ්‍යාපන අරමුණු ප්‍රධාන වශයෙන් ඉගැන්වීමේ ගුණාත්මකභාවය ඉහළ නැංවීම, පාසල්වල යටිතල පහසුකම් වැඩිදියුණු කිරීම සහ විෂයමාලා සංවර්ධනය යන කරුණු මත පදනම් වී ඇත. \nමෙම අරමුණු සාක්ෂාත් කර ගැනීම සඳහා මූලික අඩිතාලම ලෙස ගුණාත්මක ප්‍රාථමික අධ්‍යාපනයක් ක්‍රියාත්මක කළ යුතු අතර ප්‍රාථමික සිසුන්ගේ ආරක්ෂාව සැලසීම ප්‍රමුඛ කාර්යයක් වේ. \nමේ සඳහා ප්‍රාථමික පාසල් ජාලය තුළ ආරක්ෂිත පරිසරයක් නිර්මාණය කිරීම විශේෂ අවශ්‍යතාවකි.\nඒ අනුව අධ්‍යාපන අමාත්‍යාංශය විසින්  පළමු අදියර ලෙස දිවයිනේ ප්‍රාථමික අංශය පමණක් සහිත පාසල්වල (type 3) සිසුන්ගේ ආරක්ෂාව තහවුරු කිරීමට අත්‍යවශ්‍ය මූලික අවශ්‍යතා සම්පූර්ණ කිරීම ප්‍රමුඛ කාර්යයක් ලෙස හඳුනාගෙන ඇත. \nඒ සඳහා එම පාසල්වල සිදු කළ යුතු සංවර්ධන කාර්යයන් සොයා බැලිය යුතු වේ. \nමේ යටතේ ක්‍රියාත්මක කළ යුතු සංවර්ධන කාර්යයන් කිහිපයක් පහත දක්වා ඇත.\nඅබලන් ගොඩනැගිලි ප්‍රතිසංස්කරණය.\nඉවත් කළ යුතු ගොඩනැගිලි කඩා ඉවත් කිරීම.\nඅක්‍රමවත්ව පවතින විදුලිබල සැපයුම් විධිමත් පරිදි සකස් කිරීම.\nඅවදානම් සහිත භාවිතයට නොගන්නා ළිං ආදිය වසා දැමීම.\nඅවදානම් සහිත පැරණි වැසිකිළි වලවල් වැසීම.\nපාසල් භූමියේ පවතින අවදානම් සහිත බෑවුම් සඳහා පැති බැමි/වැටවල් සකස් කිරීම."
    ,
      "පළාත් අධ්‍යාපන අධ්‍යක්ෂ, \nපළාත් අධ්‍යාපන දෙපාර්තමේන්තුව,\nත්‍රිකුණාමලය.\nපාසල් අභ්‍යන්තර ඇගයීම් පිලිබඳ නියාමන වැඩසටහන.\nඅධ්‍යාපන අමාත්‍යාංශයෙහි කළමනාකරණ හා ප්‍රමිති ශාඛාව මගින් 2015 වර්ෂයේ සිට හඳුන්වා දෙන ලද නව අභ්‍යන්තර ඇගයීම් ක්‍රමවේදය 2015 ජනවාරි මස සිට සියලු පාසල්වල ක්‍රියාත්මක කරන ලෙස උපදෙස් ලබා දී ඇති අතර පාසල්වල අභ්‍යන්තර ඇගයීම් ක්‍රියාත්මක කිරීමේ ප්‍රගතිය නියාමනය සඳහා කලාප මට්ටමෙන් වැඩසහටනක් පැවැත්වීමට අධ්‍යාපන අමාත්‍යාංශය සැලසුම් කර ඇත.\nඒ සඳහා ඔබ පළාතේ මඩකලපුව කලාපය තෝරාගෙන ඇති අතර 2016.07.01 වන දින පෙ.ව. 8.30 සිට මෙම වැඩසටහන පැවත්වීමට සැලසුම් කර ඇත.\n කලාප කළමනාකරණය හා ප්‍රමිති සම්බන්ධීකරණ නිලධාරී මගින් මෙම වැඩසටහන සංවිධානය කිරීමට කටයුතු කරන මෙන් කාරුණිකව ඉල්ලා සිටින අතර ඒ සම්බන්ධ පහත සඳහන් කරුණු ද ඔබගේ අවධානය සඳහා යොමු කරමි.\nඑදිනට අදාළ කලාපයේ සියලුම පාසල්වල විදුහල්පතිවරුන් සුදුසු ස්ථානයකට කැඳවීම.\nපාසල් අභ්‍යන්තර ඇගයීම් සම්බන්ධ ප්‍රධාන ගොනුව සහ ඇගයීම් ක්‍ෂේත්‍ර 08 ට අදාළ අභ්‍යන්තර ඇගයීම් ගොනු ද රැගෙන එන ලෙස දැනුවත් කිරීම.\nඅභ්‍යන්තර  ඇගයීම් හොඳින් ක්‍රියාත්මක කරන පාසලක් තෝරගෙන එම පාසල පිලිබඳ මිනිත්තු 15ක පමණ ඉදිරිපත් කිරීමක් (Power Point) කරන ලෙස එම පාසලෙහි විදුහල්පතිවරයාව දැනුවත් කිරීම.\nඉහත කරුණු පිලිබඳ ඔබගේ අවධානය යොමු කොට අවශ්‍ය කටයුතු සිදු කරන මෙන් කාරුණිකව ඉල්ලා සිටිමි.\nඑස්.යු. විජේරත්න,\nඅතිරේක ලේකම් (සැලසුම්, ප්‍රතිපත්ති හා කාර්යසාධන සමාලෝචනය),\nඅධ්‍යාපන ලේකම් වෙනුවට.\nපිටපත - කලාප අධ්‍යාපන අධ්‍යක්ෂ , මඩකලපුව කලාපය - දැන ගැනීම සඳහා හා සහභාගී  වීම සඳහා.\nපළාත් කලමනාකරණ හා ප්‍රමිති සම්බන්ධීකරණ නිලධාරී -  අවශ්‍ය කටයුතු සඳහා හා සහභාගී වීම සඳහා.\nකලාප කලමනාකරණ හා ප්‍රමිති සම්බන්ධීකරණ නිලධාරී -  අවශ්‍ය කටයුතු සඳහා හා සහභාගී වීම සඳහා."
    ,
      "මගේ අංකය - 1/14/3/1-III\nදිස්ත්‍රික් ලේකම් කාර්යාලය ,\nමාතලේ.\n2016.02.\nප්‍රාදේශීය ලේකම්,\nප්‍රාදේශීය ලේකම් කාර්යාලය,\nගලේවෙල. \nඉඩම සඳහා බලපත්‍රයක් ලබා ගැනීම සම්බන්ධවයි. \nඋක්ත කරුණට අදාලව ඔබ වෙත යොමු කළ මගේ සමාංක හා 2016.01.06 දිනැති ලිපිය සම්බන්ධයෙනි. \n02. ඒ පිළිබදව මෙතෙක් පිළිතුරු ලැබී නොමැත. \nඑම නිසා 2016.02.29 වන දිනට පෙර ඒ සඳහා පිළිතුරක් එවන ලෙස කාරුණිකව දන්වමි.\nමාතලේ දිස්ත්‍රික් ලේකම් වෙනුවට."
    ,
      "ප්‍රාදේශීය ලේකම්,\nප්‍රාදේශීය ලේකම් කාර්යාලය,\nවිල්ගමුව.\nආපදා කළමනාකරණ පිළිබඳ ග්‍රාමීය කමිටු දැනුවත් කිරීමේ වැඩසටහන. \nමාතලේ දිස්ත්‍රික් ආපදා කළමනාකරණ මධ්‍යස්ථානය මගින් ග්‍රාමීය ආපදා කළමනාකරණ කටයුතු විධිමත් කිරීමේ අරමුණින් ඔබ ප්‍රාදේශීය ලේකම් කොට්ඨාශයේ මල්ගම්මන ග්‍රාම නිළධාරි වසම සඳහා ආපදා කළමනාකරණ පිළිබඳ ග්‍රාමීය කමිටු දැනුවත් කිරීමේ වැඩසටහනක් 2016.05.17 දින පැවැත්වීමට අදහස් කර ඇත.\n02. එබැවින් හදිසි ආපදා අවස්ථාවන්හිදි ජීවිත ආරක්ෂා කර ගැනීම සඳහා ආපදා දැනුවත් කිරීමේ වැඩසටහනක් පැවැත්වීම වැදගත් වන බැවින් පහත සඳහන් නිළධාරීන්ගේ සහය ඇතිව ඉහත වැඩ සටහන පැවැත්වීම සඳහා අවශ්‍ය කටයුතු සංවිධානය කර දෙන ලෙස කාරුණිකව දන්වා සිටිමි.\n01. ආපදා කළමණාකරණ නිළධාරි\n02. සමාජ සේවා නිළධාරි\n03. ග්‍රාම නිළධාරි\n04. වනිතා සංවර්ධන නිළධාරි\n05. ළමා සංවර්ධන නිළධාරි\n06. තරුණ සේවා නිළධාරි\nතේජානි තිලකරත්න,\nඅතිරේක දිස්ත්‍රික් ලේකම්,\nමාතලේ දිස්ත්‍රික් ලේකම් වෙනුවට."
    ,
      "මාගේ අංක 4/1/18-11\nහිමිකම් ලියාපදිංචි කිරීමේ මැනුම් අංශය.\n2016.01.04\nසියලුම පළාත් සර්වේයර් ජනරාල් වරුන් සහ සියලුම ජ්‍යෙෂ්ඨ මිනින්දෝරු අධිකාරී වරුන් වෙත.\nහිමිකම් ලියාපදිංචි කිරීම සඳහා ලියාපදිංචි බලයලත් මිනින්දෝරුවරුන් මගින් මැනුම් සිදු කර කැඩැස්තර සිතියම් පිළියෙල කිරීම - 2016 වසර සඳහා.\n2016 වර්ෂයට බිම්සවිය වැඩසටහන යටතේ තෝරාගත් ප්‍රාදේශීය ලේකම් කොට්ඨාශවල කැඩැස්තර මැනුම් කිරීම සඳහා මිල ගණන් ඉදිරිපත් කරන ලෙස දිවයිනේ සියලුම බලයලත් මිනින්දෝරුවන් වෙත මේ වන විට දැනුවත් කර ඇත.\n02. මේ සම්බන්ධ විස්තර 2016.01.16 දින පුවත් පත් මඟින් ප්‍රකාශයට පත්කර ඇත.\nඑහිදී අවශ්‍ය විස්තර හා ලිපි ලේඛණ www.survey.gov.lk දෙපාර්තමේන්තු වෙබ් අඩවිය තුලින් ලබා ගත හැක. \nඒ බව එම පුවත් පත් දැන්වීමේ ද දන්වා ඇත.\n03. එහිදී එක අයදුම්කරුවෙකුට එක ප්‍රාදේශීය ලේකම් කොට්ඨාශයක් සඳහා වන අයදුම්පත් ගාස්තුව ලෙස රු.250/= ක මුදලක් ලඟම ඇති දිස්ත්‍රික් හෝ පළාත් මැනුම් කාර්යාල වෙත ගෙවා ලබා ගන්න ලදුපතෙහි පිටපතක් ලංසුපතට අමුණා එවිය යුතු බව දන්වා ඇත.\n(එක් අයදුම්කරුවෙකුට ප්‍රාදේශීය ලේකම් කොට්ඨාශ කිහිපයක් සඳහා වුවද රු. 250/= බැගින් ගෙවා අයදුම් කල හැක.). \nඅයදුම් පත් භාර ගන්න අවසන් දිනය 2016.02.08 දින වන බැවින් එදිනට පසුව ලැබෙන මුදල් භාර ගැනීමෙන් වලකින ලෙසත්, එදින දක්වා මුදල් ගෙවූ ලියාපදිංචි බලයලත් මිනින්දොරුවන්ගේ නම්, අයදුම් කරන ප්‍රාදේශීය ලේකම් කොට්ඨාශය, මුදල් ගෙවූ දිනය, ලදුපත් අංකය හා මුදල සඳහන් ලේඛනයක් 2016.02.09  වන දින වන විට මා වෙත එවීමට කටයුතු කරන ලෙසද කාරුණිකව දන්වා සිටිමි.\nඩබ්.ඒ.එස්.වික්‍රමආරච්චි,\nඅතිරේක සර්වේයර් ජනරාල්(හි.ලි.),\nසර්වේයර් ජනරාල් වෙනුවට. \nපිටපත : අධ්‍යක්ෂ මුදල් - කාරුණික දැන ගැනීම සහ අවශ්‍ය කටයුතු සඳහා."
    ,
      "මගේ අංකය : සී.පී.සී/කේ.එම්.සී/නා.ලේ/06/විග.කළ.ක\nනාගරික ලේකම් දෙපාර්තමේන්තුව,\nමහ නගර සභාව,\nමහනුවර.\n2016.06.\nවිගණන කළමණාකරණ කමිටු රැස්වීම.\nමහනුවර මහ නගර සභාවේ විගණන කළමණාකරණ කමිටු රැස්වීම ගරු නගරාධිපතිතුමාගේ අණ පරිදි 2016-06-15 වන දින ප.ව.3.00 ට මහනුවර නගර ශාලාවේදී පැවැත්වෙන බවත්, එම රැස්වීමට සහභාගි වන ලෙසත් මෙයින් කාරුණිකව දන්වා සිටිමි.\n(ඒ.එච්.එම්.ඉන්දිකා කුමාරි අබේසිංහ)\nනාගරික ලේකම්.\nන්‍යාය පත්‍රය.\n01. එක් එක් දෙපාර්තමේන්තු වලට අදාළව මෙතෙක් ලැබී ඇති විගණන වාර්තා පිළිබඳ ප්‍රගතිය සාකච්ඡා කිරීම.\n02. අභ්‍යන්තර විගණන වාර්තා වලට අදාළ ප්‍රගතිය සාකච්ඡා කිරීම.\n03. වෙනත් කරුණු."
    ,
      "පළාත් අධ්‍යාපන අධ්‍යක්ෂ, \nදකුණු පළාත.\nප්‍රසම්පාදන මාර්ගෝපදේශ 8:13:4 අනුව කොන්ත්‍රාත් මුදලින් 15% වැඩි කොන්ත්‍රාත් විචලන අනුමත කිරීම \nඔබගේ අංක දප/ප්‍රස/ඉඳි/03/අවැබාවැ/ හා 2012.12.20 දාතම ලිපිය සමග නිර්දේශ කර ඒවා තිබු පහත සඳහන් වැඩයට අදාළ කොන්ත්‍රාත් විචලනය වගුවේ දැක්වෙන පරිදි අමාත්‍යංශයේ සුළු ප්‍රසම්පාදන කමිටුව විසින් අනුමත කර ඇත.\nඒ අනුව මේ සම්බන්ධයෙන් ඉදිරි කටයුතු කරන්නේ නම් මැනවි.\nඅයි.එම්. ප්‍රනාන්දු,\nපාසල් ගොඩනැගිලි අධ්‍යක්ෂ,\nඅධ්‍යාපන ලේකම් වෙනුවට. \nපිටපත් :\n1. ප්‍රධාන ඉංජිනේරු (පා.ගො) දකුණු පළාත\n2. දිස්ත්‍රික් ඉංජිනේරු (ගොඩනැගිලි)  ගාල්ල"
    ,
      "අධ්‍යාපන අමාත්‍යාංශය, \nඉසුරුපාය, බත්තරමුල්ල.\n2013.11.13\n අනුමත සේවාවන්.\n01. ඔබගේ අංක .............. හා.......... දින දරන ලිපියෙන් ඉදිරිපත් කරන ලද මහ/ ධර්මරාජ වි. ප්‍රධාන ගොඩනැගිල්ල ඉදිරිපිට කොන්ක්‍රීට් බැම්ම ඉදිකිරිම සඳහා දැරීමට අවශ්‍ය වියදම් ඇස්තමේන්තුව අඩංගු සේවාවන් ඉටු කිරීම සඳහා වියදම් දැරීමට අදාල අනුඥාවේ නිවේදනය සහිත අනුමත ඇස්තමේන්තුව මේ සමග එවමි.\n02. මෙම වැඩයට අදාළ ඇස්තමේන්තු අංකය : පාගො 4/2013/229 වේ. \nමෙම කාර්යයට සම්බන්ධව මින් ඉදිරියට කරනු ලබන ලිපි ගනුදෙනුවලදී මෙම ඇස්තමේන්තු අංකය යෙදීමට සටහන් කර ගන්න.\n03. අවශ්‍ය ප්‍රතිපාදන ග/විගි/මුදල් විසින් යථා කාලයේදී ඔබ වෙත වෙන් කර දෙනු ඇත.\nප්‍රතිපාදන වෙන් කර දෙන තෙක් වැඩය ආරම්භ කිරීම කල් නොදැමිය යුතුය. \nවහාම වැඩය ආරම්භ කිරීමට කටයුතු කරන්න.\n04. යම්හෙයකින් කොන්ත්‍රාත්තුව යටතේ අමතර හා බාහිර වැඩ ඉටු කිරීමට සිදු වුවහොත් එම වැඩ රජයේ ප්‍රසම්පාදන මාර්ගෝපදේශ සංග්‍රහය 2006(ද්‍රව්‍ය හා වැඩ ) 813 ජේදයේ (විචලන නියෝග වැඩ) උපදෙස් අනුව කටයුතු කර අදාළ අනුමැතිය ලබා ගත යුතුය.\n05.මෙම සේවාව නිම කිරීම සඳහා අවශ්‍ය යයි හැගෙන තාත්වික හා සාධාරණ කාලසීමාවක් ලබා දී නියමිත කාලසීමාව තුළ වැඩය නිම නොකරන කොන්ත්‍රාත්තුකරුවන්ගේ අලාභය අයකර ගැනීමට කටයුතු කළයුතුය.\n06. ගිවිසුම අනුව කොන්ත්‍රාත් අත්සන් කිරීමෙන් අනතුරුව නියම කරනු ලබන කාලසීමාව ඇතුළත වැඩ ආරම්භ කිරීමට කොන්ත්‍රාත්කරු ක්‍රියා කරන්නේ දැයි වගබලා ගත යුතුවේ.\n08. මෙම වැඩය පිළිබඳව ඔබගේ ප්‍රගති වාර්තාවල මෙම ලිපිය ලද මාසයේ සිටම සඳහන් කළයුතුවේ.\nඅයි.එම්.ප්‍රනාන්දු,\nපාසල් ගොඩනැගිලි අධ්‍යක්ෂ,\nඅධ්‍යාපන ලේකම් වෙනුවට.\nපිටපත් :\n1.ප්‍රධාන ඉංජිනේරු (පා.ගො)  මධ්‍යම පළාත. \n2. විදුහල්පති  මහ/ ධර්මරාජ වි.  කරුණාකර මෙම වැඩ ආරම්භ කරන දිනය සහ අවසන් කරන දිනය වාර්තා කිරීම සඳහා."
    ,
      "භාෂා/7/17/ත්‍රි/පොදු/01\nප්‍රාදේශීය ලේකම් මගින්, \nජාතික ඒකාබද්ධතා ප්‍රවර්ධන සහයක,\nප්‍රාදේශීය ලේකම් කාර්යාලය,\nකැකිරාව.\nභාෂා සංගම් ජාතික ව්‍යාපාරයේ ගැටලු පිළිබඳ සාකච්ඡාව.\nඋක්ත කරුණ සම්බන්ධයෙන් 2013.06.13 දින පවතී සාකච්ඡාව හා බැඳේ.\n2. මෙහිදී මතුවූ කරුණු අනුව කැකිරාව ප්‍රාදේශීය ලේකම් කොට්ඨාශයට අදාළ භාෂා සංගම් සම්බන්ධ කටයුතු ගැටලු සහගත් බවත් ඊට අදාළ විස්තර සම්පූර්ණ නැති බවත් පෙනී යයි.\n මේ හේතුවෙන් කැකිරාව ප්‍රාදේශීය ලේකම් කොට්ඨාශයට අදාළ භාෂා පන්ති පිළිබදව එකින් එක විමසා බලා සවිස්තර වාර්තාවක් අවශ්‍ය බව අමාත්‍යංශ ලේකම්තුමිය දන්වන ලදී.\n3. ඒ අනුව ඉහත තොරතුරු සියල්ල සමග 2013.06.27 දින පෙ.ව. 09.30 ට අමාත්‍යංශයට පැමිණෙන  මෙන් කාරුණිකව දැන්වා සිටිමි.\n4.මෙම සාකච්ඡාව සදහා ඔබගේ පැමිණීම අනිවාර්ය බව කාරුණිකව සලකන්න.\nඑච්. හේවගේ,\nසහකාර ලේකම් (ජාතික භාෂා),\nජාතික භාෂා හා සමාජ ඒකාබද්ධතා අමාත්‍යාංශයේ ලේකම් වෙනුවට."
    ,
      "රාජ්‍ය භාෂා කොමසාරිස්,\nරාජ්‍ය භාෂා දෙපාර්තමේන්තුව,\nභාෂා මන්දිරය,\n341/7,කෝට්ටේ පාර,\nරාජගිරිය.\nරාජ්‍ය ආයතනවලින් නිකුත් කරන ලිපි ලේඛනවල පවතින අක්ෂර වින්‍යාසය හා ව්‍යාකරණ දෝෂ පිළිබඳ අධ්‍යයනය.\nඋක්ත හිසින් යුත් ඔබගේ අංක DOL/REU/09/03 HAA 2014.07.04 දිනැති ලිපිය හා බැඳේ.\nඑම කාර්යය කාලෝචිත ක්‍රියාවක් වන බැවින් අධ්‍යපන අමාත්‍යාංශය ලිපි ලේඛන පරීක්ෂා කිරීම සඳහා 2014.08.11 හා 12 දෙදින පෙ.ව 9.00 ටආධ්‍යපන අමාත්‍යාංශය වෙත පැමිණීමට වේලාව වෙන් කළ හැකි බව කාරුණිකව දන්වමි.\nආර්.එම්.එම්. රත්නායක,\nඅධ්‍යාපන අධ්‍යක්ෂ,\nජාතික භෂා හා මානව ශාස්ත්‍ර. \nඑච්.යූ ප්‍රේමතිලක.\nඅතිරේක ලේකම්(අධ්‍යාපන ගුණාත්මක සංවර්ධනය) ලේකම් වෙනුවට."
    ,
      "මගේ අංකය : ED/07/99/3/6/1/6\nදිනය : 2014.04.02 දින\nගොඩනැගිලි අධ්‍යක්ෂ මගින්,\nදිස්ත්‍රික් ඉංජිනේරු,\nමාතර.\nඋසස් පෙළ විද්‍යාගාරයේ වහලය අලුත්වැඩියාව - මාර/රාහුල විද්‍යාලය \nඋක්ත කරුණට සම්බන්ධයෙන් විදුහල්පතිගේ 2014.03.07 දිනැති ලිපිය හා බැඳේ .\n(පිටපතක් අමුණා ඇත ).\nවිද්‍යාලයේ උසස්පෙළ විද්‍යාගාරයේ වහලය අලුත්වැඩියා කිරීම සඳහා කඩිනමින් ඇස්තමේන්තු සකස් කර එවන මෙන් කාරුණිකව දන්වමි.\nඅයි.එම්. ප්‍රනාන්දු,\nපාසල් ගොඩනැගිලි අධ්‍යක්ෂ,\nඅධ්‍යාපන ලේකම් වෙනුවට.\nපිටපත් :-\n1. දකුණු පළාත් අධ්‍යාපන අධ්‍යක්ෂ, දකුණු පළාත් අධ්‍යාපන දෙපාර්තමේන්තුව.\n2. දිස්ත්‍රික් ඉංජිනේරු (ගොඩනැගිලි), මාතර - අතිරේක පිටපතකි.\n3. විදුහල්පති, මාර/රාහුල විද්‍යාලය."
    ,
      "උපකරණ අංශය,\nමිනින්දෝරු දෙපාර්තමේන්තුව,\nකොළඹ 05.\n2016.02.19\nඅධ්‍යක්ෂ,\nමැනුම් හා සිතියම් ගතකිරීමේ ආයතනය.\nකොට්ඨාශ භාර ජ්‍යෙෂ්ඨ මිනින්දෝරු අධිකාරීන් 2016 වර්ෂයේ පූර්ණමාන සේවා කිරීම උපකරණ අංශය - සර්වේයර් ජනරාල් කාර්යාලය. \n2016 වර්ෂයේ පූර්ණමාන සේවා කිරීම සදහා යෝජිත දින දැක්වෙන ලේඛනය මේ සමග එවමි.\nඑහි අංක සඳහන් පූර්ණමාන කුමන කාර්යාලයකට අයත් වුවත් අදාළ වකවානුවේදී ඒවා භාරව සිටින නිලධාරින් උපකරණය සේවා කටයුතු සඳහා උපකරණ අංශය වෙත එවීම ඔවුන්ගේ වගකීම බව ඔවුන් වෙත දන්වන්න. \nවිද්‍යුත් තැපෑල මගින්ද ඔවුන් වෙත මෙම විස්තර සැපයීමට කටයුතු කර ඇති බවද කාරුණිකව දන්වා සිටිමි.\n02. අළුතින් ක්ෂේත්‍රයට නිකුත් කරන ලද OX 108 වර්ගයේ පූර්ණමාන කුමන රාජකාරී කටයුතු තිබුන __________________ සිට 2016.06.30 දින අතරතුරදී උපකරණ අංශය වෙත ඒවා සේවා කරගැනීමට අනිවාර්යයෙන්ම කටයුතු කලයුතු බව අදාළ නිලධාරීන් වෙත දන්වමි.\n03. 2016 වර්ෂයේදී සේවා කටයුතු සඳහා උපකරණ අංශය වෙත එවා ඇති උපකරණ (* ලකුණ යොදා ඇති )  දින රහිතව සඳහන් කර ඇති අතර, SGO යන්නෙන් සඳහන් කර ඇති උපකරණ සම්බන්ධයෙන් තවදුරටත් පරීක්ෂාකර බලා  ක්ෂේත්‍රයට නිකුත් කළ හැකිද යන්න තීරණය කිරීමෙන් අනතුරුව සේවා කිරීම සඳහා දිනයක් ලබාදීමට කටයුතු කරනු ඇත.\n04. උපකරණ සේවා කිරීම සඳහා එවනවිට සෑම උපකරණයක් සම්බන්ධයෙන්ම (උපාංග නොවේ)  ඒ සඳහා ඇති ආකෘති පත්‍රයේ අදාළ විස්තර සඳහන් කර එවිය යුතු අතර උපකරණ/ උපාංග, ආපසු රැගෙන යාමේදී මෙම අංශයෙන් නිකුත් කර ඇති භාරගැනීමේ ලදුපතෙහි විස්තර පුරවා එවීමෙන් උපකරණය නැවත ලබාගත හැකි බැවින් වෙනත් අමතර ලිපි ලිවීම අවශ්‍ය නොවේ.\n05. පූර්ණමාන සේවා කටයුතු සඳහා භාරදීමට රැගෙන ඒමට ප්‍රථම එහි ඇතුලත් දත්තයන් (Data) ගබඩා කරගෙන (Save) ඇත් ද ? යන්න මිනින්දෝරු අධිකාරිවරුන් විසින් විමසා බැලිය යුතුය. \nඉහත සඳහන් සේවා ලේඛනයේ දින සඳහන් කර ඇත්තේ පෙර වර්ෂයේ සේවා කරන ලද දිනයට අනුකූලව වර්ෂයක් ඉක්මවන දිනය සැලකීමෙනි. \nකෙසේ නමුත් උපකරණ ප්‍රවාහන වියදම හා ක්ෂේත්‍ර රාජකාරී සඳහා වන බාධා සැලකිල්ලට ගෙන උපලේඛනයේ සඳහන් දිනට ආසන්න දිනයක (මාසයක් තුල ) සේවා කටයුතු සඳහා උපකරණය එවීමට කොට්ඨාශ භාර ජ්‍යෙෂ්ඨ මිනින්දෝරු අධිකාරීන් තීරණ ගැනීම සම්බන්ධයෙන් විරුද්ධත්වයක් නොමැත."
    ,
      "මගේ අංකය - 1/8/1/1  \n දිනය  - 2016-01-20 \nඅධ්‍යක්ෂ ,\nජාතික ආපදා සහන සේවා මධ්‍යස්ථානය, \nවිද්‍යා මාවත,\nකොළඹ  07. \nආපදා සහන සැලසීම සඳහා ප්‍රතිපාදන ඉල්ලීම - 2016 මුදල් වර්ෂය \nමාතලේ දිස්ත්‍රික්කයේ ආපදා සිදුවීම් නිසා ආපදා සහන සැලසීම වෙනුවෙන් පහත පරිදි ප්‍රතිපාදන අදාළ ප්‍රාදේශීය ලේකම් කොට්ඨාශ මගින් ඉල්ලුම් කර ඇත.\n02. අදාළ ජාආසසේම 01 ආකෘතිද මේ සමග අමුණා ඇති අතර අදාළ ප්‍රතිපාදන ලබා දෙන ලෙස කාරුණිකව ඉල්ලමි.\nතේජානි තිලකරත්න, \nඅතිරේක දිස්ත්‍රික් ලේකම්, \nමාතලේ දිස්ත්‍රික් ලේකම් වෙනුවට. \n  පිටපත් :- ප්‍රාදේශීය ලේකම්           - කරු . දැ . ගැ . පි"
    ,
      "මගේ අංකය - 7/4/3/5/12  \nප්‍රාදේශීය ලේකම් කාර්යාලය ,\nපල්ලේපොල.\n2016.05.04.\nගරු සභාපති/ලේකම්/භාණ්ඩාගාරික, \nග්‍රාම සංවර්ධන සමිතිය.\nග්‍රාම සංවර්ධන බල මණ්ඩලය ප්‍රතිසංවිධානය.\nපල්ලේපොල ප්‍රාදේශීය ලේකම් කොට්ඨාශයේ ලියාපදිංචි ග්‍රාම සංවර්ධන බල මණ්ඩල මහ සභා රැස්වීම 2016.05.09 දින පෙ.ව.9.00 ට මෙම කාර්යාලයීය ශ්‍රවණාගාරයෙහි  පැවැත්වීමට කටයුතු යොදා ඇත.\n මෙදින 2016-2018 වර්ෂ සඳහා නව නිලධාරී මණ්ඩලයක් පත් කිරීම සහ බල මණ්ඩල ව්‍යාපෘති යෝජනා ලබා ගැනීමට අපේක්ෂිත බැවින් ඔබගේ යෝජනා රැගෙන එන ලෙස කාරුණිකව දන්වා සිටිමි.\n02. ග්‍රාම සංවර්ධන සමිති කාර්ය සාධන තරගය පිළිබඳ සමිති ප්‍රජා නායකයින් දැනුවත් කිරීම සහ තරග අයදුම්පත ලබාදීම මෙදින සිදුකරන බැවින්  සමිතියෙහි ගරු සභාපති, ලේකම් ,භාණ්ඩාගාරික යන නිලධාරීන් තිදෙනාම අනිවාර්යෙන්ම සහභාගී වන ලෙස වැඩි දුරටත් දන්වා සිටිමි.\nග්‍රාම සංවර්ධන කටයුතු වැඩි දියුණු කිරීමෙහිලා ඔබ දක්වන සහයෝගය ඉතා අගය කොට සලකමි.\nප්‍රාදේශීය ලේකම් ,\nපල්ලේපොල."
    ,
      "ලේකම්,\nඅධ්‍යාපන අමාත්‍යංශය,\nඉසුරුපාය,\nබත්තරමුල්ල.\nවිශේෂ අවශ්‍යතා සහිත සිසුන්ට ඉගැන්වීම සඳහා පාසල් වලට පත් කර ඇති විශේෂ අධ්‍යාපන ගුරුවරුන් එම කාර්යෙහිම යෙදවීම.\nඋක්ත කරුණ සම්බන්ධයෙන් ඔබතුමාගේ  EP/9/18/8/2/6 අංක දරන හා 2016.05.24 දිනැති ලිපිය හා බැඳේ.\n02. ස්ථානගත කිරීම අවසන් ගුරුවරුන්ගේ නාම ලේඛනය ඇමුණුම 1 මගින් ඉදිරිපත් කරමි.\n03. සේවය නොකර ගුරුවරුන් ලෙස තවදුරටත් හඳුනා ගන්නා ගුරුවරුන් සම්බන්ධව සහ නිලධාරීන් සම්බන්ධව ඉදිරි ක්‍රියාමාර්ග ගනු ලැබේ. ( ඇමුණුම 2සහ 3)\n04. මේ සම්බන්ධ කටයුතු සියල්ල අවසන් කර, ඔබතුමා වෙත සවිස්තර වාර්තාවක් මෙම වර්ෂය නිමවීමට පෙර ඉදිරිපත් කරමි.\nඒ.ටි.එම්. නිසාම්,\nපළාත් අධ්‍යාපන අධ්‍යක්ෂ ,\nපළාත් අධ්‍යාපන දෙපාර්තමේන්තුව,\nපිටපත්\n01. ලේකම්, නැගෙනහිර පළාත් අධ්‍යාපන අමාත්‍යංශය, ත්‍රිකුණාමලය-කරු. දැ. ගැ. පි.\n02.නැගෙනහිර පළාත් සියලුම අධ්‍යාපන අධ්‍යක්ෂවරුන් වෙත - කරු. අ.ක.ස."
    ,
      "මගේ අංකය - ඩි.එම්.යු./මාලේ/16/7/5/2-01\nදිනය - 2016.06.15\nප්‍රාදේශීය ලේකම්,\nප්‍රාදේශීය ලේකම් කාර්යාලය,\nපල්ලේපොල.\nප්‍රාදේශීය ආපදා කළමනාකරණ පෙර සූදානම් හා සැලසුම් සකස් කිරීමේ වැඩසටහන.\nඋක්ත කරුණ සම්බන්ධව 2016.06.14 දින දිස්ත්‍රික් ආපදා කළමනාකරණ මධ්‍යස්ථානයේ සහකාර අධ්‍යක්ෂ සමග පැවති දුරකථන සාකච්ඡාව හා බැඳේ.\n02. ඒ අනුව දිස්ත්‍රික් ආපදා කළමනාකරණ මධ්‍යස්ථානය මගින් ප්‍රාදේශීය ආපදා කළමනාකරණ කමිටු සාමාජිකයින්ගේ සහභාගිත්වයෙන් ප්‍රාදේශීය ලේකම් කොට්ඨාශය සඳහා විධිමත් ආපදා පෙර  සූදානම් සැලැසුමක් සකස් කිරීමේ වැඩ සටහනක් 2016.06.21 දින ඔබ කාර්යාලීය රැස්වීම් ශාලාවේදී පැවැත්වීමට අදහස් කර ඇත.\n03. ඒ අනුව ආපදා කළමනාකරණ කටයුතු සාර්ථකව සිදුකර ගැනීමට විධිමත් සැලැස්මක් තිබීම වැදගත් වන බැවින් ඉහත වැඩ සටහන පැවැත්වීම සඳහා අවශ්‍ය කටයුතු සංවිධානය කර දෙන ලෙස හා කොට්ඨාශයේ රාජ්‍ය ආයතන ප්‍රධානීන්, ආපදා බහුල වසම්වල සංවර්ධන නිළධාරින් සහ ස්වේච්ඡා සංවිධාන නියෝජිතයින්ද ඇතුළුව 100 දෙනකු සහභාගි කරවීමට කටයුතු කරන ලෙස කාරුණිකව ඉල්ලා සිටිමි.\nතේජානි තිලකරත්න,\nඅතිරේක දිස්ත්‍රික් ලේකම්,\nමාතලේ දිස්ත්‍රික් ලේකම් වෙනුවට ."
    ,
      "මගේ අංකය - PL\nදිනය - 2016.01.04\nප්‍රධාන ලේකම්,\nප්‍රධාන ලේකම් කාර්යාලය,\nමධ්‍යම පළාත.\nකාර්ය මණ්ඩල තොරතුරු - 2016\nමෙම අමාත්‍යාංශය විසින් සම්පාදනය කරනු ලැබ ඇති පළාත් සභා කාර්ය මණ්ඩල දත්ත පදනම ( Data Base) යාවත්කාලීන කිරීම සඳහා අවශ්‍ය තොරතුරු විධිමත් ලෙස ලබා දීම මඟින් මෙතෙක් ඔබ විසින් දෙන ලද සහයෝගය බෙහෙවින් අගය කරන අතර, මෙම දත්ත පද්ධතිය තව දුරටත් වැඩි දියුණු කිරීම පිණිස ඔබගේ කාරුණික සහාය අවශ්‍ය වී ඇත.\nඒ අනුව පළාත් සභා කාර්ය මණ්ඩල සම්බන්ධයෙන් මෙම අමාත්‍යාංශය විසින් නඩත්තු කරනු ලබන දත්ත පදනම 2016 වර්ෂයට තවදුරටත් පුළුල් කිරීමට යෝජිත හෙයින් 2016.01.01 දිනට ඔබ පළාත් සභාවේ කාර්ය මණ්ඩල තොරතුරු මේ සමඟ ඔබ වෙත එවනු ලබන ආකෘතියට අනුව වර්ගීකරණය කර 2016.01.31 දිනට පෙර මා වෙත එවන මෙන් කාරුණිකව ඉල්ලා සිටිමි.\nඑස්.ඩී.ඒ.බී.බොරලැස්ස,\nඅතිරේක ලේකම්(ප්‍රතිපත්ති   ),\nලේකම් වෙනුවට ."
    ,
      "මගේ අංකය - ඩි.එම්.යු./මාලේ/16/07/05/03-04\nදිනය - 2016.06.22\nප්‍රාදේශීය ලේකම්,\nප්‍රාදේශීය ලේකම් කාර්යාලය,\nපල්ලේපොල.\nග්‍රාමීය ආපදා පෙර සූදානම් සැලසුම් සකස් කිරීමේ වැඩ සටහන් සම්බන්ධව.\nමාතලේ දිස්ත්‍රික් ආපදා කළමනාකරණ මධ්‍යස්ථානය මගින් ග්‍රාමීය ආපදා කළමනාකරණ කටයුතු විධිමත් කිරීමේ අරමුණින් ඔබ ප්‍රාදේශීය ලේකම් කොට්ඨාශයේ දිඹුල්ගමුව ග්‍රාම නිලධාරී වසමේ නායයෑම් ආපදා පෙර සූදානම් වැඩ සටහනක් 2016.06.30 වන දින පැවැත්වීමට අදහස් කර ඇත.\n02. එබැවින් හදිසි ආපදා අවස්ථාවන්හිදි ජීවිත හා දේපල ආරක්ෂා කර ගැනීම සඳහා ආපදා පෙර සූදානම් වැඩසටහන් පැවැත්වීම වැදගත් වන බැවින් පහත සඳහන් නිළධාරීන්ගේ සහය ඇතිව ඉහත වැඩ සටහන පැවැත්වීම සඳහා අවශ්‍ය කටයුතු සංවිධානය කර දෙන ලෙස කාරුණිකව දන්වා සිටිමි.\n01. ආපදා කළමණාකරණ නිළධාරි\n02. සමාජ සේවා නිළධාරි\n03. ග්‍රාම නිළධාරි\n04. වනිතා සංවර්ධන නිළධාරි\n05. ළමා සංවර්ධන නිළධාරි\n06. තරුණ සේවා නිළධාරි\nතේජානි තිලකරත්න,\nඅතිරේක දිස්ත්‍රික් ලේකම්,\nමාතලේ දිස්ත්‍රික් ලේකම් වෙනුවට.."
    ,
      "ප්‍රධාන විධායක නිලධාරී,\nජාතික රක්ෂණ භාර අරමුදල,\nඅංක 97 මරදාන පාර,\nකොළඹ - 10.\nමහත්මයාණෙනි,\nඅග්‍රහාර  වෛද්‍ය රක්ෂණ ප්‍රතිලාභ ඉල්ලීම -2016. \nE.G. හේමන්ත උදේනිශාන්ත මයා - කාර්යාල සේවක සේවය.\nමෙම කාර්යාලයට අනුයුක්තව සේවය කරන ඉහත නම් සඳහන් සේවක මහතා විසින් මා වෙත ඉදිරිපත් කර ඇති දරු උපතක් සඳහා වූ අග්‍රහාර වෛද්‍ය රක්ෂණ  ප්‍රතිලාභ අයදුම්පත , ඊට අදාළ වෛද්‍ය බිල්පත් හා පහත සඳහන් ලියවිලි මගේ නිර්දේශය සහිතව අවශ්‍ය කටයුතු සඳහා මේ සමග ඉදිරිපත් කරන බව කාරුණිකව දන්වමි.\n01. රෝග නිශ්චය කාඩ්පතේ සහතික කරන ලද පිටපත.\n02. දරුවාගේ උප්පැන්න සහතිකයේ සහතික කරන ලද පිටපත.\n03. මවගේ උප්පැන්න සහතිකයේ සහතික කරන ලද පිටපත.\n04. පියාගේ උප්පැන්න සහතිකයේ සහතික කරන ලද පිටපත.\n05. විවාහ සහතිකයේ සහතික කරන ලද පිටපත.\n06. මවගේ ජාතික හැදුනුම්පතේ සහතික කරන ලද පිටපත.\n07. පියාගේ ජාතික හැදුනුම්පතේ සහතික කරන ලද පිටපත.\nමෙයට - ඔබේ සේවයට කැපවූ ,\nඑච්.එම්.එම්.සී.පී.හිඳගොඩ.\nසහකාර ප්‍රාදේශීය ලේකම්,\nපල්ලේපොල ප්‍රාදේශීය ලේකම් වෙනුවට ,"
    ,
      "පළාත් අධ්‍යාපන අධ්‍යක්ෂ, \nපළාත් අධ්‍යාපන දෙපාර්තමේන්තුව,\n............... පළාත.\nවිදු ලොවට මං කියාදෙන සොබාදහම සමස්ථ ලංකා විද්‍යා තරඟාවලිය 2016.\nජාතික මූලික අධ්‍යාපන ආයතනයේ විද්‍යා අධ්‍යාපන හා ව්‍යාප්ති අංශය විසින් සංවිධානය කරනු ලබන ඉහත සඳහන් දීප ව්‍යාප්ත තරඟාවලිය සඳහා අනුමැතිය ඉල්ලා ඇත.\nමෙම දීප ව්‍යප්ත තරඟාවලිය සඳහා අනුමැතිය ලබා දෙන අතර මෙම වැඩසටහන්  පාසල්වල වාර විභාග කාල සීමාවේ නොපැවත්විය යුතු බව හා පාසලේ  අධ්‍යාපන කටයුතුවලට බාධාවක් නොවන පරිදි සිදු කිරීමට අවශ්‍ය සහයෝගය ලබා දෙන ලෙස ද විදුහල්පතිවරුන් දැනුවත් කරන මෙන් කාරුණිකව දන්වමි.\nමෙම කර්තව්‍යය  ඉටු කිරීමට ඔබගෙන් ලැබෙන සහයෝගය ඉතා අගේ කොට සලකුණු ලබන අතර අදාළ විස්තර මේ සමග අමුණා ඇත.\nඑච්. යූ. ප්‍රේමතිලක \nඅතිරේක ලේකම් (අධ්‍යාපන ගුණාත්මක සංවර්ධන)\nඅධ්‍යාපන ලේකම් වෙනුවට.\nපිටපත් :\n1. පළාත් අධ්‍යාපන ලේකම්, පළාත් අධ්‍යාපන අමාත්‍යාංශය. ....................................... පළාත. - දැ. ගැ. පි\n2.මහාචාර්ය යූ.එල්.බී.ජයසිංහ, වැඩබලන අධ්‍යක්ෂ. ජාතික මූලික අධ්‍යයන ආයතනය,මහනුවර. - අ.ක.ස."
    ,
      "මිනින්දෝරු අධිකාරී,\nදි. මිනින්දෝරු  කාර්යාලය\nමඩකලපුව.\n2015.12.31 දිනට දුම්රිය බිල්පත් සඳහා අය වීමට ඇති හිඟ මුදල් ගිණුම් අංක: 4806\nඔබ අමාත්‍යාංශයෙන්/ දෙපාර්තමේන්තුවෙන්/ ආයතනයෙන් නිකුත් කර ඇති දුම්රිය බලපත්‍ර වෙනුවෙන් 2015.12.31 දිනට අයවීමට ඇති හිඟ මුදල් මේ සමග අමුණා ඇති උපලේණයේ පරිදි රු  128,183.70 වේ.\n02. ඔබ විසින් දුම්රිය බලපත්‍ර නිරවුල් කිරීමේදී අප විසින් දී ඇති උපදෙස් අනුව ක්‍රියා නොකිරීම නිසා ගෙවනු ලබන චෙක්පත් වල සඳහන් මුදලට අදාළ විස්තර නොමැති වීමෙන් ගිණුම් ගත කිරීම දුෂ්කර වී ඇත. \nඑම නිසා ඔබ විසින් බිල්පත් නිරවුල් කිරීමේදී ගෙවනු ලබන මුදලට අදාළ ඉන්වොයිස් අංකය අනිවාර්යයෙන්ම සඳහන් කල යුතු වේ."
    ,
      "මගේ අංකය-  1/8/1/1   \nදිනය -  2016-05-26 \nප්‍රාදේශීය ලේකම් ,\nප්‍රාදේශීය ලේකම් කාර්යාලය. \nකඩිනම් ආපදා සහන සැළසීම -2016.\nමාතලේ දිස්ත්‍රික්කයට බලපෑ අයහපත් කාලගුණික තත්ත්වය හේතුවෙන් ඔබ ප්‍රාදේශීය ලේකම් කොට්ඨාශයේ  ආපදාවන් නිසා විපතට පත් ජනතාව වෙත වියළි සළාක හා පිසූ ආහාර ලබා දීම වෙනුවෙන් ඔබ විසින් ඉල්ලුම් කර ඇති මුදල් ජාතික ආපදා සහන සේවා මධ්‍යස්ථානය මගින් ලබා දී ඇත.\n02. එම මුදල් වලට අදාළ චෙක් පත් පහත පරිදි ඔබ වෙත එවීමට කටයුතු කරමි.\nඅනු අංක       ප්‍රා.ලේ. කාර්යාලය          චෙක් පත් අංකය           චෙක් පතේ වටිනාකම (රු)\n 01.               උකුවෙල                                     646029                               11400.00\n02.               යටවත්ත                                    646030                                 4600.00\n03.               නාඋල                                         646031                                 4100.00\n04.               දඹුල්ල                                         646032                                 9000.00\n05.              රත්තොට                                    646033                               20250.00\n06.               මාතලේ                                       646034                                 2200.00\n07.              ගලේවෙල                                     646035                               15300.00\n03. අදාළ ගෙවීම් කඩිනමින් සිදු කර ආපදා කළමනාකරණ අමාත්‍යංශ ලේකම්ගේ අංක NDRSC/01/04/01 දරණ 2016-05-16 දිනැති ලිපියට අනුව එහි ඇමුණුමේ පරිදි වියදම් වාර්තාව සකසා 2016-06-05 දිනට පෙර වියදම් වාර්තාව එවීමට කටයුතු කරන ලෙස කාරුණිකව දන්වමි.\nතේජානි තිලකරත්න, \n අතිරේක දිස්ත්‍රික් ලේකම්, \n මාතලේ දිස්ත්‍රික් ලේකම්/දිසාපති. \n  පිටපත :  1. අධ්‍යක්‍ෂ,ජාතික ආපදා සහන සේවා මධ්‍යස්ථානය.             \n\t   2. ප්‍රධාන ගණකාධිකාරී , මාතලේ."
    ,
      "මගේ අංකය :- ED/07/99/4/2/13    \nදිනය :- 2014.06.13\nනියෝජ්‍ය ප්‍රධාන ලේකම්(ඉංජිනේරු සේවා),\nසබරගමු පළාත් ඉංජිනේරු දෙපාර්තමේන්තුව,\nනව නගරය, \nරත්නපුර.\nඋ.පෙළ විද්‍යාගාරයේ අබලන් වී ඇති වහලය සකස්කර ගැනීම සම්බන්ධවයි - කෑගලු විද්‍යාලය.\nඋක්ත කරුණ සම්බන්ධයෙන් විදුහල්පති විසින් අධ්‍යාපන අමාත්‍යතුමන් වෙත යොමු කරන ලදුව මා වෙත යොමු කරන ලද 2014.05.07 දිනැති ලිපිය හා බැඳේ(පිටපතක් අමුණා ඇත).\nඒ අනුව, අබලන් වී ඇති වහලය අලුත්වැඩියා කර ගැනීම සඳහා වන ඇස්තමේන්තුව කඩිනමින් සකස්කර එවන මෙන් කාරුණිකව දන්වා සිටිමි.\nඅයි.එම්.ප්‍රනාන්දු,\nපාසල් ගොඩනැගිලි අධ්‍යක්ෂ,\nඅධ්‍යාපන ලේකම් වෙනුවට.\nපිටපත් :-\n01.විදුහල්පති,කෑගලු විද්‍යාලය,කෑගල්ල."
    ,
      "මගේ අංකය : ඊඩි/4/58/2/R/220\nදිනය : 2013.07.26\nඑස්.පී. රාජසේකරම් මහතා,\nශ්‍රී.ලං.අ.ප.සේ. 111,\nකලාප අධ්‍යාපන කාර්යාලය,\nනුවරඑළිය.\nකාර්යක්ෂමතා කඩඉම සම්පුර්ණ කිරීම සඳහා සහන කාලයක් ඉල්ලීම.\nඋක්ත කරුණ සම්බන්ධයෙන් මධ්‍යම පළාත් අධ්‍යාපන ලේකම් විසින් එවන ලද අංක සීපිසී/සිඑම්එස්/1/19/1/6 හා 2012.04.02 දිනැති නිර්දේශිත ලිපිය හා බැඳේ.\n02. ඒ අනුව ඔබට පළමු කාර්යක්ෂමතා කඩඉම් පරීක්ෂණය සමත්වීම සඳහා 2002.01.04 දින සිට 2003.03.24 දින දක්වා සහන කාලයක් ලබා දීමට රාජ්‍ය සේවා කොමිෂන් සභාව විසින් අනුමත කර ඇති බව කාරුණිකව දන්වා සිටිමි.\nපි.එස්.එස්. විජේරත්න,\nසහකාර ලේකම්,\n(අධ්‍යාපන සේවා ආයතන),\nඅධ්‍යාපන ලේකම් වෙනුවට.\n පිටපත් :\n1. ලේකම් - රාජ්‍ය සේවා කොමිෂන් සභාව - ඔබගේ අංක PSC/APP/05/16(111)/20 හා 2013.06.20 දිනැති ලිපිය අනුව දැ.ගැ.පි\n2. අධ්‍යාපන ලේකම් - මධ්‍යම පළාත් අධ්‍යාපන අමාත්‍යාංශය \n3. පළාත් අධ්‍යාපන අධ්‍යක්ෂ - මධ්‍යම පළාත\n4. කලාප අධ්‍යාපන අධ්‍යක්ෂ - නුවරඑළිය"
    ,
      "03. අතිගරු ජනාධිපතිතුමාගේ උපදෙස් පරිදි, ශ්‍රී ලංකාව තුළ ජගත් පරිසර දිනය සැමරීම සඳහා සෑම වසරකම ජුනි මස 05 වන දින යොදා ගැනීමටත්, මැයි 30 සිට ජුනි 05 දක්වා පූර්ව සතිය ''ජාතික පරිසර සතිය'' ලෙස ප්‍රකාශයට පත්කිරීමටත්, ජාතික පරිසර සතිය තුළ සියළු රාජ්‍ය හා රාජ්‍ය නොවන ආයතනවල දායකත්වයෙන් විශේෂ පරිසර සංරක්ෂණ වැඩසටහන් ක්‍රියාත්මක කිරීමටත් අමාත්‍ය මණ්ඩල අනුමැතිය ලබා දී ඇත.\n04. ඒ අනුව 2016 මැයි මස 30 වන දින සිට ජුනි මස 05 වන දින දක්වා ක්‍රියාත්මක පරිසර සතිය තුළ පහත සඳහන් පරිදි වැඩසටහන් සංවිධානය කොට ක්‍රියාත්මක කිරීමට කටයුතු කරන ලෙස දන්වමි.\ni. සියළුම රාජ්‍ය ආයතනවල ආයතන ප්‍රධානීන් විසින් ජගත් පරිසර දිනයේ වැදගත්කම පිළිබඳව කාර්ය මණ්ඩලය දැනුවත් කිරීම, කාර්යාල පරිශ්‍ර පවිත්‍ර කිරීමේ ශ්‍රමදාන පැවැත්වීම, කාර්යාල පරිශ්‍ර තුළ කසළ වර්ගීකරණය කර (ප්ලාස්ටික්/පොලිතීන්/කඩදාසි/වීදුරු/ලෝහ/කාබනික ද්‍රව්‍ය ආදී ලෙස) බැහැර කිරීම ආරම්භ කිරීම සහ පරිසර සංරක්ෂණ වැඩසටහන් ක්‍රියාත්මක කිරීම.\nii. පාසල්වල විදුහල්පතිවරුන් විසින් ලෝක පරිසර දිනයේ වැදගත්කම පිළිබඳව ශිෂ්‍ය ශිෂ්‍යාවන් දැනුවත් කිරීම හා පාසල් සිසුන්ගේ සහභාගීත්වයෙන් පරිසර සංරක්ෂණ වැඩසටහන් ක්‍රියාත්මක කිරීම.\niii. පළාත් පාලන ආයතනවල මූලිකත්වයෙන් එම පළාත් පාලන බල ප්‍රදේශ තුළ අපද්‍රව්‍ය කළමනාකරණ වැඩසටහන් ඇතුළු පරිසර සංරක්ෂණ වැඩසටහන් ක්‍රියාත්මක කිරීම. \niv. පොලිතීන් හා ප්ලාස්ටික් භාවිතය අවම කිරීම සහ පරිසර හිතකාමී ඇසුරුම් භාවිතය ප්‍රවර්ධනය කිරීම.\nv. නීල හරිත ලස්සන ලංකා වැඩසටහන යටතේ පරිසර හිතකාමී ''හරිත ගම්මාන'' සහ ''හරිත නගර'' වැඩසටහන් ක්‍රියාත්මක කිරීම.\nvi. ප්‍රජා සහභාගීත්වයෙන් පරිසර සංවේදී භූමි සංරක්ෂණ, රුක් රෝපණ වැඩසටහන් ඇතුළු විවිධ පරිසර සංරක්ෂණ වැඩසටහන් ක්‍රියාත්මක කිරීම.\nvii. හානියට පත් භූමි පුනරුත්ථාපනය කිරීම.\nviii. පරිසර සංරක්ෂණය පිළිබඳව ප්‍රජාව දැනුවත් කිරීමට වැඩසටහන් ක්‍රියාත්මක කිරීම.\nix. පාරිසරික ප්‍රදර්ශන පැවැත්වීම.\nx. හරිත රැකියා හඳුනා ගැනීම හා ප්‍රවර්ධන වැඩසටහන් ක්‍රියාත්මක කිරීම."
    ,
      "වෙත  :- ගණකාධිකාරි(ගෙවීම්)\nමගින්  :- අධ්‍යක්ෂ( සැපයීම්)\nවෙතින්  :- ගණකාධිකාරි(මුදල්)\nදිනය :- 2013.04.03\nමගේ අංකය  :- ඊඩී/07/97/05/02/01\nවිෂය :- අධීක්ෂණ හා සහායක නිලධාරි සංචිතයේ සඳහන් නම නිවැරදි ලෙස සංශෝධනය කිරීම.\nසැපයීම් ශාඛාව මගින් විෂයානුබද්ධ උපකරණ අප්‍රමාදව සිසුදරු දැරියන් අතට පත් කිරීමේ වැඩසටහන සඳහා මූල්‍ය අනුමැතිය ලබාගැනීම යටතේ 2013.01.01 දින සිට 2013.12.31 දින දක්වා සංයුක්ත දීමනා සහ බටා නිර්දේශිත අධීක්ෂණ හා සහායක නිලධාරින්ගේ සංචිත සඳහා අනුමත කරන ලද ලේඛනයෙහි ආර්.එස්.කේ.පෙරේරා මයා ලෙස සඳහන්ව ඇති නම ආර්.එස්.කේ.ජයවර්ධන මයා ලෙස සංශෝධනය විය යුතු බව කාරුණිකව සලකන්න.\nඅදාල අනුමත පිටපත් මේ සමග අමුණා ඇත.\nබී.ඩී.සී.බියන්විල,\nඅධ්‍යක්ෂ(සැපයීම්)."
    ,
      "මගේ අංකය - ED/07/74/5/02(3)\nදිනය - 2013.12.12\nකළමනාකරු,\nසී/ස - එලිවේටර්ස්(පුද්.) සමාගම,\n9 වන මහල,\nවොක්ෂෝල් වීදිය,\nකොළඹ - 02.\nඉසුරුපාය ගොඩනැගිල්ලේ භාණ්ඩ විදුලි සෝපානය 5 වන මහල දක්වා (අතිරේක මහල දෙක) දීර්ඝ කර සවිකිරීම.\nඉසුරුපාය පරිශ්‍රයේ භාණ්ඩ විදුලි සෝපානය සවිකිරීම සඳහා 2013.12.09 දින සිට ඔබ ආයතනය මගින් දැනටමත් වැඩ ආරම්භ කර ඇත.\n02. එම විදුලි සෝපානයේ ගලවා ඉවත්කරන ලද කොටස් ලේඛනගත කොට අප අමාත්‍යාංශයේ ගොඩනැගිලි ශාඛාව වෙත භාර දෙන මෙන් කාරුණිකව දන්වා සිටිමි.\nකමල් අතුකෝරල,\nගොඩනැගිලි කළමනාකරු."
    ,
      "මගේ අංකය : ED/07/99/7/9/8\nපාසල් ගොඩනැගිලි ශාඛාව. \n2013-03-18.\nගණකාධිකාරී (ප්‍රසම්පාදන),\nප්‍රසම්පාදන ඒකකය.\nප්‍රසම්පාදන මාර්ගෝපදේශ 8:13:4 අනුව කොන්ත්‍රාත් මුදලින් 10% වැඩි කොන්ත්‍රාත් විචලන ලේඛන ඇගයීම සඳහා තාක්‍ෂණ ඇගයීම් කමිටුවක් පත් කරව ගැනීම.\nගා/අමරසුරිය ගුරු විදුහල ( වලව්ව හා පිරිමි නේවාසිකාගාර ගොඩනැගිල්ල අලුත්වැඩියාව )\nඉහත වැඩ දකුණු පළාත් අධ්‍යාපන අධ්‍යක්ෂ විසින් රු. 1,179,471.00 මුදලකට කොන්ත්‍රාත් පවර ඉටු කර ගෙන යාමේ දී කොන්ත්‍රාත් මුදලින් 32.52% ක පමණ කොන්ත්‍රාත් විචලන ඉටු කිරීමට සිදු වු බවත්, ඒ සඳහා අනුමැතිය ලබා දෙන ලෙසත් දකුණු පළාත් අධ්‍යාපන අධ්‍යක්ෂ ඉල්ලා ඇත.\n ඔහු විසින් අදාළ ලේඛන ද මා වෙත යොමු කර ඇත.\nඉහත මාර්ගෝපදේශයට අනුව මෙම විචලන ලේඛන අනුමැතියට ප්‍රසම්පාදන කමිටුවට ඉදිරිපත් කිරීමට සිදු වේ. \nඒ අනුව මෙය තාක්‍ෂණ ඇගයීම් කමිටුවට යොමු කළ යුතුව ඇති බැවින් මේ සඳහා ඇගයීම් කමිටුවක් පත් කර දෙන මෙන් කාරුණිකව ඉල්ලා සිටිමි.\nතවද මෙම ඇගයීම් කමිටුව සඳහා මෙම ශාඛාවේ පහත සඳහන් නිළධාරීන් නම් කරමි.\n1. ඩබ්.එන්. අජන්ත මිය - විදුලි ඉංජිනේරු (සභාපති)\n2. කමල් අතුකෝරල මහතා - ජෞෂ්ඨ කාර්මික නිළධාරී (සාමාජික) \nඅයි.එම්. ප්‍රනාන්දු,\nපාසල් ගොඩනැගිලි අධ්‍යක්ෂ."
    ,
      "මගේ අංකය : ඊඩි/4/58/8/1/5\nඔබේ අංකය : PSC/APP/07/619/2014 \nදිනය : 2014.04.24\nලේකම්,\nරාජ්‍ය සේවා කොමිෂන් සභාව. \nසබරගමුව පළාත් අධ්‍යාපන දෙපාර්තමේන්තුවේ පළාත් අධ්‍යාපන අධ්‍යක්ෂක තනතුර සඳහා  නිලධාරියෙකු පත් කිරීම.\nඋක්ත කරුණ සම්බන්ධයෙන් ඔබේ සමාංක හා 2014.03.05 දිනැති ලිපිය හා බැඳේ.\n02. ඒ අනුව පුරප්පාඩු වී ඇති සබරගමුව පළාත් අධ්‍යාපන අධ්‍යක්ෂ තනතුරට සුදුසු නිලධාරියෙකු පත් කිරීම සඳහා වු සම්මුඛ පරීක්ෂණය පවත්වන ලද අතර එම සම්මුඛ පරීක්ෂණ ප්‍රතිපල මත ශ්‍රී ලංකා අධ්‍යාපන පරිපාලන සේවයේ 1 පන්තියේ නිලධාරී එස්. කුරුප්පුආරච්චි මහතා එම තනතුරට පත් කිරීම නිර්දේශ කරමි.\nඅනුර දිසානායක,\nලේකම්, \nඅධ්‍යාපන අමාත්‍යාංශය."
    ,
      "පළාත් අධ්‍යාපන අධ්‍යක්ෂ, \nපළාත් අධ්‍යාපන දෙපාර්තමේන්තුව,\nනැගෙනහිර/ උතුරුමැද/ උතුර / වයඹ පළාත.\nසමාජ ඒකාග්‍රතා සාම අධ්‍යපන විෂය වඩාත් විධිමත් හා සක්‍රීය මට්ටමින් පාසල් තුල ක්‍රියාවට නැගීම සඳහාත් ඒ හා බැඳෙන ජාතික ප්‍රතිසන්ධාන ක්‍රියාවලිය වඩාත් ශක්තිමත් මට්ටමින් ක්‍රියාවට නැගීම සඳහාත් පළාත් හා  කලාප මට්ටමින් ගුරු උපදේශකවරුන් දැනුවත් කිරීම.\n2016 වසරේ සමාජ ඒකාග්‍රතා හා සාම  අධ්‍යපන විෂය වඩාත් විධිමත් හා සක්‍රීය මට්ටමින්  පාසල් පද්ධතිය තුළ ක්‍රියාවට නැගීම අරමුණු කර ගනිමින්  විෂය භාර පළාත් හා කලාප සම්බන්ධීකරණ නිලධාරීන් සහ පළාත් හා කලාප මට්ටමින් තෝරා ගත් ප්‍රාථමික ගුරු උපදේශකවරුන් සඳහා ධාරිතා සංවර්ධන වැඩසටහන් ක්‍රියාවට නැගීමට සැලසුම් කර තිබේ.\nමෙම පළාත් කලාප සම්බන්ධීකරණ නිළධාරීන්ට සමාජ ඒකාග්‍රතා හා සාම අධ්‍යාපන විෂයයත් එකී අරමුණු සාධනය සඳහාත් එම වැඩසටහන් වඩාත් සක්‍රීයව ක්‍රියාවට නැගීම සඳහා සහාය වීමට අදාළ විෂයය භාර ප්‍රාථමික ගුරු උපදේශකවරුන් තුලද විෂය පිලිබඳ දැනුම සංවර්ධනය කිරීමට අපේක්ෂිතය. \nඒ අනුව සියලුම පළාත් හා කලාප මට්ටමින් තෝරාගත් එක් ප්‍රාථමික ගුරු උපදේශකවරයකු සඳහා විෂය පිලිබඳ ධාරිතා සංවර්ධනය කිරීමේ වැඩමුළුවක් ක්‍රියාවට නැඟීමට සැලසුම් කර තිබේ.\nඑම වැඩමුළුව 2016 ජුනි 30, ජූලි 01 හා 02 දිනයන්හි ගිරිතලේ  wood side Tour Inn හිදී පැවැත්වේ. \nඑම වැඩමුළුව සඳහා ඔබ පළාත්හි මේ සමග අමුණා ඇති ලේඛණයේ සඳහන් අධ්‍යාපන කලාපයන්හි තෝරාගත් ප්‍රාථමික ගුරු උපදේශකවරයෙකු නම් කර අනිවාර්යයෙන්ම සහභාගී කරවීමට කටයුතු කරන මෙන් කාරුණිකව දන්වා සිටිමි.\nමෙම වැඩමුළුව 2016 ජුනි මස 30 දින පෙ.ව. 8.00 ට ආරම්භ වී 2016 ජූලි මස 02 දින ප.ව. 4.00 අවසන් වේ. \n2016 ජූනි 29 වන දින රාත්‍රී නේවාසික පහසුකම් සලසා ඇති බැවින් එදින රාත්‍රී නේවාසික පහසුකම් ලබාගෙන වැඩමුළුවට නියමිත වේලාවට සහභාගී වන ලෙස අදාළ ගුරු උපදේශකවරයා / ගුරුඋපදේශකවරිය දැනුවත් කරන්න.\nඔබ විසින් වැඩමුළුවට සහභාගී වීමට නම් කරන ගුරු උපදේශකවරයා/ උපදේශකවරිය පිලිබඳ තොරතුරු පහත දුරකථන අංක වලට ඇමතුමක් හෝ කෙටි පණිවිඩයක් ලබාදීමෙන් හෝ විද්‍යුත් ලිපිනයට ලබා දී පැමිණීම තහවුරු කරන ලෙස වැඩිදුරටත් දන්වා සිටිමි.\nසහකාර අධ්‍යාපන අධ්‍යක්ෂ බී.ඒ.ඩබ්ලිව්.ආර්. පද්මශාන්ත  071-4486911, 076-8644911 email-pdmshantha@gmail.com\nගමන් මාර්ගය - මහනුවර - පොළොන්නරුව මාර්ගයේ ගිරිතලේ\nඒ. එස්. හේවගේ,\nඅතිරේක ලේකම් (පාසල් කටයුතු),\nලේකම් වෙනුවට .\nපිටපත්1. කලාප අධ්‍යාපන අධ්‍යක්ෂ,  .... අධ්‍යාපන කලාපය - ගුරු උපදේශකවරයෙකු අනිවාර්යෙන්ම සහභාගී කරවීම සඳහා.\n    2.සමාජ ඒකාග්‍රතා හා සාම  අධ්‍යාපන සම්බන්ධීකරණ නිලධාරී.... පළාත් / අධ්‍යාපන කලාපය - කලාප අධ්‍යාපන අධ්‍යක්ෂතුමා දැනුවත් කර සුදුසු ගුරු උපදේශකවරයෙකු නම්කර නියමිත දින අනිවාර්යයෙන්ම සහභාගී කර වීම සඳහා."
    ,
      "මගේ අංකය - 7/4/3/5/12  \nප්‍රාදේශීය ලේකම් කාර්යාලය ,\nපල්ලේපොල.\n2016.05.04\nගරු සභාපති/ලේකම්/භාණ්ඩාගාරික, \nග්‍රාම සංවර්ධන සමිතිය ,\nග්‍රාම සංවර්ධන බල මණ්ඩලය ප්‍රතිසංවිධානය.\nපල්ලේපොල ප්‍රාදේශීය ලේකම් කොට්ඨාශයේ ලියාපදිංචි ග්‍රාම සංවර්ධන බල මණ්ඩල මහ සභා රැස්වීම 2016.05.09 දින පෙ.ව.9.00 ට මෙම කාර්යාලයීය ශ්‍රවණාගාරයෙහි  පැවැත්වීමට කටයුතු යොදා ඇත. \nමෙදින 2016-2018 වර්ෂ සඳහා නව නිලධාරී මණ්ඩලයක් පත් කිරීම සහ බල මණ්ඩල ව්‍යාපෘති යෝජනා ලබා ගැනීමට අපේක්ෂිත බැවින් ඔබගේ යෝජනා රැගෙන එන ලෙස කාරුණිකව දන්වා සිටිමි.\n02. ග්‍රාම සංවර්ධන සමිති කාර්ය සාධන තරඟය පිළිබඳ සමිති ප්‍රජා නායකයින් දැනුවත් කිරීම සහ තරඟ අයදුම්පත ලබාදීම මෙදින සිදුකරන බැවින්  සමිතියෙහි ගරු සභාපති, ලේකම් , භාණ්ඩාගාරික යන නිලධාරීන් තිදෙනාම අනිවාර්යෙන්ම සහභාගී වන ලෙස වැඩි දුරටත් දන්වා සිටිමි.\nග්‍රාම සංවර්ධන කටයුතු වැඩි දියුණු කිරීමෙහිලා ඔබ දක්වන සහයෝගය ඉතා අගය කොට සලකමි.\nප්‍රාදේශීය ලේකම් ,\nපල්ලේපොල."
    ,
      "මගේ අංකය  7/2/9/2 \nප්‍රාදේශීය ලේකම් කාර්යාලය ,\nපල්ලේපොල. \n2016.01.20.\nනියෝජ්‍ය ලේකම්,\nප්‍රධාන ලේකම් කාර්යාලය ,\nමහනුවර .\n2015 දෙසැම්බර් මස බැංකු සැසදුම් වාර්තාව. \nපල්ලේපොල මහජන බැංකුවේ ගිණුම් අංක 211100130000271 අදාල 2015.12.31 දිනට බැංකු  සැසදුම් වාර්තාව මේ සමග එවන බව කාරුණිකව දන්වමි.\nගණකාධිකාරි,\nප්‍රාදේශීය ලේකම් වෙනුවට ,\nපල්ලේපොල .\nපිටපත් \t1 සහකාර විගණකාධිපති රජයේ විගණක අංශය දිස්ත්‍රික් ලේකම් කාර්යාලය මාතලේ. \n\t\t2 මුදල් පොත (විෂය ලිපිකරු)."
    ,
      "මාගේ අංකය  SLEAS/2016/21\nදිනය  2016.04.19\nලේකම්, අධ්‍යාපන අමාත්‍යංශය,\nරාජ්‍ය ලේකම්, අධ්‍යාපන අමාත්‍යංශය,\nසියලුම පළාත් ප්‍රධාන/අධ්‍යාපන අමාත්‍යංශයවල ප්‍රධාන අධ්‍යාපන ලේකම්වරුන්,\nවිභාග කොමසාරිස් ජනරාල්, විභාග දෙපාර්තමේන්තුව,\nඅධ්‍යාපන ප්‍රකාශන කොමසාරිස් ජනරාල්, අධ්‍යාපන ප්‍රකාශන දෙපාර්තමේන්තුව,\nසියලුම පළාත් අධ්‍යාපන අධ්‍යක්ෂවරුන් ,\nඅධ්‍යක්ෂ ජනරාල්, ජාතික අධ්‍යාපන ආයතනය,\nසියලුම කලාප අධ්‍යාපන අධ්‍යක්ෂවරුන්,\nසියලුම ජාතික පාසල් විදුහල්පතිවරුන්.\nසේවා ව්‍යවස්ථාවට පටහැනිව පත්වීම් හා උසස්වීම ලබා දීමට ගන්නා උත්සාහයට විරෝධය දැක්වීමේ වෘත්තීය ක්‍රියාමාර්ග පිළිබඳව දැනුවත් කිරීම.\nඅධ්‍යාපන ලේකම්ට යොමු කරන ලද SLEAS/2016/20 හා 2016.04.12 දිනැති ශ්‍රී ලංකා අධ්‍යාපන පරිපාලන සේවයේ සේවා ව්‍යවස්ථාව උල්ලංඝනය කරමින් උසස්වීම් හා පත්වීම් ලබා දීමේ උත්සාහය පිළිබඳ ලිපියට අනුව එම ලිපියේ සඳහන් කරුණු පිළිබඳව අප සංගමය වෙත සෑහීමකට පත්විය හැකි පිළිතුරක් නොලැබුණු හෙයින් එම ලිපියේ සඳහන් පරිදි වෘත්තීය ක්‍රියාමාර්ගයකට එළඹීමට අප සංගමයේ විධායක කමිටුව තීරණය කරන ලදී.\nඑබැවින් 2016.04.20 දින අප සංගමයේ සාමාජිකයන් වන ශ්‍රී ලංකා අධ්‍යාපන පරිපාලන සේවයේ සියලුම නිලධාරීන් වැඩ වර්ජනයක යෙදෙන බව දන්වා සිටිමි.\nතව ද එදින පෙ.ව. 10.00 ට අධ්‍යාපන අමාත්‍යංශ පරිශ්‍රයේ ශ්‍රී ලංකා අධ්‍යාපන පරිපාලන සේවා ව්‍යවස්ථාවට පටහැනිව පත්වීම් හා උසස්වීම් ලබා දීම පිලිබඳ විශේෂ සමුළුවක් පැවැත්වෙන බව ද ඉන් අනතුරුව සාමකාමී විරෝධතාවක් පැවැත්වෙන බව ද දන්වා සිටිමි.\nමෙයට විශ්වාසී,\nගරු ලේකම් ,\nශ්‍රී ලංකා අධ්‍යාපන පරිපාලන සේවා වෘත්තීය සංගමය.\nපිටපත අධ්‍යාපන පරිපාලන සේවයේ සියලුම නිලධාරීන් වෙත, සහභාගී වීම පිණිස."
    ,
      "සියලුම පළාත් අධ්‍යාපන ලේකම්වරුන්, \nසියලුම පළාත් අධ්‍යාපන අධ්‍යක්ෂවරුන්, \nසියලුම කලාප අධ්‍යාපන අධ්‍යක්ෂවරුන් ,\nවිභාග කොමසාරිස් ජනරාල් ,\nඅධ්‍යාපන ප්‍රකාශන කොමසාරිස් ජනරාල් ,\nඅධ්‍යක්‍ෂ ජනරාල් - ජාතික අධ්‍යාපන ආයතනය. \nදීප ව්‍යාප්ත සේවාවන්හි නිලධාරීන්ගේ තොරතුරු පද්ධතිය සකස් කිරීම. \nදීප ව්‍යාප්ත සේවාවල රාජ්‍ය නිලධාරීන්ගේ තොරතුරු පද්ධතියක් සකස් කිරීම සඳහා රාජ්‍ය සේවා කොමිෂන් සභාව විසින් තීරණයක් ගෙන ඇති බව එම ලේකම්ගේ අංක PSC/EST/09-03/02/2015 හා  2016.05.17 දිනැති ලිපියෙන් මා වෙත දන්වා එවා ඇත.\n මේ සමග අමුණා ඒවා ඇති ආකෘතිය පරිදි එම විස්තරය ඉල්ලා ඒවා ඇති අතර ඔබ පළාතේ/ආයතනයේ ශ්‍රී ලංකා අධ්‍යාපන පරිපාලන සේවයේ නිලධාරීන්ගේ තොරතුරු අප්‍රමාදව මා වෙත ලැබීමට සලස්වන ලෙස කාරුණිකව දන්වමි.\nෆැක්ස් අංක : 0112787383 \nඑච්.යු.ප්‍රේමතිලක \nඅතිරේක ලේකම් (රා.ආ.)\n අධ්‍යාපන සේවා ආයතන,\nඅධ්‍යාපන ලේකම් වෙනුවට"
    ,
      "ලේකම්, ප්‍රධාන අමාත්‍යාංශය, මධ්‍යම පළාත,\nලේකම්, කෘෂිකර්ම අමාත්‍යාංශය, මධ්‍යම පළාත,\nලේකම්, සෞඛ්‍ය අමාත්‍යාංශය, මධ්‍යම පළාත,\nලේකම්, කර්මාන්ත අමාත්‍යාංශය, මධ්‍යම පළාත,\nලේකම්, මාර්ග අමාත්‍යාංශය, මධ්‍යම පළාත.\nජාතික අයවැයෙන් මුදල් ප්‍රතිපාදන ලබා දෙනු ලබන ක්‍රියාකාරකම් හි මූල්‍ය හා භෞතික ප්‍රගතිය අධීක්ෂණය කිරීම.\n(Monitoring of Financial and Physical Performance of Activities Financed by the National Budget)\n02. ඒ සමඟ ඔබ වෙත එවන ලද මුදල් අමාත්‍යාංශයේ 2015.12.29 වන දින නිකුත් කරන ලද අයවැය චක්‍රලේඛ අංක 06/2015 අනුව,2016 වර්ෂයේ අය හා වැය ඇස්තමේන්තුව මගින් වෙන් කර ඇති ප්‍රාග්ධන ප්‍රතිපාදන යටතේ සඳහන් වැය විෂයන් වලට අනුකූලව Form 1 ආකෘති පත්‍රය සම්පූර්ණ කර එවීම පිළිබඳව ස්තූතිවන්ත වෙමි. \nඑසේ වුවද 2016 පළාත් නිශ්චිත සංවර්ධන සැලැස්මට අනුකූලව, Form 3 a සංශෝධනය සිදු වූ බැවින් ඊට අනුගතව Form 1 ද නැවත සකස් කල යුතු බව ඉහත 2016.04.18 දිනැති ලිපියෙන් ඔබ වෙත දන්වන ලදී.\n03.Form 1 ආකෘති පත්‍රයෙන් සපයන තොරතුරු වලට අනුකූලව Form 2b මගින් ප්‍රගතිය වාර්තා කලයුතු බැවින්, Form 1 ආකෘති පත්‍රය නිවැරදිව සකස් කර 2016.05.13 දිනට පෙර මා වෙත එවීමට කටයුතු කරන ලෙස කාරුණිකව දන්වා සිටිමි.\nකේ.පී.එස්.නිශ්ශංක\nනියෝජ්‍ය ප්‍රධාන ලේකම්(සැලසුම්)\nප්‍රධාන ලේකම් වෙනුවට."
    ,
      "උපදෙස් පත්‍රිකාව\nපළාත් පාලන ආයතනයන්හි මානව, භෞතික හා මූල්‍ය කාර්ය සාධනය පිළිබඳව ඇගැයීම සඳහා තොරතුරු කැඳවීම.\nපළාත් පාලන ආයතනයන් පිළිබඳව කරුණු රැස් කිරීමටත්, එම ආයතන විසින් ඉටු කරනු ලබන කාර්ය සාධනය පිළිබඳව ඇගැයීමක් කර ඒවායේ සාර්ථකභාවය අනුව පෙළගැස්වීමක් කිරීමටත් එම කරුණු පදනම් කරගෙන එම ආයතන කාරක සභාව ඉදිරියට කැඳවීමටත් රාජ්‍ය ගිණුම් කාරක සභාව තීරණය කර ඇත. \nඒ සඳහා කාරක සභාව විසින් අනුමත කළ ඇගැයීම් පත්‍රිකාවක් මේ සමඟ එවා ඇති අතර  එය ttp://www.parliment.lk/files/committees/docs/pac-performance-ev-form-lg-insti-si.pdf මගින් බා ගත කළ හැක.\nඑම ඇගැයීම් පත්‍රයේ අදාළ ප්‍රශ්නවලට පහත සඳහන් උපදෙස් ද සැලකිල්ලට ගෙන නිවැරදි හා සාධාරණ පිළිතුරු ලබා දිය යුතුය. \nඅදාළ ආයතනයන් විසින් පවත්වා ගෙන යනු ලබන ලේඛන, පොත්පත්, ගිණුම් වාර්තා, සැසඳුම් ප්‍රකාශ, වාර්ෂික පරිපාලන වාර්තා හා සභා වාර්තා වලින් එම තොරතුරුවල නිවැරදිභාවය සනාථ විය යුතුය. \nඑම වාර්තා ඕනෑම විටෙක විගණකාධිපතිවරයාගේ පරීක්ෂාවට ලක් කළ හැකි වන පරිදි සූදානම්ව තැබිය යුතුය.\n ඇගැයීම් පත්‍රය සම්පූර්ණ කිරීම සඳහා පහත සඳහන් උපදෙස් අනුගමනය කරන්න.\n1.ඇගැයීම් පත්‍රයේ සඳහන් එක් ප්‍රශ්නයකට අදාළ එක් පිළිතුරක් පමණක්     ලකුණින් අදාළ කොටුවේ සලකුණු කරන්න.\n2.ඔබගේ ආයතනයට අදාළ නොවන ප්‍රශ්න මෙම ප්‍රශ්නාවලියේ තිබිය හැක. \nඑසේ අදාළ නොවන ප්‍රශ්න සඳහා අදාළ නොවේ යන කොටුවේ     යන සලකුණෙන් සටහන් කරන්න.\n3. ඇගැයීම් පත්‍රිකාව සඳහා පිළිතුරු, ආවරණ ලිපියේ 05වන ඡේදයේ සඳහන් දිනට හෝ ඊට පෙර ලේකම්, රාජ්‍ය ගිණුම් කාරක සභාව, ශ්‍රී ලංකා පාර්ලිමේන්තුව, ශ්‍රී ජයවර්ධනපුර, කෝට්ටේ යන ලිපිනය වෙත එවිය යුතුය. \n4. සම්පූර්ණ කරන ලද ඇගැයීම් පත්‍රවල එම තොරතුරු නිවැරදි බවට ගණකාධිකාරී/කළමනාකරණ සහකාර, නාගරික කොමසාරිස්/ලේකම් හා නගරාධිපති/සභාපති පෞද්ගලිකව මැදිහත් වී සහතික කළ යුතුය.\n5. මෙම ඇගැයීම් පත්‍ර සම්පූර්ණ කිරීමේදී පැහැදිලි කර ගැනීම් අවශ්‍ය වන්නේ නම් පහත සඳහන් නිලධාරීන් වෙත දුරකථනයෙන් දැනුම් දී අවශ්‍ය කරුණු පැහැදිලි කර ගැනීමට හැකි බවද දන්වමි."
    ,
      "සියළුම අමාත්‍යාංශ ලේකම්වරුන්,\nසියළුම පළාත් ප්‍රධාන ලේකම්වරුන්,\nසියළුම දෙපාර්තමේන්තු ප්‍රධානීන්,\nසියළුම දිස්ත්‍රික් ලේකම්වරුන්,\nසියළුම පළාත් පාලන ආයතන ප්‍රධානීන්,\nසියළුම රාජ්‍ය සංස්ථා, ව්‍යවස්ථාපිත මණ්ඩල ප්‍රධානීන්,\nසියළුම රාජ්‍ය සමාගම්වල සභාපතිවරුන්,\nජගත් පරිසර දින සැමරුම් වැඩසටහන - 2016 ජුනි 05\n1972 වර්ෂයේ ස්ටොක්හෝම් නුවර පැවති ජාත්‍යන්තර පරිසර සමුළුවේදී ගන්නා ලද තීරණයකට අනුව බිහිවූ එක්සත් ජාතීන්ගේ පරිසර වැඩසටහනේ (UNEP) සමාරම්භක දිනය සැමරීම උදෙසා සෑම වසරකම ජුනි 05 වන දිනය ''ජගත් පරිසර දිනය'' වශයෙන් නම් කරන ලදී. \nඒ අනුව පරිසර සංරක්ෂණය පිළිබඳව ලෝකවාසී ජනතාව දැනුවත් කිරීමේ හා විවිධ පරිසර ගැටළු පිළිබඳව ප්‍රතිපත්ති සම්පාදකයින්ගේ අවධානය යොමු කරවීමේ අරමුණින් සෑම වසරකම ජුනි මස 05 වන දින ''ජගත් පරිසර දිනය'' සමරනු ලබයි.\n02. ලෝකයේ දැනට වෙසෙන බිලියන 07ක ජනගහනය ඉදිරියේදී තවත් වේගයෙන් වර්ධනය වනු ඇත. \nඒ අනුව, ඔවුන්ගේ අවශ්‍යතා ද දිනෙන් දින වැඩි වනු ඇත. \nමේ නිසා සිදුවන විශාල සංවර්ධන ව්‍යාපෘති, අවිධිමත් පරිභෝජනය සහ නීති විරෝධී වනසත්ව හා ශාඛ ජාවාරම් මෙන්ම, ගිනි තැබීම් හා ලැව්ගිනි හේතුවෙන්ද ලෝකයේ පවතින වනගහනය හා වනසත්ත්වගහනය ඉතා වේගයෙන් අඩුවෙමින් පවතී.\n ඇතැම් ජීවී විශේෂ වඳවීමේ තර්ජනයට ද ලක් වී ඇත. \nවනාන්තර විනාශය, වායු ගෝලයේ කාබන් ඩයොක්සයිඩ් ඇතුළු හරිතාගාර වායු අධිකවීම ආදී කරුණු හේතු කොටගෙන ගෝලීය උණුසුම ඉහළ  යාම නිසා අද වන විට ලෝකය දේශගුණික විපර්යාසවලට මුහුණ දෙමින් සිටී. \nමේ පිළිබඳව දැඩි අවධානයක් යොමු නොකළහොත් මිනිසා ඇතුළු සමස්ත සත්ත්ව සංහතියම නුදුරු අනාගතයේදී ගැටළු රැසකට මුහුණ දෙනු ඇත.\nඑබැවින් වනසම්පත හා වනජීවීන් සුරැකීමත්, වන ව්‍යාප්තිය වැඩිකිරීමත්, ජෛව විවිධත්වය ආරක්ෂා කරගනිමින් පරිසර සංරක්ෂණ වැඩසටහන් ක්‍රියාත්මක කිරීමත් මගින් මෙම ගැටළුවලට යම්තාක්දුරට විසඳුම් ලබාදිය හැකිය.\n මේ පිළිබඳව ලෝකවාසී ජනතාවගේ දැඩි අවධානය යොමු කිරීම පිණිස ''වනජීවී සම්පත් නීතිවිරෝධී ලෙස අලෙවි කිරීමට එරෙහි වෙමු'' (FIGHT AGAINST THE ILLEGAL TRADE IN WILDLIFE) යන තේමාව යටතේ 2016 වසරේ ජගත් පරිසර දිනය ලොව පුරා සමරනු ලබයි."
    ,
      "කළමණාකරු,\nඑම්.ජේ.ආර්.පීරිස් සහ පුත්‍රයෝ,\nඅංක 194,\nකටුගස්තොට පාර,\nමහනුවර.\nඅංක පී.ඒ. 4899 දරණ කැබ් රථයට බැටරියක් ලබා ගැනීම සම්බන්ධයෙනි.\nප්‍රධාන ලේකම් කාර්යාලයේ සැලසුම් හා ප්‍රගති පාලන අංශයට අයත් අංක පී.ඒ. 4899 දරණ කැබ් රථයට බැටරියක් මිලදී ගැනීම සඳහා මිල ගණන් කැඳවීමේ දී ඔබ ආයතනය විසින් ඉදිරිපත් කරන ලද  මිල ගණන් පිලිගත් බවත්, ඒ අනුව 10531 R/L AMARON (M/F) වර්ගයේ බැටරියක් (රු.22,500/=) ලබා දීමට කටයුතු කරන ලෙසත් මෙයින් කාරුණිකව දන්වමි.\n  අදාළ බිල්පත් ගෙවීම සඳහා මේ සමඟ අමුණා ඇති පොදු 35 වවුචරය අත්සන් කර බිල්පත සමඟ ඉදිරිපත් කරන ලෙස වැඩිදුරටත් දන්වමි.\nකේ.පී.එස්.නිශ්ශංක,\nනියෝජ්‍ය ප්‍රධාන ලේකම් (සැලසුම්)"
    ,
      "මගේ අංකය - 7/2/5/1 \nප්‍රාදේශීය ලේකම් කාර්යලය ,\nපල්ලේපොල .\n2016-06-06 \nදිස්ත්‍රික් ලේකම් ,\nදිස්ත්‍රික් ලේකම් කාර්යාලය, \nපල්ලේපොල .\n2016 මැයි මස ගිණුම් සාරාංශ වාර්තාව.\n2016 මැයි මස ගිණුම් සාරාංශ වාර්තාව මේ සමග එවන බව කාරුණිකව දන්වමි.\nගණකාධිකාරී ,\nප්‍රාදේශීය ලේකම් වෙනුවට, \nපල්ලේපොල .\nපිටපත් -     විගණක අධිකාරී. රජයේ විගණක අංශය . දිස්ත්‍රික් ලේකම් කාර්යාලය . මාතලේ. \n\tප්‍රධාන අභ්‍යන්තර විගණක . දිස්ත්‍රික් ලේකම් කාර්යාලය මාතලේ ."
    ,
      "ලේකම්, අධ්‍යාපන රාජ්‍ය අමාත්‍යංශය,\nසියළුම අතිරේක ලේකම්වරු - අධ්‍යාපන අමාත්‍යංශය,\nවිභාග කොමසාරිස් ජනරාල්,\nඅධ්‍යාපන ප්‍රකාශන කොමසාරිස් ජනරාල්,\nනියෝජ්‍ය අධ්‍යක්‍ෂ ජනරාල් - ජාතික අධ්‍යාපන ආයතනය,\nසියලුම පළාත් අධ්‍යාපන අධ්‍යක්ෂවරු,\nසියලුම කලාප අධ්‍යාපන අධ්‍යක්ෂවරු,\nසියලුම විදුහල්පතිවරු / නියෝජ්‍ය විදුහල්පතිවරු,\nසියලුම නියෝජ්‍ය / සහකාර අධ්‍යාපන අධ්‍යක්‍ෂවරු ඇතුළු සියලුම අධ්‍යාපන පරිපාලන සේවයේ නිලධාරීන් වෙත,\nශ්‍රී ලංකා අධ්‍යාපන පරිපාලන සේවා වෘත්තීය සංගමයේ මහා සභා රැස්වීම - 2015/16\nශ්‍රී ලංකා අධ්‍යාපන පරිපාලන සේවා වෘත්තීය සංගමයේ මහා සභා රැස්වීම 2016 මාර්තු 3 වන බ්‍රහස්පතින්දා දින පෙ.ව. 9.30 සිට මහරගම වාණිජ විද්‍යා ජාතික අධ්‍යාපන විද්‍යාපීඨ ප්‍රධාන ශාලාවේදී පැවැත්වීමට කටයුතු යොදා ඇත.\nමේ සඳහා සහභාගී වන ලෙස ශ්‍රී ලංකා අධ්‍යාපන පරිපාලන සේවයේ සියලුම නිලධාරීන්ට ආරාධනා කර සිටිමු.\nසැළකිය යුතුයි\n01. න්‍යාය පත්‍ර සහ වැඩි තොරතුරු 2016 මාර්තු 01 වන දින සිට සංගමයේ නිල වෙබ් අඩවිය වන sleas.org හි ප්‍රකාශයට පත් කෙරෙනු ඇත.\n02. ආයතන සංග්‍රහයේ xxv වැනි පරිච්ඡේදයේ 2.1 වගන්තිය අනුව මෙදින සහභාගී වීම සඳහා සියළුම අධ්‍යාපන පරිපාලන සේවයේ නිලධාරීන් සඳහා අධ්‍යාපන ලේකම් විසින් රාජකාරි නිවාඩු අනුමත කර ඇත.\nඋප ලේකම්, \nප්‍රභාත් අයි. විතානගේ,\nලේකම් වෙනුවට."
    ,
      "කලාප අධ්‍යාපන අධ්‍යක්ෂ, \nකලාප අධ්‍යාපන කාර්යාලය.\nවිදු ලොවට මං කියාදෙන සොබාදහම සමස්ථ ලංකා විද්‍යා තරගාවලිය 2016\nසිංහල/ දෙමළ / ඉංග්‍රීසි මාධ්‍ය - 9 වන ශ්‍රේණිය\nමහත්මයාණෙනි/ මහත්මියනි\nජාතික මුලික අධ්‍යාපන ආයතනයේ විද්‍යා අධ්‍යාපන හා ව්‍යාප්ති අංශය විසින් සංවිධානය කරනු ලබන සමස්ථ ලංකා විදු ලොවට මං කියාදෙන සොබාදහම තරඟාවලිය පාසැල් සිසුන් අතර සොබාදහම තම පැවැත්ම සඳහා භාවිතා කරන ක්‍රම විධි හා ඒවා මානව වර්ගයාගේ සුභසිද්ධිය සඳහා අනුකරණය (ජෛවානුකරණය) කර ඇති ආකාරය පිළිබඳව අවබෝධය ලබා දීම තුලින් සොබාදහම දෙස නවතාවයකින් බැලීමට උනන්දු කරවීම මෙම තරඟාවලියේ අරමුණයි.\nසිසුන් සිදු කල යුත්තේ, ඔවුන්ගේ නෙත ගැටෙන පරිසරයෙහි සිදුවන සංසිද්ධීන් ඇතිවන විවිධ ක්‍රම විධි (යාන්ත්‍රණයන්) හඳුනා ගැනීමයි. \nඉන් අනතුරුව එම යාන්ත්‍රණ (ස්වාභාවික නිර්මාණ) මිනිසා විසින් තමන්ගේ එදිනෙදා කටයුතු සඳහා මෙන්ම, මානව සංවර්ධනය සඳහා යොදා ගෙන ඇති(අනුකරණය කර ඇති) ආකාරය නිරීක්ෂණය කළ යුතුය. \nඉන් පසු ඔබ නෙත ගැටුණු  අනුකරණ වලින් වඩාත්ම සිත්ගත් අනුකරණ 5ක් තෝරා ඒ පිළිබඳව විස්තර අප විසින් හඳුන්වා දී ඇති ආදර්ශ වාර්තාවේ ආකෘති පත්‍රයට අනුකූලව වාර්තාවක් ලෙස සකස් කර අප වෙත එවිය යුතුය..\nතරඟාවලිය පිලිබඳ වැඩි විස්තර මේ සමග අමුණා ඇත.\nමෙම තරඟයේ 1,2,3 ස්ථාන හා විශේෂ කුසලතා සඳහා ත්‍යාග හා සහතික ප්‍රදානය කරනු ලැබේ.\nඅධ්‍යාපන අමාත්‍යාංශයේ අනුමැතිය සහිතව මෙම දීප ව්‍යාප්ත තරඟාවලිය පැවැත්වේ .\nඅදාළ ලිපිය අමුණා ඇත.\n ඔබගේ කලාපයට අයත් පාසැල් වල සිසු සිසුවියන් හා ගුරුභවතුන් දිරිගන්වා එයට ඉදිරිපත් කිරීමට අවශ්‍ය කටයුතු කරන මෙන් ඉල්ලා සිටිමි. \nඔබ කලාපයේ පාසල් වලට බෙදා දීම සඳහා අදාළ විස්තර අවශ්‍ය පමණ මේ සමග ඒවා ඇත.\nස්තුතියි,\nආචාර්ය කුමාරි තිලකරත්න,\nඅංශ ප්‍රධානි/විද්‍යා අධ්‍යාපන හා ව්‍යාප්ති අංශය, \nජාතික මුලික අධ්‍යාපන ආයතනය \nමහනුවර"
    ,
      "දිස්ත්‍රික් ලේකම් ,\nදිස්ත්‍රික් ලේකම් කාර්යාලය, \nමාතලේ .\nමහජනාධාර මුදල වැඩිකර ගැනීම. \nඋක්ත මැයෙන් යුත්  ඔබගේ සමාංක හා 2014.07.30 දිනැති ලිපිය හා බැඳේ.\n02. අදාල චක්‍ර ලේඛණ අනුව තනි පුද්ගලයකු සඳහා ලබාදෙන මහජනාධාර මුදල රු.250/-වන ඉක්මවූ පසු රු.1000 /- දීමනාව ලබාදිය හැකි බව කාරුණිකව සඳහන් කර සිටිමි.\n03. වෙනත් යම් සහනාධාරයක්  ලබාදිය හැකි අවස්ථාවක අයි. ජි. ටී. එස්. කරුණාරත්න මහතාටද ආධාර ලබාදීමට සටහන් කරගත් බව වැඩිදුරටත් සඳහන් කරමි.\nප්‍රාදේශීය ලේකම් ,\nපල්ලේපොල .\n01.ගරු ඉඩම් හා ඉඩම් සංවර්ධන අමාත්‍ය ජනක බණ්ඩාර තෙන්නකෝන් මැතිතුමා - කා.දැ.ස.\n02. අයි.ජි.ටී.එස්.කරුණාරත්න මයා -   කා.දැ.ස."
    ,
      "මගේ අංකය :  ED/07/101/08/TEC/2013\nදිනය : 2013 - 04 - 09\nඩබ්.එන්. අජන්ත මිය (විදුලි ඉංජිනේරු),\nගීතා රූබසිංහ මිය (ගණකාධිකාරී ගෙවීම් 1),\nකමල් අතුකෝරල මහතා (ජෞෂ්ඨ තාක්‍ෂණ නිලධාරී  පාසල් ගොඩනැගිලි).\nතාක්‍ෂණ ඇගයීම් කමිටු පත්කිරීම - 2013\nකමිටුවේ නම - ගා/ අමරසුරිය ගුරු විදුහල වලව්ව හා පිරිමි නේවාසිකාගාර ගොඩනැගිලි අලුත්වැඩියාව සම්බන්ධව.\nප්‍රසම්පාදන මාර්ගෝපදේශ 8.13.4. අනුව කොන්ත්‍රාත් මුදලින් 10% වැඩි කොන්ත්‍රාත් විචලන ලේඛන ඇගයීම සඳහා තාක්‍ෂණ ඇගයීම් කමිටුවේ සභාපති/ සාමාජිකයකු වශයෙන් ඔබව පත්කර ඇති බව සතුටින් දන්වමි.\nඑම තාක්‍ෂණ ඇගයීම් කමිටුවේ සංයුතිය පහත සඳහන් පරිදි වේ.\nසාමාජික - ඩබ්.එන්. අජන්ත මිය (විදුලි ඉංජිනේරු)\nසාමාජික - ගීතා රූබසිංහ මිය (ගණකාධිකාරී - ගෙවීම් 1)\nසාමාජික - කමල් අතුකෝරල මහතා (ජෞෂ්ඨ තාක්‍ෂණ නිලධාරී - පාසල් ගොඩනැගිලි)\n02. උක්ත කාර්යය ප්‍රශස්ත අයුරින් ක්‍රියාත්මක කිරීම සඳහා ඔබ දක්වන සක්‍රීය දායකත්වය ඉතා අගයමි.\nවයි.එම්.එස්. ගුණසේකර \n(ගණකාධිකාරී - ප්‍රසම්පාදන අංශය)\nඅ.ක : එස්.එම්. ගෝඨාභය ජයරත්න\nලේකම් අධ්‍යාපන අමාත්‍යාංශය \nපිටපත් : 1. විගණකාධිකාරී - දැ.ගැ.ස.\n2. අධ්‍යක්ෂ පාසල් ගොඩනැගිලි - දැ.ගැ.ස."
    ,
      "මගේ අංකය  :- ED/07/74/5/02(3)\nදිනය  :- 2014.07.17\nඅධ්‍යක්ෂ,\nසී/ස - එලිවේටර්ස්(පුද්.) සමාගම,\n9 වන මහල,\n315,\nවොක්ෂෝල් වීදිය,\nකොළඹ - 02.\nඉසුරුපාය ගොඩනැගිල්ලේ භාණ්ඩ විදුලි සෝපානය 5 වන මහල දක්වා (අතිරේක මහල් දෙක) දීර්ඝ කර සවිකිරීම.\nඅංක: ED/5/74/5/2/9(ii) දරන ඉහත කොන්ත්‍රාත්තුව සම්බන්ධව එළඹි ගිවිසුමට අනුවයි.\n02. ඉහත කාර්යය ඔබ ආයතනය විසින් 2014 ජූනි මස 09 වන දින අවසන් කර අප වෙත භාරදී ඇත. \nමෙම විදුලි සෝපානය බඩු පටවා ඉහළ පහළ යාමේ දී පහත සඳහන් අඩුපාඩු පවතින බැව් හෙළිදරව් විය.\nනතර කිරීමේ දී ගැස්සීමක් ඇති වීම.\nපළමු මහලේ දී නිසියාකාරව විදුලි සෝපානයේ දොර නොවැසීම.\nඑකවර මහල් 3 කට වඩා බොත්තම් සහිත අංක තේරීම කළහොත් එම අංක සියල්ල සඳහන් බොත්තම් අක්‍රිය වීම.\nඅදාළ මහලට සෝපාන තට්ටුව විවෘත වුවද සෝපාන තට්ටුව මහලට වඩා මඳක් උස්ව පැවතීම.\n 03. ඉහත සඳහන් අඩුපාඩු හැකි ඉක්මණින් සාදා භාණ්ඩ -  විදුලි සෝපානය යථා තත්ත්වයට පත්කර දෙන මෙන් කාරුණිකව ඉල්ලා සිටිමි.\nකමල් අතුකෝරල, \nගොඩනැගිලි කළමනාකරු."
    ,
      "දිස්ත්‍රික් සංඛ්‍යාලේඛනඥ, \nමාතලේ දිස්ත්‍රික්කය. \nසිතියම් සකස් කිරීමේ කාර්යය සම්බන්ධයෙනි.\nජන හා නිවාස සංගණනය  2021\nජන හා නිවාස සංගණනය 2021 ට අදාලව සිතියම් සකස් කිරීමේ කටයුතු සිතියම් විද්‍යා අංශයේදි ආරම්භ කර ඇත.\nමෙවර සංගණනයේදි  චන්ද්‍රිකා ඡායාරූප මත අංකිත කරනු ලබන සිතියමක් ලබාදීමට බලාපොරොත්තු වන අතර එම කටයුත්ත සඳහා 2011 ජන හා  නිවාස සංගණනයට අදාලව සකස් කරනු ලැබූ සිතියම් පාදක කර ගනු ඇත.\nසිතියම් සකස් කිරීමේ දී ශ්‍රී ලංකාවේ කුඩාම පරිපාලන ඒකකය වන ග්‍රාම නිලධාරි වසම පදනම් කොට ගෙන සංගණන කාණ්ඩ වෙන් කිරීමට සැලසුම් කර ඇත. \nමේ අනුව ග්‍රාම නිලධාරි වසම තුල නාගරික ,ග්‍රාමීය හා වතු  අංශ 3 සඳහාම වෙන වෙනම සංගණන කාණ්ඩ වෙන් කිරීම සිදුවේ. \nදිස්ත්‍රික්කවලින් ලබාගන්නා ලද දත්තයන් උපයෝගීකර ගෙන ජන සංගණන හා  ප්‍රජා විද්‍යා අංශයේ  අධ්‍යක්ෂකගේ ප්‍රධානත්වයෙන් එම අංශයේ, නියැදි සමීක්ෂණ අංශයේ හා සිතියම් විද්‍යා අංශයේ නිලධාරින්ගේ සහභාගිත්වයෙන් පැවැත්වූ සාකච්ඡාවේදී යෝජිත සංගණන කාණ්ඩවල තරම පහත පරිදි වේ. \nනාගරික අංශය- ගොඩනැගිලි ඒකක 100-120 අතර \nග්‍රාමීය අංශය- ගොඩනැගිලි ඒකක 80-100 අතර \nවතු  අංශය- ගොඩනැගිලි ඒකක 80-100 අතර \nසිතියම් සකස් කිරීම සම්බන්ධයෙන් ඉහත අනුගමනය කරනු ලබන ක්‍රමවේදය පිලිබඳව දිස්ත්‍රික්කයේ ක්ෂේත්‍ර කාර්යය මණ්ඩලය දැනුවත් කර ඒ සම්බන්ධයෙන් අදහස් හා යෝජනා 2016 මාර්තු මස 31 වෙනිදාට පෙර ලැබනසේ  නියෝජ්‍ය අධ්‍යක්ෂ, සිතියම් විද්‍යා අංශය වෙත එවීමට කටයුතු කරන්න.\nඑසේම දිස්ත්‍රික් සංඛ්‍යාලේඛන අංශ ප්‍රධනීන්ගේ වැඩමුළුවේදී සිතියම් සකස් කිරීමේ  ක්‍රමවේදය පිළිබඳව සාකච්ඡා කිරීමට  නියමිත බවද  දන්වමි.\nඩී.ඩී.ජී.සෙනවිරත්න,\nනියෝජ්‍ය අධ්‍යක්ෂ,\nසිතියම් විද්‍යා අංශය.\nඅත්/කලේ \nඒ.ආර්.සතරසිංහ,\nඅධ්‍යක්ෂ ජනරාල්,\nජනලේඛන හා සංඛ්‍යාලේඛන දෙපාර්තමේන්තුව."
    ,
      "දිනය 2016/05/26\nනාගරික කොමසාරිස්, \nමහා නගර සභාව ,\nමාතලේ.\nමාතලේ වී.ටී.නානායක්කාර උද්‍යානය සංචාරක ආකර්ෂණීය ස්ථානයක් ලෙස සංවර්ධනය කිරීම.\nඋක්ත කරුණ සම්බන්ධයෙන් ඔබගේ අංක සීපීසී/එම්එම්සී/3/2/2/1/2016  හා 2016.05.23 දිනැති, ලිපිය හා බැඳේ.\n02. ඒ අනුව, මෙම ව්‍යාපෘතිය සඳහා මෙයට පෙර ඔබ වෙත යොමු කර ඇති මගේ සමාංක හා 2016.02.24 දරන ලිපියෙන් වෙන් කර ඇති රු. මි. 5.0 ක මුදල සංශෝධනය කර රු. මි. 4.5 ක මුදලක් වෙන් කර ඇත.\n03. ඒ සඳහා ඔබ විසින් ඉදිරිපත් කර ඇති ව්‍යාපෘතියට අදාල ඇස්තමේන්තු වැඩවිෂයන් ක්‍රියාත්මක කිරීම අනුමත කරන අතර, වෙන්කර ඇති මුදල ඉක්මවා නොයන පරිදි කඩනමින් ව්‍යාපෘති කටයුතු ආරම්භ කරන ලෙස කාරුණිකව දන්වමි.\n ව්‍යාපෘති කාර්යය ඉටුකිරීමේ දී මුදල් රෙගුලාසි ආයතන සංග්‍රහවිධිවිධාන වලටත්, ප්‍රධාන ලේකම් විසින් කලින් කලට නිකුත් කරනු ලබන උපදෙස් වලටත්, පළාත්සභා මුදල් රෙගුලාසි වලටත් මහා නගරසභා ආඥා පනතටත් යටත්ව එකී විධිවිධාන දැඩි ලෙස පිළිපැදිය යුතු බව දන්වමි. \nමෙම වෙන් කළ මුදල අදාල නිශ්චිත කාර්යය වෙනුවෙන් පමණක් වන බැවින් වෙනත් කිසිඳු කාර්යයක් සඳහා වියදම් දැරීම හෝ වෙනත් දෙපාර්තමේන්තුවකට පැවරීම හෝ නොකළ යුතුය.\n04.ව්‍යාපෘති කාලය දීර්ග නොකරන බවත් අවිච්ඡේද නොකරන බවත් දන්වමි.\n 2016.09.30 දින වන විට වැඩ අවසන් කර ගෙවීම් සඳහා බිල්පත් හා වවුචර්පත් ඉදිරිපත් කරන ලෙස කාරුණිකව දන්වමි.\nඑච්.එච්.එම්.යූ.බී.හේරත්,\nපළාත් අධ්‍යක්‍ෂ(වැ.බ.),\nවෙළඳ වාණිජ හා සංචාරක දෙපාර්තමේන්තුව,\nමධ්‍යම පළාත.\nපිටපත\n1.පළාත් පාලන කොමසාරිස් - මධ්‍යම පළාත - අදාළ ඇස්තමේන්තු වල පිටපත් මේ සමඟ එවමි.\n මෙම කාර්යය නියමිත දිනට ඉටු කර ගැනීමට අවශ්‍ය මගපෙන්වීම සිදුකරන මෙන් කාරුණිකව ඉල්ලා සිටිමි.\n2.පළාත් පාලන සහකාර කොමසාරිස්, මාතලේ - අදාළ ඇස්තමේන්තු වල පිටපත් මේ සමඟ එවමි. \nමෙම කාර්යය නියමිත දිනට ඉටු කර ගැනීමට අවශ්‍ය මගපෙන්වීම සිදුකරන මෙන් කාරුණිකව ඉල්ලා සිටිමි."
    ,
      "ඔබේ අංකය \n2016.05.27 \nනාගරික කොමසාරිස් ,\nමාතලේ මහා නගර සභාව ,\nමාතලේ .\nමහත්මයාණනි/මහත්මියනි,\nව්‍යාපෘති කළමනාකරණය පිළිබඳ සහතිකපත්‍ර පාඨමාලාව - 2016 \nපළාත් පාලන හා පළාත් සභා අමාත්‍යාංශයේ උපදෙස් මත පළාත් පාලනය පිළිබඳ ශ්‍රී ලංකා ආයතනය විසින් ලංකාවේ පළාත් පාලන ආයතනවල මහජන නියෝජිතයන් හා නිලධාරීන් සඳහා ව්‍යාපෘති කළමනාකරණය (Project Management) පිළිබඳ සහතිකපත්‍ර පාඨමාලාවක් 2016 වර්ෂය සඳහා පැවැත්වීමට සැළසුම් කර තිබේ.\nමෙහි මුලික අරමුණ වනුයේ රාජකාරිමය ලෙස පැවරෙන ව්‍යාපෘතියක හෝ ඒ හා සහ සම්බන්ධ වන වැඩකටයුතු කරගෙන යාමේදී සඵලදායි ලෙස කිරීමට අවශ්‍ය දැනුම ලබා දීමත් , ඇතිවන ගැටළු වලට වඩා සාර්ථක විසදුම් ප්‍රයෝගිකව සොයා ගැනීමට උපකාර වීමත්ය.\nඅන්තර්ගතය ;\nව්‍යාපෘතියක් හදුනා ගැනීම. \nව්‍යාපෘතියක ගුණාංග හා ගති ලක්ෂණ. \nව්‍යාපෘති සැළසුම්කරණය හා අයවැයකරණය.\nව්‍යාපෘති තුළ උපාංග හා මුලධර්ම. \nක්‍රියාත්මක කරවීම. \nපසු විපරම. \nවර්තමානයේදී පළාත් පාලන ආයතන ව්‍යාපෘති කළමනාකරණය සඳහා වැඩි නැඹුරුවක් පවතින බැවින් මෙය කාලීන වැදගත්කමින් යුතු බව අපගේ හැගීමයි . \nපාඨමාලාවේ විස්තර පහතින් දක්වා ඇත.\nඉලක්ක කණ්ඩායම    - පළාත් පාලන ආයතනයන්හි මහජන නියෝජතයින් සහ නිලධාරීන් \nමුලික සුදුසුකම්            - කළමනාකරණ සහකාර සේවයේ තනතුරු , ඊට සමාන්තර හෝ ඊට   ඉහළ තනතුරු දරණ නිලධාරීන් සහ මහජන නියෝජිතයින් \nපාඨමාලා ගාස්තුව     - එක් පුහුණුලාභියෙකු සඳහා සම්පුර්ණ පාඨමාලා ගාස්තුව රු. 15,000/- කි (ආහාරපාන සහිතව)\nභාෂා මාධ්‍ය                 - සිංහල \nකාලය                          - මාස 03 (අඟහරුවාදා දින 12 ක් පමණි)"
    ,
      "8. වෙනත් කරුණු\n8.01. ක්‍රීඩා ශක්ති වැඩසටහන යටතේ දීමනා ගෙවනු ලබන්නේ මෙහි සඳහන් පරිදි ඒ ඒ ක්‍රීඩා සඳහා ඒ ඒ සංචිතයන්හි සිටින ක්‍රීඩක ක්‍රීඩිකාවන් සහ තෝරා ගත් පුහුණුකරුවන් සඳහා පමණි. \nඊට අමතරව එම ක්‍රීඩා සඳහා සංචිත පවත්වා ගැනීම, පුහුණු කිරීම බාධාවක් නොවන අතර ඔවුන්ට දීමනාව නොලැබේ. \nඑසේම වෙනත් ක්‍රීඩා සඳහාද මේ ආකාරයට දීමනා වලින් තොරව සංචිත පවත්වාගෙන යා හැකිය. \nඔවුන්ගේ කාර්ය සාධන මට්ටම් අනුව දීමනාවක් ගෙවීමේ හැකියාව ඔවුන්ගේ ඉල්ලීම මත සලකා බලනු ඇත.\n8.02. ක්‍රීඩක ක්‍රීඩිකාවන් තෝරා ගැනීමේ දී සුදුසු පරිදි අධ්‍යාපන අමාත්‍යාංශයේ හෝ පළාත් අධ්‍යාපන අමාත්‍යාංශයේ, පළාත් අධ්‍යාපන අධ්‍යක්ෂකගේ හෝ විදුහල්පතිවරුන්ගේ අනුමැතිය සුදුසු පරිදි ලබා ගත යුතු වේ. \nඑසේම අධ්‍යාපන ක්‍ෂේත්‍රයේ ශාරීරික උපදේශක වරුන්ගේ සේවය හැකි සෑම විටම ලබාගත යුතු වේ.\nපළාත් ක්‍රීඩා අමාත්‍යාංශයේ හෝ පළාත් ක්‍රීඩා අධ්‍යක්ෂකගේ අනුමැතිය සහ සේවයද හැකි සෑම විටම ලබා ගත යුතු වේ.\n8.03. මෙම චක්‍රලේඛණයේ ආවරණය නොවන යම් කරුණක් ඇත්නම් ඒ පිළිබඳව අධ්‍යක්ෂ (ක්‍රීඩා) අවධානයට යොමු කල යුතු අතර අධ්‍යක්ෂ ජනරාල්ගේ පූර්ව අනුමැතිය මත ඔහු වහාම එම කාරණය පිළිබඳව පිළිතුරක් සාධාරණ කාලයක් තුල ලබා දිය යුතුය.\nකේ.ඩි.එස් රුවන්චන්ද්‍ර,\nඅධ්‍යක්ෂ ජනරාල්.\nපිටපත් - \n1. ලේකම් - ක්‍රීඩා අමාත්‍යාංශය\n2. ලේකම් - අධ්‍යාපන අමාත්‍යාංශය\n3. පළාත් ප්‍රධාන ලේකම්වරුන්\n4. ලේකම් - පළාත් අධ්‍යාපන / ක්‍රීඩා අමාත්‍යාංශය\n5. දිස්ත්‍රික් ලේකම්වරුන්\n6. පළාත් අධ්‍යාපන අධ්‍යක්ෂවරුන්\n7. පළාත් ක්‍රීඩා අධ්‍යක්ෂවරුන්\n8. සියලුම ජාතික ක්‍රීඩා සංගම්වල ලේකම්වරුන්\n9. සියලුම මාණ්ඩලික නිලධාරින්\n10. සියලුම ව්‍යපෘති නිලධාරින්\n11. දිස්ත්‍රික් ක්‍රීඩා නිලධාරින්"
    ,
      "මාතලේ දිස්ත්‍රික් සම්බන්ධීකරණ කමිටු රැස්වීම \n2016.05.27  ප.ව.1.30\nමාතලේ දිස්ත්‍රික් සම්බන්ධීකරණ කමිටුව, එහි සම සභාපතිත්වය දරණ මධ්‍යම පළාත් ප්‍රධාන අමාත්‍ය ගරු සරත් ඒකනායක මැතිතුමා, මාතලේ දිස්ත්‍රික් පාර්ලිමේන්තු මන්ත්‍රීතුමන්ලා වන ගරු ජනක බණ්ඩාර තෙන්නකෝන් මැතිතුමා සහ ගරු රංජිත් අලුවිහාරේ මැතිතුමාගේ ප්‍රධානත්වයෙන් 2016.05.27 දින ප.ව.1.30 ට මාතලේ දිස්ත්‍රික් ලේකම් කාර්යාල ශ්‍රවණාගාරයේදී පැවැත්වේ.\nමෙම රැස්වීමට සහභාගී වන මෙන් ඔබතුමාට ගෞරවයෙන් ආරාධනා කර සිටිමි.\nමෙම රැස්වීමට ඔබ අමාත්‍යාංශ / ප්‍රාදේශීය ලේකම් කාර්යාලය/ දෙපාර්තමේන්තුව/ ආයතනය විසින් ඉදිරිපත් කිරීමට අදහස් කරන ගැටළු හා යෝජනා ඇතොත් 2016.05.20 දිනට පෙර ඉදිරිපත් කරන ලෙස කාරුණිකව ඉල්ලන අතර , අදාල යෝජනා ප්‍රාදේශීය සම්බන්ධීකරණ කමිටුවලදී විසඳා ගත නොහැකි හා ප්‍රාදේශීය ලේකම් කොට්ඨාස කිහිපයකට හෝ දිස්ත්‍රික්කයකටම පොදුවේ බලපාන යෝජනා පමණක් වියයුතු බව සිහිගන්වමි. \nඑසේම එදිනෙන් පසුව ඉදිරිපත්වන යෝජනා මෙම රැස්වීමේදී සාකච්ඡාවට ගැනීමට නොහැකි වන බව ද වැඩිදුරටත් දන්වමි.\nමෙයට,\nවිශ්වාසී,\nනීල් ද අල්විස්.\nමාතලේ දිස්ත්‍රික් ලේකම් /දිසාපති හා මාතලේ දිස්ත්‍රික් සම්බන්ධීකරණ කමිටුවේ ලේකම් සහ ස්වදේශ කටයුතු අමාත්‍යාංශයේ අතිරේක ලේකම්."
    ,
      "ගරු මැතිතුමණි,\nමාතලේ දිස්ත්‍රික් සම්බන්ධීකරණ කමිටු රැස්වීම \n2016.05.27 - ප.ව.1.30\nමාතලේ දිස්ත්‍රික් සම්බන්ධීකරණ කමිටුව, එහි සම සභාපතිත්වය දරණ මධ්‍යම පළාත් ප්‍රධාන අමාත්‍ය ගරු සරත් ඒකනායක මැතිතුමා, මාතලේ දිස්ත්‍රික් පාර්ලිමේන්තු මන්ත්‍රීතුමන්ලා වන ගරු ජනක බණ්ඩාර තෙන්නකෝන් මැතිතුමා සහ ගරු රංජිත් අලුවිහාරේ මැතිතුමාගේ ප්‍රධානත්වයෙන් 2016.05.27 දින ප.ව.1.30 ට මාතලේ දිස්ත්‍රික් ලේකම් කාර්යාල ශ්‍රවණාගාරයේදී පැවැත්වේ.\nමෙම රැස්වීමට සහභාගී වන මෙන් ඔබතුමාට ගෞරවයෙන් ආරාධනා කර සිටිමි.\nමෙදින පැවැත්වෙන දිස්ත්‍රික් සම්බන්ධීකරණ කමිටුව සඳහා ඉදිරිපත් කිරීමට යෝජනා ඇතොත් 2016.05.20 දිනට පෙර ඉදිරිපත් කරන ලෙස කාරුණිකව ඉල්ලන අතර , අදාල යෝජනා ප්‍රාදේශීය සම්බන්ධීකරණ කමිටුවලදී විසඳා ගත නොහැකි හා ප්‍රාදේශීය ලේකම් කොට්ඨාස කිහිපයකට හෝ දිස්ත්‍රික්කයකටම පොදුවේ බලපාන යෝජනා පමණක් වියයුතු බව සිහිගන්වමි. \nඑසේම එදිනෙන් පසුව ඉදිරිපත්වන යෝජනා මෙම රැස්වීමේදී සාකච්ඡාවට ගැනීමට නොහැකි වන බව ද වැඩිදුරටත් දන්වමි.\nමෙයට,\nවිශ්වාසී,\nනීල් ද අල්විස්\nමාතලේ දිස්ත්‍රික් ලේකම් /දිසාපති හා මාතලේ දිස්ත්‍රික් සම්බන්ධීකරණ කමිටුවේ ලේකම් සහ ස්වදේශ කටයුතු අමාත්‍යාංශයේ අතිරේක ලේකම්."
    ,
      "ජිවිත සහතිකය -2016\nප්‍රාදේශීය ලේකම් කාර්යාලය - පල්ලේපොල \n(සෑම විශ්‍රාමිකයෙකුම වර්ෂය ආරම්භයේදීම මෙම සහතිකය විශ්‍රාම වැටුප් ලිපි ගොනුව පවතින ප්‍රාදේශීය ලේකම් කාර්යාලය වෙත ඉදිරිපත් කල යුතුයි )\n1. විශ්‍රාම වැටුප් අංකය -\n2. විශ්‍රාමිකයාගේ සම්පුර්ණ නම -\n3. විශ්‍රාමිකයාගේ උපන් දිනය -\n4. විශ්‍රාම ගිය දිනය - \n5. ජාතික හැදුනුම්පත් අංකය -\n6. විශ්‍රාමිකයාගේ  වැ.අ.වි.වැ. අංකය -\n7. සෙබල අංකය -\n8. ස්ථිර පදිංචි ලිපිනය හා ග්‍රාම නිලධාරී වසම -\n9. විශ්‍රාම වැටුප් ලබා ගන්නා බැංකු ශාඛාව හා ගිණුම් අංකය -\n10. විශ්‍රාම වැටුප් ලබා ගන්නා තැපැල් කාර්යාලය -\nදිනය-                                              විශ්‍රාමිකයාගේ අත්සන \n........................................... විශ්‍රාමික මහතා / මහත්මිය / මෙනවිය මා ඉදිරිපිටදී අත්සන් තැබූ බව සහතික කරමි.\nදිනය -                             සහතික කරන්නාගේ අත්සන හා නිලමුද්‍රව \n(මෙම සහතිකය ග්‍රාම නිලධාරී විසින් සහතික කල යුතුයි)\nමෙම අයදුම්පත පුරවා 2016.01.31 දිනට පෙර ආපසු එවන්න."
    ,
      "බැවින් අමර උදාගම, නැටිය, මලගනේ යන ලිපිනයේ පදිංචි ආර්. එම්. දයාරත්න යන අයගේ කාර්මික කැණීම් බලපත්‍රය සදහා මාගේ නිර්දේශය ලබාදිය නොහැකි බව කාරුණිකව දන්වා සිටිමි.\nඅංක 275, සුභසිංහපුර ,ආලංකූලම, ගල්වල.\nකැණීම් බලපත්‍රය අළුත් කිරීම පිළිබඳව ඉදිරිපත් කරන ලද ඔබගේ ඉල්ලීම් ලිපිය හා බැඳේ.\n2. ජ්‍යෙෂ්ඨ මිනින්දෝරු අධිකාරීගේ අංක කු/මැ/3/රස්නායකපුර/විවිධ/2/ සහ 2013.12.03 දිනැති ලිපියෙන් උක්ත ගල්වල පිහිටි ඉඩම පෞද්ගලික ඉඩමක් බව තහවුරු කර ඇති බැවින් එම ඉඩමේ අයිතිය තහවුරු කිරීම සඳහා පත්ඉරු සමග ඉඩමේ ඔප්පුවඉදිරිපත් කරන ලෙස දන්වමි.\n3. එසේ කිරීමට ඔබ අපොහොසත් වන්නේ නම් ඔබගේ ඉල්ලීම ඉටුකිරීමට හැකියාවක් නොමැති බව වැඩි දුරටත් කාරුණිකව දන්වමි.\nකාර්මික කළුගල් කැඩීමේ ව්‍යාපෘතිය  - තූනිගල , කුරුවිකුලම.\nඋක්ත කරුණ සම්බන්ධයෙන් ඔබේ අංක.......  හා 2015.07.22 දිනැති ලිපිය හා බැඳේ.\n02. ඒ අනුව 271 කුරුවිකුලම ග්‍රාම නිලධාරි වසමේ තෝනිගල ඉඩමේ කළුගල් කැඩීම සඳහා මධ්‍යම පරිසර අධිකාරි නිර්දේශය ලැබී ඇති අතර පුරාවිද්‍යා දෙපාර්තමේන්තුවේ නිර්දේශයට යටත්ව කුරුවිකුලම, කඩිගාව යන ලිපිනයේ පදිංචි ඊ. ඒ. මහින්ද එදිරිසිංහ මහතාට මගේ අවසරය ලබා දෙන බව කාරුණිකව දන්වමි."
    ,
      "සියළුම දිස්ත්‍රික් සංඛ්‍යාලේඛන අංශ ප්‍රධානීන් වෙත,\nකෘෂිකර්ම සංඛ්‍යාති එක් රැස් කිරීම විධිමත් කිරීම -2016\n2013/14 ආර්ථික සංගණනයේ කෘෂිකාර්මික ගණන් ගැනීමෙන් පසු 2015 වර්ෂය තුල කෘෂිකාර්මික සංඛ්‍යාලේඛන එක් රැස් කිරීම හා වාර්තා කිරීම ප්‍රමාදයකින් තොරව සිදුකිරීම සම්බන්ධව ඔබට ස්තූතිවන්ත වෙමි.\n2016  වර්ෂය සඳහා කරන එක් රැස් කිරීම මෙහි සඳහන් කරන දින වකවානුවට ප්‍රථමව මා වෙත ලබාදීමට කටයුතු කරන මෙන්  කාරුණිකව දන්වන අතර මේ පිළිබඳව ඔබගේ ක්ෂේත්‍ර නිලධාරින් දැනුවත් කිරීමට කටයුතු කරන මෙන් දන්වා සිටිමි.\nවී සංඛ්‍යාලේඛන  යල කන්නය මහ කන්නය \t\t\t\t\t\nවගා කිරීමට අපේක්ෂිත බිම් ප්‍රමාණය \t\t\nඔක්තොම්බර් 31 ට පෙර  මැයි 31 ට පෙර \nවගාකළ බිම් ප්‍රමාණය හා නෙලීමට අපේක්ෂිත බිම් ප්‍රමාණය නොවැම්බර් 30 ට පෙර ජූනි 30 ට පෙර \nවගාකළ බිම් ප්‍රමාණය හා වගා හානි ජනවාරි15 ට පෙර අගෝස්තු 01 ට පෙර  වූ බිම් ප්‍රමාණය\nවගාකළ බිම් ප්‍රමාණය ජනවාරි 15 ට පෙර  අගෝස්තු 01 ට පෙර \nකපා නොගත් බිම් ප්‍රමාණය මැයි 30 ට පෙර  නොවැම්බර් 10 ට පෙර\nගොඩ ගොවිතැන්  සංඛ්‍යාලේඛන වාර්තා ජුනි 30 ට පෙර  නොවැම්බර් 30ට පෙර  වාර්ෂික\t\t\t\t\t\n සතුන් පිළිබඳ සංඛ්‍යාලේඛන සැප්තැම්බර් 15 ට පෙර \nමස් පිණිස සතුන් මැරීම පිළිබඳ සංඛ්‍යාලේඛන මාර්තු 30 ට පෙර \n ලොකු ළූණු සමීක්ෂණ  උපලේඛන ගෙන්වා ගැනීම  ජුනි 10 ට පෙර \nඋපලේඛන ගෙන්වා ගැනීම සැප්තැම්බර් 15 ට පෙර \nබී ළූණු වගා කරන ගොවීන්/බිම් ප්‍රමාණය\t ඔක්තෝබර් 01 ට පෙර \n පොල් නිෂ්පාදන  සමීක්ෂණ  සැප්තැම්බර් 15 ට පෙර\nඅර්තාපල් සමීක්ෂණ සමීක්ෂණ වටයන්  ජනවාරි /පෙබරවාරි මාර්තු  31 ට පෙර අප්‍රේල් 15 ට පෙර  අප්‍රේල්/මැයි   ජුලි 15 ට පෙර ජූලි 31 ට පෙර අගෝස්තු/සැප්තැම්බර් ඔක්තෝබර් 31 ට පෙර  නොවැම්බර් 15 ට පෙර \nඅධ්‍යක්ෂ,\nකෘෂිකර්ම හා පාරිසරික සංඛ්‍යාලේඛන අංශය,"
    ,
      "මගේ අංකය : DSKU/SWD/SS/\nදිනය : 2016.\nදායක අංකය:\nවිශ්‍රාම වැටුප් යෝජනා ක්‍රමයෙන් ඉවත්වීම/ ඔප්පුව ලබා ගැනීම/ ඔප්පුව සංශෝධනය කිරීම .\nසමාජ ආරක්ෂණ විශ්‍රාම වැටුප්  ක්‍රමයේ  මුදල් ආපසු ලබා ගැනීම/ ඔප්පුව ලබා ගැනීම/ ඔප්පුව සංශෝධනය කිරීම සදහා ඔබ විසින් යොමු කරන ලද ........................... දිනැති ලිපිය හා බැඳේ.\nඒ සම්බන්ධයෙන් කටුයුතු කිරීම සඳහා පහත ලේඛන සති දෙකක් තුළ දිස්ත්‍රික් ලේකම් කාර්යාලයේ පිහිටි ශ්‍රී ලංකා සමාජ ආරක්ෂණ මණ්ඩල දිස්ත්‍රික් කාර්යාලය වෙත මෙම ලිපියේ පිටපතක් සමග භාර දීමට කටයුතු කරන ලෙස කාරුණිකව දන්වමි.\nමුදල් වාරික සටහන් පොතේ මුල් පිටපත.\nසමාජ ආරක්ෂණ ඔප්පුවේ මුල් පිටපත.\nපත්වීම් සහතිකයේ ජයා පිටපත.\nමුදල් වාරික සටහන් පොතේ ජයා පිටපත.\nජාතික හැඳුනුම්පතේ ජය පිටපත. \nපදිංචිය සනාථ කිරීමේ ග්‍රාම සේවක සහතිකය .\nආර්ථික අපහසුතාවය සනාථ කිරීමේ ග්‍රාම සේවක සහතිකය. \nඑච්.කේ.එම්. රංජිත් දිසානායක,\nදිස්ත්‍රික් සම්බන්ධීකරණ නිලධාරී."
    ,
      "6. ක්‍රීඩා ශක්ති සඳහා තෝරා ගන්න ලද ක්‍රීඩක ක්‍රීඩිකාවන් සහ පුහුණු කරුවන්ට ගෙවීම්.\n6.01. දිස්ත්‍රික් සංචිතයේ ක්‍රීඩක ක්‍රීඩිකාවන්ට ගෙවීම\nපැයකට රු.60.00 බැගින් පැය 2 කට දිනකට රු. 120.00\nදිනකට රු.120.00 බැගින් සතියකට දින 3 කට රු. 360.00\nමාසයකට පැය 24 ක් සඳහා රු. 1,440.00\nමාසයකට රු.1440.00 බැගින් අවුරුද්දකට රු. 17,280.00\nදිස්ත්‍රික් ප්‍රධාන පුහුණුකරුට ගෙවීම\nපැයකට රු. 200.00 බැගින් දිනකට පැය 2 කට රු..400.00\nදිනකට රු. 400.00 බැගින් සතියකට දින 3 කට රු.1,200.00\nමාසයකට පැය 24 සඳහා රු.4,800.00\nමාසයකට රු.4800.00 බැගින් අවුරුද්දකටරු.57,600.00\n6.02 දිස්ත්‍රික් සංචිතයේ ක්‍රීඩක ක්‍රීඩිකාවන් ගෙවීම.\nපැයකට රු. 90.00 බැගින් දිනකට පැය 2 කට රු..180.00\nදිනකට රු. 180.00 බැගින් සතියකට දින 03 කට රු.540.00\nමාසයකට පැය 30 සඳහා රු.2,700.00\nමාසයකට රු.2700.00 බැගින් අවුරුද්දකට රු.32,400.00\nදිස්ත්‍රික් ප්‍රධාන පුහුණුකරුට ගෙවීම්\nපැයකට \tරු. 300.00 බැගින් දිනකට පැය 2 කට  රු..600.00\nදිනකට \tරු. 600.00 බැගින් සතියකට දින 3 කට රු.1,800.00\nසතියක\tරු..1, 800.00  බැගින් පැය 30 කට රු.9,000.00\nමාසයකට \tරු.9,000.00 බැගින් අවුරුද්දකට රු.108,000.00\n6.03. ක්‍රීඩා ශක්ති ජාතික සංවර්ධන සංචිතයේ ක්‍රීඩක ක්‍රීඩිකාවන් ගෙවීම.\nපැයකට රු. 120.00 බැගින් දිනකට පැය 2 කට රු..240.00\nදිනකට රු. 240.00 බැගින් සතියකට දින 3 කට රු.720.00\nමාසයකට දින 2 කට  රු.400.00\nමාසයකට රු.2,800.00 බැගින් අවුරුද්දකට රු.33,600.00\nප්‍රධාන පුහුණුකරුට මාසයකට රු. 10,000.00 බැගින් අවුරුද්දකට රු.120,000.00"
    ,
      "දිනය - 2015.12.23\nසුපිරි වෙළඳ සංකීර්ණයේ දෙවන මහලේ කඩ කාමර වෙන්කර දීම.\nඉහත සඳහන් වෙළඳ සංකීර්ණයේ කඩ කාමර බදු ගැනීම පිණිස ඉදිරිපත් කරන ලද මිල ගණන් පිළිගෙන අංක  ......................................... දරණ කඩ කාමරය ඔබගේ නමින් වෙන් කර ඇති බව සතුටින් දන්වමි.\nදැනට ඔබ විසින් ගිවිසුම් වශයෙන් මාස 03 ක ඇප තැන්පත් කර ඇති බවින් එම කඩ කාමරයේ ඉතිරි වැඩ කොටස් නිම කරගෙන හැකි ඉක්මනින් ඔබගේ වෙළඳ ව්‍යාපාරය ආරම්භ කරන මෙන් කාරුණිකව දන්වමි.\nනාගරික කොමසාරිස්."
    ,
      "පිටපත් දෙකකින් ඉදිරිපත් කල යුතුයි.\nභාණ්ඩ ඉල්ලුම් පත්‍රය.\n ඉහත සඳහන් භාණ්ඩ මා වෙත ලබා දීමට අවශ්‍ය කටයුතු සලසන ලෙස කාරුණිකව ඉල්ලා සිටිමි.\n දිනය :- 2013. 09.16\nනිලධාරියාගේ අත්සන\nගබඩා භාරකරු\nසටහන් කර ඇති ද්‍රව්‍ය නිකුත් කරන්න.\nදිනය : 2013 09.\nඉහත සඳහන් භාණ්ඩ භාර ගතිමි.\nප්‍රාදේශීය ලේකම්\nදිනය \nනිලධාරිවරයාගේ අත්සන\nග්‍රාමනිලධාරි, \nඅංක 276,\nරස්නායකපුර වසම.\nග්‍රාම නිලධාරි වාර්තා ලබාගැනීම.\nඔබගේ වසමට අයත් පහළ මාවතගම ගමේ පිහිටි දංගහවත්ත යන ඉඩම් රජයට අයත් ද නැතහොත් සින්නක්කර හෝ පරවේණි ඉඩමක්ද යන වග 2015.03.03 දිනට පෙර මා වෙත වාර්තා කරන ලෙස කාරුණිකව දන්වමි.\nඔබගේ පහසුව පිණිස අදාළ ඔප්පු පිටපතක් මීට අමුණා ඇත.\nදීමනාපත්‍රවල උරුම හිමිකම ලබාගැනීම \n1. ඉල්ලුම්කරුගේ / ඉල්ලුම් කරුවන් පිලිබඳ විස්තර.\nඅනු අංකය \nඉල්ලුම් කරුගේ / ඉල්ලුම් කරුවන්ගේ සම්පූර්ණ නම (උප්පැන්න සහතිකය / ජාතික )\nස්ථීර පදිංචි ලිපිනය \nජාතික හැදුනුම්පත් අංකය \nදීමනාපත්‍ර කරුට ඇති ඥාතිත්වය"
    ,
      "මාතලේ දිස්ත්‍රික්කයේ සියළුම ප්‍රාදේශීය ලේකම්වරුන් වෙත,\nරාජ්ය පරිපාලන සේවක සංඛා තොරතුරු පද්ධතිය(PACIS)මෘදුකාංගය හදුන්වාදීම.\nඋක්ත කරුණ සම්බන්ධයෙන් රාජ්‍ය පරිපාලන හා කළමනාකරණ අමාත්‍යාංශ ලේකම්ගේ අංක CS/Rec/Vacancies/2013 හා 2015.05.12 දිනැති ලිපිය හා බැඳේ.\n02. ඒ අනුව , අමාත්‍යාංශ හා දෙපාර්තමේන්තුවලින් රාජ්‍ය පරිපාලන හා කළමනාකරණ අමාත්‍යාංශයට අයත් දීප ව්‍යාප්ත සේවාවන්ගේ හා ඒකාබද්ධ සේවාවන්ගේ නිලධාරින්ගේ සේවක සංඛ්‍යා තොරතුරු ගෙන්වා ගැනීම වරින්වර රාජ්‍ය පරිපාලන හා කළමනාකරණ අමාත්‍යාංශය මඟින් සිදුකලද සේවක සංඛ්‍යාවේ ප්‍රමාණය හා පුරප්පාඩු පිරවීමේ කටයුතුවලදී අවශ්‍ය කටයුතු සිදුකිරීමට දැනට පවත්නා ක්‍රමවේදය ප්‍රමාණවත් නොවන බව පෙනී ගොස් ඇති බවත් ඊට විසඳුමක් ලෙස 2016.05.02 දින සිට ඉහත මෘදුකාංගයට අදාළ සබැඳිය (link) රාජ්‍ය පරිපාලන අමාත්‍යාංශයේ වෙබ් අඩවියේ පලකර ඇති බව මා වෙත දන්වා ඇත.\n03.තවද, ඔබ කාර්යාලයේ දැනට පවත්නා අනුමත, සේවයේ සිටින හා පුරප්පාඩු සේවක සංඛ්‍යාව සම්බන්ධ තොරතුරු ඔබ විසින්ම යාවත්කාලීන කිරීම සඳහා අතුරුමුහුණතක් (Interface) ඉහත මෘදුකාංගය මගින් ලබාදෙනු ලැබේ, \nඑයට ඇතුලත් වී එමඟින් ලබාදෙන උපදෙස් අනුව සේවක සංඛ්‍යා තොරතුරු ඇතුලත් කිරීම සිදුකල යුතු අතර ඉදිරියේදී අන් ආකාරයකින් එම තොරතුරු කැඳවීම සිදු නොකරනු ලබන බවත් එම තොරතුරු දීපව්‍යාප්ත හා ඒකාබද්ධසේවා පුරප්පාඩු පිරවීමේදී අනිවාර්ය සැලකිල්ලට ගනු ලබන බවත් දන්වා සිටිමි.\n04. එබැවින්, මා වෙත ලබා දී ඇති ඊට අදාල උපදෙස් පත්‍රිකාව සහ මෘදුකාංගය සඳහා ඇතුලත්වීමට අවශ්‍ය මුරපදය මේ සමඟ අමුණා එවන අතර එම උපදෙස් පත්‍රිකාවට අනුව  කටයුතු කිරීම සඳහා සුදුසු සම්බන්ධීකරණ නිලධාරියෙකු පත් කර මා දැනුවත් කරන ලෙසත් වැඩිදුරටත් කාරුණිකව දන්වා සිටිමි.\nතේජානි තිලකරත්න,\nඅතිරේක දිස්ත්‍රික්ලේකම්.\nඅත්/කලේ. නීල් ද අල්විස්,\nදිස්ත්‍රික් ලේකම්/දිසාපති,\nමාතලේ."
    ,
      "සභාපති,\nඅරුණළු ප්‍රජා මුල සංවිධානය,\nරජ්ජම්මන, මඩවල උල්පත.\nගම්මාන සංවර්ධන වැඩසටහන - 2016\nඋක්ත වැඩසටහන යටතේ මොරගහමඩ ග්‍රාම නිලධාරී වසමේ ක්‍රියාත්මක කිරීමට නියමිත මොරගහමඩ ගමට ජලය ලබාදීම නැමති ජල ව්‍යාපෘතිය හරහා පවුල් විස්සක් පමණ ජලය ලබා ගැනීමට අපේක්ෂාවෙන් පසුවන අතර ඒ සඳහා ජල නල ලබාදීම පිණිස රු. 200,000.00 ක මුදලක් වෙන්කර ඇත.\n02. මෙම ජල ව්‍යාපෘතියේ ඉදිකිරීම් කටයුතු අරුණළු ප්‍රජා මුල සංවිධානය මගින් ක්‍රියාත්මක කරන බැවින් ප්‍රදේශයේ ජනතාවගේ ඉල්ලීම පරිදි අදාළ ජල නල ඔබ ප්‍රජා  මුල සංවිධානය වෙත ලබාදීමට තීරණය කර ඇත. \nඑසේ නමුත් ජල මීටර් සවිකර ජලය ලබා ගැනීම සදහා එක පවුලකින් රු. 15,000.00 ක පමණ මුදලක් ය කිරීමට නියමිත බව දැන ගන්නට ලැබී ඇත. \nජල ව්‍යාපෘතිය සඳහා රජය මඟින් රු. 200,000.00 ක මුදලක් ලබා දෙන බැවින් හා පවුල් විස්සක් පමණ ජලය ලබා ගැනීමට අදහස් කර ඇති බැවින් අදාල ගාස්තුව එක පවුලක් සඳහා රු.5000.00 ක් පමණ විය යුතු බවට පෙනී යයි. \nඑබැවින් මේ සම්බන්ධයෙන් තීරණයක් ගැනීම සඳහා 2016.06.27 වන දින පෙ.ව. 9.00 ට මා හමුවන මෙන් කාරුණිකව දන්වා සිටිමි. \nඑම්.ඒ.එස්. කුලසුරිය,\nසහකාර අධ්‍යක්ෂ ( සැලසුම් .)\nඅත්/කලේ, කේ.පී.කේ.එල්.පී. මධුවන්ති,\nප්‍රාදේශීය ලේකම් - මාතලේ."
    ,
      "01. රැස්වීම කැඳවීමේ නිවේදනය කියවීම.\nනාගරික ලේකම් විසින් රැස්වීම කැඳවීමේ නිවේදනය කියවන ලදී.\n02. i. 2016.03.23 වන දින පැවති විශේෂ මහා සභා රැස්වීමේ වාර්තාව සලකා බලන ලදී.\nඉහත වාර්තාව පිළිගත යුතු යයි ගරු මහා නගර සභා මන්ත්‍රී ටි.එම්.අයි. ආබ්දීන් මහතා යෝජනා කළේය.\nගරු මහා නගර සභා මන්ත්‍රී රාජා පුෂ්ප කුමාර මහතා ස්ථිර කරන ලදී.\nii. 2016.03.28 වන දින පැවති සාමාන්‍ය මහා සභා රැස්වීම් වාර්තාව සලකා බලන ලදී.\nගරු මහා නගර සභා මන්ත්‍රී ටි.එම්.අයි. ආබ්දීන් මහතා මෙසේ පැවසීය.\nපිටු අංක 90 හි අංක : 65 හි වාර්තාව සමග වැටුප් වැඩිවීමට අදාලව තිබෙන සියළුම නිර්දේශයන් ද ගරු ආණ්ඩුකාරතුමියගේ අනුමැතියට යැවිය යුතුයි යනුවෙන්ද, ගරු මහා නගර සභා මන්ත්‍රී ජනක ගාමිණිරත්න මහතා  මෙසේ පැවසීය.\nපිටු අංක 81 හි අංක : 59 යටතේ, අයිතම 11 පිළිගත් සමිතියක් මඟින් ඉටු කරවා ගත යුතුය  යන්න විශේෂ ව්‍යාපෘතිය මඟින් ඉටු කරවා ගත යුතුයලෙස ද, පිටු අංක 83 හි අංක : 59 යටතේ, අයිතම 20 දෙපාර්තමේන්තුව මඟින් ඉටු කරවා ගත යුතුය යන්න විශේෂ ව්‍යාපෘතිය මඟින් ඉටු කරවා ගත යුතුය ලෙසද සංශෝධනය විය යුතුය.\nඉහත සංශෝධනයන්ට යටත්ව වාර්තාව පිළිගත යුතු යයි ගරු මහා නගර සභා මන්ත්‍රී විපුල ලාල් රණසිංහ මහතා යෝජනා කළේය.\nගරු මහා නගර සභා මන්ත්‍රී ටි.එම්.අයි. ආබ්දීන් මහතා ස්ථිර කරන ලදී.\nඒකමතිකව සම්මත විය.\n03. ගරු නගරාධිපතිතුමාගේ ප්‍රකාශන/නිවේදන. \n1. මෙම මහා නගර සභාවේ 150 වන සංවත්සරය වෙනුවෙන් සංවිධානය කරන ලද සර්ව ආගමික වැඩසටහන් මාලාව 2016.03.30 වන දින ද 150 අභිමානී ප්‍රදර්ශනය 2016.03.31 වන දින සිට 2016.04.03 වන දින දක්වා පුරා දින 4ක් නාගරික රගහලේ ඉහල මාලයේ මධ්‍යම පළාත්  ගරු ප්‍රධාන අමාත්‍යතුමාගේ ප්‍රධානත්වයෙන් ආරම්භ කොට පැවැත්වූ බව ද සඳහන් කිරීමට මෙය අවස්ථාවක් කර ගනිමි.\n2. සිංහල හා හින්දු අලුත් අවුරුද්ද වෙනුවෙන් ඝන අපද්‍රව්‍ය අංශයේ සේවකයන් වෙනුවෙන් වියලි ආහාර මල්ලක් ලබාදීමේ අවස්ථාව මෙවර ද 2016.04.12 වන දින නගර ශාලාවේ දී සිදුකෙරිණි. \nමෙහිදී සේවකයන් 400 ක් සඳහා වියලි ආහාර මලු ලබාදුන් බව සඳහන් කරමි.\n3. මහනුවර මහා නගර සභාව සංවිධානය කරන ලද බක්මහ උළෙල 2016.04.19 වන දින වේවැල්පිටිය පී.ටි. රාජන් ක්‍රීඩාංගනයේ දී උත්සවාකාරයෙන් පැවතී අතර, ඒ සඳහා අනුග්‍රහය දැක්වූ නෙස්ලේ ආයතනයට මෙම මහා නගර සභාව වෙනුවෙන් ස්තුතිය පිරිනැමීමට ද මෙය අවස්ථාවක් කරගනිමි.\n4. මහජනාධාර දෙපාර්තමේන්තුව මගින් සංවිධානය කරනු ලබන නාගරික පෙර පාසල් දරුවන්ගේ සිඟිති බක්මහ උළෙල 2016.04.29 වන දින එනම් හෙට වේවැල්පිටිය පී.ටි. රාජන් ක්‍රීඩාංගනයේ දී පැවැත්වීමට කටයුතු සංවිධානය කර ඇති අතර, ඒ සඳහා සහභාගී වන ලෙසට ඔබතුමන්ලා සියළුදෙනාට  ආරාධනා කිරීමට ද මෙය අවස්ථාවක් කරගනිමි.\n5. ගරු මහා නගර සභා මන්ත්‍රී ජී. එල්. එම්. නුහුමාන් මහතා මක්කම උම්රා වන්දනාවේ යෙදෙන බැවින් අද දින පැවැත්වෙන මහා සභා රැස්වීමට සහභාගී විය නොහැකි බව දන්වා ඇති අතර ඒ මහතාට නිවාඩු අනුමත කරමි.\n6. ගරු මහා නගර සභා මන්ත්‍රී උපාලි ජයසේකර මහතා අද දින පැවැත්වෙන මහා  සභා රැස්වීමට සහභාගී විය නොහැකි බව ලිඛිතව දන්වා ඇති අතර ඒ මහතාට  නිවාඩු අනුමත කරමි.\n7. ගරු මහා නගර සභා මන්ත්‍රී සංගීත් රංගන සිරි සිල්වා මහතා අද දින පැවැත්වෙන මහා සභා රැස්වීමට සහභාගී විය නොහැකි බව ලිඛිතව දන්වා ඇති අතර ඒ මහතාට නිවාඩු අනුමත කරමි.\n04. මාසික ආදායම් වියදම් ප්‍රගති වාර්තා යනාදිය ඉදිරිපත් කිරීම.\n1. 2016 මාර්තු මස ලැබීම් හා ගෙවීම් ගිණුම.\n2. 2016 මාර්තු මස වැඩසටහන් වියදම් විශ්ලේෂණය.\n3. 2016 මාර්තු මස වැටුප් පාලන වියදම් විශ්ලේෂණය.\n4. 2016 මාර්තු මස අගරු චෙක්පත් ලැයිස්තුව ( පොදු ).\n5. 2016 මාර්තු මස අගරු චෙක්පත් ලැයිස්තුව (ජල).\n6. ජල කර්මාන්ත දෙපාර්තමේන්තුව - 2016 මාර්තු මස ප්‍රගති වාර්තාව.\n7. ජල කර්මාන්ත දෙපාර්තමේන්තුව - 2016.03.31 දිනට හිඟ ආදායම් වාර්තාව.\n05. මුළු සභාවම කාරක සභාවක් බවට පත්විය යුතු යැයි ගරු නගරාධිපතිතුමා යෝජනා කළේය.\nගරු මහා නගර සභා මන්ත්‍රී රාජා පුෂ්ප කුමාර මහතා ස්ථිර කළේය.\nසම්මත විය.\n06. 2016.04.18 වන දින පැවති මුදල් හා ආයතන කටයුතු පිළිබඳ ස්ථාවර කාරක සභා රැස්වීමේ නිර්දේශයන් සහිත වාර්තාව සලකා බැලීම.\nමේ අවස්ථාවේදී ගරු නගරාධිපති සේන දිසානායක මහතා \nගරු නියෝජ්‍ය නගරාධිපති අසංක දොඩම්වල මහතා \nගරු මහා නගර සභා මන්ත්‍රී ටි.එම්.අයි.ආබ්දීන් මහතා \nගරු මහා නගර සභා මන්ත්‍රී සරත් විජේසිංහ මහතා \nගරු මහා නගර සභා මන්ත්‍රී විපුල ලාල් රණසිංහ මහතා\n ගරු මහා නගර සභා මන්ත්‍රී අනුර කුලතුංග මහතා \n ගරු මහා නගර සභා මන්ත්‍රී රාජා පුෂ්ප කුමාර මහතා \n ගරු මහා  නගර සභා මන්ත්‍රී කේසර ඩී. සේනානායක මහතා\n ගරු මහා නගර සභා මන්ත්‍රී  කේ. එල්. සුමින්ද වික්‍රමසිංහ මහතා \n ගරු මහා නගර සභා මන්ත්‍රී ඒ. සී. එම්. ඒ. මරික්කාර් මහතා\n ගරු මහා නගර සභා මන්ත්‍රී ලලන්ත සංජීව පීරිස් මහතා\n ගරු මහා නගර සභා මන්ත්‍රී රෝහිත ලාල්  ධර්මසිරි මහතා\n ගරු මහා නගර සභා මන්ත්‍රී ආනන්ද සිරිවර්ධන මහතා\nගරු මහා නගර සභා මන්ත්‍රීඑම් ජී. සුගත් ධම්මික මහතා\nගරු මහා නගර සභා මන්ත්‍රී ජනක ගාමිණිරත්න මහතා\nගරු මහා නගර සභා මන්ත්‍රී ටී. ටී. කේ. දුනුවීර මහතා\nගරු මහා නගර සභා මන්ත්‍රී කේ. ඒ. ආනන්ද සනත් කුමාර මහතා\nගරු මහා නගර සභා මන්ත්‍රී වරුණ ලසන්ත ලියනගේ මහතා\nගරු මහා නගර සභා මන්ත්‍රී විජිත් සමන්ත පෙරේරා මහතා\nගරු මහා නගර සභා මන්ත්‍රී ආනන්ද විජේරත්න මහතා\nගරු මහා නගර සභා මන්ත්‍රී එම්. එම්. සිරාජ් හසන් මහතා සභා ගර්භයේ රැදී සිටියහ.\n07. යෝජනා ස්ථිරත්වයෙන් නැවත මහා සභාවක් බවට පරිවර්තනය වීම.\n08. කාරක සභා අවස්ථාවේදී ගන්නා ලද විවිධ තීරණයන් පිළිගත යුතු යයි ගරු  නගරාධිපතිතුමා යෝජනා කළේය.\nගරු මහා නගර සභා මන්ත්‍රී විපුල ලාල් රණසිංහ මහතා ස්ථිර කළේය."
    ,
      "මාතලේ දිස්ත්‍රික්කයේ සියළුම ප්‍රාදේශීය ලේකම්වරුන් වෙත,\nපෝෂණය සඳහා වන බහු ආශික ක්‍රියාකාරී සැලැස්ම - 2016\nඋක්ත කරුණ සම්බන්ධව ජනාධිපති ලේකම්ගේ අංක PCMD/Nu/Ms/SCN(Vol III) හා 2016.05.19 දිනැති ලිපිය හා බැඳේ.\n2. ඒ අනුව උක්ත වැඩසටහනේ ඉදිරි ක්‍රියාකාරකම් ක්‍රියාත්මක කිරීම සඳහා 001-1-4-21-2202(3)-(මූල්‍යකරණය 13) වැය විෂය යටතේ රු.35,000.00 ක ප්‍රතිපාදන වෙන්කර ඇත.\nමෙම ප්‍රතිපාදන වැය කිරීමේ දී  මු.රෙ.145(3) හා 146 ට ඔබගේ විශේෂ අවධානය යොමු කරවන අතර, එයට අමතරව පහත සඳහන් විධි විධාන ද අනුගමනය කරමින් ඇමුණුම 1 හි සදහා පමණක් වැය කරන ලෙස කාරුණිකව දන්වා සිටිමි.\n1.ආයතන සංග්‍රහයේ විධි විධාන\n2. මුදල් රෙගුලාසි විධි විධාන\n3.භාණ්ඩාගාර චක්‍රලේඛන උපදෙස්\n4.ප්‍රසම්පාදන මාර්ගෝපදේශ සංග්‍රහයේ විධි විධාන\n5.විශේෂ ගෙවීම් කිරීම සඳහා ආයතන අධ්‍යක්ෂ ජනරාල්ගේ අනුමැතිය\nතේජානි තිලකරත්න,\nඅතිරේක දිස්ත්‍රික්ලේකම්.\nදිස්ත්‍රික් ලේකම්/දිසාපති මාතලේ වෙනුවට"
    ,
      "ක්‍රීඩක ක්‍රීඩිකාවන් තෝරා ගත යුතු ක්‍රම වේදය.\n4.01. ක්‍රීඩා ශක්ති දිස්ත්‍රික් සංචිතය\n1.දැනට දිස්ත්‍රික්කයේ ඇති ක්‍රීඩා ශක්ති පුහුණු සංචිතයේ අඩුපාඩු අනුව ක්‍රීඩක ක්‍රීඩිකාවන් තෝරා ගත හැක.\n2.දැනට දිස්ත්‍රික්කයේ නොමැති ක්‍රීඩා ශක්ති පුහුණු සංචිත සඳහා ක්‍රීඩක ක්‍රීඩිකාවන් අළුතින් තෝරා ගත යුතුය.\n3.එක් එක් ක්‍රීඩාව අනුව ව්‍යපෘති නිලධාරින් විසින් ලබා දෙන උපදෙස් අනුව තෝරා ගැනීමේ කටයුතු සිදු කල යුතුය.\n4.දිස්ත්‍රික්කයේ ඇති ක්‍රීඩා ශක්ති පුහුණු සංචිත සඳහා නවක ක්‍රීඩක ක්‍රීඩිකාවන් තෝරා ගැනීමේ දී එම ක්‍රීඩාවට අයත් ව්‍යපෘති නිලධාරියා දැනුවත් කර (දිනය , ස්ථානය, වෙලාව ) අනුමැතිය ලබා ගත යුතුය.\n5. ක්‍රීඩා ශක්ති ක්‍රීඩක ක්‍රීඩිකාවන් තෝරා ගැනීමේ දී කමිටුවට ඇතුලත් විය යුතු පුද්ගලයින් සඳහා සතියකට පෙර ව්‍යපෘති නිලධාරින්ගෙන් අනුමැතිය ලබා ගත යුතුය.\nක්‍රීඩා ශක්ති පුහුණු සංචිතය සඳහා ක්‍රීඩක ක්‍රීඩිකාවන් තෝරා ගැනීමේ දී දිස්ත්‍රික් සංචිතය සඳහා පත් කල යුතු කමිටුව පහත දැක්වේ. \n1. දිස්ත්‍රික් සම්බන්ධීකරණ නිලධාරි.\n2.ව්‍යපෘති නිලධාරියා විසින් පත් කරනු ලැබූ නිලධාරියෙකු.\n3.තෝරා ගනු ලබන ක්‍රීඩාවට අදාලව පළාත් සභාවේ සිටින පුහුණු කරුවකු/ ක්‍රීඩා නිලධාරිවරයෙකු.\n4.එම ක්‍රීඩාවට අනුව පාසල් සම්මේලනයේ පුහුණුකරු / නිලධාරියෙකු / ශාරීරික අධ්‍යාපන ගුරුවරයෙකු.\n5.එම තෝරා ගන්නා ක්‍රීඩාව අනුව එම මවු සංගමයට අනුබද්ධ දිස්ත්‍රික් /පළාත්/ ලීග් / සංගමයක පුහුණුකරුවකු.\nක්‍රීඩා ශක්ති පුහුණු සංචිත සඳහා ක්‍රීඩක ක්‍රීඩිකාවන් තෝරා ගැනීමේ දී පත් කල යුතු කමිටුව පහත දැක්වේ.\n1. පළාත් ක්‍රීඩා ලේකම්/ අධ්‍යක්ෂ දිස්ත්‍රික් ක්‍රීඩා නිලධාරින්.\n2.දිස්ත්‍රික් ක්‍රීඩා සම්බන්ධීකරණ නිලධාරි.\n3. එම ක්‍රීඩාවට අදාලව දිස්ත්‍රික්ක වල මෙතෙක් පුහුණු කල පුහුණුකරුවන්.\n4.ව්‍යපෘති නිලධාරින් විසින් නම් කරනු ලබන නිලධාරියෙකු.\n4.02. ක්‍රීඩා ශක්ති පළාත් සංචිත\nදිස්ත්‍රික් ක්‍රීඩා ශක්ති පුහුණු සංචිතයට ක්‍රීඩක ක්‍රීඩිකාවන් තෝරා ගැනීමේ දී පළාතට අයත් දිස්ත්‍රික් ක්‍රීඩා සම්බන්ධීකරණ නිලධාරින්ගේ එකඟත්වය අනුව දිනය, ස්ථානය, වෙලාව තීරණය කොට ඒ පිළිබඳව ව්‍යපෘති නිලධාරියා දැනුවත් කර නිර්දේශ ලබා ගත යුතුය."
    ,
      "දිනය - 2016.06.20\nදිස්ත්‍රික් ලේකම් - මාතලේ\nඅධ්‍යක්‍ෂ  - පිළිසරු ව්‍යාපෘතිය මධ්‍යම පරිසර අධිකාරිය.\nසහකාර අධ්‍යක්‍ෂ  - මධ්‍යම පරිසර අධිකාරිය - මාතලේ.\nගරු නගරාධිපති ඇතුළු - සියලුම නාගරික මන්ත්‍රීවරු \nපිළිසරු ව්‍යාපෘතියේ මූල්‍යාධාර මත විස්තීරණය කරණු ලබන හිග්ගොල්ල කොම්පෝස්ට් පොහොර නිෂ්පාදන ව්‍යාපෘතිය - විවෘත කිරීම. 2016.06.23 දින පෙ:ව: 9.30 \nමධ්‍යම පරිසර අධිකාරියේ පිළිසරු ව්‍යාපෘතියේ මූල්‍යාධාර මත විස්තීරණය කරණු ලැබූ මාතලේ මහා නගර සභාවේ කොම්පෝස්ට් පොහොර නිෂ්පාදන ව්‍යාපෘතිය 2016.06.23 දින පෙ:ව 9.30 ට විවෘත කිරීමට කටයුතු සංවිධානය කර ඇත.\nමෙම විවෘත කිරීමේ අවස්ථාවට ඔබගේ සහභාගීත්වය ගෞරවයෙන් අපේක්ෂා කරමි.\nනාගරික කොමසාරිස් \nමාතලේ."
    ,
      "මගේ අංකය : DSKU/SWD/SS/REFUN/6/39\nදිනය : 2016.\nදායක අංකය:\nයෝජනා ක්‍රමයෙන් ඉල්ලා අස්වීම\nඉහත කාරණය සම්බන්ධයෙන් ................ දින අප වෙත කරන ලද ඉල්ලීම් හා බැඳේ.\nවිශ්‍රාම වැටුප් යෝජනා ක්‍රමයේ නීති රීති අනුව මුදල් අපසු ලබා ගැනීම සඳහා ගෙවිය යුතු වාරික සංඛ්‍යාවෙන් 25% ක වාරික සංඛ්‍යාවක් ගෙවිය යුතුය. \nඔබගේ ගිණුම පරික්ෂා කිරීමේදී 25% ක වාරික සංඛ්‍යාව ගෙවීම සදහා ඔබ විසින් ................ගෙවිය යුතු නමුත් ඔබ විසින් ගෙවා ඇත්තේ වාරික ......................... පමණි. \nඒ අනුව ඔබ වෙත මුදල් ආපසු ගෙවීමේ හැකියාවක් නොමැති බව කණගාටුවෙන් දන්වමි.\nඔබ වෙත මුදල් ආපසු ගෙවිය නොහැකි වුවත් ඔබට ඉසුරු හෝ සුරැකුම් යෝජනා ක්‍රම වල නැවත දායකත්වය ලබා ගෙන පෙර විශ්‍රාම වැටුප් යෝජනා ක්‍රමයට ගෙවූ මුදල් නව ක්‍රමයට බැරකර ගත හැක. \nඑසේත් නොමැති නම් වෙනත් අයෙකු සුරැකුම් යෝජනා ක්‍රමයට ග්‍රාම නිලධාරී මගින් හෝ අප කාර්යාලය මගින් දායක වී ඔබ විසින් ගෙවන ලද මුදල් එම ගිණුමට බැර කළ හැකි බව දන්වමි. \nඔබ අපවෙත ඒවා ඇති පහත ආකෘතිපත්‍ර මේ සමග එවමි.\nමුදල් වාරික සටහන් පොත\nඔප්පුව\nනැවත දායකත්වය ලබන්නේ නම් හෝ වෙනත් අයෙකු දායක කර ගෙවූ මුදල් නව ගිණුමට බැර කරන්නේ නම් එම මුදල් නව ගිණුමට බැර කරන ලෙස ඉල්ලීම් ලිපිය, ඔප්පුව, මුදල් වාරික සටහන්  පොත, නව අයදුම්පත්‍රය සමග ලබා දිය යුතු බවද දන්වමි.\nඑච්.කේ.එම්. රංජිත් දිසානායක,\nදිස්ත්‍රික් සම්බන්ධීකරණ නිලධාරී ."
    ,
      "දිනය -  2013.06.26\nප්‍රධාන තක්සේරුකරු,\nතක්සේරු දෙපාර්තමේන්තුව,\nමහනුවර.\nසුපිරි වෙළඳ  සංකීර්ණයේ දෙවන මහල හා තුන්වන මහල තක්සේරු කිරීම.\nමාතලේ මහනගර සභාවට අයත් සුපිරි වෙළඳ සංකීර්ණයේ දෙවන මහල හා තුන්වන මහල ඉදිකර අවසන් කර ඇත.\nඑම මහල්වල කඩකාමර ටෙන්ඩර් කර භාරදීමට සූදානම් කර ඇති බැවින්, වර්ග අඩියක මිළ තක්සේරුකර දීමට කටයුතු සලසා දෙන මෙන් කාරුණිකව ඉල්ලමි.\nතක්සේරු කිරීමට නියමිත මහල්වල සැලැස්ම මේ සමඟ එවමි.\nනාගරික කොමසාරිස්,\nමාතලේ."
    ,
      "රාජ්‍ය පරිපාලන චක්‍රලේඛ : 10/2012(II)\nමාගේ අංක: CS/DOS/02/05/01\nරාජ්‍ය පරිපාලන කළමනාකරණ අමාත්‍යාංශය,\nනිදහස් චතුරශ්‍රය,\nකොළඹ 07.\n2016.04.04\n සියලුම අමාත්‍යාංශ ලේකම්වරුන්,\nදෙපාර්තමේන්තු ප්‍රධානින්,\nදිස්ත්‍රික් ලේකම්වරුන් /ප්‍රාදේශීය ලේකම්වරුන් \nආයතන ප්‍ර‍ධානීන් වෙත.\nසංවර්ධන නිලධාරි සේවා ව්‍යවස්ථාව ක්‍රියාත්මක කිරීම.\nඉහත කරුණ සම්බන්ධයෙන් නිකුත් කරන ලද 2012.05.08 දිනැති රාජ්‍ය පරිපාලන චක්‍රලේඛ 10/2012 හා 2012.06.21 දිනැති රාජ්‍ය පරිපාලන චක්‍රලේඛ 10/2012(I) කෙරෙහි ඔබගේ කාරුණික අවධානය යොමු කරවමි.\n02. සංවර්ධන නිලධාරි සේවා ව්‍යවස්ථාව පිළිබදව නිසි අවබෝධයක් නොතිබීම හේතුවෙන් එම සේවයට අන්තර්ග්‍රහණය වීම සඳහා මීට පෙර මනාපය පළ නොකරන ලද නිලධාරින්ට සංවර්ධන  සේවයට අන්තර්ග්‍රහණය වීමට මනාපය පළ කල හැකි අවසාන දිනය 2016.05.31 ලෙස තීරණය කර ඇත. \nඒ අනුව සංවර්ධන නිලධාරි සේවය සඳහා අන්තර්ග්‍රහණයට මනාප පළ කරන ලද නිලධාරින්ගේ ලිපිගොනු හා අවශ්‍ය ලේඛණ අංක 10/2012 දරන රාජ්‍ය පරිපාලන චක්‍රලේඛයේ සදහන් උපදෙස් පරිදි සකස් කර 2016.06.30 දිනට පෙර ලැබෙන සේ එකාබද්ධ සේවා අධ්‍යක්ෂ ජනරාල් වෙත එවිය යුතුය.\n03. තවද සංවර්ධන නිලධාරි සේවයට අන්තර්ග්‍රහණය වීමට මනාපය පළ කරන ලද නිලධාරින් එම සේවයට අන්තර්ග්‍රහණය කරන තෙක් අන්තර්කාලීන විධි විධානයක් ලෙස සංවර්ධන නිලධාරි සේවයේ තනතුරකට ස්ථාන මාරුවීමට අවස්තාව ලබා දිය හැකි බැවින් එවැනි ඉල්ලීම් අදාල ආයතන ප්‍රධානියාගේ නිර්දේශය සහිතව එකාබද්ධ සේවා අධ්‍යක්ෂ ජනරාල් වෙත  20161.12.31 දක්වා යොමු කල හැකි බව වැඩිදුරටත් දැනුම් දෙමි.\nජේ. දඩල්ලගේ,\nලේකම්,\nරාජ්‍ය පරිපාලන හා කළමනාකරණ අමාත්‍යාංශය."
    ,
      "දිනය 30.05.2016 \nප්‍රධාන ගිනි නිවන නිළධාරී, \nගිනි නිවීමේ ඒකකය. \nවෘත්තීමය සෞඛ්‍යය හා ආරක්ෂාව පිළිබඳ පාඨමාලාව.\n(Course on Occupational Health and Safety)\nශ්‍රී ලංකා වරාය අධිකාරියට අයත් මහපොළ අභ්‍යාස ආයතනය පුහුණු කිරීමේ ක්ෂේත්‍රය තුළ සක්‍රීය දායකත්වයකින් යුතු රාජ්‍ය ආයතනයක් වන අතර මෙහි වෘත්තීය සෞඛ්‍ය හා ආරක්ෂාව, ගිනි ආරක්ෂණය, නාවික කළමනාකරණය සහ තොරතුරු තාක්ෂණය වැනි ආයතනික ප්‍රගමනයට සෘජුවම දායක වන විෂයානුබද්ධ පුහුණු පාඨමාලා රැසක් දැනටමත් පවත්වාගෙන යනු ලබන අතර බොහෝ පාඨමාලා සඳහා ජාත්‍යන්තර තත්ව සහතිකය (ISO 9001-2008) ලබා තිබේ.\nතවද අප ආයතනය මඟින් මෙම ක්ෂේත්‍ර පිළිබඳව ප්‍රවීනයන් රැසකගේ සම්පත් දායකත්වය යටතේ ඉහත පාඨමාලාව දැනට පවත්වාගෙන යනු ලබන අතර මෙහි මූලික පරමාර්ථය වනුයේ මෙවැනි පාඨමාලාවක් ආයතනික වශයෙන් ඉතා වැදගත් වීම මෙන්ම මෙම ක්ෂේත්‍ර තුළ පුහුණු අවස්ථා ඉතා විරලව පැවතීම හේතුවෙන් එය කාලීන අවශ්‍යතාවයක් වන බැවිනි.\nඑසේම මෙවැනි පාඨමාලාවක් ආරම්භ කිරීමේ මූලික අරමුණ වන්නේ කර්මාන්තශාලා ආඥා පනත හා අනෙකුත් ජ්‍යාත්‍යන්තර සම්මත වලට අනුකූල වන සේ ආයතනික වශයෙන් මානව සම්පතෙහි වෘත්තීය සෞඛ්‍ය හා ආරක්ෂාව තහවුරු කරීම මඟින් ආයතනය තුළ සිදුවිය හැකි වෘත්තීය අනතුරු සහ රෝග අවම කිරීම, සුබවාදී සේවාදායකයින් මෙන්ම යහපත් වැඩ පරිසරයක් බිහි කිරීම, ආයතනයේ ඵලදායීතාවය හා කාර්යක්ෂමතාවය වර්ධනය කිරීම වැනි කරුණු සඳහා වෙයි.\nමෙම පුහුණු පාඨමාලාව සකස් කර ඇත්තේ පරිපාලන, ඉංජිනේරුමය හා නිෂ්පාදනය යන ක්ෂේත්‍ර වල නිළධාරීන් සහ සේවක සේවිකාවන් ඉලක්ක කරගනිමින් හා ඔවුන්ගේ සුරක්ෂිතභාවය පිළිබඳව වන අවශ්‍යතා හඳුනා ගනිමිනි. \nඒ අනුව පාඨමාලාවේ ආරම්භය 2016 ජුනි මස හතරවන සතියේ සෙනසුරාදා පැවැත්වීමට කටයුතු සම්පාදනය කර ඇත.\nතවද මේ සමඟ වන අත් පත්‍රිකාව  මඟින් පාඨමාලාව පිළිබඳව වැඩිමනක් තොරතුරු දක්වා ඇත. \nපාඨමාලා ගාස්තුව පාඨමාලා ආරම්භයට සතියකට පෙර ගෙවිය යුතු අතර සහභාගිවන්නන් සඳහා දිවා ආහාර/තේ පැන්/පාඨමාලා අධ්‍යයන පත්‍රිකා සහ අදාල වෙනත් පහසුකම්ද සපයනු ලබන බව කරුණාවෙන් දන්වමි.\nමෙයට,\nවිශ්වාසීවූ,\nරංජිත් සේපාල, \nප්‍රධාන අභ්‍යාස කළමණාකරු."
    ,
      "නාගරික කොමසාරිස්,\nමහනගර සභාව,\nමාතලේ.\nනිවසට අනතුරුදායක ලෙස පිහිටා ඇති ගස් සම්බන්ධවයි.\nමාතලේ පුල්ලේයාර් කෝවිල්පාර, අංක 55/1, දරණ නිවසේ පදිංචි ඩබ්.ජී.එන්.ප්‍රේමනායක යන අය විසින් ඔහුගේ නිවසට සහ තාප්පයට ඉතාමත් අනතුරුදායක තත්වයේ පවතින යාබද ඉඩමේ පිහිටි මී ගසක්,තේක්ක ගසක්,සහ දෙල් ගසක් ඉවත්කර ගැනීමට ඔබ වෙත ඉල්ලීම් කර ඇත. \nමේ වනතෙක් එම ගස් කපා බිම හෙලීමට කටයුතු කර නොමැති බව වසමේ ග්‍රාම නිලධාරි විසින් මා වෙත වාර්තා කර ඇති අතර මෙම ගස් හේතුවෙන් දැඩි අනතුරුදායක තත්වයක් උද්ගතවී ඇති බවද වාර්තා කර ඇත.\n02. මේ දිනවල පවතින අධික වර්ෂාව සහ සුලං නිසා අනතුරු සිදුවිය හැකි බැවින් කඩිනමින් අවශ්‍ය ඉදිරි ක්‍රියා මාර්ග ගෙන මෙම ගස් ඉවත් කිරීමට කටයුතු කරන ලෙසට  කාරුණිකව දන්වමි.\nකේ.පී.කේ.එල්.පී.මධුවන්ති,\nප්‍රදේශීය ලේකම්,\nමාතලේ."
    ,
      "මාතලේ දිස්ත්‍රික් ගරු පාර්ලිමේන්තු මන්ත්‍රී,\nජනක බණ්ඩාර තෙන්නකෝන් මැතිතුමා,\nරංගිරි, යාපාගම,\nදඹුල්ල.\nවිමධ්‍යගත වැඩසටහන ක්‍රියාත්මක කිරීම - 2016\nශීර්ෂගත  වැඩසටහන යටතේ ඔබතුමන් විසින් හඳුනාගත් පහත සදහන් ව්‍යාපෘති හා සම්බන්ධවයි.\nමාතලේ ප්‍රාදේශීය ලේකම් කොට්ඨාශයේ ලියාපදිංචි ස්වේච්ඡා සංවිධාන සඳහා ප්ලාස්ටික් පුටු ලබාදීම. - රු. 400000.00\n02. ඉහත වැඩසටහන ක්‍රියාත්මක කිරීම සදහා ප්ලාස්ටික් පුටු ලබාදිය යුතු ස්වේච්ඡා සංවිධාන නම්  කර දෙන ලෙස කාරුණිකව ඉල්ලා සිටින අතර ලියාපදිංචි ස්වේච්ඡා සංවිධානවල නාම ලේඛණයක්ද ඔබතුමාගේ පහසුව සදහා මෙයට යාකර ඉදිරිපත් කරමි. \nහැකි නම් ලබාදිය යුත්තේ ඇදි රහිත/සහිත ප්ලාස්ටික් පුටුද යන්න නම්කර එවන ලෙස වැඩිදුරටත් ඉල්ලා සිටිමි.\nකේ.පී.කේ.එල්.පී. මධුවන්ති,\nප්‍රාදේශීය ලේකම්,\nමාතලේ."
    ,
      "ඉඩම් අත්කර ගැනීමේ පනතේ ( 460  වැනි පරිච්ඡේදය ) \n15 වගන්තිය යටතේ දැන්වීමයි.\nඉඩම් අත්කර ගැනීමේ පනතේ ( 460  වැනි පරිච්ඡේදය ) 7  වැනි වගන්තියෙන් නියම කර තිබෙන ප්‍රකාර 2016 ක් වූ ජනවාරි මස 21 වන දින දරණ දැන්වීමෙන් මාතලේ දිස්ත්‍රික්කයේ මාතලේ ප්‍රාදේශීය ලේකම් කොට්ඨාශයේ කොහොන්සියපත්තුව සුළු කොට්ඨාශයේ ගොංගොවිල කොට්ඨාශ අංක 10  දියබුබුල ( මාතලේ මහා නගර සභා සීමාව තුල ) ගමේ පිහිටි සර්වෙයර් ජනරාල් විසින් මැන සකස් කරන ලද මුලික පිඹුරු අංක මා 1627 හි පහත විස්තර සඳහන් ඉඩම් පොදු කටයුත්තක් සදහා ලබා ගැනීමට අදහස් කරන බැව් මා විසින් මහජනයාට නිවේදනය කර, එකී ඉඩම ගැන ඉදිරිපත් වීමට හිමිකම් ඇති සියළු දෙනාටම 2016 ක් වූ මැයි මස 02 දින තම තමන්ගේ හෝ  ඔවුන්ගේ විධි වූ ප්‍රකාර ලියවිලි වලින් බලය ලත් අයවලුන් මා ඉදිරියේ පෙනී සිටින්නටය කියා නියම කරන ලද්දේ වී නමුත් එබදු කිසිවකු ඉදිරිපත් නොවුයෙන්,  එකී ඉඩම ගැන අයිතිවාසිකම් දක්වන සියළුම අය තම තමන් හෝ  ඔවුන්ගෙන් විධි වූ ප්‍රකාර ලියවිලි වලින් බලය ලත් අයවලුන් 2016 ක් වූ ජුනි මස 27 දින පෙරවරු 9.00 වේලාවට මාතලේ ප්‍රාදේශීය ලේකම් කාර්යාලයේදී මා ඉදිරියේ පෙනී සිටින්නට ඕනෑය කියා දන්වන අතරම, ඉහත කී ඉඩම පිළිබඳ ඔවුන්ට ඇති අයිතිවාසිකම් ආදියේ ස්වභාවයත් වන්දි ඉල්ලීම පිළිබඳ තොරතුරු හා වන්දි මුදල් ගණනත් මාවෙත කලින් දන්වා නැත්නම් එකී තොරතුරු 2016 ක් වූ ජුනි මස 20 දිනට මත්තෙන් මා වෙත ලියවිල්ලකින් දන්වා එවිය යුතු බැව් මෙයින්  මහජනයාට නිවේදනය කරනු ලැබේ.\nඉහතින් සඳහන් කරන ලද ප්‍රකාරයට අයිතිවාසිකම් ඇති අය පැමිණියත් නොපැමිණියත් එකී ඉඩම ලබා ගැනීම සම්බන්ධයෙන් ගෙවිය යුතු වන්දි මුදල නිශ්චිත කිරීමට විභාගයක් පැවැත්වීමට මා විසින් ක්‍රියා කරගෙන යනවා ඇත.\nකැබලි අංකය \t\nඉඩමේ නම\nවරිපනම් අංක 76 A, රජ වීදිය\nවරිපනම් අංක 76 , රජ වීදිය\nවරිපනම් අංක 76/1/1, රජ වීදිය\nවරිපනම් අංක 74 , රජ වීදිය\nවරිපනම් අංක 74, රජ වීදිය\nකේ. පී. කේ.එල්.පී. මධුවන්ති,\nප්‍රාදේශීය ලේකම්/ මාතලේ.\n2016 ක් වූ ....................................... මස ............................. දින,\nමාතලේ ප්‍රාදේශීය ලේකම් කාර්යාලයේදී."
    ,
      "පළාත් ආයුර්වේද කොමසාරිස්,\nමධ්‍යම පළාත් ආයුර්වේද දෙපාර්තමේන්තුව,\nපල්ලේකැලේ,\nකුණ්ඩසාලය.\nආයුර්වේද නිෂ්පාදනයක් ලියාපදිංචි කිරීම සඳහා ඉල්ලීම සම්බන්ධවයි.\nමාතලේ හුලංගමුව උතුරු ග්‍රාම නිලධාරි වසමේ අංක 35/12 A ලිපිනයේ පදිංචි එච්.බී.ආර්.එම්. අනුරාධ සමරසේකර මහතා විසින් දේශීය ඖෂධ උපයෝගී කර ගනිමින් ස්වභාවික මදුරු ආලේපනයක් නිෂ්පාදනය කර ඇත.\n02. එම නිෂ්පාදනයන් ලියාපදිංචි කිරීමේ ඉදිරි කටයුතු සඳහා කාරුණිකව ඉදිරිපත් කරමි.\nප්‍රාදේශිය ලේකම්,\nමාතලේ."
    ,
      "රාජ්‍යය නොවන සංවිධාන/ ස්වේච්ඡා සංවිධාන ලියාපදිංචි කිරීම.\nක්‍රියාත්මක බල ප්‍රදේශය දිස්ත්‍රික් එකකට වැඩිනම් රා.නො.ස. පිළිබඳ ජාතික ලේකම් කාර්යාලය.\nක්‍රියාත්මක බල ප්‍රදේශය ප්‍රාදේශීය ලේකම් කොට්ඨාශ එකකට වැඩිනම් දිස්ත්‍රික් ලේකම් කාර්යාලය.\nක්‍රියාත්මක බල ප්‍රදේශය එක් ප්‍රාදේශීය ලේකම් කොට්ඨාශයක් නම් අදාල ප්‍රාදේශීය ලේකම් කාර්යාලයේ ලියාපදිංචි කල යුතුය.\nකුරුණෑගල දිස්ත්‍රික් ලේකම් කාර්යාලයේ රාජ්‍යය නොවන සංවිධාන/ ස්වේච්ඡා සංවිධාන ලියාපදිංචි කිරීම සඳහා ඉදිරිපත් කල යුතු ලිපි ලේඛන වසරකට ආසන්න කාල සීමාවක් ක්‍රියාත්මක කර තිබිය යුතු  අතර ප්‍රශස්ත කාර්ය භාරයක් ඉටුකර තිබිය යුතුය.\nලියාපදිංචිය සඳහා ඉල්ලුම් කිරීමේ ලිපිය හා සම්පුර්ණ කරන ලද ඉල්ලුම් පත්‍රය ද සමග පහත සඳහන් ලිපි ලේඛන ඉදිරිපත් කල යුතුය.\n1. සංවිධානයේ ව්‍යවස්ථා මාලාව.\n2. ආරම්භක රැස්වීම් වාර්තාව.\n3. ව්‍යවස්ථාව පිළිගත් රැස්වීම් වාර්තාව.\n4. වත්මන් නිලධාරී මණ්ඩලය පත් කර ගත් රැස්වීම් වාර්තාව.\n5. ස්වේච්ඡා සංවිධාන ලියාපදිංචි කිරීමේ පනත යටතේ ලියාපදිංචි කිරීම සඳහා යෝජනා සම්මත කර ගත රැස්වීම් වාර්තාව.\n6. ලිපිනයන් සහිත නාම ලේඛනය.\n7. පෙර වර්ෂය සඳහා ආදායම් වියදම් ගිණුම් හා ශේෂ පත්‍රය (මෙය රජයේ පිළිගත් ගණකාධිවරයකු විසින් නිකුත් කළ වාර්තාවක් විය යුතුය.)\n8. බැංකු ගිණුමේ ශේෂ පත්‍රය (අවසන් ශේෂය පෙන්විය යුතුය.)\n9. ඉදිරියේදී කිරීමට යෝජිත වැඩ සැලැස්ම.\n10. මෙතෙක් කළ වැඩසටහන්/ ව්‍යාපෘති වල ප්‍රගති වාර්තාව. (මෙය වසරක පමණ කාලසීමාවක් සඳහා සකස් කල යුතුය) \n11. සංවිධානයේ ක්‍රියාකාරී ස්වභාවය.\n12.සංවිධානයේ ව්‍යාපෘති ක්‍රියාත්මක කරනු ලබන ප්‍රාදේශීය ලේකම් කොට්ඨාශ දෙකක ප්‍රාදේශීය ලේකම්වරු අනු අත්සන් කරන ලද ග්‍රාම නිලධාරී සහතික.\n13. පොලිස් වාර්තාව."
    ,
      "මාගේ අංකය : පුවි/නැ/4/4/53\nපළාත් පුරාවිද්‍යා කාර්යාලය,\nඅම්පාර.\n2015.06.03 \nදිසාපති,\nදිස්ත්‍රික් ලේකම් කාර්යාලය,\nමඩකලපුව.\n කෝවිල්කුලම් හඳුනා නොගත් ස්මාරක පාදා ගැනීමේ කැණීම් ව්‍යපෘති සම්බන්ධවයි.\n මඩකලපුව කාත්තන්කුඩි කෝවිල්කුලම් නමැති හඳුනා නොගත් ස්මාරකය පාදා ගැනීමේ කැණීම් ව්‍යපෘතියට අදාළ කැණීම් අංශය මගින් යොමු කර ඇති ඇස්තමේන්තුවට අදාළ ප්‍රතිපාදන පුරාවිද්‍යා දෙපාර්තමේන්තුවේ පොදු ගිණුම් තැන්පත් කිරීමට කටයුතු කරන මෙන් කාරුණිකව දන්වා සිටිමි.\nපළාත් සහකාර අධ්‍යක්ෂ (නැගෙනහිර)\nයු. බි. විරාජ් එන්. බාලසූරිය"
    ,
      "மாநகர ஆணையாளர்,\nமாத்தளை மாநகர .\nமாத்தளை வீ,டி.நாநயக்கார பூங்காவைச் சுற்றுலாப்பயணிகளை ஈர்க்கும் நிலையமாக அபிவிருத்தி செய்தல்.\nமேற் குறித்த விடயம் தொடர்பாக எனக்கும் ஒரு பிரதி உட்பட உங்களுக்கு அனுப்பப்பட்ட வர்த்தக மற்றும் வியாபார சுற்றுலாத் திணைக்களத்தின் மாகாணப் பணிப்பாளரின் epc/DTCT/DP/05/08 மற்றும் 2016.05.26 ஆந் திகதிய கடிதம் சம்பந்தமாக .\n02. அதற்கமைய உங்கள் உள்ளூராட்சி நிறுவனத்துக்காக முன்வைக்கப்பட்டுள்ள கருத்திட்டத்தை ஒழுங்கான மேற்பார்வையின் கீழ் மற்றும் வழங்கல் செயல்நெறிகளை பின்பற்றி உரிய திகதிக்கும் பூர்த்தி செய்ய நடவடிக்கை மேற்கொள்ளுமாறு தயவுடன் அறியத்திருகிறேன்.\n03. மற்றும் உரிய கருத்திட்டங்களின் முன்னேற்றம் பற்றிய அறிக்கை ஒவ்வொரு மாதமும் 30 ம் திகதிக்கு முன்னர் எனக்கும் ஒரு பிரதியுடன் வியாபார வர்த்தக மற்றும் சுற்றுலாத் திணைக்களத்துக்கு அனுப்புமாறு மேலும் தயவாய் அறிவிக்கிறேன் .\nபீ.எச்.என்.ஜயவிக்ரம,\nஉள்ளூராட்சி ஆணையாளர்,\nமத்திய மாகாணம்.\nபிரதிகள்.\n01.மாகாணப்பணிப்பாளர் வியாபார வர்த்தக மற்றும் சுற்றுலாத் திணைக்களம்- அ.கொ\n02.உள்ளூராட்சி உதவி ஆணையாளர் - மாத்தளை"
    ,
      "දිනය -  2016.12.17\nසහකාර පළාත් පාලන කොමසාරිස් මගින්,\nපළාත් පාලන කොමසාරිස්(මධ්‍යම පළාත) මඟින්,\nආර්. වල්පොල මහතා,\nව්‍යාපෘති අධ්‍යක්‍ෂ,\nපිළිසරු ව්‍යාපෘතිය,\nමධ්‍යම පරිසර අධිකාරිය - පරිසර පියස,\nඅංක 104, ඩෙන්සිල් කොබ්බෑකඩුව මාවත,\nබත්තරමුල්ල.\nපිළිසරු ව්‍යාපෘතියේ මූල්‍යාධාර මත විස්තීරණය කරණු ලබන හිග්ගොල්ල කොම්පෝස්ට් පොහොර නිෂ්පාදන ව්‍යාපෘතිය. \nඉහත සඳහන් ව්‍යාපෘතියේ ඉදිකිරීම් කටයුතු සඳහා කැනීම් කිරීමේදී භූමියේ විශාල ප්‍රමාණයක් ගල් තිබුණු අතර කොටසක් පුරවන ලද භූමියක් නිසා කැනීම් ප්‍රමාණය වැඩි කිරීමට සිදු විය. \nතවද පාෂාණ කඩා ඉවත් කිරීමේ ප්‍රමාණයද වැඩි විය. \nඑසේම ගොඩනැගිල්ලේ ගබඩා කාමරය කොටසේ සිමෙන්ති අච්චුගල් බිත්ති සඳහා වැරගැන්වීම් සහිත කුළුණු හා තලාද යෙදීමට සිදුවිය. \nඅත්තිවාරම ශක්තිමත් කිරීම සඳහා  වැරගැන්වූ කයිරු තලාද යෙදීමට සිදු විය.\nවහලේ කාප්ප අතර අන්තර් සම්බන්ධතාවය ඇති කිරීම සඳහා කාප්ප සම්බන්ධ කිරීමට කේබල් සවි කිරීමටද සිදු විය.\nඑබැවින් ඉහත සඳහන් අතිරේක කාර්යයන් ඇතුළත් කර සකස් කරන ලද සංශෝධිත ඇස්තමේන්තුව සහ මිනුම් වාර්තාව (පිටපත් 03 බැගින්) අනුමැතිය සඳහා මේ සමඟ ඉදිරිපත් කරමි.\nනාගරික කොමසාරිස්,\nමහා නගර සභාව, \nමාතලේ.\nBill Amount - Rs. 8,625,970.50"
    ,
      "ගරු නගරාධිපතිතුමා,\nමාතලේ මහා නගර සභාව.\nගරු නගරාධිපතිතුමණි,\nපළාත් පාලන ආයතන කාර්යාල කළමනාකරණය සම්බන්ධයෙන් නවක කළමනාකරණ සහකාරවරුන් දැනුවත් කිරීමේ වැඩමුළුව - මධ්‍යම පළාත \nපළාත් පාලන ආයතන කාර්යාල කළමනාකරණයේදී කාර්යාල සැලසුම් කිරීම, කාර්යාල පිරිසැකසුම හා වැඩ බෙදීම,බලතල හා කාර්යය පැවරීම, කාර්යාල තොරතුරු ක්‍රම පවත්වාගෙන යාම මඟින් ඉහළ කාර්ය සාධනයක් සඳහා කාර්යාල සූදානම්ව තැබීම සම්බන්ධයෙන් දැනුවත් කිරීමේ අරමුණ ඇතිව දෙදින වැඩමුළුවක් පළාත් කළමනාකරණ සංවර්ධන පුහුණු ඒකකය(MDTU)අනුග්‍රහයෙන්  පළාත් පාලන පුහුණු මධ්‍යස්ථානයේ මෙහෙයවීමෙන් පැවැත්වීමට කටයුතු යොදා ඇති බව කාරුණිකව දන්වනු කැමැත්තෙමි.\nපැවැත්වෙන දිනය - 2016 ජුනි මස 16 හා 17 යන දෙදින පෙ.ව. 08.30 සිට ප.ව. 05.00 දක්වා \nස්ථානය - පළාත් පාලන පුහුණු මධ්‍යස්ථානය, නවයාලතැන්න, කටුගස්තොට.\n02.මෙම වැඩමුළුවට පහත නම් සඳහන් නවක කළමනාකරණ සහකාරවරුන් සහභාගී කරවන ලෙස කාරුණිකව ඉල්ලමි. \nමෙය වැදගත් වැඩමුළුවක් වන බැවින් මේ සඳහා අදාල නිලධාරීන් සහභාගී කරවන ලෙස වැඩිදුරටත් ඉල්ලමි.\n මෙම වැඩමුළුවට සහභාගී වන අය පිළිබඳ විස්තර 2016-06-15 දිනට පෙර ස්ථානභාර නිලධාරී, පළාත් පාලන පුහුණු මධ්‍යස්ථානය, නවයාලතැන්න, කටුගස්තොට යන ලිපිනයට එවන මෙන් ඉල්ලන අතර වැඩමුළුවට සහභාගී වන නිලධාරීන්ට අවශ්‍ය දේශන සටහන් කර ගැනීම සඳහා සී.ආර්. පොතක්ද රැගෙන එන ලෙස උපදෙස් දෙන්නේ නම් මැනවි.\n03.වැඩමුළුවේ කටයුතු සාර්ථක කර ගැනීම සඳහා ඔබතුමා දක්වන සහාය අතිශයින් අගය කරමි.\n01. S.රම්‍යා ප්‍රභා මිය \n02. H.M.N.සෙව්වන්දි මිය \nමෙයට විශ්වාසී,  \nබී.කේ.ජී.ධනපාල,\t\t\t\nස්ථානභාර නිලධාරී.\nඅ/කලේබී.එච්.එන්.ජයවික්‍රම, \t\t\t\t\nපළාත් පාලන කොමසාරිස්.\nපළාත්පාලන පුහුණු මධ්‍යස්ථානය, නවයාලතැන්න, කටුගස්තොට.\nපිටපත්   :-\t\n01.නියෝජ්‍ය ප්‍රධාන ලේකම් (පිරිස් හා පුහුණු), මධ්‍යම පළාත.-දැ.ගැ.පි.\n02.අධ්‍යක්ෂ පළාත් පාලනය පිළිබඳ ශ්‍රී ලංකා ආයතනය (SLILG)-දැ.ගැ.පි.\n03.පළාත් පාලන සහකාර කොමසාරිස් (මහනුවර/මාතලේ/නුවරඑළිය)-දැ.ගැ.පි.\n04.පරිපලාන නිලධාරී පළාත් පාලන දෙපාර්තමේන්තුව (ම.ප)\t-දැ.ගැ.පි."
    ,
      "ප්‍රාදේශීය ලේකම්, \nප්‍රාදේශීය ලේකම් කාර්යාලය,\n මන්මුනේපත්තු,\n ආරයම්පදී.\n කෝවිල් කුලම් පුරාවස්තු ඉඩම් සම්බන්ධවයි\nපුරාවස්තු දෙපාර්තමේන්තුවේ අධ්‍යක්ෂ ජනරාල්ගේ අංක : H/EXP/DIS/25/2014 හා 2014.12.10 දිනැති ලිපිය අනුව ප්‍රාදේශීය පුරාවස්තු සහකාර අධ්‍යක්ෂ මාර්ගයෙන් කෝවිල්කුලම් පුරාවස්තු ප්‍රදේශ මායිම් නිර්ණය කර ඉහත සඳහන් අධ්‍යක්ෂ ජනරාල්ගේ කොන්දේසි වලට එකඟව අවශ්‍ය කටයුතු කඩිනමින් ගන්නා මෙන් ඉල්ලා සිටිමි.\nපුරාවස්තු දෙපාර්තමේන්තුවේ ලිපියේ පිටපත් අමුණා ඇත.\nඑස්. සසිදරන්,\nඅතිරේක දිස්ත්‍රික් ලේකම්, \nමඩකලපුව.\nඅත්සන: පී. එස්. එම්. චාර්ල්ස් මිය,\nදිසාපති සහ දිස්ත්‍රික් ලේකම්,\nමඩකලපුව.\nපිටපත් : 1. අධ්‍යක්ෂ ජනරාල් පුරාවස්තු දෙපාර්තමේන්තුව. \n2. සහකාර අධ්‍යක්ෂ නැ. ප. පුරාවස්තු දෙපාර්තමේන්තුව."
    ,
      "ක්‍රීඩක ක්‍රීඩිකාවන් ක්‍රීඩා සංචිතයට ඇතුලත් වීම සඳහා සපුරාලිය යුතු සුදුසුකම් \n3.01). දිස්ත්‍රික් සංචිතය\nඅමණුම් 2 සඳහන් පරිදි ඒ ඒ ක්‍රීඩාව සඳහා ක්‍රීඩා ශක්ති සංචිත ආරම්භ කළ යුතුය. \nඑසේ ආරම්භ කරන ක්‍රීඩා දිස්ත්‍රික් සම්බන්ධීකරණ නිලධාරින් විසින් තෝරා ගත යුතුය. \nඑසේ ආරම්භ කරන පුහුණු සංචිත දිගටම පවත්වා යාමේ වගකීම දිස්ත්‍රික් සම්බන්ධීකරණ නිලධාරින් සතු වේ.\n1.  තෝරා ගනු ලබන ක්‍රීඩක ක්‍රීඩිකාවන් වයස අවු 14 සහ වයස අවු 18 ට අඩු විය යුතුය. (2002 - 1998) \nක්‍රීඩා කිහිපයකට පමණක් මෙම වයස් සීමාව වෙනස් විය හැක.\nඒ පිළිබද විස්තරයක් ව්‍යපෘති නිලධාරින් විසින් සපයනු ලැබේ.\n2.තෝරා ගනු ලබන ක්‍රීඩක ක්‍රීඩිකාවන් එම දිස්ත්‍රික්කයේ ස්ථීර පදිංචිකරුවන් විය යුතුය.\n3.2015 වර්ෂයේ ක්‍රීඩා ශක්ති පුහුණු සංචිතයේ සිටින ලද ක්‍රීඩක ක්‍රීඩිකාවන්ට දැනට වයස අවු 18 වැඩිනම් ඔවුන් පළාත් සංචිතයට ඇතුලත් කළ හැකිය.\n4. 2015 වර්ෂය ක්‍රීඩා ශක්ති පුහුණු සංචිතයේ සිටින ලද මුත් දක්ෂතා වර්ධනය නොවූ ක්‍රීඩක ක්‍රීඩිකාවන් ඉවත් කල යුතුය. \nඔවුන්ගේ දක්ෂතා වර්ධනය වූ වහාම කැමති නම් නැවත සංචිතට ඇතුලත් විය හැකිය.\n5.තව ද 2015 වර්ෂයේ පැවති ක්‍රීඩා ශක්ති තරග මාලාවට සහභාගී නොවූ ක්‍රීඩක ක්‍රීඩිකාවන් සාධාරණ හේතුන් දක්වා නොමැතිනම් ඉවත් කල යුතුය.\n6. පදිංචි දිස්ත්‍රික්කයෙන් පුහුණු සංචිතයට සහභාගී නොවී වෙනත් දිස්ත්‍රික්කයකට ඇතුලත් වන්නේ නම් එය කලින් දැනුම් දී අධ්‍යක්ෂක (ක්‍රීඩා) වෙතින් අනුමැතිය ලබා ගත යුතුය.\n7.දැනට පවත්වාගෙන යනු ලබන ක්‍රීඩා ශක්ති පුහුණු සංචිත වලින් ඉවත් කරනු ලබන ක්‍රීඩක ක්‍රීඩිකාවන්ගේ අඩුව නවක ක්‍රීඩක ක්‍රීඩිකාවන් මගින් සම්පුර්ණ කල යුතුය.\n(2016 මැයි 20 වන දිනට ප්‍රථම එය සිදු කල යුතුය. )\n3.02  ජාතික සංවර්ධන සංචිතය\nමෙම සංචිතය සඳහා ක්‍රීඩක ක්‍රීඩිකාවන් තෝරා ගනු ලබන්නේ ශ්‍රී ලංකාවම ආවරණය වන පරිදි එක් සංචිතයක් ලෙසය.\n1.වයස අවු 21 ට වැඩි සහ වයස අවු 23 ට අඩු විය යුතුය.(1995 - 1993)\n2. ජාතික සංවර්ධන පුහුණු සංචිතයට ක්‍රීඩක ක්‍රීඩිකාවන් තෝරා ගැනීමේ දී පළාත් සංචිත වල සිටින දක්ෂතා ඇති ක්‍රීඩක ක්‍රීඩිකාවන්ට මූලිකත්වය ලබා දිය යුතුය.\n3. පළාත් ක්‍රීඩක ක්‍රීඩිකාවන් පුහුණුව ලබන දිනවල ඔවුන් සමග පුහුණු වීම කළ යුතුය.\n4.ජාතික සංවර්ධන සංචිතයේ ක්‍රීඩක ක්‍රීඩිකාවන්ට ජාත්‍යන්තර තලයේ අත්දැකීම් ලබා ගැනීමට ඉඩ සලස්වනු ඇත\n5.ජාත්‍යන්තර තලයේ පුහුණුකරුවන්ගේ මග පෙන්වීම හා පුහුණුසැසි ක්‍රීඩා සංවර්ධන දෙපාර්තමේන්තුවේ අධ්‍යක්ෂ(ක්‍රීඩා) විසින් සංවිධානය කල යුතුය."
    ,
      "සුපිරි වෙළඳ සංකීර්ණයේ 2 වන මහල බදුදීම සම්බන්ධවයි. \nමාතලේ සුපිරි වෙළඳ සංකීර්ණයේ දෙවන හා තෙවන මහලේ  කඩ කාමර බදුදීම සඳහා කැඳවන ලද ටෙන්ඩරය අනුව, ටෙන්ඩර්පත් ඉදිරිපත් කර තිබූ ඔබට දෙවන මහලෙන්  .....ක වර්ග අඩි ප්‍රමාණයක් වෙන් කර ඇත. \nඑම කඩ කාමරය වෙන්කර ගැනීමට ප්‍රථම ඔබ විසින් ඉදිරිපත් කර තිබූ ටෙන්ඩර් මුදල අනුව රු. ...............  (.....................  )ක මුදලක් (මාස 03 ක කුලිය) තැන්පත් කල යුතුව ඇත.\nඑම මුදල ... දිනට පෙර ගෙවන ලෙස කාරුණිකව දන්වා සිටිමි.\nනාගරික කොමසාරිස්."
    ,
      "නියෝජ්‍ය අධ්‍යක්ෂ,\nපර්යේෂණ හා විශේෂ අධ්‍යයන අංශය,\nජනලේඛන හා සංඛ්‍යාලේඛන දෙපාර්තමේන්තුව.\nමහත්මයාණෙනි,\nරාජ්‍ය හා අර්ධ රාජ්‍ය අංශයේ සේවා නියුක්තිය පිළිබඳ සංගණනය - 2016\nජනලේඛන හා සංඛ්‍යාලේඛන දෙපාර්තමේන්තුව.\nඋක්ත කරුණ සම්බන්ධයෙන් ජනලේඛන හා සංඛ්‍යාලේඛන අධ්‍යක්ෂ ජනරාල්ගේ අංක 15/EMPCEN2016/2015 හා 2016.05.26 දිනැති ලිපිය  හා බැඳේ.\n02.ඒ අනුව මාතලේ දිස්ත්‍රික්යට අදාල ප්‍රාදේශීය ලේකම් කාර්යාලවල සම්බන්ධීකරණ නිලධාරින්ගේ තොරතුරු ඇතුලත් ලේඛණය ඔබ විසින් එවා ඇති C3 ආකෘතියට අනුව සම්පූර්ණ කර මේ සමඟ එවන බව කාරුණිකව දන්වා සිටිමි.\nතේජානි තිලකරත්න,\nඅතිරේක දිස්ත්‍රික්ලේකම්.\nමාතලේ දිස්ත්‍රික් ලේකම් වෙනුවට."
    ,
      "සාමාන්‍යාධිකාරී ,\nජාතික ජල සම්පාදන හා ජලප්‍රවාහන මණ්ඩලය,\nගැටඹේ, පේරාදෙණිය.\nගම්මාන සංවර්ධන වැඩසටහන - 2016\nජලනල දීර්ඝ කිරීම වල්ලිවෙල - ඒ 9 මාර්ගය මැයෙන් ඔබ විසින් ම වෙත යොමු කළ 2016.06.09 දිනැති ලිපිය හා බැඳේ.\n02. ඒ අනුව වල්ලිවෙල වසමේ යෝජිත ජල ව්‍යාපෘති සඳහා වන ඇස්තමේන්තුව මා වෙත ලැබුණ බව සඳහන් කර සිටින අතර උක්ත වැඩසටහන යටතේ සෑම ව්‍යාපෘතියක් සඳහාම 1% ක් පරිපාලන වියදම් වෙනුවෙන් වෙන්කර ඇති බැවින් එම ඇස්තමේන්තුව රු. 990,000 ක මුදල් ප්‍රමාණයට සකස්කර එවන මෙන් කාරුණිකව ඉල්ලමි.\n03. එසේම උක්ත වැඩසටහනට අදාලව මා විසින් ඔබ වෙත යොමු කළ මාගේ සමාංක හා 2016.06.06 දිනැති ලිපියේ සඳහන් පහත ජල ව්‍යාපෘති සඳහා ද පහත සඳහන් මුදල් ප්‍රමාණයට ඇස්තමේන්තු සකස් කර එවන මෙන් කාරුණිකව ඉල්ලමි.\nඉඹුලන්දණ්ඩ  සිරිමල්වත්ත පාර  ජල නල දීර්ඝ කිරීම රු.198,000.00\nදික්කිරිය    කනත්ත වටරවුම් පාර පානීය ජල පද්ධතිය ලබාදීම. රු.198,000.00 \nකේ.පී.කේ.එල්.පී. මධුවන්ති,\nප්‍රාදේශීය ලේකම්,\nමාතලේ."
    ,
      "ග්‍රාම නිලධාරි,\nමාලිගාතැන්න,\nමාතලේ.\nමියගිය තැන්පත්කරුවන්ගේ ගිණුම් සම්බන්ධ වාර්තා -  ලිපිගොනු අංක ඩීසී/එම්ටී/2016/25\nඋක්ත කරුණ සම්බන්ධයෙන් මාතලේ ජාතික ඉතිරි කිරීමේ බැංකුවේ කළමනාකරු  විසින් මා වෙත එවා ඇති හා 2016.05.23 දිනැති ලිපිය හා බැඳේ.\n2. ඒ අනුව ඔබ ග්‍රාම නිලධාරි වසමේ පහත නම් සඳහන් අයගේ ගිණුමට අදාලව මේ සමග අමුණා ඇති ආකෘති පත්‍ර සම්පුර්ණ කර දින 03 ක් ඇතුලත මෙම කාර්යාලයට ලබාදෙන ලෙස කාරුණිකව දන්වමි.\nලිපිනය \n කුරුවිටගේ අභයජීව අල්විස් මයා\nඅංක 04,\nරාහුල මාවත\nඑම්.සී.පාර\nමාතලේ"
    ,
      "ග්‍රාම නිලධාරි,\n.................. වසම\nවෛද්‍ය ප්‍රතිකාර ලබා ගැනීම සඳහා ජනාධිපති අරමුදලින් මුල්‍යධාරයක් ඉල්ලීම.\n ..................................   මයා\nඔබ වසමේ අංක.............................. ලිපිනයෙහි පදිංචි..........................  මහතා තම අසනීප තත්වයට ප්‍රතිකාර ලබා ගැනීම සදහා ජනාධිපති අරමුදලින් මුල්‍යධාරයක් ඉල්ලුම් කර ඇත.\n02. එම අයදුම් පත්‍රයේ  ජායා පිටපතක් මේ සමග ඉදිරිපත් කරන අතර එහි ජේද අංක 08,09,10 හා 12 යටතේ සදහන් විස්තර වලට අදාලව තොරතුරු ලබාගෙන මාවෙත ලබාදෙන ලෙස කාරුණිකව දන්වමි.\nසහකාර ප්‍රාදේශීය ලේකම්,\nප්‍රාදේශීය ලේකම් මාතලේ වෙනුවට."
    ,
      "දිස්ත්‍රික් ලේකම්/දිසාපති,\nදිස්ත්‍රික් ලේකම් කාර්යාලය,\nමාතලේ.\nගම්මාන සංවර්ධන වැඩසටහන - 2016\nඋක්ත කරුණ සම්බන්ධයෙන් ජාතික ප්‍රතිපත්ති හා ආර්ථික කටයුතු අමාත්‍යංශ ලේකම් විසින් ඔබ වෙත යොමුකළ අංක MNPEA/PLN/RIDP/2016 හා 2016.05.18 දිනැති ලිපිය හා බැඳේ.\n02. ඒ අනුව මාතලේ ප්‍රාදේශීය ලේකම් කොට්ඨාශය තුළ ක්‍රියාත්මක කිරීමට නියමිතව තිබු ප්‍රජා ශාලා ඉදිකිරීම්, ග්‍රාම ලේකම් කාර්යාල ඉදිකිරීම් හා ඒවාට යටිතල පහසුකම් සැපයීම සම්බන්ධ සංවර්ධන යෝජනා ඉවත් කළ අතර ඒ සඳහා ග්‍රාමීය කමිටු වෙතින් ලබාගත් නව සංශෝධිත සංවර්ධන යෝජනා ඔබගේ අනුමැතිය සඳහා කාරුණිකව ඉදිරිපත් කරමි. \nඅදාල නව සංශෝධිත යෝජනා ලැයිස්තුව මේ සමග අමුණා එවන බවද කාරුණිකව සඳහන් කර සිටිමි.\nකේ.පී.කේ.එල්.පී. මධුවන්ති,\nප්‍රාදේශීය ලේකම්,\nමාතලේ."
    ,
      "ඒකාබද්ධ සේවා අධ්‍යක්ෂජනරාල්,\nරාජ්‍ය පරිපාලන හා කළමනාකරණ අමාත්‍යාංශය,\nනිදහස් චතුරශ්‍රය,\nකොළඹ 07.\nඋපාධිධාරී කාර්ය මණ්ඩල තොරතුරු සමාලෝචනය කිරීම.\nඋක්ත කරුණ සම්බන්ධයෙන් මා ඇමතූ ඔබේ අංක  CS/DOS/24/02 හා 2016.04.06 දිනැති ලිපිය හා බැඳේ. \n02. ඒ අනුව අප දිස්ත්‍රික් ලේකම් කාර්යාලයේ හා ප්‍රාදේශිය ලේකම් කාර්යාලවල සේවයේ යෙදී සිටින උපාධිධාරී කාර්ය මණ්ඩලයන් පිළිබඳ තොරතුරු මේ වන විටත් අප විසින් රැස් කරමින් පවතින අතර එම තොරතුරු සාරංශ ගත කොට ආකෘති පත්‍රයට අනුව සැකසීමේ කටයුතු මේ දින වල සිදුවෙමින් පවතින බවද කාරුණිකව දන්වා සිටිමි.\n03. එම නිසා අදාල තොරතුරු විධිමත් පරිදි සැකසීමට යම් කාලසීමාවක් ගත වන බැවින් එම කටයුතු අවසන් වූ වහාම ඔබ වෙත යොමු කිරීමට කටයුතු කරන බව වැඩිදුරටත් කාරුණිකව දන්වා සිටිමි.\nතේජානි තිලකරත්න,\nඅතිරේක දිස්ත්‍රික් ලේකම්.\nමාතලේ දිස්ත්‍රික් ලේකම් වෙනුවට."
    ,
      "ක්‍රීඩා ශක්ති චක්‍රලේඛය - 02/2016\nසියලුම දිස්ත්‍රික් සම්බන්ධීකරණ නිලධාරීන් වෙත.\nක්‍රීඩා ශක්ති වැඩ සටහන සවිබල ගැන්වීම. \nක්‍රීඩා අමාත්‍යාංශය හා ක්‍රීඩා සංවර්ධන දෙපාර්තමේන්තුව මගින් සංවිධානය කරනු ලබන ක්‍රීඩා ශක්ති වැඩසටහන. \n2016 වර්ෂයේ නව මුහුණුවරකින් දීපව්‍යාප්තව විසිරී සිටින ක්‍රීඩා කෞශල්‍යයන්ගෙන් හෙබි ක්‍රීඩක ක්‍රීඩිකාවන් හඳුනා ඔවුන්ගේ ක්‍රීඩා දක්ෂතා සඳහා සංවිධානය කරනු ලබන මෙම ක්‍රීඩා ශක්ති වැඩසටහන එක් එක් ක්‍රීඩාව සඳහා පත් කර ඇති ව්‍යපෘති නිලධාරීන් සහ දිස්ත්‍රික් ලේකම් කාර්යාලයට අනුයුක්ත සේවය කරන දිස්ත්‍රික් සම්බන්ධීකරණ නිලධාරීන්, පුහුණුකරුවන් හා ක්‍රීඩා නිලධාරීන් මගින් ක්‍රියාත්මක වේ.\n මෙම වැඩසටහන පළාත් ක්‍රීඩා ලේකම්, පළාත් ක්‍රීඩා අධ්‍යක්ෂ, දිස්ත්‍රික් ක්‍රීඩා නිලධාරීන්, පළාත් ක්‍රීඩා පුහුණුකරුවන්, පළාත් ක්‍රීඩා නිලධාරීන්, අධ්‍යාපන අමාත්‍යාංශයේ ශාරීරික උපදේශකවරුන්, ක්‍රීඩා සංගම්වල පුහුණුකරුවන්ගේ ක්‍රියාකාරී සහයෝගය සහ දායකත්වය උසස් මට්ටමින් ලබා දෙනු ඇතැයි අපේක්ෂා කෙරේ.\n02) අදියරයන්\n1 අදියර \nදිස්ත්‍රික්කය තුල සිටින, වයස අවු 14 ට වැඩි සහ වයස අවු 18 ට අඩු ක්‍රීඩා කෞශල්‍යයන්ගෙන් හෙබි ක්‍රීඩක ක්‍රීඩිකාවන්ගෙන් සමන්විත එක් එක් ක්‍රීඩාවට ඇමණුම 01 සඳහන් පරිදි සංචිත 51ක් ඇති කරනු ලැබේ.\n2 අදියර\nසෑම දිස්ත්‍රික්කයක්ම ආවරණය වන පරිදි වයස අවු 18 ට වැඩි සහ වයස අවු 21 ට අඩු ක්‍රීඩක ක්‍රීඩිකාවන්ගෙන් සැදුම්ලත් පළාත් සංචිත ඇති කරන අතර එය ක්‍රීඩා සංවර්ධන සංචිතයක් ලෙස පවත්වනු ලැබේ. \nමෙහිදී ක්‍රීඩා ශක්ති දිස්ත්‍රික් සංචිත වල සිටින ලද වයස 18 වැඩි දක්ෂතා පූර්ණ ක්‍රීඩක ක්‍රීඩිකාවන්ට මුල් තැන දිය යුතුය. \nඔවුන්ගේ දක්ෂතා අනුව තෝරා ගෙන අඩු පාඩු සඳහා නවක ක්‍රීඩක ක්‍රීඩිකාවන් තෝරා සම්පූර්ණ කළ යුතුය.\n3 අදියර\nසෑම දිස්ත්‍රික්කයක්ම ආවරණය වන පරිදි වයස අවු 21 ට වැඩි සහ වයස අවු 23 ට අඩු ක්‍රීඩක ක්‍රීඩිකාවන් මෙම ප්‍රවර්ධන සංචිතයට ඇතුලත් කරනු ලැබේ. \nමේ සඳහා එක් සංචිතයක් පමණක් පවත්වා ගෙන යනු ලැබේ."
    ,
      "2016/06/10\n01. S.රම්‍යා ප්‍රභා මිය - කලමනාකරණ සහකාර \n02. H.M.N.සෙව්වන්දි මිය - කලමනාකරණ සහකාර \nපළාත් පාලන ආයතන කාර්යාල කළමනාකරණය සම්බන්ධයෙන් නවක කළමනාකරණ සහකාරවරුන් දැනුවත් කිරීමේ වැඩමුළුව - මධ්‍යම පළාත. \nඋක්ත කරුණ සම්බන්ධයෙන් මධ්‍යම පළාත් පාලන කොමසාරිස්ගේ අංක සීපීසී/සීඑල්ජී/01/AC/පුහුණු/1/2/2016-07 හා 2016.05.06 දිනැති ලිපිය හා බැඳේ.\n02. ඒ අනුව 2016 ජුනි මස 16 සහ 17 දෙදින පෙ.ව. 8.30 සිට ප.ව. 5.00 දක්වා පළාත් පාලන පුහුණු මධ්‍යස්ථානය, නවයාලතැන්න දී පැවැත්වීමට නියමිත ඉහත වැඩමුළුව සඳහා මාතලේ මහා නගර සභාව නියෝජනය කරමින්  සහභාගී වන ලෙස කාරුණිකව දන්වමි.\n03. පුහුණු වැඩසටහන අවසානයේ ඔබ ලැබූ පුහුණුව සම්බන්ධව වාර්තාවක් පහත ආකෘතිය පරිදි සම්පූර්ණකර ආයතන අංශය වෙත අනිවාර්යයෙන්ම ලබා දෙන මෙන් ද දන්වමි.\nනම:-\nතනතුර:-\nසේවය කරන දෙපාර්තමේන්තුව:- \nපුහුණු වැඩමුළුවේ නම:- \nපැවැත්වූ දිනය:- \nපැවැත්වූ ස්ථානය:- \nසම්පත් දායකයා/දායකයන්ගේ නම්:- \nවැඩමුළුවේ අන්තර්ගතය (සවිස්තරව)\nරාජකාරී ඉටු කිරීමේ දී ඉවහල්කර ගන්නා අයුරු (සවිස්තරව)\nමෙම වැඩමුළුවට සහභාගී වීමේදී අවශ්‍ය දේශන සටහන් කර ගැනීම සඳහා සී.ආර්. පොතක්ද රැගෙන යන ලෙස වැඩිදුරටත් මෙයින් දන්වමි.\nනාගරික කොමසාරිස්, \nමාතලේ."
    ,
      "සහකාර අධ්‍යක්ෂ (නැගෙනහිර)\nකෝවිල්කුලම් පුරාවිද්‍යා ස්ථානය සම්බන්ධව.\nඉහත ස්ථානය සම්බන්ධයෙන් 2015.03.12 දිනැතිව මා වෙත ලැබුණු ඇමතුම හා සබැඳේ.\n02. ඒ අනුව සෙල්වරාජා නැමැති මන්ත්‍රීවරයෙකු විසින් දන්වා සිටියේ මෙම ස්ථානය මේ වන විට යම්කිසි පිරිසක් විනාසකාරි ක්‍රියාකාරකම් සිදු කරන බවයි. \nඑබැවින් මේ සම්බන්ධයෙන් පරික්ෂාකොට බලා සුදුසු ක්‍රියාමාර්ග ගන්න මෙන් කාරුණිකව දන්වා සිටිමි.\nඑන්. බී. කරව්ගහන්ග\nවැ.ආ. සහකාර අධ්‍යක්ෂ (නිශ්චළ දේපල )\nඅ. කලේ / ආචාර්ය සෙනරත් දිසානායක \nපුරාවිද්‍යා අධ්‍යක්ෂ ජනරාල්"

]

# Generate embeddings for documents and save for later use
document_embeddings = model.encode(documents, convert_to_tensor=True)
doc_embeddings = normalize(document_embeddings.cpu().numpy(), axis=1)

dimension = doc_embeddings.shape[1]
index = faiss.IndexFlatIP(dimension)  # Inner product (cosine similarity)
index.add(doc_embeddings)

# Save documents and embeddings to disk
# with open('documents.pkl', 'wb') as f:
#     pickle.dump((documents, doc_embeddings), f)
# print("Document embeddings precomputed and saved.")

In [ ]:
def retrieve_documents(query, k=3):
    # Encode query
    query_embedding = model.encode([query], convert_to_tensor=True)
    query_embedding = normalize(query_embedding.cpu().numpy(), axis=1)

    # Search index
    distances, indices = index.search(query_embedding, k)

    # Return results
    results = []
    for idx, score in zip(indices[0], distances[0]):
        results.append({
            "text": documents[idx],
            "score": float(score)
        })
    return results

# Example usage
# query = "පරිසර දිනය සැමරීම සඳහා උත්සවයකට අවසර ලබා ගැනීමට අදාළ පාර්ශවයන්ට ලිපියක් ලියන්න."
query = "අසනීප තත්වයක් නිසා උත්සවයට සහභාගී වීමට නොහැකි වන බව දන්වා පාසල් විද්‍යා සංගමයේ සභාපති තුමාට ලිපියක් ලියන්න"
results = retrieve_documents(query, k=2)

print("Query:", query)
print("\nTop results:")
for i, res in enumerate(results):
    print(f"\nResult {i+1}:")
    # print(f"Type: {res['type']}")
    print(f"Similarity Score: {res['score']:.4f}")
    print(f"Text: {res['text']}")

Query: අසනීප තත්වයක් නිසා උත්සවයට සහභාගී වීමට නොහැකි වන බව දන්වා පාසල් විද්‍යා සංගමයේ සභාපති තුමාට ලිපියක් ලියන්න

Top results:

Result 1:
Similarity Score: 0.3725
Text: මාගේ අංකය  SLEAS/2016/21
දිනය  2016.04.19
ලේකම්, අධ්‍යාපන අමාත්‍යංශය,
රාජ්‍ය ලේකම්, අධ්‍යාපන අමාත්‍යංශය,
සියලුම පළාත් ප්‍රධාන/අධ්‍යාපන අමාත්‍යංශයවල ප්‍රධාන අධ්‍යාපන ලේකම්වරුන්,
විභාග කොමසාරිස් ජනරාල්, විභාග දෙපාර්තමේන්තුව,
අධ්‍යාපන ප්‍රකාශන කොමසාරිස් ජනරාල්, අධ්‍යාපන ප්‍රකාශන දෙපාර්තමේන්තුව,
සියලුම පළාත් අධ්‍යාපන අධ්‍යක්ෂවරුන් ,
අධ්‍යක්ෂ ජනරාල්, ජාතික අධ්‍යාපන ආයතනය,
සියලුම කලාප අධ්‍යාපන අධ්‍යක්ෂවරුන්,
සියලුම ජාතික පාසල් විදුහල්පතිවරුන්.
සේවා ව්‍යවස්ථාවට පටහැනිව පත්වීම් හා උසස්වීම ලබා දීමට ගන්නා උත්සාහයට විරෝධය දැක්වීමේ වෘත්තීය ක්‍රියාමාර්ග පිළිබඳව දැනුවත් කිරීම.
අධ්‍යාපන ලේකම්ට යොමු කරන ලද SLEAS/2016/20 හා 2016.04.12 දිනැති ශ්‍රී ලංකා අධ්‍යාපන පරිපාලන සේවයේ සේවා ව්‍යවස්ථාව උල්ලංඝනය කරමින් උසස්වීම් හා පත්වීම් ලබා දීමේ උත්සාහය පිළිබඳ ලිපියට අනුව එම ලිපියේ සඳහන් කරුණු පිළිබඳව අප සංගමය වෙත සෑහීමකට පත්විය හැකි පිළිතුරක් නො